<a href="https://colab.research.google.com/github/OrbitPortal/ARINC424_Converter/blob/master/ARINC424_Converter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import re
import shutil
import sqlite3
import string
import time
import urllib
from decimal import ROUND_HALF_UP, Decimal

import msaccessdb
import pandas
from sqlalchemy import create_engine

# addtional dependency: sqlalchemy-access
# addtional dependency: pyodbc

# Regex for per-row slicing in 424-file
as_grid = r"^(S|T)[ ]{3}(A)(S).{7}(.)(.{2})(.)(.{3})(.).{9}(.{3})(.{3})(.{3})(.{3})(.{3})(.{3})(.{3})(.{3})(.{3})(.{3})(.{3})(.{3})(.{3})(.{3})(.{3})(.{3})(.{3})(.{3})(.{3})(.{3})(.{3})(.{3})(.{3})(.{3})(.{3})(.{3})(.{3})(.{3})(.{3})(.{3}).{3}([0-9]*)([0-9]{4})$"
vor = r"^(S|T)(.{3})(D)( )(.{4})(.{2}).(.{4}).{2}(.{2})(0|1)(.{5})(.{5})(N|S| )(.{2})(.{2})(.{2})(.{2})(E|W| )(.{3})(.{2})(.{2})(.{2})(.{4})(N|S| )(.{2})(.{2})(.{2})(.{2})(E|W| )(.{3})(.{2})(.{2})(.{2})(.{5})(.{5})(.)(.{2})(.{3})(.{3})(.{25}).{3}(.)(.)([0-9]*)([0-9]{4})$"
db = r"^(S|T)(.{3})(D)(B)(.{4})(.{2}).(.{4}).{2}(.{2})(0|1)(.{5})(.{5})(N|S)(.{2})(.{2})(.{2})(.{2})(E|W)(.{3})(.{2})(.{2})(.{2}).{23}(E|W)(.{4}).{11}(.{3})(.{30})([0-9]*)([0-9]{4})$"
ea = r"^(S|T)(.{3})(E)(A)(.{4})(.{2})( )(.{5})[ ](.{2})(0|1)[ ]{4}(.{3})(.{2})[ ](N|S)(.{2})(.{2})(.{2})(.{2})(E|W)(.{3})(.{2})(.{2})(.{2})[ ]{23}(E|W)(.{4})[ ]{5}(.{3})[ ]{11}(.{25})([0-9]*)([0-9]{4})$"
ep = r"^(S|T)(.{3})(E)(P)(.{4})(.{2}).{15}(.{2})(.{5})(.{2})(.)(.)(0|1)(.{4})(.)(.{3})(.{2})(.{5})(.{5})(.{3})(.{3})(.{6})(.{3})(.{3})(.{3}).{18}(.{25})([0-9]*)([0-9]{4})$"
er = r"^(S|T)(.{3})(E)(R).{7}(.{5})..{6}(.{4})(.{5})(.{2})(.{1})(.{1})(0|1)(.{4})(.)(.)(.)(.)(..)(.)(.{4})(.{2})(.{3}).{3}(.{4})(.{4})(.{4})(.{4})(.{4}).(.{5})(.{5})(.{5})(.{3})(.{3})(.{3})(.{3}).{4}(.{2}).{4}.{3}([0-9]*)([0-9]{4})$"
pa = r"^(S|T)(.{3})(P).(.{4})(.{2})(A)(.{3}).{2}.{3}(0|1)(.{5})(.{3})(.)(.)(N|S)(.{2})(.{2})(.{2})(.{2})(E|W)(.{3})(.{2})(.{2})(.{2})(E|W)(.{4})(.{5})(.{3})(.{4})(.{2})(.{5})(.{5})(.)(.{3})(.)(.)(.{3}).{4}(.{30})([0-9]*)([0-9]{4})$"
pc = r"^(S|T)(.{3})(P)( )(.{4})(.{2})(C)(.{5})[ ](.{2})(0|1)[ ]{4}(.{3})(.{2})[ ](N|S)(.{2})(.{2})(.{2})(.{2})(E|W)(.{3})(.{2})(.{2})(.{2})[ ]{23}(E|W)(.{4})[ ]{5}(.{3})[ ]{11}(.{25})([0-9]*)([0-9]{4})$"
pdef = r"^(S|T)(.{3})(P).(.{4})(.{2})(D|E|F)(.{6})(.)(.{5}).(.{3})(.{5})(.{2})(.)(.)(0|1)(.{4})(.)(.{3})(.{2})(.)(.{4})(.{2})(.{6})(.{4})(.{4})(.{4})(.{4})(.)(.).{2}(.)(.)(.{5})(.{5})(.{5})(.{3})(.{4})(.{5})(.)(.{2})(.)(.)(.)(.)(.)(.).{3}([0-9]*)([0-9]{4})$"
pg = r"^(S|T)(.{3})(P).(.{4})(.{2})(G)(.{5}).{3}(0|1)(.{5})(.{4}).(N|S)(.{2})(.{2})(.{2})(.{2})(E|W)(.{3})(.{2})(.{2})(.{2})(.)(.{4}).{4}(.{6})(.{5})(.{4})(.{2})(.{3})(.)(.{4})(.)(.{4})(.{4})(.).{6}(.{22})([0-9]*)([0-9]{4})$"
pi = r"^^(S|T)(.{3})(P).(.{4})(.{2})(I)(.{4})(.).{3}(0|1)(.{5})(.{5})(N|S| )(.{2})(.{2})(.{2})(.{2})(E|W| )(.{3})(.{2})(.{2})(.{2})(.{4})(N|S| )(.{2})(.{2})(.{2})(.{2})(E|W| )(.{3})(.{2})(.{2})(.{2})(.{4})(.)(.{4})(.{4})(.{3})(E.{4}|W.{4})(.{2})(.{5})(.{4})(.{2})(.)(.).{13}([0-9]*)([0-9]{4})$"
pm = r"^(S|T)(.{3})(P).(.{4})(.{2})(M)(.{4})(.{3}).(0|1)(.{5})(.{5})(N|S| )(.{2})(.{2})(.{2})(.{2})(E|W| )(.{3})(.{2})(.{2})(.{2})(.{4})(N|S| )(.{2})(.{2})(.{2})(.{2})(E|W| )(.{3})(.{2})(.{2})(.{2})(.{5})(.{5})(.{4}).{2}(E.{4}|W.{4}).{2}(.{5}).{21}([0-9]*)([0-9]{4})$"
pn = r"^(S|T)(.{3})(P)(N)(.{4})(.{2}).(.{4}).{2}(.{2})(0|1)(.{5})(.{5})(N|S)(.{2})(.{2})(.{2})(.{2})(E|W)(.{3})(.{2})(.{2})(.{2}).{23}(E|W)(.{4}).{11}(.{3})(.{30})([0-9]*)([0-9]{4})$"
pt = r"(S|T)(.{3})(P).(.{4})(.{2})(T)(.{4})(.).{3}(0|1)(.{5})(.{5}).{19}(.{4})(N|S| )(.{2})(.{2})(.{2})(.{2})(E|W| )(.{3})(.{2})(.{2})(.{2})(.{4}).{5}(.{2})(.{2})(.{3})(E.{4}|W.{4}).{2}(.{5})(.{3})(.{3}).{2}(.{5}).{8}([0-9]*)([0-9]{4})$"
tc = r"^(S|T)[ ]{3}(T)(C)(.{2})(.)[ ]{19}(.{4})(.{4})(M|T)[ ]{2}(.{5})(.{5})(.{5})(.{5})(.{5})(.{5})(.{5})(.{5})(.{5})(.{5})(.{5})(.{5}).{24}([0-9]*)([0-9]{4})$"
# Regex for per-row slicing in Jeehell's navrecord.dat
vor_jh_nd = r"^(D )\|([A-Z]{1,4})\|([A-Z]{2})\|[0-9.]{1,}\|.{5}\|([0-9.]{0,})\|([0-9.]{0,})\|([0-9.]{0,})\|([0-9.]{0,})\|.{0,}\|[0-9.]{0,}\|[0-9.]{0,}\|.{0,}\|$"
db_jh_nd = r"^(DB)\|([A-Z]{1,4})\|([A-Z]{2})\|[0-9.]{1,}\|.{5}\|([0-9.]{0,})\|([0-9.]{0,})\|.{0,}\|$"
ea_jh_nd = r"^(EA)\|([A-Z0-9]{1,5})\|([A-Z]{2})\|[A-Z ]{3}\|([0-9.]{1,})\|([0-9.]{1,})\|[A-Z]{4}\|.{0,}\|$"
# Regex for per-row slicing in AS's Airport.txt
pa_as_nd = r"^A,(.*),(.*),(.*),(.*),(.*),(.*),(.*),(.*),(.*)$"
pg_as_nd = r"^R,(.*),(.*),(.*),(.*),(.*),(.*),(.*),(.*),(.*),(.*),(.*),(.*),1,0$"


# Options
FSLabs = True
JeeHell = True
MJC = True
ProSim = True
TFDi = True
Aerosoft = True
add_tailored = False

if add_tailored:
    fsl_create_blank = False
    jh_create_blank = False
    mjc_create_blank = False
    prosim_create_blank = False
    tfdi_create_blank = False
    as_create_blank = False
else:
    fsl_create_blank = True
    jh_create_blank = True
    mjc_create_blank = True
    prosim_create_blank = True
    tfdi_create_blank = True
    as_create_blank = True

fsl_write = True
jh_output_navrecord = True
jh_output_additional = True
mjc_write = True
prosim_write = True
tfdi_write = True
as_write = True


# Necessary Info for MJC/ProSim/TFDi when create_blank = True
CycleEndDate = '09SEP20'
CycleName = '2009'
CycleStartDate = '13AUG20'


Datafile = input('Please Input The 424-file Name (In "Input" Folder): ')
Cycle_Ident = Datafile.split('.')[0]
if os.path.isdir(Cycle_Ident):
    shutil.rmtree(Cycle_Ident)
print(time.strftime("%H:%M:%S", time.localtime()) , 'System: Converting DataFile', Datafile)

# FSLabs (Completed)
if FSLabs:
    print(time.strftime("%H:%M:%S", time.localtime()) , 'FSLabs: Start Processing DataFile')
    os.makedirs(Cycle_Ident + '/FSLabs')
    if fsl_create_blank != True:
        shutil.copy2('Input/FSLabs/' + Cycle_Ident + '.rom', Cycle_Ident + '/FSLabs/' + Cycle_Ident + '.rom')
    # Set Variable of FSLabs
    if 1:
        mora_fsl = []
        vhf_fsl = []
        ndb_fsl = []
        wpt_fsl = []
        hold_fsl = []
        awy_fsl = []
        arpt_fsl = []
        proc_fsl = []
        rwy_fsl = []
        loc_fsl = []
        crz_fsl = []
        if add_tailored:
            FILE_RECD_NR = 2800000
        else:
            FILE_RECD_NR = 0
    # Process Data
    with open('Input/' + Datafile, 'r', encoding='ISO-8859-1') as ARINC424:
        # Process of FSLabs
        for line in ARINC424: 
            mora_se = re.search(as_grid, line)
            vhf_se = re.search(vor, line)
            db_se = re.search(db, line)
            ea_se = re.search(ea, line)
            hold_se = re.search(ep, line)
            awy_se = re.search(er, line)
            arpt_se = re.search(pa, line)
            pc_se = re.search(pc, line)
            proc_se = re.search(pdef, line)
            rwy_se = re.search(pg, line)
            loc_se = re.search(pi, line)
            pn_se = re.search(pn, line)
            crz_se = re.search(tc, line)
            if mora_se:
                if add_tailored:
                    RECD_TYPE = "T"
                else:
                    RECD_TYPE = mora_se.group(1)
                SECT_CODE = mora_se.group(2)
                SUBS_CODE = mora_se.group(3)
                if mora_se.group(4) == "N":
                    MORA_LAT = float(mora_se.group(5))
                else:
                    MORA_LAT = 0 - float(mora_se.group(5))
                if mora_se.group(6) == "E":
                    MORA_LON = float(mora_se.group(7))
                else:
                    MORA_LON = 0 - float(mora_se.group(7))
                MORA_TYPE = mora_se.group(8)
                if mora_se.group(9) == "UNK":
                    MORA_00 = None
                else:
                    MORA_00 = mora_se.group(9)
                if mora_se.group(10) == "UNK":
                    MORA_01 = None
                else:
                    MORA_01 = mora_se.group(10)
                if mora_se.group(11) == "UNK":
                    MORA_02 = None
                else:
                    MORA_02 = mora_se.group(11)
                if mora_se.group(12) == "UNK":
                    MORA_03 = None
                else:
                    MORA_03 = mora_se.group(12)
                if mora_se.group(13) == "UNK":
                    MORA_04 = None
                else:
                    MORA_04 = mora_se.group(13)
                if mora_se.group(14) == "UNK":
                    MORA_05 = None
                else:
                    MORA_05 = mora_se.group(14)
                if mora_se.group(15) == "UNK":
                    MORA_06 = None
                else:
                    MORA_06 = mora_se.group(15)
                if mora_se.group(16) == "UNK":
                    MORA_07 = None
                else:
                    MORA_07 = mora_se.group(16)
                if mora_se.group(17) == "UNK":
                    MORA_08 = None
                else:
                    MORA_08 = mora_se.group(17)
                if mora_se.group(18) == "UNK":
                    MORA_09 = None
                else:
                    MORA_09 = mora_se.group(18)
                if mora_se.group(19) == "UNK":
                    MORA_10 = None
                else:
                    MORA_10 = mora_se.group(19)
                if mora_se.group(20) == "UNK":
                    MORA_11 = None
                else:
                    MORA_11 = mora_se.group(20)
                if mora_se.group(21) == "UNK":
                    MORA_12 = None
                else:
                    MORA_12 = mora_se.group(21)
                if mora_se.group(22) == "UNK":
                    MORA_13 = None
                else:
                    MORA_13 = mora_se.group(22)
                if mora_se.group(23) == "UNK":
                    MORA_14 = None
                else:
                    MORA_14 = mora_se.group(23)
                if mora_se.group(24) == "UNK":
                    MORA_15 = None
                else:
                    MORA_15 = mora_se.group(24)
                if mora_se.group(25) == "UNK":
                    MORA_16 = None
                else:
                    MORA_16 = mora_se.group(25)
                if mora_se.group(26) == "UNK":
                    MORA_17 = None
                else:
                    MORA_17 = mora_se.group(26)
                if mora_se.group(27) == "UNK":
                    MORA_18 = None
                else:
                    MORA_18 = mora_se.group(27)
                if mora_se.group(28) == "UNK":
                    MORA_19 = None
                else:
                    MORA_19 = mora_se.group(28)
                if mora_se.group(29) == "UNK":
                    MORA_20 = None
                else:
                    MORA_20 = mora_se.group(29)
                if mora_se.group(30) == "UNK":
                    MORA_21 = None
                else:
                    MORA_21 = mora_se.group(30)
                if mora_se.group(31) == "UNK":
                    MORA_22 = None
                else:
                    MORA_22 = mora_se.group(31)
                if mora_se.group(32) == "UNK":
                    MORA_23 = None
                else:
                    MORA_23 = mora_se.group(32)
                if mora_se.group(33) == "UNK":
                    MORA_24 = None
                else:
                    MORA_24 = mora_se.group(33)
                if mora_se.group(34) == "UNK":
                    MORA_25 = None
                else:
                    MORA_25 = mora_se.group(34)
                if mora_se.group(35) == "UNK":
                    MORA_26 = None
                else:
                    MORA_26 = mora_se.group(35)
                if mora_se.group(36) == "UNK":
                    MORA_27 = None
                else:
                    MORA_27 = mora_se.group(36)
                if mora_se.group(37) == "UNK":
                    MORA_28 = None
                else:
                    MORA_28 = mora_se.group(37)
                if mora_se.group(38) == "UNK":
                    MORA_29 = None
                else:
                    MORA_29 = mora_se.group(38)
                FILE_RECD_NR = FILE_RECD_NR + 1
                CYCLE = mora_se.group(40)
                mora_fsl.append((RECD_TYPE,SECT_CODE,SUBS_CODE,MORA_LAT,MORA_LON,MORA_TYPE,MORA_00,MORA_01,MORA_02,MORA_03,MORA_04,MORA_05,MORA_06,MORA_07,MORA_08,MORA_09,MORA_10,MORA_11,MORA_12,MORA_13,MORA_14,MORA_15,MORA_16,MORA_17,MORA_18,MORA_19,MORA_20,MORA_21,MORA_22,MORA_23,MORA_24,MORA_25,MORA_26,MORA_27,MORA_28,MORA_29,FILE_RECD_NR,CYCLE))
            if vhf_se:
                if add_tailored:
                    RECD_TYPE = "T"
                else:
                    RECD_TYPE = vhf_se.group(1)
                CUST_AREA = vhf_se.group(2)
                SECT_CODE = vhf_se.group(3)
                SUBS_CODE = vhf_se.group(4)
                ARPT_IDENT = vhf_se.group(5).strip()
                ARPT_ICAO_CODE = vhf_se.group(6)
                VOR_IDENT = vhf_se.group(7).strip()
                VHF_ICAO_CODE = vhf_se.group(8)
                CONT_NR = vhf_se.group(9)
                VOR_FREQ = float(vhf_se.group(10))/100
                NAVAID_CLASS = vhf_se.group(11)
                if vhf_se.group(12).strip() == '':
                    VOR_LAT = None
                elif vhf_se.group(12) == "N":
                    VOR_LAT = float(vhf_se.group(13)) + float(vhf_se.group(14))/60 + float(vhf_se.group(15))/3600 + float(vhf_se.group(16))/360000
                else:
                    VOR_LAT = 0 - float(vhf_se.group(13)) - float(vhf_se.group(14))/60 - float(vhf_se.group(15))/3600 - float(vhf_se.group(16))/360000
                if vhf_se.group(17).strip() == '':
                    VOR_LON = None
                elif vhf_se.group(17) == "E":
                    VOR_LON = float(vhf_se.group(18)) + float(vhf_se.group(19))/60 + float(vhf_se.group(20))/3600 + float(vhf_se.group(21))/360000
                else:
                    VOR_LON = 0 - float(vhf_se.group(18)) - float(vhf_se.group(19))/60 - float(vhf_se.group(20))/3600 - float(vhf_se.group(21))/360000
                DME_IDENT = vhf_se.group(22).strip()
                if vhf_se.group(23).strip() == '':
                    DME_LAT = None
                elif vhf_se.group(23) == "N":
                    DME_LAT = float(vhf_se.group(24)) + float(vhf_se.group(25))/60 + float(vhf_se.group(26))/3600 + float(vhf_se.group(27))/360000
                else:
                    DME_LAT = 0 - float(vhf_se.group(24)) - float(vhf_se.group(25))/60 - float(vhf_se.group(26))/3600 - float(vhf_se.group(27))/360000
                if vhf_se.group(28).strip() == '':
                    DME_LON = None
                elif vhf_se.group(28) == "E":
                    DME_LON = float(vhf_se.group(29)) + float(vhf_se.group(30))/60 + float(vhf_se.group(31))/3600 + float(vhf_se.group(32))/360000
                else:
                    DME_LON = 0 - float(vhf_se.group(29)) - float(vhf_se.group(30))/60 - float(vhf_se.group(31))/3600 - float(vhf_se.group(32))/360000
                STATION_DECLINATION = vhf_se.group(33)
                if vhf_se.group(34).strip() == '':
                    DME_ELEV = None
                else:
                    DME_ELEV = vhf_se.group(34)
                FIGURE_OF_MERIT = vhf_se.group(35)
                if vhf_se.group(36).strip() == '':
                    ILS_DME_BIAS = None
                else:
                    ILS_DME_BIAS = vhf_se.group(36)
                FREQ_PROTECTION = vhf_se.group(37)
                DATUM_CODE = vhf_se.group(38)
                VOR_NAME = vhf_se.group(39).replace('VOR/DME','       ').replace('(','').replace(')','')
                ROUTE_INAPPROPRIATE_DME = vhf_se.group(40)
                DME_OPERATIONAL_SERVICE_VOLUME = vhf_se.group(41)
                FILE_RECD_NR = FILE_RECD_NR + 1
                CYCLE = vhf_se.group(43)
                vhf_fsl.append((RECD_TYPE,CUST_AREA,SECT_CODE,SUBS_CODE,ARPT_IDENT,ARPT_ICAO_CODE,VOR_IDENT,VHF_ICAO_CODE,CONT_NR,VOR_FREQ,NAVAID_CLASS,VOR_LAT,VOR_LON,DME_IDENT,DME_LAT,DME_LON,STATION_DECLINATION,DME_ELEV,FIGURE_OF_MERIT,ILS_DME_BIAS,FREQ_PROTECTION,DATUM_CODE,VOR_NAME,ROUTE_INAPPROPRIATE_DME,DME_OPERATIONAL_SERVICE_VOLUME,FILE_RECD_NR,CYCLE))
            if db_se:
                if add_tailored:
                    RECD_TYPE = "T"
                else:
                    RECD_TYPE = db_se.group(1)
                CUST_AREA = db_se.group(2)
                SECT_CODE = db_se.group(3)
                SUBS_CODE = db_se.group(4)
                ARPT_IDENT = db_se.group(5).strip()
                ARPT_ICAO_CODE = db_se.group(6)
                NDB_IDENT = db_se.group(7).strip()
                NDB_ICAO_CODE = db_se.group(8)
                CONT_NR = db_se.group(9)
                NDB_FREQ = float(db_se.group(10))/10
                NDB_CLASS = db_se.group(11)
                if db_se.group(12) == "N":
                    NDB_LAT = float(db_se.group(13)) + float(db_se.group(14))/60 + float(db_se.group(15))/3600 + float(db_se.group(16))/360000
                else:
                    NDB_LAT = 0 - float(db_se.group(13)) - float(db_se.group(14))/60 - float(db_se.group(15))/3600 - float(db_se.group(16))/360000
                if db_se.group(17) == "E":
                    NDB_LON = float(db_se.group(18)) + float(db_se.group(19))/60 + float(db_se.group(20))/3600 + float(db_se.group(21))/360000
                else:
                    NDB_LON = 0 - float(db_se.group(18)) - float(db_se.group(19))/60 - float(db_se.group(20))/3600 - float(db_se.group(21))/360000
                if db_se.group(22) == "E":
                    MAG_VAR = float(db_se.group(23))/100
                else:
                    MAG_VAR = 0 - float(db_se.group(23))/100
                DATUM_CODE = db_se.group(24)
                NDB_NAME = db_se.group(25).replace('NDB','   ').replace('(','').replace(')','')
                FILE_RECD_NR = FILE_RECD_NR + 1
                CYCLE = db_se.group(27)
                ndb_fsl.append((RECD_TYPE,CUST_AREA,SECT_CODE,SUBS_CODE,ARPT_IDENT,ARPT_ICAO_CODE,NDB_IDENT,NDB_ICAO_CODE,CONT_NR,NDB_FREQ,NDB_CLASS,NDB_LAT,NDB_LON,MAG_VAR,DATUM_CODE,NDB_NAME,FILE_RECD_NR,CYCLE))
            if ea_se:
                if add_tailored:
                    RECD_TYPE = "T"
                else:
                    RECD_TYPE = ea_se.group(1)
                CUST_AREA = ea_se.group(2)
                SECT_CODE = ea_se.group(3)
                SUBS_CODE = ea_se.group(4)
                REGION_CODE = ea_se.group(5)
                REGION_ICAO_CODE = ea_se.group(6)
                REGION_SUBS_CODE = ea_se.group(7)
                WAYPOINT_IDENT = ea_se.group(8).strip()
                WAYPOINT_ICAO_CODE = ea_se.group(9)
                CONT_NR = ea_se.group(10)
                WAYPOINT_TYPE = ea_se.group(11)
                WAYPOINT_USAGE = ea_se.group(12)
                if ea_se.group(13) == "N":
                    WAYPOINT_LAT = float(ea_se.group(14)) + float(ea_se.group(15))/60 + float(ea_se.group(16))/3600 + float(ea_se.group(17))/360000
                else:
                    WAYPOINT_LAT = 0 - float(ea_se.group(14)) - float(ea_se.group(15))/60 - float(ea_se.group(16))/3600 - float(ea_se.group(17))/360000
                if ea_se.group(18) == "E":
                    WAYPOINT_LON = float(ea_se.group(19)) + float(ea_se.group(20))/60 + float(ea_se.group(21))/3600 + float(ea_se.group(22))/360000
                else:
                    WAYPOINT_LON = 0 - float(ea_se.group(19)) - float(ea_se.group(20))/60 - float(ea_se.group(21))/3600 - float(ea_se.group(22))/360000
                if ea_se.group(23) == "E":
                    DYNAMIC_MAG_VAR = float(ea_se.group(24))/100
                else:
                    DYNAMIC_MAG_VAR = 0 - float(ea_se.group(24))/100
                DATUM_CODE = ea_se.group(25)
                WAYPOINT_NAME_DESCR = ea_se.group(26).replace('(','').replace(')','')
                FILE_RECD_NR = FILE_RECD_NR + 1
                CYCLE = ea_se.group(28)
                wpt_fsl.append((RECD_TYPE,CUST_AREA,SECT_CODE,SUBS_CODE,REGION_CODE,REGION_ICAO_CODE,REGION_SUBS_CODE,WAYPOINT_IDENT,WAYPOINT_ICAO_CODE,CONT_NR,WAYPOINT_TYPE,WAYPOINT_USAGE,WAYPOINT_LAT,WAYPOINT_LON,DYNAMIC_MAG_VAR,DATUM_CODE,"   ",WAYPOINT_NAME_DESCR,FILE_RECD_NR,CYCLE))
            if hold_se:
                if add_tailored:
                    RECD_TYPE = "T"
                else:
                    RECD_TYPE = hold_se.group(1)
                CUST_AREA = hold_se.group(2)
                SECT_CODE = hold_se.group(3)
                SUBS_CODE = hold_se.group(4)
                REGION_CODE = hold_se.group(5)
                REGION_ICAO_CODE = hold_se.group(6)
                DUPLICATE_IDENT = hold_se.group(7)
                FIX_IDENT = hold_se.group(8).strip()
                FIX_ICAO_CODE = hold_se.group(9)
                FIX_SECT_CODE = hold_se.group(10)
                FIX_SUBS_CODE = hold_se.group(11)
                CONT_NR = hold_se.group(12)
                if hold_se.group(13).lstrip(string.digits) == "T":
                    INBOUND_HOLDING_COURSE = float(Decimal(hold_se.group(13).rstrip('T')))
                else:
                    INBOUND_HOLDING_COURSE = float(Decimal(hold_se.group(13)[0:3] + '.' + hold_se.group(13)[3:]).quantize(Decimal('0'), rounding=ROUND_HALF_UP))
                TURN_DIR = hold_se.group(14)
                if hold_se.group(15).strip() == '':
                    LEG_LENGTH = None
                else:
                    LEG_LENGTH = float(Decimal(hold_se.group(15)[0:2] + '.' + hold_se.group(15)[2:]).quantize(Decimal('0'), rounding=ROUND_HALF_UP))
                LEG_TIME = hold_se.group(16)
                if hold_se.group(17).strip() == '':
                    MINIMUM_ALT = None
                elif hold_se.group(17).strip() == "NESTB":
                    MINIMUM_ALT = None
                elif hold_se.group(17).rstrip(string.digits) == "FL":
                    MINIMUM_ALT = hold_se.group(17).lstrip(string.ascii_uppercase) + "00"
                else:
                    MINIMUM_ALT = hold_se.group(17)
                if hold_se.group(18).strip() == '':
                    MAXIMUM_ALT = None
                elif hold_se.group(18).strip() == "UNLTD":
                    MAXIMUM_ALT = None
                elif hold_se.group(18).rstrip(string.digits) == "FL":
                    MAXIMUM_ALT = hold_se.group(18).lstrip(string.ascii_uppercase) + "00"
                else:
                    MAXIMUM_ALT = hold_se.group(18)
                if hold_se.group(19).strip() == '':
                    HOLDING_SPEED = None
                else:
                    HOLDING_SPEED = hold_se.group(19)
                if hold_se.group(20).strip() == '':
                    RNP = None
                else:
                    # RNP = float(hold_se.group(20)[0:2])/(10 ** float(hold_se.group(20)[2:]))
                    RNP = float(hold_se.group(20)[0:2])/(10 ** float(hold_se.group(20)[2:])) * 10
                if hold_se.group(21).strip() == '':
                    ARC_RADIUS = None
                else:
                    ARC_RADIUS = hold_se.group(21).strip().lstrip('0')
                VERTICAL_SCALE_FACTOR = hold_se.group(22)
                if hold_se.group(23).strip() == '':
                    RVSM_MINIMUM_LEVEL = None
                else:
                    RVSM_MINIMUM_LEVEL = hold_se.group(23)
                if hold_se.group(24).strip() == '':
                    RVSM_MAXIMUM_LEVEL = None
                else:
                    RVSM_MAXIMUM_LEVEL = hold_se.group(24)
                NAME = hold_se.group(25).replace('(','').replace(')','')
                FILE_RECD_NR = FILE_RECD_NR + 1
                CYCLE = hold_se.group(27)
                hold_fsl.append((RECD_TYPE,CUST_AREA,SECT_CODE,SUBS_CODE,REGION_CODE,REGION_ICAO_CODE,DUPLICATE_IDENT,FIX_IDENT,FIX_ICAO_CODE,FIX_SECT_CODE,FIX_SUBS_CODE,CONT_NR,INBOUND_HOLDING_COURSE,TURN_DIR,LEG_LENGTH,LEG_TIME,MINIMUM_ALT,MAXIMUM_ALT,HOLDING_SPEED,RNP,ARC_RADIUS,VERTICAL_SCALE_FACTOR,RVSM_MINIMUM_LEVEL,RVSM_MAXIMUM_LEVEL,NAME,FILE_RECD_NR,CYCLE))
            if awy_se:
                if add_tailored:
                    RECD_TYPE = "T"
                else:
                    RECD_TYPE = awy_se.group(1)
                CUST_AREA = awy_se.group(2)
                SECT_CODE = awy_se.group(3)
                SUBS_CODE = awy_se.group(4)
                ROUTE_IDENT = awy_se.group(5).strip()
                SEQ_NR = awy_se.group(6)
                FIX_IDENT = awy_se.group(7).strip()
                FIX_ICAO_CODE = awy_se.group(8)
                FIX_SECT_CODE = awy_se.group(9)
                FIX_SUBS_CODE = awy_se.group(10)
                CONT_NR = awy_se.group(11)
                WAYPOINT_DESCR_CODE = awy_se.group(12)
                BOUNDARY_CODE = awy_se.group(13)
                ROUTE_TYPE = awy_se.group(14)
                LEVEL = awy_se.group(15)
                DIRECTION_RESTRICTION = awy_se.group(16)
                CRUISE_TABLE_IND = awy_se.group(17)
                EU_IND = awy_se.group(18)
                RECOMMENDED_NAVAID = awy_se.group(19)
                RECOMMENDED_NAVAID_ICAO_CODE = awy_se.group(20)
                if awy_se.group(21).strip() == '':
                    RNP = None
                else:
                    # RNP = float(awy_se.group(21)[0:2])/(10 ** float(awy_se.group(21)[2:]))
                    RNP = float(awy_se.group(21)[0:2])/(10 ** float(awy_se.group(21)[2:])) * 10
                if awy_se.group(22).strip() == '':
                    THETA = None
                else:
                    THETA = awy_se.group(22)
                if awy_se.group(23).strip() == '':
                    RHO = None
                else:
                    RHO = awy_se.group(23)
                if awy_se.group(24).strip() == '':
                    OUTBOUND_MAG_COURSE = None
                elif awy_se.group(24).lstrip(string.digits) == "T":
                    # OUTBOUND_MAG_COURSE = float(Decimal(awy_se.group(24).rstrip('T')))
                    OUTBOUND_MAG_COURSE = float(Decimal(awy_se.group(24)[0:2] + '.' + awy_se.group(24)[2:3]))
                else:
                    # OUTBOUND_MAG_COURSE = float(Decimal(awy_se.group(24)[0:3] + '.' + awy_se.group(24)[3:]).quantize(Decimal('0'), rounding=ROUND_HALF_UP))
                    OUTBOUND_MAG_COURSE = float(Decimal(awy_se.group(24)[0:2] + '.' + awy_se.group(24)[2:3]))
                ROUTE_DISTANCE_FROM = awy_se.group(25)
                if awy_se.group(26).strip() == '':
                    INBOUND_MAG_COURSE = None
                elif awy_se.group(26).lstrip(string.digits) == "T":
                    # INBOUND_MAG_COURSE = float(Decimal(awy_se.group(26).rstrip('T')))
                    INBOUND_MAG_COURSE = float(Decimal(awy_se.group(26)[0:2] + '.' + awy_se.group(26)[2:3]))
                else:
                    # INBOUND_MAG_COURSE = float(Decimal(awy_se.group(26)[0:3] + '.' + awy_se.group(26)[3:]).quantize(Decimal('0'), rounding=ROUND_HALF_UP))
                    INBOUND_MAG_COURSE = float(Decimal(awy_se.group(26)[0:2] + '.' + awy_se.group(26)[2:3]))
                if awy_se.group(27).strip() == '':
                    MINIMUM_ALT = None
                elif awy_se.group(27).strip() == "NESTB":
                    MINIMUM_ALT = None
                elif awy_se.group(27).rstrip(string.digits) == "FL":
                    MINIMUM_ALT = awy_se.group(27).lstrip(string.ascii_uppercase) + "00"
                else:
                    MINIMUM_ALT = awy_se.group(27)
                if awy_se.group(28).strip() == '':
                    MINIMUM_ALT_1 = None
                elif awy_se.group(28).strip() == "NESTB":
                    MINIMUM_ALT_1 = None
                elif awy_se.group(28).rstrip(string.digits) == "FL":
                    MINIMUM_ALT_1 = awy_se.group(28).lstrip(string.ascii_uppercase) + "00"
                else:
                    MINIMUM_ALT_1 = awy_se.group(28)
                if awy_se.group(29).strip() == '':
                    MAXIMUM_ALT = None
                elif awy_se.group(29).strip() == "UNLTD":
                    MAXIMUM_ALT = None
                elif awy_se.group(29).rstrip(string.digits) == "FL":
                    MAXIMUM_ALT = awy_se.group(29).lstrip(string.ascii_uppercase) + "00"
                else:
                    MAXIMUM_ALT = awy_se.group(29)
                FIX_RADIUS_TRANSITION_IND = awy_se.group(30)
                VERTICAL_SCALE_FACTOR = awy_se.group(31)
                if awy_se.group(32).strip() == '':
                    RVSM_MINIMUM_LEVEL = None
                else:
                    RVSM_MINIMUM_LEVEL = awy_se.group(32)
                if awy_se.group(33).strip() == '':
                    VSF_RVSM_MAXIMUM_LEVEL = None
                else:
                    VSF_RVSM_MAXIMUM_LEVEL = awy_se.group(33)
                ICAO_CODE_2 = awy_se.group(34)
                FILE_RECD_NR = FILE_RECD_NR + 1
                CYCLE = awy_se.group(36)
                awy_fsl.append((RECD_TYPE,CUST_AREA,SECT_CODE,SUBS_CODE,ROUTE_IDENT,SEQ_NR,FIX_IDENT,FIX_ICAO_CODE,FIX_SECT_CODE,FIX_SUBS_CODE,CONT_NR,WAYPOINT_DESCR_CODE,BOUNDARY_CODE,ROUTE_TYPE,LEVEL,DIRECTION_RESTRICTION,CRUISE_TABLE_IND,EU_IND,RECOMMENDED_NAVAID,RECOMMENDED_NAVAID_ICAO_CODE,RNP,THETA,RHO,OUTBOUND_MAG_COURSE,ROUTE_DISTANCE_FROM,INBOUND_MAG_COURSE,MINIMUM_ALT,MINIMUM_ALT_1,MAXIMUM_ALT,FIX_RADIUS_TRANSITION_IND,VERTICAL_SCALE_FACTOR,RVSM_MINIMUM_LEVEL,VSF_RVSM_MAXIMUM_LEVEL,ICAO_CODE_2,FILE_RECD_NR,CYCLE))
            if arpt_se:
                if add_tailored:
                    RECD_TYPE = "T"
                else:
                    RECD_TYPE = arpt_se.group(1)
                CUST_AREA = arpt_se.group(2)
                SECT_CODE = arpt_se.group(3)
                ARPT_IDENT = arpt_se.group(4)
                ARPT_ICAO_CODE = arpt_se.group(5)
                SUBS_CODE = arpt_se.group(6)
                IATA_IDENT = arpt_se.group(7).strip()
                CONT_NR = arpt_se.group(8)
                if arpt_se.group(9).strip() == '':
                    SPEED_LIMIT_ALT = None
                else:
                    SPEED_LIMIT_ALT = arpt_se.group(9)
                LONGEST_RUNWAY = arpt_se.group(10)
                IFR_CAPABILITY = arpt_se.group(11)
                LONGEST_RUNWAY_SURFACE_CODE = arpt_se.group(12)
                if arpt_se.group(13) == "N":
                    ARPT_LAT = float(arpt_se.group(14)) + float(arpt_se.group(15))/60 + float(arpt_se.group(16))/3600 + float(arpt_se.group(17))/360000
                else:
                    ARPT_LAT = 0 - float(arpt_se.group(14)) - float(arpt_se.group(15))/60 - float(arpt_se.group(16))/3600 - float(arpt_se.group(17))/360000
                if arpt_se.group(18) == "E":
                    ARPT_LON = float(arpt_se.group(19)) + float(arpt_se.group(20))/60 + float(arpt_se.group(21))/3600 + float(arpt_se.group(22))/360000
                else:
                    ARPT_LON = 0 - float(arpt_se.group(19)) - float(arpt_se.group(20))/60 - float(arpt_se.group(21))/3600 - float(arpt_se.group(22))/360000
                if arpt_se.group(23) == "E":
                    MAG_VAR = float(arpt_se.group(24))/100
                else:
                    MAG_VAR = 0 - float(arpt_se.group(24))/100
                ARPT_ELEV = arpt_se.group(25)
                if arpt_se.group(26).strip() == '':
                    SPEED_LIMIT = None
                else:
                    SPEED_LIMIT = arpt_se.group(26)
                RECOMMENDED_NAVAID = arpt_se.group(27)
                RECOMMENDED_NAVAID_ICAO_CODE = arpt_se.group(28)
                if arpt_se.group(29).strip() == '':
                    TRANSITIONS_ALT = None
                else:
                    TRANSITIONS_ALT = arpt_se.group(29)
                if arpt_se.group(30).strip() == '':
                    TRANSITION_LEVEL = None
                else:
                    TRANSITION_LEVEL = arpt_se.group(30)
                PUBLIC_MILITARY_IND = arpt_se.group(31)
                TIME_ZONE = arpt_se.group(32)
                DAYLIGHT_IND = arpt_se.group(33)
                MAG_TRUE_IND = arpt_se.group(34)
                DATUM_CODE = arpt_se.group(35)
                ARPT_NAME = arpt_se.group(36).replace('(','').replace(')','')
                FILE_RECD_NR = FILE_RECD_NR + 1
                CYCLE = arpt_se.group(38)
                arpt_fsl.append((RECD_TYPE,CUST_AREA,SECT_CODE,ARPT_IDENT,ARPT_ICAO_CODE,SUBS_CODE,IATA_IDENT,CONT_NR,SPEED_LIMIT_ALT,LONGEST_RUNWAY,IFR_CAPABILITY,LONGEST_RUNWAY_SURFACE_CODE,ARPT_LAT,ARPT_LON,MAG_VAR,ARPT_ELEV,SPEED_LIMIT,RECOMMENDED_NAVAID,RECOMMENDED_NAVAID_ICAO_CODE,TRANSITIONS_ALT,TRANSITION_LEVEL,PUBLIC_MILITARY_IND,TIME_ZONE,DAYLIGHT_IND,MAG_TRUE_IND,DATUM_CODE,ARPT_NAME,FILE_RECD_NR,CYCLE))
            if pc_se:
                if add_tailored:
                    RECD_TYPE = "T"
                else:
                    RECD_TYPE = pc_se.group(1)
                CUST_AREA = pc_se.group(2)
                SECT_CODE = pc_se.group(3)
                SUBS_CODE = pc_se.group(4)
                REGION_CODE = pc_se.group(5)
                REGION_ICAO_CODE = pc_se.group(6)
                REGION_SUBS_CODE = pc_se.group(7)
                WAYPOINT_IDENT = pc_se.group(8).strip()
                WAYPOINT_ICAO_CODE = pc_se.group(9)
                CONT_NR = pc_se.group(10)
                WAYPOINT_TYPE = pc_se.group(11)
                WAYPOINT_USAGE = pc_se.group(12)
                if pc_se.group(13) == "N":
                    WAYPOINT_LAT = float(pc_se.group(14)) + float(pc_se.group(15))/60 + float(pc_se.group(16))/3600 + float(pc_se.group(17))/360000
                else:
                    WAYPOINT_LAT = 0 - float(pc_se.group(14)) - float(pc_se.group(15))/60 - float(pc_se.group(16))/3600 - float(pc_se.group(17))/360000
                if pc_se.group(18) == "E":
                    WAYPOINT_LON = float(pc_se.group(19)) + float(pc_se.group(20))/60 + float(pc_se.group(21))/3600 + float(pc_se.group(22))/360000
                else:
                    WAYPOINT_LON = 0 - float(pc_se.group(19)) - float(pc_se.group(20))/60 - float(pc_se.group(21))/3600 - float(pc_se.group(22))/360000
                if pc_se.group(23) == "E":
                    DYNAMIC_MAG_VAR = float(pc_se.group(24))/100
                else:
                    DYNAMIC_MAG_VAR = 0 - float(pc_se.group(24))/100
                DATUM_CODE = pc_se.group(25)
                WAYPOINT_NAME_DESCR = pc_se.group(26).replace('(','').replace(')','')
                FILE_RECD_NR = FILE_RECD_NR + 1
                CYCLE = pc_se.group(28)
                wpt_fsl.append((RECD_TYPE,CUST_AREA,SECT_CODE,SUBS_CODE,REGION_CODE,REGION_ICAO_CODE,REGION_SUBS_CODE,WAYPOINT_IDENT,WAYPOINT_ICAO_CODE,CONT_NR,WAYPOINT_TYPE,WAYPOINT_USAGE,WAYPOINT_LAT,WAYPOINT_LON,DYNAMIC_MAG_VAR,DATUM_CODE,"   ",WAYPOINT_NAME_DESCR,FILE_RECD_NR,CYCLE))
            if proc_se:
                if add_tailored:
                    RECD_TYPE = "T"
                else:
                    RECD_TYPE = proc_se.group(1)
                CUST_AREA = proc_se.group(2)
                SECT_CODE = proc_se.group(3)
                ARPT_IDENT = proc_se.group(4)
                ARPT_ICAO_CODE = proc_se.group(5)
                SUBS_CODE = proc_se.group(6)
                PROC_IDENT = proc_se.group(7).strip()
                ROUTE_TYPE = proc_se.group(8)
                TRANSITION_IDENT = proc_se.group(9).strip()
                SEQ_NR = proc_se.group(10)
                FIX_IDENT = proc_se.group(11).strip()
                FIX_ICAO_CODE = proc_se.group(12)
                FIX_SECT_CODE = proc_se.group(13)
                FIX_SUBS_CODE = proc_se.group(14)
                CONT_NR = proc_se.group(15)
                WAYPOINT_DESCR_CODE = proc_se.group(16)
                TURN_DIR = proc_se.group(17)
                if proc_se.group(18).strip() == '':
                    RNP = None
                else:
                    # RNP = float(proc_se.group(18)[0:2])/(10 ** float(proc_se.group(18)[2:]))
                    RNP = float(proc_se.group(18)[0:2])/(10 ** float(proc_se.group(18)[2:])) * 10
                PATH_AND_TERMINATION = proc_se.group(19)
                TURN_DIR_VALID = proc_se.group(20)
                RECOMMENDED_NAVAID = proc_se.group(21)
                RECOMMENDED_NAVAID_ICAO_CODE = proc_se.group(22)
                if proc_se.group(23).strip() == '':
                    ARC_RADIUS = None
                else:
                    ARC_RADIUS = proc_se.group(23).strip().lstrip('0')
                if proc_se.group(24).strip() == '':
                    THETA = None
                else:
                    THETA = proc_se.group(24)
                if proc_se.group(25).strip() == '':
                    RHO = None
                else:
                    RHO = proc_se.group(25)
                if proc_se.group(26).strip() == '':
                    MAG_COURSE = None
                elif proc_se.group(26).lstrip(string.digits) == "T":
                    MAG_COURSE = float(Decimal(proc_se.group(26).rstrip('T')))
                else:
                    MAG_COURSE = float(Decimal(proc_se.group(26)[0:3] + '.' + proc_se.group(26)[3:]).quantize(Decimal('0'), rounding=ROUND_HALF_UP))
                ROUTE_DISTANCE_HOLDING_DISTANCE_OR_TIME = proc_se.group(27)
                RECOMMENDED_NAVAID_SECT_CODE = proc_se.group(28)
                RECOMMENDED_NAVAID_SUBS_CODE = proc_se.group(29)
                ALT_DESCR = proc_se.group(30)
                ATC_IND = proc_se.group(31)
                if proc_se.group(32).strip() == '':
                    ALT_1 = None
                elif proc_se.group(32).rstrip(string.digits) == "FL":
                    ALT_1 = proc_se.group(32).lstrip(string.ascii_uppercase) + "00"
                else:
                    ALT_1 = proc_se.group(32)
                if proc_se.group(33).strip() == '':
                    ALT_2 = None
                elif proc_se.group(33).rstrip(string.digits) == "FL":
                    ALT_2 = proc_se.group(33).lstrip(string.ascii_uppercase) + "00"
                else:
                    ALT_2 = proc_se.group(33)
                if proc_se.group(34).strip() == '':
                    TRANSITION_ALT = None
                else:
                    TRANSITION_ALT = proc_se.group(34)
                SPEED_LIMIT = proc_se.group(35)
                if proc_se.group(36).strip() == '':
                    VERTICAL_ANGLE = None
                else:
                    VERTICAL_ANGLE = float(proc_se.group(36))/100
                CENTER_FIX_OR_TAA_PROCEDURE_TURN_IND = proc_se.group(37)
                MULTIPLE_CODE_OR_TAA_SECTOR_IDENT = proc_se.group(38).strip()
                MULTIPLE_CODE_OR_TAA_SECTOR_ICAO_CODE = proc_se.group(39)
                MULTIPLE_CODE_OR_TAA_SECTOR_SECT_CODE = proc_se.group(40)
                MULTIPLE_CODE_OR_TAA_SECTOR_SUBS_CODE = proc_se.group(41)
                GNSS_FMS_INDICATION = proc_se.group(42)
                SPEED_LIMIT_DESCR = proc_se.group(43)
                APCH_ROUTE_QUALIFIER_1 = proc_se.group(44)
                APCH_ROUTE_QUALIFIER_2 = proc_se.group(45)
                FILE_RECD_NR = FILE_RECD_NR + 1
                CYCLE = proc_se.group(47)
                proc_fsl.append((RECD_TYPE,CUST_AREA,SECT_CODE,ARPT_IDENT,ARPT_ICAO_CODE,SUBS_CODE,PROC_IDENT,ROUTE_TYPE,TRANSITION_IDENT,SEQ_NR,FIX_IDENT,FIX_ICAO_CODE,FIX_SECT_CODE,FIX_SUBS_CODE,CONT_NR,WAYPOINT_DESCR_CODE,TURN_DIR,RNP,PATH_AND_TERMINATION,TURN_DIR_VALID,RECOMMENDED_NAVAID,RECOMMENDED_NAVAID_ICAO_CODE,ARC_RADIUS,THETA,RHO,MAG_COURSE,ROUTE_DISTANCE_HOLDING_DISTANCE_OR_TIME,RECOMMENDED_NAVAID_SECT_CODE,RECOMMENDED_NAVAID_SUBS_CODE,ALT_DESCR,ATC_IND,ALT_1,ALT_2,TRANSITION_ALT,SPEED_LIMIT,VERTICAL_ANGLE,CENTER_FIX_OR_TAA_PROCEDURE_TURN_IND,MULTIPLE_CODE_OR_TAA_SECTOR_IDENT,MULTIPLE_CODE_OR_TAA_SECTOR_ICAO_CODE,MULTIPLE_CODE_OR_TAA_SECTOR_SECT_CODE,MULTIPLE_CODE_OR_TAA_SECTOR_SUBS_CODE,GNSS_FMS_INDICATION,SPEED_LIMIT_DESCR,APCH_ROUTE_QUALIFIER_1,APCH_ROUTE_QUALIFIER_2,"   ",FILE_RECD_NR,CYCLE))
            if rwy_se:
                if add_tailored:
                    RECD_TYPE = "T"
                else:
                    RECD_TYPE = rwy_se.group(1)
                CUST_AREA = rwy_se.group(2)
                SECT_CODE = rwy_se.group(3)
                ARPT_IDENT = rwy_se.group(4)
                ARPT_ICAO_CODE = rwy_se.group(5)
                SUBS_CODE = rwy_se.group(6)
                RUNWAY_IDENT = rwy_se.group(7).strip()
                CONT_NR = rwy_se.group(8)
                RUNWAY_LENGTH = rwy_se.group(9)
                if rwy_se.group(10).lstrip(string.digits) == "T":
                    RUNWAY_MAG_BEARING = float(Decimal(rwy_se.group(10).rstrip('T')))
                else:
                    RUNWAY_MAG_BEARING = float(Decimal(rwy_se.group(10)[0:3] + '.' + rwy_se.group(10)[3:]).quantize(Decimal('0'), rounding=ROUND_HALF_UP))
                if rwy_se.group(11) == "N":
                    RUNWAY_LAT = float(rwy_se.group(12)) + float(rwy_se.group(13))/60 + float(rwy_se.group(14))/3600 + float(rwy_se.group(15))/360000
                else:
                    RUNWAY_LAT = 0 - float(rwy_se.group(12)) - float(rwy_se.group(13))/60 - float(rwy_se.group(14))/3600 - float(rwy_se.group(15))/360000
                if rwy_se.group(16) == "E":
                    RUNWAY_LON = float(rwy_se.group(17)) + float(rwy_se.group(18))/60 + float(rwy_se.group(19))/3600 + float(rwy_se.group(20))/360000
                else:
                    RUNWAY_LON = 0 - float(rwy_se.group(17)) - float(rwy_se.group(18))/60 - float(rwy_se.group(19))/3600 - float(rwy_se.group(20))/360000
                if rwy_se.group(21).strip() == '':
                    RUNWAY_GRADIENT = None
                elif rwy_se.group(21) == "-":
                    RUNWAY_GRADIENT = 0 - float(rwy_se.group(22))/100
                else:
                    RUNWAY_GRADIENT = float(rwy_se.group(22))/100
                if rwy_se.group(23).strip() == '':
                    ELLIPSOID_HEIGHT = None
                else:
                    ELLIPSOID_HEIGHT = rwy_se.group(23)
                LANDING_THLD_ELEV = rwy_se.group(24)
                if rwy_se.group(25).strip() == '':
                    DISPLACED_THLD_DISTANCE = "0"
                else:
                    DISPLACED_THLD_DISTANCE = rwy_se.group(25)
                if rwy_se.group(26).strip() == '':
                    THLD_CROSSING_HEIGHT = "50"
                else:
                    THLD_CROSSING_HEIGHT = rwy_se.group(26)
                RUNWAY_WIDTH = rwy_se.group(27)
                TCH_VALUE_IND = rwy_se.group(28)
                LOCALIZER_MLS_GLS_REF_PATH_IDENT = rwy_se.group(29).strip()
                LOCALIZER_MLS_GLS_CATEGORY_CLASS = rwy_se.group(30)
                STOPWAY = rwy_se.group(31)
                SECOND_LOCALIZER_MLS_GLS_REF_PATH_IDENT = rwy_se.group(32).strip()
                SECOND_LOCALIZER_MLS_GLS_CATEGORY_CLASS = rwy_se.group(33)
                RUNWAY_DESCR = rwy_se.group(34)
                FILE_RECD_NR = FILE_RECD_NR + 1
                CYCLE = rwy_se.group(36)
                rwy_fsl.append((RECD_TYPE,CUST_AREA,SECT_CODE,ARPT_IDENT,ARPT_ICAO_CODE,SUBS_CODE,RUNWAY_IDENT,CONT_NR,RUNWAY_LENGTH,RUNWAY_MAG_BEARING,RUNWAY_LAT,RUNWAY_LON,RUNWAY_GRADIENT,ELLIPSOID_HEIGHT,LANDING_THLD_ELEV,DISPLACED_THLD_DISTANCE,THLD_CROSSING_HEIGHT,RUNWAY_WIDTH,TCH_VALUE_IND,LOCALIZER_MLS_GLS_REF_PATH_IDENT,LOCALIZER_MLS_GLS_CATEGORY_CLASS,STOPWAY,SECOND_LOCALIZER_MLS_GLS_REF_PATH_IDENT,SECOND_LOCALIZER_MLS_GLS_CATEGORY_CLASS,RUNWAY_DESCR,FILE_RECD_NR,CYCLE))
            if loc_se:
                if add_tailored:
                    RECD_TYPE = "T"
                else:
                    RECD_TYPE = loc_se.group(1)
                CUST_AREA = loc_se.group(2)
                SECT_CODE = loc_se.group(3)
                ARPT_IDENT = loc_se.group(4)
                ARPT_ICAO_CODE = loc_se.group(5)
                SUBS_CODE = loc_se.group(6)
                LOCALIZER_IDENT = loc_se.group(7).strip()
                ILS_CATEGORY = loc_se.group(8)
                CONT_NR = loc_se.group(9)
                LOCALIZER_FREQ = float(loc_se.group(10))/100
                RUNWAY_IDENT = loc_se.group(11).strip()
                if loc_se.group(12).strip() == '':
                    LOCALIZER_LAT = None
                elif loc_se.group(12) == "N":
                    LOCALIZER_LAT = float(loc_se.group(13)) + float(loc_se.group(14))/60 + float(loc_se.group(15))/3600 + float(loc_se.group(16))/360000
                else:
                    LOCALIZER_LAT = 0 - float(loc_se.group(13)) - float(loc_se.group(14))/60 - float(loc_se.group(15))/3600 - float(loc_se.group(16))/360000
                if loc_se.group(17).strip() == '':
                    LOCALIZER_LON = None
                elif loc_se.group(17) == "E":
                    LOCALIZER_LON = float(loc_se.group(18)) + float(loc_se.group(19))/60 + float(loc_se.group(20))/3600 + float(loc_se.group(21))/360000
                else:
                    LOCALIZER_LON = 0 - float(loc_se.group(18)) - float(loc_se.group(19))/60 - float(loc_se.group(20))/3600 - float(loc_se.group(21))/360000
                if loc_se.group(22).lstrip(string.digits) == "T":
                    LOCALIZER_BEARING = float(Decimal(loc_se.group(22).rstrip('T')))
                else:
                    LOCALIZER_BEARING = float(Decimal(loc_se.group(22)[0:3] + '.' + loc_se.group(22)[3:]).quantize(Decimal('0'), rounding=ROUND_HALF_UP))
                if loc_se.group(23).strip() == '':
                    GLIDE_SLOPE_LAT = None
                elif loc_se.group(23) == "N":
                    GLIDE_SLOPE_LAT = float(loc_se.group(24)) + float(loc_se.group(25))/60 + float(loc_se.group(26))/3600 + float(loc_se.group(27))/360000
                else:
                    GLIDE_SLOPE_LAT = 0 - float(loc_se.group(24)) - float(loc_se.group(25))/60 - float(loc_se.group(26))/3600 - float(loc_se.group(27))/360000
                if loc_se.group(28).strip() == '':
                    GLIDE_SLOPE_LON = None
                elif loc_se.group(28) == "E":
                    GLIDE_SLOPE_LON = float(loc_se.group(29)) + float(loc_se.group(30))/60 + float(loc_se.group(31))/3600 + float(loc_se.group(32))/360000
                else:
                    GLIDE_SLOPE_LON = 0 - float(loc_se.group(29)) - float(loc_se.group(30))/60 - float(loc_se.group(31))/3600 - float(loc_se.group(32))/360000
                LOCALIZER_POS = loc_se.group(33)
                LOCALIZER_POS_REFERENCE = loc_se.group(34)
                GLIDE_SLOPE_POS = loc_se.group(35)
                LOCALIZER_WIDTH = float(loc_se.group(36))
                if loc_se.group(37).strip() == '':
                    GLIDE_SLOPE_ANGLE = None
                else:
                    GLIDE_SLOPE_ANGLE = float(loc_se.group(37))/100
                STATION_DECLINATION = loc_se.group(38)
                if loc_se.group(39).strip() == '':
                    GLIDE_SLOPE_HEIGHT_AT_LANDING_THLD = None
                else:
                    GLIDE_SLOPE_HEIGHT_AT_LANDING_THLD = loc_se.group(39)
                if loc_se.group(40).strip() == '':
                    GLIDE_SLOPE_ELEV = None
                else:
                    GLIDE_SLOPE_ELEV = loc_se.group(40)
                SUPPORTING_FACILITY_IDENT = loc_se.group(41)
                SUPPORTING_FACILITY_ICAO_CODE = loc_se.group(42)
                SUPPORTING_FACILITY_SECT_CODE = loc_se.group(43)
                SUPPORTING_FACILITY_SUBS_CODE = loc_se.group(44)
                FILE_RECD_NR = FILE_RECD_NR + 1
                CYCLE = loc_se.group(46)
                loc_fsl.append((RECD_TYPE,CUST_AREA,SECT_CODE,ARPT_IDENT,ARPT_ICAO_CODE,SUBS_CODE,LOCALIZER_IDENT,ILS_CATEGORY,CONT_NR,LOCALIZER_FREQ,RUNWAY_IDENT,LOCALIZER_LAT,LOCALIZER_LON,LOCALIZER_BEARING,GLIDE_SLOPE_LAT,GLIDE_SLOPE_LON,LOCALIZER_POS,LOCALIZER_POS_REFERENCE,GLIDE_SLOPE_POS,LOCALIZER_WIDTH,GLIDE_SLOPE_ANGLE,STATION_DECLINATION,GLIDE_SLOPE_HEIGHT_AT_LANDING_THLD,GLIDE_SLOPE_ELEV,SUPPORTING_FACILITY_IDENT,SUPPORTING_FACILITY_ICAO_CODE,SUPPORTING_FACILITY_SECT_CODE,SUPPORTING_FACILITY_SUBS_CODE,FILE_RECD_NR,CYCLE))
            if pn_se:
                if add_tailored:
                    RECD_TYPE = "T"
                else:
                    RECD_TYPE = pn_se.group(1)
                CUST_AREA = pn_se.group(2)
                SECT_CODE = pn_se.group(3)
                SUBS_CODE = pn_se.group(4)
                ARPT_IDENT = pn_se.group(5).strip()
                ARPT_ICAO_CODE = pn_se.group(6)
                NDB_IDENT = pn_se.group(7).strip()
                NDB_ICAO_CODE = pn_se.group(8)
                CONT_NR = pn_se.group(9)
                NDB_FREQ = float(pn_se.group(10))/10
                NDB_CLASS = pn_se.group(11)
                if pn_se.group(12) == "N":
                    NDB_LAT = float(pn_se.group(13)) + float(pn_se.group(14))/60 + float(pn_se.group(15))/3600 + float(pn_se.group(16))/360000
                else:
                    NDB_LAT = 0 - float(pn_se.group(13)) - float(pn_se.group(14))/60 - float(pn_se.group(15))/3600 - float(pn_se.group(16))/360000
                if pn_se.group(17) == "E":
                    NDB_LON = float(pn_se.group(18)) + float(pn_se.group(19))/60 + float(pn_se.group(20))/3600 + float(pn_se.group(21))/360000
                else:
                    NDB_LON = 0 - float(pn_se.group(18)) - float(pn_se.group(19))/60 - float(pn_se.group(20))/3600 - float(pn_se.group(21))/360000
                if pn_se.group(22) == "E":
                    MAG_VAR = float(pn_se.group(23))/100
                else:
                    MAG_VAR = 0 - float(pn_se.group(23))/100
                DATUM_CODE = pn_se.group(24)
                NDB_NAME = pn_se.group(25).replace('NDB','   ').replace('(','').replace(')','')
                FILE_RECD_NR = FILE_RECD_NR + 1
                CYCLE = pn_se.group(27)
                ndb_fsl.append((RECD_TYPE,CUST_AREA,SECT_CODE,SUBS_CODE,ARPT_IDENT,ARPT_ICAO_CODE,NDB_IDENT,NDB_ICAO_CODE,CONT_NR,NDB_FREQ,NDB_CLASS,NDB_LAT,NDB_LON,MAG_VAR,DATUM_CODE,NDB_NAME,FILE_RECD_NR,CYCLE))
            if crz_se:
                if add_tailored:
                    RECD_TYPE = "T"
                else:
                    RECD_TYPE = crz_se.group(1)
                SECT_CODE = crz_se.group(2)
                SUBS_CODE = crz_se.group(3)
                CRUISE_TABLE_IDENT = crz_se.group(4)
                SEQUENCE_NUMBER = crz_se.group(5)
                COURSE_FROM = crz_se.group(6)
                COURSE_TO = crz_se.group(7)
                MAG_TRUE = crz_se.group(8)
                CRUISE_LEVEL_FROM_0 = crz_se.group(9)
                VERTICAL_SEPARATION_0 = crz_se.group(10)
                CRUISE_LEVEL_TO_0 = crz_se.group(11)
                CRUISE_LEVEL_FROM_1 = crz_se.group(12)
                VERTICAL_SEPARATION_1 = crz_se.group(13)
                CRUISE_LEVEL_TO_1 = crz_se.group(14)
                CRUISE_LEVEL_FROM_2 = crz_se.group(15)
                VERTICAL_SEPARATION_2 = crz_se.group(16)
                CRUISE_LEVEL_TO_2 = crz_se.group(17)
                CRUISE_LEVEL_FROM_3 = crz_se.group(18)
                VERTICAL_SEPARATION_3 = crz_se.group(19)
                CRUISE_LEVEL_TO_3 = crz_se.group(20)
                FILE_RECD_NR = FILE_RECD_NR + 1
                CYCLE = crz_se.group(22)
                crz_fsl.append((RECD_TYPE,SECT_CODE,SUBS_CODE,CRUISE_TABLE_IDENT,SEQUENCE_NUMBER,COURSE_FROM,COURSE_TO,MAG_TRUE,CRUISE_LEVEL_FROM_0,VERTICAL_SEPARATION_0,CRUISE_LEVEL_TO_0,CRUISE_LEVEL_FROM_1,VERTICAL_SEPARATION_1,CRUISE_LEVEL_TO_1,CRUISE_LEVEL_FROM_2,VERTICAL_SEPARATION_2,CRUISE_LEVEL_TO_2,CRUISE_LEVEL_FROM_3,VERTICAL_SEPARATION_3,CRUISE_LEVEL_TO_3,FILE_RECD_NR,CYCLE))
        # Convert list to Pandas DataFrame
        if 1:
            mora_fsl_tb = pandas.DataFrame(mora_fsl, columns=['RECD_TYPE','SECT_CODE','SUBS_CODE','MORA_LAT','MORA_LON','MORA_TYPE','MORA_00','MORA_01','MORA_02','MORA_03','MORA_04','MORA_05','MORA_06','MORA_07','MORA_08','MORA_09','MORA_10','MORA_11','MORA_12','MORA_13','MORA_14','MORA_15','MORA_16','MORA_17','MORA_18','MORA_19','MORA_20','MORA_21','MORA_22','MORA_23','MORA_24','MORA_25','MORA_26','MORA_27','MORA_28','MORA_29','FILE_RECD_NR','CYCLE'])
            vhf_fsl_tb = pandas.DataFrame(vhf_fsl, columns=['RECD_TYPE','CUST_AREA','SECT_CODE','SUBS_CODE','ARPT_IDENT','ARPT_ICAO_CODE','VOR_IDENT','VHF_ICAO_CODE','CONT_NR','VOR_FREQ','NAVAID_CLASS','VOR_LAT','VOR_LON','DME_IDENT','DME_LAT','DME_LON','STATION_DECLINATION','DME_ELEV','FIGURE_OF_MERIT','ILS_DME_BIAS','FREQ_PROTECTION','DATUM_CODE','VOR_NAME','ROUTE_INAPPROPRIATE_DME','DME_OPERATIONAL_SERVICE_VOLUME','FILE_RECD_NR','CYCLE'])
            ndb_fsl_tb = pandas.DataFrame(ndb_fsl, columns=['RECD_TYPE','CUST_AREA','SECT_CODE','SUBS_CODE','ARPT_IDENT','ARPT_ICAO_CODE','NDB_IDENT','NDB_ICAO_CODE','CONT_NR','NDB_FREQ','NDB_CLASS','NDB_LAT','NDB_LON','MAG_VAR','DATUM_CODE','NDB_NAME','FILE_RECD_NR','CYCLE'])
            awy_fsl_tb = pandas.DataFrame(awy_fsl, columns=['RECD_TYPE','CUST_AREA','SECT_CODE','SUBS_CODE','ROUTE_IDENT','SEQ_NR','FIX_IDENT','FIX_ICAO_CODE','FIX_SECT_CODE','FIX_SUBS_CODE','CONT_NR','WAYPOINT_DESCR_CODE','BOUNDARY_CODE','ROUTE_TYPE','LEVEL','DIRECTION_RESTRICTION','CRUISE_TABLE_IND','EU_IND','RECOMMENDED_NAVAID','RECOMMENDED_NAVAID_ICAO_CODE','RNP','THETA','RHO','OUTBOUND_MAG_COURSE','ROUTE_DISTANCE_FROM','INBOUND_MAG_COURSE','MINIMUM_ALT','MINIMUM_ALT_1','MAXIMUM_ALT','FIX_RADIUS_TRANSITION_IND','VERTICAL_SCALE_FACTOR','RVSM_MINIMUM_LEVEL','VSF_RVSM_MAXIMUM_LEVEL','ICAO_CODE_2','FILE_RECD_NR','CYCLE'])
            hold_fsl_tb = pandas.DataFrame(hold_fsl, columns=['RECD_TYPE','CUST_AREA','SECT_CODE','SUBS_CODE','REGION_CODE','REGION_ICAO_CODE','DUPLICATE_IDENT','FIX_IDENT','FIX_ICAO_CODE','FIX_SECT_CODE','FIX_SUBS_CODE','CONT_NR','INBOUND_HOLDING_COURSE','TURN_DIR','LEG_LENGTH','LEG_TIME','MINIMUM_ALT','MAXIMUM_ALT','HOLDING_SPEED','RNP','ARC_RADIUS','VERTICAL_SCALE_FACTOR','RVSM_MINIMUM_LEVEL','RVSM_MAXIMUM_LEVEL','NAME','FILE_RECD_NR','CYCLE'])
            arpt_fsl_tb = pandas.DataFrame(arpt_fsl, columns=['RECD_TYPE','CUST_AREA','SECT_CODE','ARPT_IDENT','ARPT_ICAO_CODE','SUBS_CODE','IATA_IDENT','CONT_NR','SPEED_LIMIT_ALT','LONGEST_RUNWAY','IFR_CAPABILITY','LONGEST_RUNWAY_SURFACE_CODE','ARPT_LAT','ARPT_LON','MAG_VAR','ARPT_ELEV','SPEED_LIMIT','RECOMMENDED_NAVAID','RECOMMENDED_NAVAID_ICAO_CODE','TRANSITIONS_ALT','TRANSITION_LEVEL','PUBLIC_MILITARY_IND','TIME_ZONE','DAYLIGHT_IND','MAG_TRUE_IND','DATUM_CODE','ARPT_NAME','FILE_RECD_NR','CYCLE'])
            wpt_fsl_tb = pandas.DataFrame(wpt_fsl, columns=['RECD_TYPE','CUST_AREA','SECT_CODE','SUBS_CODE','REGION_CODE','REGION_ICAO_CODE','REGION_SUBS_CODE','WAYPOINT_IDENT','WAYPOINT_ICAO_CODE','CONT_NR','WAYPOINT_TYPE','WAYPOINT_USAGE','WAYPOINT_LAT','WAYPOINT_LON','DYNAMIC_MAG_VAR','DATUM_CODE','NAME_FORMAT_IND','WAYPOINT_NAME_DESCR','FILE_RECD_NR','CYCLE'])
            proc_fsl_tb = pandas.DataFrame(proc_fsl, columns=['RECD_TYPE','CUST_AREA','SECT_CODE','ARPT_IDENT','ARPT_ICAO_CODE','SUBS_CODE','PROC_IDENT','ROUTE_TYPE','TRANSITION_IDENT','SEQ_NR','FIX_IDENT','FIX_ICAO_CODE','FIX_SECT_CODE','FIX_SUBS_CODE','CONT_NR','WAYPOINT_DESCR_CODE','TURN_DIR','RNP','PATH_AND_TERMINATION','TURN_DIR_VALID','RECOMMENDED_NAVAID','RECOMMENDED_NAVAID_ICAO_CODE','ARC_RADIUS','THETA','RHO','MAG_COURSE','ROUTE_DISTANCE_HOLDING_DISTANCE_OR_TIME','RECOMMENDED_NAVAID_SECT_CODE','RECOMMENDED_NAVAID_SUBS_CODE','ALT_DESCR','ATC_IND','ALT_1','ALT_2','TRANSITION_ALT','SPEED_LIMIT','VERTICAL_ANGLE','CENTER_FIX_OR_TAA_PROCEDURE_TURN_IND','MULTIPLE_CODE_OR_TAA_SECTOR_IDENT','MULTIPLE_CODE_OR_TAA_SECTOR_ICAO_CODE','MULTIPLE_CODE_OR_TAA_SECTOR_SECT_CODE','MULTIPLE_CODE_OR_TAA_SECTOR_SUBS_CODE','GNSS_FMS_INDICATION','SPEED_LIMIT_DESCR','APCH_ROUTE_QUALIFIER_1','APCH_ROUTE_QUALIFIER_2','VERTICAL_SCALE_FACTOR','FILE_RECD_NR','CYCLE'])
            rwy_fsl_tb = pandas.DataFrame(rwy_fsl, columns=['RECD_TYPE','CUST_AREA','SECT_CODE','ARPT_IDENT','ARPT_ICAO_CODE','SUBS_CODE','RUNWAY_IDENT','CONT_NR','RUNWAY_LENGTH','RUNWAY_MAG_BEARING','RUNWAY_LAT','RUNWAY_LON','RUNWAY_GRADIENT','ELLIPSOID_HEIGHT','LANDING_THLD_ELEV','DISPLACED_THLD_DISTANCE','THLD_CROSSING_HEIGHT','RUNWAY_WIDTH','TCH_VALUE_IND','LOCALIZER_MLS_GLS_REF_PATH_IDENT','LOCALIZER_MLS_GLS_CATEGORY_CLASS','STOPWAY','SECOND_LOCALIZER_MLS_GLS_REF_PATH_IDENT','SECOND_LOCALIZER_MLS_GLS_CATEGORY_CLASS','RUNWAY_DESCR','FILE_RECD_NR','CYCLE'])
            loc_fsl_tb = pandas.DataFrame(loc_fsl, columns=['RECD_TYPE','CUST_AREA','SECT_CODE','ARPT_IDENT','ARPT_ICAO_CODE','SUBS_CODE','LOCALIZER_IDENT','ILS_CATEGORY','CONT_NR','LOCALIZER_FREQ','RUNWAY_IDENT','LOCALIZER_LAT','LOCALIZER_LON','LOCALIZER_BEARING','GLIDE_SLOPE_LAT','GLIDE_SLOPE_LON','LOCALIZER_POS','LOCALIZER_POS_REFERENCE','GLIDE_SLOPE_POS','LOCALIZER_WIDTH','GLIDE_SLOPE_ANGLE','STATION_DECLINATION','GLIDE_SLOPE_HEIGHT_AT_LANDING_THLD','GLIDE_SLOPE_ELEV','SUPPORTING_FACILITY_IDENT','SUPPORTING_FACILITY_ICAO_CODE','SUPPORTING_FACILITY_SECT_CODE','SUPPORTING_FACILITY_SUBS_CODE','FILE_RECD_NR','CYCLE'])
            crz_fsl_tb = pandas.DataFrame(crz_fsl, columns=['RECD_TYPE','SECT_CODE','SUBS_CODE','CRUISE_TABLE_IDENT','SEQUENCE_NUMBER','COURSE_FROM','COURSE_TO','MAG_TRUE','CRUISE_LEVEL_FROM_0','VERTICAL_SEPARATION_0','CRUISE_LEVEL_TO_0','CRUISE_LEVEL_FROM_1','VERTICAL_SEPARATION_1','CRUISE_LEVEL_TO_1','CRUISE_LEVEL_FROM_2','VERTICAL_SEPARATION_2','CRUISE_LEVEL_TO_2','CRUISE_LEVEL_FROM_3','VERTICAL_SEPARATION_3','CRUISE_LEVEL_TO_3','FILE_RECD_NR','CYCLE'])
        print(time.strftime("%H:%M:%S", time.localtime()) , 'FSLabs: Data Process Finished')
    # Write Data
    if fsl_write:
        rom = sqlite3.connect(Cycle_Ident + '/FSLabs/' + Cycle_Ident + '.rom')
        rom_cur = rom.cursor()
        if fsl_create_blank:
            rom_cur.execute('DROP TABLE IF EXISTS AIRPORT;')
            rom_cur.execute('DROP TABLE IF EXISTS AIRPORT_FLIGHT_PLANNING_CONTINUATION;')
            rom_cur.execute('DROP TABLE IF EXISTS AIRPORT_GATE;')
            rom_cur.execute('DROP TABLE IF EXISTS AIRPORT_LOCALIZER_AND_GLIDE_SLOPE;')
            rom_cur.execute('DROP TABLE IF EXISTS AIRPORT_PROCEDURE;')
            rom_cur.execute('DROP TABLE IF EXISTS COMPANY_ROUTE;')
            rom_cur.execute('DROP TABLE IF EXISTS CRUISING_TABLE;')
            rom_cur.execute('DROP TABLE IF EXISTS CYCLE_INFO;')
            rom_cur.execute('DROP TABLE IF EXISTS ENROUTE_AIRWAYS;')
            rom_cur.execute('DROP TABLE IF EXISTS GRID_MORA;')
            rom_cur.execute('DROP TABLE IF EXISTS HDR_ROWS;')
            rom_cur.execute('DROP TABLE IF EXISTS HOLDING_PATTERN;')
            rom_cur.execute('DROP TABLE IF EXISTS NDB_NAVAID;')
            rom_cur.execute('DROP TABLE IF EXISTS NDB_NAVAID_FLIGHT_PLANNING_CONTINUATION;')
            rom_cur.execute('DROP TABLE IF EXISTS REJECTED_ROWS;')
            rom_cur.execute('DROP TABLE IF EXISTS RUNWAY;')
            rom_cur.execute('DROP TABLE IF EXISTS TACAN_ONLY_NAVAID;')
            rom_cur.execute('DROP TABLE IF EXISTS VHF_NAVAID;')
            rom_cur.execute('DROP TABLE IF EXISTS VHF_NAVAID_FLIGHT_PLANNING_CONTINUATION;')
            rom_cur.execute('DROP TABLE IF EXISTS WAYPOINT;')
            rom_cur.execute('DROP TABLE IF EXISTS WAYPOINT_FLIGHT_PLANNING_CONTINUATION;')
            rom_cur.execute('CREATE TABLE AIRPORT (RECD_TYPE varchar(1), CUST_AREA varchar(3), SECT_CODE varchar(1), ARPT_IDENT varchar(4), ARPT_ICAO_CODE varchar(2), SUBS_CODE varchar(1), IATA_IDENT varchar(3), CONT_NR varchar(1), SPEED_LIMIT_ALT real, LONGEST_RUNWAY varchar(3), IFR_CAPABILITY varchar(1), LONGEST_RUNWAY_SURFACE_CODE varchar(1), ARPT_LAT real, ARPT_LON real, MAG_VAR real, ARPT_ELEV real, SPEED_LIMIT real, RECOMMENDED_NAVAID varchar(4), RECOMMENDED_NAVAID_ICAO_CODE varchar(2), TRANSITIONS_ALT real, TRANSITION_LEVEL real, PUBLIC_MILITARY_IND varchar(1), TIME_ZONE varchar(3), DAYLIGHT_IND varchar(1), MAG_TRUE_IND varchar(1), DATUM_CODE varchar(3), ARPT_NAME varchar(30), FILE_RECD_NR real, CYCLE real);')
            rom_cur.execute('CREATE TABLE AIRPORT_FLIGHT_PLANNING_CONTINUATION (RECD_TYPE varchar(1), CUST_AREA varchar(3), SECT_CODE varchar(1), ARPT_IDENT varchar(4), ARPT_ICAO_CODE varchar(2), SUBS_CODE varchar(1), IATA_IDENT varchar(3), CONT_NR real, APPLICATION_TYPE varchar(1), FIR_IDENT varchar(4), UIR_IDENT varchar(4), START_END_IND varchar(1), CONTROLLED_A_S_IND varchar(1), CONTROLLED_A_S_ARPT_IDENT varchar(4), CONTROLLED_A_S_ARPT_ICAO varchar(2), FILE_RECD_NR real, CYCLE real);')
            rom_cur.execute('CREATE TABLE AIRPORT_GATE (RECD_TYPE varchar(1), CUST_AREA varchar(3), SECT_CODE varchar(1), ARPT_IDENT varchar(4), ARPT_ICAO_CODE varchar(2), SUBS_CODE varchar(1), GATE_IDENT varchar(5), CONT_NR varchar(1), GATE_LAT real, GATE_LON real, NAME varchar(25), FILE_RECD_NR real, CYCLE real);')
            rom_cur.execute('CREATE TABLE AIRPORT_LOCALIZER_AND_GLIDE_SLOPE (RECD_TYPE varchar(1), CUST_AREA varchar(3), SECT_CODE varchar(1), ARPT_IDENT varchar(4), ARPT_ICAO_CODE varchar(2), SUBS_CODE varchar(1), LOCALIZER_IDENT varchar(4), ILS_CATEGORY varchar(1), CONT_NR varchar(1), LOCALIZER_FREQ real, RUNWAY_IDENT varchar(5), LOCALIZER_LAT real, LOCALIZER_LON real, LOCALIZER_BEARING real, GLIDE_SLOPE_LAT real, GLIDE_SLOPE_LON real, LOCALIZER_POS varchar(4), LOCALIZER_POS_REFERENCE varchar(1), GLIDE_SLOPE_POS varchar(4), LOCALIZER_WIDTH real, GLIDE_SLOPE_ANGLE real, STATION_DECLINATION varchar(5), GLIDE_SLOPE_HEIGHT_AT_LANDING_THLD real, GLIDE_SLOPE_ELEV real, SUPPORTING_FACILITY_IDENT varchar(4), SUPPORTING_FACILITY_ICAO_CODE varchar(2), SUPPORTING_FACILITY_SECT_CODE varchar(1), SUPPORTING_FACILITY_SUBS_CODE varchar(1), FILE_RECD_NR real, CYCLE real);')
            rom_cur.execute('CREATE TABLE AIRPORT_PROCEDURE (RECD_TYPE varchar(1), CUST_AREA varchar(3), SECT_CODE varchar(1), ARPT_IDENT varchar(4), ARPT_ICAO_CODE varchar(2), SUBS_CODE varchar(1), PROC_IDENT varchar(6), ROUTE_TYPE varchar(1), TRANSITION_IDENT varchar(5), SEQ_NR real, FIX_IDENT varchar(5), FIX_ICAO_CODE varchar(2), FIX_SECT_CODE varchar(1), FIX_SUBS_CODE varchar(1), CONT_NR varchar(1), WAYPOINT_DESCR_CODE varchar(4), TURN_DIR varchar(1), RNP RNP3, PATH_AND_TERMINATION varchar(2), TURN_DIR_VALID varchar(1), RECOMMENDED_NAVAID varchar(4), RECOMMENDED_NAVAID_ICAO_CODE varchar(2), ARC_RADIUS real, THETA real, RHO real, MAG_COURSE real, ROUTE_DISTANCE_HOLDING_DISTANCE_OR_TIME varchar(4), RECOMMENDED_NAVAID_SECT_CODE varchar(1), RECOMMENDED_NAVAID_SUBS_CODE varchar(1), ALT_DESCR varchar(1), ATC_IND varchar(1), ALT_1 real, ALT_2 real, TRANSITION_ALT real, SPEED_LIMIT varchar(3), VERTICAL_ANGLE real, CENTER_FIX_OR_TAA_PROCEDURE_TURN_IND varchar(5), MULTIPLE_CODE_OR_TAA_SECTOR_IDENT varchar(1), MULTIPLE_CODE_OR_TAA_SECTOR_ICAO_CODE varchar(2), MULTIPLE_CODE_OR_TAA_SECTOR_SECT_CODE varchar(1), MULTIPLE_CODE_OR_TAA_SECTOR_SUBS_CODE varchar(1), GNSS_FMS_INDICATION varchar(1), SPEED_LIMIT_DESCR varchar(1), APCH_ROUTE_QUALIFIER_1 varchar(1), APCH_ROUTE_QUALIFIER_2 varchar(1), VERTICAL_SCALE_FACTOR varchar(3), FILE_RECD_NR real, CYCLE real);')
            rom_cur.execute('CREATE TABLE COMPANY_ROUTE (RECD_TYPE varchar(1), CUSTOMER varchar(3), SECT_CODE varchar(1), SUBS_CODE varchar(1), FROM_ARPT varchar(5), FROM_ARPT_ICAO_CODE varchar(2), FROM_ARPT_SECT_CODE varchar(1), FROM_ARPT_SUBS_CODE varchar(1), TO_ARPT varchar(5), TO_ARPT_ICAO_CODE varchar(2), TO_ARPT_SECT_CODE varchar(1), TO_ARPT_SUBS_CODE varchar(1), COMPANY_ROUTE_ID varchar(10), SEQUENCE_NO varchar(3), VIA varchar(3), SID_STAR_APP_AWY varchar(6), AREA_CODE varchar(3), TO_FIX varchar(6), TO_ICAO_CODE varchar(2), TO_SECT_CODE varchar(1), TO_SUBS_CODE varchar(1), RUNWAY_TRANS varchar(5), ENRT_TRANS varchar(5), CRUISE_ALT real, TERMINAL_ALTN_ARPT varchar(4), TERMINAL_ALTN_ARPT_ICAO_CODE varchar(2), ALTN_DISTANCE real, COST_INDEX varchar(3), ENROUTE_ALTN_ARPT varchar(4), SID_STAR_APP_AWY_ROUTE_TYPE varchar(1), S_S_A_ROUTE_TYPE_QUALIFIER_1 varchar(1), S_S_A_ROUTE_TYPE_QUALIFIER_2 varchar(1), FILE_RECD_NR real, CYCLE real);')
            rom_cur.execute('CREATE TABLE CRUISING_TABLE (RECD_TYPE varchar(1), SECT_CODE varchar(1), SUBS_CODE varchar(1), CRUISE_TABLE_IDENT varchar(2), SEQUENCE_NUMBER varchar(1), COURSE_FROM varchar(4), COURSE_TO varchar(4), MAG_TRUE varchar(1), CRUISE_LEVEL_FROM_0 varchar(5), VERTICAL_SEPARATION_0 varchar(5), CRUISE_LEVEL_TO_0 varchar(5), CRUISE_LEVEL_FROM_1 varchar(5), VERTICAL_SEPARATION_1 varchar(5), CRUISE_LEVEL_TO_1 varchar(5), CRUISE_LEVEL_FROM_2 varchar(5), VERTICAL_SEPARATION_2 varchar(5), CRUISE_LEVEL_TO_2 varchar(5), CRUISE_LEVEL_FROM_3 varchar(5), VERTICAL_SEPARATION_3 varchar(5), CRUISE_LEVEL_TO_3 varchar(5), FILE_RECD_NR real, CYCLE real);')
            rom_cur.execute('CREATE TABLE CYCLE_INFO (VARG, PARAMETER, VALUE);')
            rom_cur.execute('CREATE TABLE ENROUTE_AIRWAYS (RECD_TYPE varchar(1), CUST_AREA varchar(3), SECT_CODE varchar(1), SUBS_CODE varchar(1), ROUTE_IDENT varchar(5), SEQ_NR real, FIX_IDENT varchar(5), FIX_ICAO_CODE varchar(2), FIX_SECT_CODE varchar(1), FIX_SUBS_CODE varchar(1), CONT_NR varchar(1), WAYPOINT_DESCR_CODE varchar(4), BOUNDARY_CODE varchar(1), ROUTE_TYPE varchar(1), LEVEL varchar(1), DIRECTION_RESTRICTION varchar(1), CRUISE_TABLE_IND varchar(2), EU_IND varchar(1), RECOMMENDED_NAVAID varchar(4), RECOMMENDED_NAVAID_ICAO_CODE varchar(2), RNP RNP3, THETA real, RHO real, OUTBOUND_MAG_COURSE real, ROUTE_DISTANCE_FROM varchar(4), INBOUND_MAG_COURSE real, MINIMUM_ALT real, MINIMUM_ALT_1 real, MAXIMUM_ALT real, FIX_RADIUS_TRANSITION_IND varchar(3), VERTICAL_SCALE_FACTOR varchar(3), RVSM_MINIMUM_LEVEL real, VSF_RVSM_MAXIMUM_LEVEL real, ICAO_CODE_2 varchar(2), FILE_RECD_NR real, CYCLE real);')
            rom_cur.execute('CREATE TABLE GRID_MORA (RECD_TYPE varchar(1), SECT_CODE varchar(1), SUBS_CODE varchar(1), MORA_LAT real, MORA_LON real, MORA_TYPE varchar(1), MORA_00 real, MORA_01 real, MORA_02 real, MORA_03 real, MORA_04 real, MORA_05 real, MORA_06 real, MORA_07 real, MORA_08 real, MORA_09 real, MORA_10 real, MORA_11 real, MORA_12 real, MORA_13 real, MORA_14 real, MORA_15 real, MORA_16 real, MORA_17 real, MORA_18 real, MORA_19 real, MORA_20 real, MORA_21 real, MORA_22 real, MORA_23 real, MORA_24 real, MORA_25 real, MORA_26 real, MORA_27 real, MORA_28 real, MORA_29 real, FILE_RECD_NR real, CYCLE real);')
            rom_cur.execute('CREATE TABLE HDR_ROWS (HEADER varchar(132));')
            rom_cur.execute('CREATE TABLE HOLDING_PATTERN (RECD_TYPE varchar(1), CUST_AREA varchar(3), SECT_CODE varchar(1), SUBS_CODE varchar(1), REGION_CODE varchar(4), REGION_ICAO_CODE varchar(2), DUPLICATE_IDENT real, FIX_IDENT varchar(5), FIX_ICAO_CODE varchar(2), FIX_SECT_CODE varchar(1), FIX_SUBS_CODE varchar(1), CONT_NR varchar(1), INBOUND_HOLDING_COURSE real, TURN_DIR varchar(1), LEG_LENGTH real, LEG_TIME varchar(2), MINIMUM_ALT real, MAXIMUM_ALT real, HOLDING_SPEED real, RNP RNP3, ARC_RADIUS real, VERTICAL_SCALE_FACTOR varchar(3), RVSM_MINIMUM_LEVEL real, RVSM_MAXIMUM_LEVEL real, NAME varchar(25), FILE_RECD_NR real, CYCLE real);')
            rom_cur.execute('CREATE TABLE NDB_NAVAID (RECD_TYPE varchar(1), CUST_AREA varchar(3), SECT_CODE varchar(1), SUBS_CODE varchar(1), ARPT_IDENT varchar(4), ARPT_ICAO_CODE varchar(2), NDB_IDENT varchar(4), NDB_ICAO_CODE varchar(2), CONT_NR varchar(1), NDB_FREQ real, NDB_CLASS varchar(5), NDB_LAT real, NDB_LON real, MAG_VAR real, DATUM_CODE varchar(3), NDB_NAME varchar(30), FILE_RECD_NR real, CYCLE real);')
            rom_cur.execute('CREATE TABLE NDB_NAVAID_FLIGHT_PLANNING_CONTINUATION (RECD_TYPE varchar(1), CUST_AREA varchar(3), SECT_CODE varchar(1), SUBS_CODE varchar(1), ARPT_IDENT varchar(4), ARPT_ICAO_CODE varchar(2), NDB_IDENT varchar(4), NDB_ICAO_CODE varchar(2), CONT_NR real, APPLICATION_TYPE varchar(1), FIR_IDENT varchar(4), UIR_IDENT varchar(4), START_END_IND varchar(1), FILE_RECD_NR real, CYCLE real);')
            rom_cur.execute('CREATE TABLE REJECTED_ROWS (RECD_TYPE varchar(1), CUST_AREA varchar(3), SECT_CODE varchar(1), DATA varchar(118), FILE_RECD_NR real, CYCLE real);')
            rom_cur.execute('CREATE TABLE RUNWAY (RECD_TYPE varchar(1), CUST_AREA varchar(3), SECT_CODE varchar(1), ARPT_IDENT varchar(4), ARPT_ICAO_CODE varchar(2), SUBS_CODE varchar(1), RUNWAY_IDENT varchar(5), CONT_NR varchar(1), RUNWAY_LENGTH real, RUNWAY_MAG_BEARING bearing, RUNWAY_LAT real, RUNWAY_LON real, RUNWAY_GRADIENT real, ELLIPSOID_HEIGHT real, LANDING_THLD_ELEV real, DISPLACED_THLD_DISTANCE real, THLD_CROSSING_HEIGHT real, RUNWAY_WIDTH real, TCH_VALUE_IND varchar(1), LOCALIZER_MLS_GLS_REF_PATH_IDENT varchar(4), LOCALIZER_MLS_GLS_CATEGORY_CLASS varchar(1), STOPWAY varchar(4), SECOND_LOCALIZER_MLS_GLS_REF_PATH_IDENT varchar(4), SECOND_LOCALIZER_MLS_GLS_CATEGORY_CLASS varchar(1), RUNWAY_DESCR varchar(22), FILE_RECD_NR real, CYCLE real);')
            rom_cur.execute('CREATE TABLE TACAN_ONLY_NAVAID (RECD_TYPE varchar(1), CUST_AREA varchar(3), SECT_CODE varchar(1), SUBS_CODE varchar(1), ARPT_IDENT varchar(4), ARPT_ICAO_CODE varchar(2), VOR_IDENT varchar(4), CONT_NR varchar(1), VOR_FREQ real, NAVAID_CLASS varchar(5), TACAN_IDENT varchar(4), TACAN_LAT real, TACAN_LON real, THRU79_STATION_DECLINATION varchar(5), TACAN_ELEV real, FIGURE_OF_MERIT varchar(1), FREQ_PROTECTION varchar(3), DATUM_CODE varchar(3), TACAN_NAME varchar(30), FILE_RECD_NR real, CYCLE real);')
            rom_cur.execute('CREATE TABLE VHF_NAVAID (RECD_TYPE varchar(1), CUST_AREA varchar(3), SECT_CODE varchar(1), SUBS_CODE varchar(1), ARPT_IDENT varchar(4), ARPT_ICAO_CODE varchar(2), VOR_IDENT varchar(4), VHF_ICAO_CODE varchar(2), CONT_NR varchar(1), VOR_FREQ real, NAVAID_CLASS varchar(5), VOR_LAT real, VOR_LON real, DME_IDENT varchar(4), DME_LAT real, DME_LON real, STATION_DECLINATION varchar(5), DME_ELEV real, FIGURE_OF_MERIT varchar(1), ILS_DME_BIAS real, FREQ_PROTECTION varchar(3), DATUM_CODE varchar(3), VOR_NAME varchar(25), ROUTE_INAPPROPRIATE_DME varchar(1), DME_OPERATIONAL_SERVICE_VOLUME varchar(1), FILE_RECD_NR real, CYCLE real);')
            rom_cur.execute('CREATE TABLE VHF_NAVAID_FLIGHT_PLANNING_CONTINUATION (RECD_TYPE varchar(1), CUST_AREA varchar(3), SECT_CODE varchar(1), SUBS_CODE varchar(1), ARPT_IDENT varchar(4), ARPT_ICAO_CODE varchar(2), VOR_IDENT varchar(4), VHF_ICAO_CODE varchar(2), CONT_NR real, APPLICATION_TYPE varchar(1), FIR_IDENT varchar(4), UIR_IDENT varchar(4), START_END_IND varchar(1), FILE_RECD_NR real, CYCLE real);')
            rom_cur.execute('CREATE TABLE WAYPOINT (RECD_TYPE varchar(1), CUST_AREA varchar(3), SECT_CODE varchar(1), SUBS_CODE varchar(1), REGION_CODE varchar(4), REGION_ICAO_CODE varchar(2), REGION_SUBS_CODE varchar(1), WAYPOINT_IDENT varchar(5), WAYPOINT_ICAO_CODE varchar(2), CONT_NR varchar(1), WAYPOINT_TYPE varchar(3), WAYPOINT_USAGE varchar(2), WAYPOINT_LAT real, WAYPOINT_LON real, DYNAMIC_MAG_VAR real, DATUM_CODE varchar(3), NAME_FORMAT_IND varchar(3), WAYPOINT_NAME_DESCR varchar(25), FILE_RECD_NR real, CYCLE real);')
            rom_cur.execute('CREATE TABLE WAYPOINT_FLIGHT_PLANNING_CONTINUATION (RECD_TYPE varchar(1), CUST_AREA varchar(3), SECT_CODE varchar(1), SUBS_CODE varchar(1), REGION_CODE varchar(4), REGION_ICAO_CODE varchar(2), REGION_SUBS_CODE varchar(1), WAYPOINT_IDENT varchar(5), WAYPOINT_ICAO_CODE varchar(2), CONT_NR real, APPLICATION_TYPE varchar(1), FIR_IDENT varchar(4), UIR_IDENT varchar(4), START_END_IND varchar(1), FILE_RECD_NR real, CYCLE real);')
            rom_cur.execute('CREATE INDEX IX_AIRPORT_ARPT_IDENT ON AIRPORT(ARPT_IDENT);')
            rom_cur.execute('CREATE INDEX IX_AIRPORT_ARPT_LAT_ARPT_LON ON AIRPORT(ARPT_LAT, ARPT_LON);')
            rom_cur.execute('CREATE INDEX IX_AIRPORT_FLIGHT_PLANNING_CONTINUATION_ARPT_IDENT ON AIRPORT_FLIGHT_PLANNING_CONTINUATION(ARPT_IDENT);')
            rom_cur.execute('CREATE INDEX IX_AIRPORT_FLIGHT_PLANNING_CONTINUATION_RECD_TYPE ON AIRPORT_FLIGHT_PLANNING_CONTINUATION(RECD_TYPE);')
            rom_cur.execute('CREATE INDEX IX_AIRPORT_GATE_ARPT_IDENT ON AIRPORT_GATE(ARPT_IDENT);')
            rom_cur.execute('CREATE INDEX IX_AIRPORT_GATE_RECD_TYPE ON AIRPORT_GATE(RECD_TYPE);')
            rom_cur.execute('CREATE INDEX IX_AIRPORT_LOCALIZER_AND_GLIDE_SLOPE_ARPT_IDENT ON AIRPORT_LOCALIZER_AND_GLIDE_SLOPE(ARPT_IDENT);')
            rom_cur.execute('CREATE INDEX IX_AIRPORT_LOCALIZER_AND_GLIDE_SLOPE_RECD_TYPE ON AIRPORT_LOCALIZER_AND_GLIDE_SLOPE(RECD_TYPE);')
            rom_cur.execute('CREATE INDEX IX_AIRPORT_PROCEDURE_ARPT_IDENT ON AIRPORT_PROCEDURE(ARPT_IDENT);')
            rom_cur.execute('CREATE INDEX IX_AIRPORT_PROCEDURE_RECD_TYPE ON AIRPORT_PROCEDURE(RECD_TYPE);')
            rom_cur.execute('CREATE INDEX IX_AIRPORT_RECD_TYPE ON AIRPORT(RECD_TYPE);')
            rom_cur.execute('CREATE INDEX IX_COMPANY_ROUTE_COMPANY_ROUTE_ID ON COMPANY_ROUTE(COMPANY_ROUTE_ID);')
            rom_cur.execute('CREATE INDEX IX_COMPANY_ROUTE_RECD_TYPE ON COMPANY_ROUTE(RECD_TYPE);')
            rom_cur.execute('CREATE INDEX IX_CRUISING_TABLE_RECD_TYPE ON CRUISING_TABLE(RECD_TYPE);')
            rom_cur.execute('CREATE INDEX IX_ENROUTE_AIRWAYS_RECD_TYPE ON ENROUTE_AIRWAYS(RECD_TYPE);')
            rom_cur.execute('CREATE INDEX IX_ENROUTE_AIRWAYS_ROUTE_IDENT_SEQ_NR_FIX_IDENT_FIX_ICAO_CODE_FIX_SECT_CODE_FIX_SUBS_CODE ON ENROUTE_AIRWAYS(ROUTE_IDENT, SEQ_NR, FIX_IDENT, FIX_ICAO_CODE, FIX_SECT_CODE, FIX_SUBS_CODE);')
            rom_cur.execute('CREATE INDEX IX_GRID_MORA_RECD_TYPE ON GRID_MORA(RECD_TYPE);')
            rom_cur.execute('CREATE INDEX IX_HOLDING_PATTERN_RECD_TYPE ON HOLDING_PATTERN(RECD_TYPE);')
            rom_cur.execute('CREATE INDEX IX_NDB_NAVAID_FLIGHT_PLANNING_CONTINUATION_NDB_IDENT_NDB_ICAO_CODE_CUST_AREA_ARPT_IDENT ON NDB_NAVAID_FLIGHT_PLANNING_CONTINUATION(NDB_IDENT, NDB_ICAO_CODE, CUST_AREA, ARPT_IDENT);')
            rom_cur.execute('CREATE INDEX IX_NDB_NAVAID_FLIGHT_PLANNING_CONTINUATION_RECD_TYPE ON NDB_NAVAID_FLIGHT_PLANNING_CONTINUATION(RECD_TYPE);')
            rom_cur.execute('CREATE INDEX IX_NDB_NAVAID_NDB_FREQ ON NDB_NAVAID(NDB_FREQ);')
            rom_cur.execute('CREATE INDEX IX_NDB_NAVAID_NDB_IDENT ON NDB_NAVAID(NDB_IDENT);')
            rom_cur.execute('CREATE INDEX IX_NDB_NAVAID_NDB_LAT_NDB_LON ON NDB_NAVAID(NDB_LAT, NDB_LON);')
            rom_cur.execute('CREATE INDEX IX_NDB_NAVAID_RECD_TYPE ON NDB_NAVAID(RECD_TYPE);')
            rom_cur.execute('CREATE INDEX IX_RUNWAY_ARPT_IDENT ON RUNWAY(ARPT_IDENT);')
            rom_cur.execute('CREATE INDEX IX_RUNWAY_RECD_TYPE ON RUNWAY(RECD_TYPE);')
            rom_cur.execute('CREATE INDEX IX_TACAN_ONLY_NAVAID_RECD_TYPE ON TACAN_ONLY_NAVAID(RECD_TYPE);')
            rom_cur.execute('CREATE INDEX IX_VHF_NAVAID_DME_LAT_DME_LON ON VHF_NAVAID(DME_LAT, DME_LON);')
            rom_cur.execute('CREATE INDEX IX_VHF_NAVAID_FLIGHT_PLANNING_CONTINUATION_RECD_TYPE ON VHF_NAVAID_FLIGHT_PLANNING_CONTINUATION(RECD_TYPE);')
            rom_cur.execute('CREATE INDEX IX_VHF_NAVAID_FLIGHT_PLANNING_CONTINUATION_VOR_IDENT_VHF_ICAO_CODE_CUST_AREA_ARPT_IDENT ON VHF_NAVAID_FLIGHT_PLANNING_CONTINUATION(VOR_IDENT, VHF_ICAO_CODE, CUST_AREA, ARPT_IDENT);')
            rom_cur.execute('CREATE INDEX IX_VHF_NAVAID_RECD_TYPE ON VHF_NAVAID(RECD_TYPE);')
            rom_cur.execute('CREATE INDEX IX_VHF_NAVAID_VOR_FREQ ON VHF_NAVAID(VOR_FREQ);')
            rom_cur.execute('CREATE INDEX IX_VHF_NAVAID_VOR_IDENT ON VHF_NAVAID(VOR_IDENT);')
            rom_cur.execute('CREATE INDEX IX_VHF_NAVAID_VOR_LAT_VOR_LON ON VHF_NAVAID(VOR_LAT, VOR_LON);')
            rom_cur.execute('CREATE INDEX IX_WAYPOINT_FLIGHT_PLANNING_CONTINUATION_RECD_TYPE ON WAYPOINT_FLIGHT_PLANNING_CONTINUATION(RECD_TYPE);')
            rom_cur.execute('CREATE INDEX IX_WAYPOINT_FLIGHT_PLANNING_CONTINUATION_WAYPOINT_IDENT_WAYPOINT_ICAO_CODE ON WAYPOINT_FLIGHT_PLANNING_CONTINUATION(WAYPOINT_IDENT, WAYPOINT_ICAO_CODE);')
            rom_cur.execute('CREATE INDEX IX_WAYPOINT_RECD_TYPE ON WAYPOINT(RECD_TYPE);')
            rom_cur.execute('CREATE INDEX IX_WAYPOINT_WAYPOINT_IDENT ON WAYPOINT(WAYPOINT_IDENT);')
            rom_cur.execute('CREATE INDEX IX_WAYPOINT_WAYPOINT_LAT_WAYPOINT_LON ON WAYPOINT(WAYPOINT_LAT, WAYPOINT_LON);')
            rom_cur.execute('CREATE UNIQUE INDEX UX_AIRPORT_FILE_RECD_NR ON AIRPORT(FILE_RECD_NR);')
            rom_cur.execute('CREATE UNIQUE INDEX UX_AIRPORT_FLIGHT_PLANNING_CONTINUATION_FILE_RECD_NR ON AIRPORT_FLIGHT_PLANNING_CONTINUATION(FILE_RECD_NR);')
            rom_cur.execute('CREATE UNIQUE INDEX UX_AIRPORT_GATE_FILE_RECD_NR ON AIRPORT_GATE(FILE_RECD_NR);')
            rom_cur.execute('CREATE UNIQUE INDEX UX_AIRPORT_LOCALIZER_AND_GLIDE_SLOPE_FILE_RECD_NR ON AIRPORT_LOCALIZER_AND_GLIDE_SLOPE(FILE_RECD_NR);')
            rom_cur.execute('CREATE UNIQUE INDEX UX_AIRPORT_PROCEDURE_FILE_RECD_NR ON AIRPORT_PROCEDURE(FILE_RECD_NR);')
            rom_cur.execute('CREATE UNIQUE INDEX UX_COMPANY_ROUTE_COMPANY_ROUTE_ID_SEQUENCE_NO ON COMPANY_ROUTE(COMPANY_ROUTE_ID, SEQUENCE_NO);')
            rom_cur.execute('CREATE UNIQUE INDEX UX_COMPANY_ROUTE_FILE_RECD_NR ON COMPANY_ROUTE(FILE_RECD_NR);')
            rom_cur.execute('CREATE UNIQUE INDEX UX_CRUISING_TABLE_FILE_RECD_NR ON CRUISING_TABLE(FILE_RECD_NR);')
            rom_cur.execute('CREATE UNIQUE INDEX UX_ENROUTE_AIRWAYS_FILE_RECD_NR ON ENROUTE_AIRWAYS(FILE_RECD_NR);')
            rom_cur.execute('CREATE UNIQUE INDEX UX_GRID_MORA_FILE_RECD_NR ON GRID_MORA(FILE_RECD_NR);')
            rom_cur.execute('CREATE UNIQUE INDEX UX_GRID_MORA_MORA_LAT_MORA_LON_MORA_TYPE ON GRID_MORA(MORA_LAT, MORA_LON, MORA_TYPE);')
            rom_cur.execute('CREATE UNIQUE INDEX UX_HOLDING_PATTERN_DUPLICATE_IDENT_FIX_IDENT_FIX_ICAO_CODE_FIX_SECT_CODE_FIX_SUBS_CODE_REGION_CODE_REGION_ICAO_CODE ON HOLDING_PATTERN(DUPLICATE_IDENT, FIX_IDENT, FIX_ICAO_CODE, FIX_SECT_CODE, FIX_SUBS_CODE, REGION_CODE, REGION_ICAO_CODE);')
            rom_cur.execute('CREATE UNIQUE INDEX UX_HOLDING_PATTERN_FILE_RECD_NR ON HOLDING_PATTERN(FILE_RECD_NR);')
            rom_cur.execute('CREATE UNIQUE INDEX UX_NDB_NAVAID_FILE_RECD_NR ON NDB_NAVAID(FILE_RECD_NR);')
            rom_cur.execute('CREATE UNIQUE INDEX UX_NDB_NAVAID_FLIGHT_PLANNING_CONTINUATION_FILE_RECD_NR ON NDB_NAVAID_FLIGHT_PLANNING_CONTINUATION(FILE_RECD_NR);')
            rom_cur.execute('CREATE UNIQUE INDEX UX_RUNWAY_FILE_RECD_NR ON RUNWAY(FILE_RECD_NR);')
            rom_cur.execute('CREATE UNIQUE INDEX UX_TACAN_ONLY_NAVAID_FILE_RECD_NR ON TACAN_ONLY_NAVAID(FILE_RECD_NR);')
            rom_cur.execute('CREATE UNIQUE INDEX UX_VHF_NAVAID_FILE_RECD_NR ON VHF_NAVAID(FILE_RECD_NR);')
            rom_cur.execute('CREATE UNIQUE INDEX UX_VHF_NAVAID_FLIGHT_PLANNING_CONTINUATION_FILE_RECD_NR ON VHF_NAVAID_FLIGHT_PLANNING_CONTINUATION(FILE_RECD_NR);')
            rom_cur.execute('CREATE UNIQUE INDEX UX_WAYPOINT_FILE_RECD_NR ON WAYPOINT(FILE_RECD_NR);')
            rom_cur.execute('CREATE UNIQUE INDEX UX_WAYPOINT_FLIGHT_PLANNING_CONTINUATION_FILE_RECD_NR ON WAYPOINT_FLIGHT_PLANNING_CONTINUATION(FILE_RECD_NR);')
        if fsl_write:
            mora_fsl_tb.to_sql('GRID_MORA', rom, if_exists='append', index=False)
            vhf_fsl_tb.to_sql('VHF_NAVAID', rom, if_exists='append', index=False)
            ndb_fsl_tb.to_sql('NDB_NAVAID', rom, if_exists='append', index=False)
            awy_fsl_tb.to_sql('ENROUTE_AIRWAYS', rom, if_exists='append', index=False)
            hold_fsl_tb.to_sql('HOLDING_PATTERN', rom, if_exists='append', index=False)
            arpt_fsl_tb.to_sql('AIRPORT', rom, if_exists='append', index=False)
            wpt_fsl_tb.to_sql('WAYPOINT', rom, if_exists='append', index=False)
            proc_fsl_tb.to_sql('AIRPORT_PROCEDURE', rom, if_exists='append', index=False)
            rwy_fsl_tb.to_sql('RUNWAY', rom, if_exists='append', index=False)
            loc_fsl_tb.to_sql('AIRPORT_LOCALIZER_AND_GLIDE_SLOPE', rom, if_exists='append', index=False)
            crz_fsl_tb.to_sql('CRUISING_TABLE', rom, if_exists='append', index=False) 
        if add_tailored:
            rom_cur.execute('UPDATE ENROUTE_AIRWAYS SET FIX_SECT_CODE = "E" AND FIX_SUBS_CODE = "A" WHERE RECD_TYPE = "T" AND FIX_IDENT = "DO" AND FIX_ICAO_CODE = "ZS" AND FIX_SECT_CODE = "D" AND FIX_SUBS_CODE = "B";')
            rom_cur.execute('UPDATE ENROUTE_AIRWAYS SET FIX_SUBS_CODE = "B" WHERE RECD_TYPE = "T" AND FIX_IDENT = "DO" AND FIX_ICAO_CODE = "ZS" AND FIX_SECT_CODE = "D" AND FIX_SUBS_CODE = " ";')
            rom_cur.execute('UPDATE ENROUTE_AIRWAYS SET FIX_ICAO_CODE = "ZJ" WHERE RECD_TYPE = "T" AND FIX_IDENT = "AGTEL" AND FIX_ICAO_CODE = "ZG";')
            rom_cur.execute('UPDATE ENROUTE_AIRWAYS SET FIX_ICAO_CODE = "ZU" WHERE RECD_TYPE = "T" AND FIX_IDENT = "GOKEB" AND FIX_ICAO_CODE = "ZL";')
            rom_cur.execute('UPDATE ENROUTE_AIRWAYS SET FIX_ICAO_CODE = "ZH" WHERE RECD_TYPE = "T" AND FIX_IDENT = "IGADA" AND FIX_ICAO_CODE = "ZL";')
            rom_cur.execute('UPDATE ENROUTE_AIRWAYS SET FIX_ICAO_CODE = "ZG" WHERE RECD_TYPE = "T" AND FIX_IDENT = "OVTAN" AND FIX_ICAO_CODE = "ZS";')
            rom_cur.execute('UPDATE ENROUTE_AIRWAYS SET FIX_ICAO_CODE = "ZY" WHERE RECD_TYPE = "T" AND FIX_IDENT = "SANKO" AND FIX_ICAO_CODE = "ZS";')
            rom_cur.execute('UPDATE ENROUTE_AIRWAYS SET FIX_ICAO_CODE = "ZB" WHERE RECD_TYPE = "T" AND FIX_IDENT = "WXI" AND FIX_ICAO_CODE = "ZH";')
            rom_cur.execute('UPDATE NDB_NAVAID SET NDB_ICAO_CODE = "ZG" WHERE RECD_TYPE = "T" AND NDB_IDENT = "DO" AND NDB_ICAO_CODE = "ZS";')
        rom_cur.close()
        rom.commit()
        rom.close()
    print(time.strftime("%H:%M:%S", time.localtime()) , 'FSLabs: Data Export Finished')

# JeeHell (Acceptable)
if JeeHell:
    print(time.strftime("%H:%M:%S", time.localtime()) , 'JeeHell: Start Processing DataFile')
    os.makedirs(Cycle_Ident + '/JeeHell')
    # Set Variable of JeeHell
    if 1:
        mora_jh = []
        vhf_jh = []
        ndb_jh = []
        wpt_jh = []
        hold_jh = []
        awy_jh = []
        arpt_jh = []
        proc_jh = []
        rwy_jh = []
        ils_jh = []
        mkr_jh = []
        gls_jh = []
        fixes_in_jh_nd = []
    # Load JeeHell's navrecord.dat
    if jh_create_blank != True:
        shutil.copy2('Input/JeeHell/navrecord.dat', Cycle_Ident + '/JeeHell/navrecord.dat')
        with open(Cycle_Ident + '/JeeHell/navrecord.dat', 'r', encoding='ISO-8859-1') as JeeHell_nd:
            for line in JeeHell_nd:
                vhf_jh_se = re.search(vor_jh_nd, line)
                db_jh_se = re.search(db_jh_nd, line)
                ea_jh_se = re.search(ea_jh_nd, line)
                if vhf_jh_se:
                    FIX_SECT_SUBS = vhf_jh_se.group(1)
                    FIX_IDENT = vhf_jh_se.group(2)
                    FIX_ICAO_CODE = vhf_jh_se.group(3)
                    if vhf_jh_se.group(4) != '':
                        FIX_LAT = vhf_jh_se.group(4)
                        FIX_LON = vhf_jh_se.group(5)
                    else:
                        FIX_LAT = vhf_jh_se.group(6)
                        FIX_LON = vhf_jh_se.group(7)
                    fixes_in_jh_nd.append((FIX_SECT_SUBS,FIX_IDENT,FIX_ICAO_CODE,FIX_LAT,FIX_LON))
                if db_jh_se:
                    FIX_SECT_SUBS = db_jh_se.group(1)
                    FIX_IDENT = db_jh_se.group(2)
                    FIX_ICAO_CODE = db_jh_se.group(3)
                    FIX_LAT = db_jh_se.group(4)
                    FIX_LON = db_jh_se.group(5)
                    fixes_in_jh_nd.append((FIX_SECT_SUBS,FIX_IDENT,FIX_ICAO_CODE,FIX_LAT,FIX_LON))
                if ea_jh_se:
                    FIX_SECT_SUBS = ea_jh_se.group(1)
                    FIX_IDENT = ea_jh_se.group(2)
                    FIX_ICAO_CODE = ea_jh_se.group(3)
                    FIX_LAT = ea_jh_se.group(4)
                    FIX_LON = ea_jh_se.group(5)
                    fixes_in_jh_nd.append((FIX_SECT_SUBS,FIX_IDENT,FIX_ICAO_CODE,FIX_LAT,FIX_LON))
        fixes_in_jh_nd_tb = pandas.DataFrame(fixes_in_jh_nd, columns=['FIX_SECT_SUBS','FIX_IDENT','FIX_ICAO_CODE','FIX_LAT','FIX_LON'])
        print(time.strftime("%H:%M:%S", time.localtime()) , 'JeeHell: navrecord.dat Load Completed')
    # Process of JeeHell Part #1
    with open('Input/' + Datafile, 'r', encoding='ISO-8859-1') as ARINC424:
        for line in ARINC424: 
            mora_se = re.search(as_grid, line)
            vhf_se = re.search(vor, line)
            db_se = re.search(db, line)
            ea_se = re.search(ea, line)
            hold_se = re.search(ep, line)
            arpt_se = re.search(pa, line)
            pc_se = re.search(pc, line)
            proc_se = re.search(pdef, line)
            rwy_se = re.search(pg, line)
            loc_se = re.search(pi, line)
            mkr_se = re.search(pm, line)
            pn_se = re.search(pn, line)
            gls_se = re.search(pt, line)
            crz_se = re.search(tc, line)
            if mora_se:
                SECT_SUBS = mora_se.group(2) + mora_se.group(3)
                MORA_LAT = mora_se.group(4) + mora_se.group(5)
                MORA_LON = mora_se.group(6) + mora_se.group(7)
                if mora_se.group(9) == "UNK":
                    MORA_00 = None
                else:
                    MORA_00 = mora_se.group(9)
                if mora_se.group(10) == "UNK":
                    MORA_01 = None
                else:
                    MORA_01 = mora_se.group(10)
                if mora_se.group(11) == "UNK":
                    MORA_02 = None
                else:
                    MORA_02 = mora_se.group(11)
                if mora_se.group(12) == "UNK":
                    MORA_03 = None
                else:
                    MORA_03 = mora_se.group(12)
                if mora_se.group(13) == "UNK":
                    MORA_04 = None
                else:
                    MORA_04 = mora_se.group(13)
                if mora_se.group(14) == "UNK":
                    MORA_05 = None
                else:
                    MORA_05 = mora_se.group(14)
                if mora_se.group(15) == "UNK":
                    MORA_06 = None
                else:
                    MORA_06 = mora_se.group(15)
                if mora_se.group(16) == "UNK":
                    MORA_07 = None
                else:
                    MORA_07 = mora_se.group(16)
                if mora_se.group(17) == "UNK":
                    MORA_08 = None
                else:
                    MORA_08 = mora_se.group(17)
                if mora_se.group(18) == "UNK":
                    MORA_09 = None
                else:
                    MORA_09 = mora_se.group(18)
                if mora_se.group(19) == "UNK":
                    MORA_10 = None
                else:
                    MORA_10 = mora_se.group(19)
                if mora_se.group(20) == "UNK":
                    MORA_11 = None
                else:
                    MORA_11 = mora_se.group(20)
                if mora_se.group(21) == "UNK":
                    MORA_12 = None
                else:
                    MORA_12 = mora_se.group(21)
                if mora_se.group(22) == "UNK":
                    MORA_13 = None
                else:
                    MORA_13 = mora_se.group(22)
                if mora_se.group(23) == "UNK":
                    MORA_14 = None
                else:
                    MORA_14 = mora_se.group(23)
                if mora_se.group(24) == "UNK":
                    MORA_15 = None
                else:
                    MORA_15 = mora_se.group(24)
                if mora_se.group(25) == "UNK":
                    MORA_16 = None
                else:
                    MORA_16 = mora_se.group(25)
                if mora_se.group(26) == "UNK":
                    MORA_17 = None
                else:
                    MORA_17 = mora_se.group(26)
                if mora_se.group(27) == "UNK":
                    MORA_18 = None
                else:
                    MORA_18 = mora_se.group(27)
                if mora_se.group(28) == "UNK":
                    MORA_19 = None
                else:
                    MORA_19 = mora_se.group(28)
                if mora_se.group(29) == "UNK":
                    MORA_20 = None
                else:
                    MORA_20 = mora_se.group(29)
                if mora_se.group(30) == "UNK":
                    MORA_21 = None
                else:
                    MORA_21 = mora_se.group(30)
                if mora_se.group(31) == "UNK":
                    MORA_22 = None
                else:
                    MORA_22 = mora_se.group(31)
                if mora_se.group(32) == "UNK":
                    MORA_23 = None
                else:
                    MORA_23 = mora_se.group(32)
                if mora_se.group(33) == "UNK":
                    MORA_24 = None
                else:
                    MORA_24 = mora_se.group(33)
                if mora_se.group(34) == "UNK":
                    MORA_25 = None
                else:
                    MORA_25 = mora_se.group(34)
                if mora_se.group(35) == "UNK":
                    MORA_26 = None
                else:
                    MORA_26 = mora_se.group(35)
                if mora_se.group(36) == "UNK":
                    MORA_27 = None
                else:
                    MORA_27 = mora_se.group(36)
                if mora_se.group(37) == "UNK":
                    MORA_28 = None
                else:
                    MORA_28 = mora_se.group(37)
                if mora_se.group(38) == "UNK":
                    MORA_29 = None
                else:
                    MORA_29 = mora_se.group(38)
                mora_jh.append((SECT_SUBS,MORA_LAT,MORA_LON,MORA_00,MORA_01,MORA_02,MORA_03,MORA_04,MORA_05,MORA_06,MORA_07,MORA_08,MORA_09,MORA_10,MORA_11,MORA_12,MORA_13,MORA_14,MORA_15,MORA_16,MORA_17,MORA_18,MORA_19,MORA_20,MORA_21,MORA_22,MORA_23,MORA_24,MORA_25,MORA_26,MORA_27,MORA_28,MORA_29,''))
            if vhf_se:
                SECT_SUBS = vhf_se.group(3) + vhf_se.group(4)
                VOR_IDENT = vhf_se.group(7).strip()
                VHF_ICAO_CODE = vhf_se.group(8)
                VOR_FREQ = float(vhf_se.group(10))/100
                NAVAID_CLASS = vhf_se.group(11)
                if vhf_se.group(12).strip() == '':
                    VOR_LAT = None
                elif vhf_se.group(12) == "N":
                    VOR_LAT = str(Decimal(Decimal(vhf_se.group(13)) + Decimal(vhf_se.group(14))/60 + Decimal(vhf_se.group(15))/3600 + Decimal(vhf_se.group(16))/360000).quantize(Decimal('0.00000000'), rounding=ROUND_HALF_UP))
                else:
                    VOR_LAT = str(Decimal(0 - Decimal(vhf_se.group(13)) - Decimal(vhf_se.group(14))/60 - Decimal(vhf_se.group(15))/3600 - Decimal(vhf_se.group(16))/360000).quantize(Decimal('0.00000000'), rounding=ROUND_HALF_UP))
                if vhf_se.group(17).strip() == '':
                    VOR_LON = None
                elif vhf_se.group(17) == "E":
                    VOR_LON = str(Decimal(Decimal(vhf_se.group(18)) + Decimal(vhf_se.group(19))/60 + Decimal(vhf_se.group(20))/3600 + Decimal(vhf_se.group(21))/360000).quantize(Decimal('0.00000000'), rounding=ROUND_HALF_UP))
                else:
                    VOR_LON = str(Decimal(0 - Decimal(vhf_se.group(18)) - Decimal(vhf_se.group(19))/60 - Decimal(vhf_se.group(20))/3600 - Decimal(vhf_se.group(21))/360000).quantize(Decimal('0.00000000'), rounding=ROUND_HALF_UP))
                if vhf_se.group(23).strip() == '':
                    DME_LAT = None
                elif vhf_se.group(23) == "N":
                    DME_LAT = str(Decimal(Decimal(vhf_se.group(24)) + Decimal(vhf_se.group(25))/60 + Decimal(vhf_se.group(26))/3600 + Decimal(vhf_se.group(27))/360000).quantize(Decimal('0.00000000'), rounding=ROUND_HALF_UP))
                else:
                    DME_LAT = str(Decimal(0 - Decimal(vhf_se.group(24)) - Decimal(vhf_se.group(25))/60 - Decimal(vhf_se.group(26))/3600 - Decimal(vhf_se.group(27))/360000).quantize(Decimal('0.00000000'), rounding=ROUND_HALF_UP))
                if vhf_se.group(28).strip() == '':
                    DME_LON = None
                elif vhf_se.group(28) == "E":
                    DME_LON = str(Decimal(Decimal(vhf_se.group(29)) + Decimal(vhf_se.group(30))/60 + Decimal(vhf_se.group(31))/3600 + Decimal(vhf_se.group(32))/360000).quantize(Decimal('0.00000000'), rounding=ROUND_HALF_UP))
                else:
                    DME_LON = str(Decimal(0 - Decimal(vhf_se.group(29)) - Decimal(vhf_se.group(30))/60 - Decimal(vhf_se.group(31))/3600 - Decimal(vhf_se.group(32))/360000).quantize(Decimal('0.00000000'), rounding=ROUND_HALF_UP))
                if vhf_se.group(33).strip() == '':
                    STATION_DECLINATION = ""
                else:
                    STATION_DECLINATION = vhf_se.group(33).strip(string.digits) + str(float(vhf_se.group(33).strip('EW'))/10)
                if vhf_se.group(34).strip() == '':
                    DME_ELEV = None
                else:
                    DME_ELEV = vhf_se.group(34).lstrip('0')
                if vhf_se.group(36).strip() == '':
                    ILS_DME_BIAS = None
                else:
                    ILS_DME_BIAS = float(vhf_se.group(36))/10
                VOR_NAME = vhf_se.group(39).replace('VOR/DME','').rstrip().replace('(','').replace(')','')
                vhf_jh.append((SECT_SUBS,VOR_IDENT,VHF_ICAO_CODE,VOR_FREQ,NAVAID_CLASS,VOR_LAT,VOR_LON,DME_LAT,DME_LON,STATION_DECLINATION,DME_ELEV,ILS_DME_BIAS,VOR_NAME,''))
            if db_se:
                SECT_SUBS = db_se.group(3) + db_se.group(4)
                NDB_IDENT = db_se.group(7).strip()
                NDB_ICAO_CODE = db_se.group(8)
                NDB_FREQ = float(db_se.group(10))/10
                NDB_CLASS = db_se.group(11)
                if db_se.group(12) == "N":
                    NDB_LAT = str(Decimal(Decimal(db_se.group(13)) + Decimal(db_se.group(14))/60 + Decimal(db_se.group(15))/3600 + Decimal(db_se.group(16))/360000).quantize(Decimal('0.00000000'), rounding=ROUND_HALF_UP))
                else:
                    NDB_LAT = str(Decimal(0 - Decimal(db_se.group(13)) - Decimal(db_se.group(14))/60 - Decimal(db_se.group(15))/3600 - Decimal(db_se.group(16))/360000).quantize(Decimal('0.00000000'), rounding=ROUND_HALF_UP))
                if db_se.group(17) == "E":
                    NDB_LON = str(Decimal(Decimal(db_se.group(18)) + Decimal(db_se.group(19))/60 + Decimal(db_se.group(20))/3600 + Decimal(db_se.group(21))/360000).quantize(Decimal('0.00000000'), rounding=ROUND_HALF_UP))
                else:
                    NDB_LON = str(Decimal(0 - Decimal(db_se.group(18)) - Decimal(db_se.group(19))/60 - Decimal(db_se.group(20))/3600 - Decimal(db_se.group(21))/360000).quantize(Decimal('0.00000000'), rounding=ROUND_HALF_UP))
                NDB_NAME = db_se.group(25).replace('NDB','').rstrip().replace('(','').replace(')','')
                ndb_jh.append((SECT_SUBS,NDB_IDENT,NDB_ICAO_CODE,NDB_FREQ,NDB_CLASS,NDB_LAT,NDB_LON,NDB_NAME,''))
            if ea_se:
                SECT_SUBS = ea_se.group(3) + ea_se.group(4)
                REGION_CODE = ea_se.group(5)
                WAYPOINT_IDENT = ea_se.group(8).strip()
                WAYPOINT_ICAO_CODE = ea_se.group(9)
                WAYPOINT_TYPE = ea_se.group(11)
                if ea_se.group(13) == "N":
                    WAYPOINT_LAT = str(Decimal(Decimal(ea_se.group(14)) + Decimal(ea_se.group(15))/60 + Decimal(ea_se.group(16))/3600 + Decimal(ea_se.group(17))/360000).quantize(Decimal('0.00000000'), rounding=ROUND_HALF_UP))
                else:
                    WAYPOINT_LAT = str(Decimal(0 - Decimal(ea_se.group(14)) - Decimal(ea_se.group(15))/60 - Decimal(ea_se.group(16))/3600 - Decimal(ea_se.group(17))/360000).quantize(Decimal('0.00000000'), rounding=ROUND_HALF_UP))
                if ea_se.group(18) == "E":
                    WAYPOINT_LON = str(Decimal(Decimal(ea_se.group(19)) + Decimal(ea_se.group(20))/60 + Decimal(ea_se.group(21))/3600 + Decimal(ea_se.group(22))/360000).quantize(Decimal('0.00000000'), rounding=ROUND_HALF_UP))
                else:
                    WAYPOINT_LON = str(Decimal(0 - Decimal(ea_se.group(19)) - Decimal(ea_se.group(20))/60 - Decimal(ea_se.group(21))/3600 - Decimal(ea_se.group(22))/360000).quantize(Decimal('0.00000000'), rounding=ROUND_HALF_UP))
                WAYPOINT_NAME_DESCR = ea_se.group(26).rstrip().replace('(','').replace(')','')
                wpt_jh.append((SECT_SUBS,WAYPOINT_IDENT,WAYPOINT_ICAO_CODE,WAYPOINT_TYPE,WAYPOINT_LAT,WAYPOINT_LON,REGION_CODE,WAYPOINT_NAME_DESCR,''))
            if hold_se:
                SECT_SUBS = hold_se.group(3) + hold_se.group(4)
                DUPLICATE_IDENT = hold_se.group(7)
                FIX_IDENT = hold_se.group(8).strip()
                FIX_ICAO_CODE = hold_se.group(9)
                FIX_SECT_SUBS = hold_se.group(10) + hold_se.group(11)
                if hold_se.group(13).lstrip(string.digits) == "T":
                    INBOUND_HOLDING_COURSE = hold_se.group(13).rstrip('T')
                else:
                    INBOUND_HOLDING_COURSE = float(hold_se.group(13))/10
                TURN_DIR = hold_se.group(14)
                if hold_se.group(15).strip() == '':
                    LEG_LENGTH = None
                else:
                    LEG_LENGTH = float(hold_se.group(15))/10
                if hold_se.group(16).strip() == '':
                    LEG_TIME = None
                else:
                    LEG_TIME = float(hold_se.group(16))/10
                if hold_se.group(19).strip() == '':
                    HOLDING_SPEED = None
                else:
                    HOLDING_SPEED = hold_se.group(19)
                hold_jh.append((SECT_SUBS,FIX_IDENT,FIX_ICAO_CODE,FIX_SECT_SUBS,DUPLICATE_IDENT,INBOUND_HOLDING_COURSE,TURN_DIR,LEG_LENGTH,LEG_TIME,HOLDING_SPEED,''))
            if arpt_se:
                SECT_SUBS = arpt_se.group(3) + arpt_se.group(6)
                ARPT_IDENT = arpt_se.group(4)
                ARPT_ICAO_CODE = arpt_se.group(5)
                if arpt_se.group(9).strip() == '':
                    SPEED_LIMIT_ALT = None
                else:
                    SPEED_LIMIT_ALT = arpt_se.group(9).lstrip('0')
                if arpt_se.group(13) == "N":
                    ARPT_LAT = str(Decimal(Decimal(arpt_se.group(14)) + Decimal(arpt_se.group(15))/60 + Decimal(arpt_se.group(16))/3600 + Decimal(arpt_se.group(17))/360000).quantize(Decimal('0.00000000'), rounding=ROUND_HALF_UP))
                else:
                    ARPT_LAT = str(Decimal(0 - Decimal(arpt_se.group(14)) - Decimal(arpt_se.group(15))/60 - Decimal(arpt_se.group(16))/3600 - Decimal(arpt_se.group(17))/360000).quantize(Decimal('0.00000000'), rounding=ROUND_HALF_UP))
                if arpt_se.group(18) == "E":
                    ARPT_LON = str(Decimal(Decimal(arpt_se.group(19)) + Decimal(arpt_se.group(20))/60 + Decimal(arpt_se.group(21))/3600 + Decimal(arpt_se.group(22))/360000).quantize(Decimal('0.00000000'), rounding=ROUND_HALF_UP))
                else:
                    ARPT_LON = str(Decimal(0 - Decimal(arpt_se.group(19)) - Decimal(arpt_se.group(20))/60 - Decimal(arpt_se.group(21))/3600 - Decimal(arpt_se.group(22))/360000).quantize(Decimal('0.00000000'), rounding=ROUND_HALF_UP))
                ARPT_ELEV = arpt_se.group(25).lstrip('0')
                if arpt_se.group(26).strip() == '':
                    SPEED_LIMIT = None
                else:
                    SPEED_LIMIT = arpt_se.group(26)
                if arpt_se.group(29).strip() == '':
                    TRANSITIONS_ALT = None
                else:
                    TRANSITIONS_ALT = arpt_se.group(29).lstrip('0')
                ARPT_NAME = arpt_se.group(36).rstrip().replace('(','').replace(')','')
                arpt_jh.append((SECT_SUBS,ARPT_IDENT,ARPT_ICAO_CODE,ARPT_LAT,ARPT_LON,ARPT_ELEV,SPEED_LIMIT,SPEED_LIMIT_ALT,TRANSITIONS_ALT,ARPT_NAME,''))
            if pc_se:
                SECT_SUBS = pc_se.group(3) + pc_se.group(7)
                REGION_CODE = pc_se.group(5)
                WAYPOINT_IDENT = pc_se.group(8).strip()
                WAYPOINT_ICAO_CODE = pc_se.group(9)
                WAYPOINT_TYPE = pc_se.group(11)
                if pc_se.group(13) == "N":
                    WAYPOINT_LAT = str(Decimal(Decimal(pc_se.group(14)) + Decimal(pc_se.group(15))/60 + Decimal(pc_se.group(16))/3600 + Decimal(pc_se.group(17))/360000).quantize(Decimal('0.00000000'), rounding=ROUND_HALF_UP))
                else:
                    WAYPOINT_LAT = str(Decimal(0 - Decimal(pc_se.group(14)) - Decimal(pc_se.group(15))/60 - Decimal(pc_se.group(16))/3600 - Decimal(pc_se.group(17))/360000).quantize(Decimal('0.00000000'), rounding=ROUND_HALF_UP))
                if pc_se.group(18) == "E":
                    WAYPOINT_LON = str(Decimal(Decimal(pc_se.group(19)) + Decimal(pc_se.group(20))/60 + Decimal(pc_se.group(21))/3600 + Decimal(pc_se.group(22))/360000).quantize(Decimal('0.00000000'), rounding=ROUND_HALF_UP))
                else:
                    WAYPOINT_LON = str(Decimal(0 - Decimal(pc_se.group(19)) - Decimal(pc_se.group(20))/60 - Decimal(pc_se.group(21))/3600 - Decimal(pc_se.group(22))/360000).quantize(Decimal('0.00000000'), rounding=ROUND_HALF_UP))
                WAYPOINT_NAME_DESCR = pc_se.group(26).rstrip().replace('(','').replace(')','')
                wpt_jh.append((SECT_SUBS,WAYPOINT_IDENT,WAYPOINT_ICAO_CODE,WAYPOINT_TYPE,WAYPOINT_LAT,WAYPOINT_LON,REGION_CODE,WAYPOINT_NAME_DESCR,''))
            if proc_se:
                SECT_SUBS = proc_se.group(3) + proc_se.group(6)
                ARPT_IDENT = proc_se.group(4)
                ARPT_ICAO_CODE = proc_se.group(5)
                PROC_IDENT = proc_se.group(7).strip()
                ROUTE_TYPE = proc_se.group(8)
                TRANSITION_IDENT = proc_se.group(9).strip()
                SEQ_NR = proc_se.group(10)
                FIX_IDENT = proc_se.group(11).strip()
                FIX_ICAO_CODE = proc_se.group(12)
                FIX_SECT_SUBS = proc_se.group(13) + proc_se.group(14)
                CONT_NR = proc_se.group(15)
                WAYPOINT_DESCR_CODE = proc_se.group(16)
                TURN_DIR = proc_se.group(17)
                if proc_se.group(18).strip() == '':
                    RNP = None
                else:
                    RNP = float(proc_se.group(18)[0:2])/(10 ** float(proc_se.group(18)[2:]))
                PATH_AND_TERMINATION = proc_se.group(19)
                TURN_DIR_VALID = proc_se.group(20)
                RECOMMENDED_NAVAID = proc_se.group(21)
                RECOMMENDED_NAVAID_ICAO_CODE = proc_se.group(22)
                if proc_se.group(23).strip() == '':
                    ARC_RADIUS = None
                else:
                    ARC_RADIUS = round(float(proc_se.group(23))/1000, 4)
                if proc_se.group(24).strip() == '':
                    THETA = None
                else:
                    THETA = proc_se.group(24)
                if proc_se.group(25).strip() == '':
                    RHO = None
                else:
                    RHO = proc_se.group(25)
                if proc_se.group(26).strip() == '':
                    MAG_COURSE = None
                elif proc_se.group(26).lstrip(string.digits) == "T":
                    MAG_COURSE = float(Decimal(proc_se.group(26).rstrip('T')))
                else:
                    MAG_COURSE = float(Decimal(proc_se.group(26)[0:3] + '.' + proc_se.group(26)[3:]).quantize(Decimal('0'), rounding=ROUND_HALF_UP))
                if proc_se.group(27).strip() == '':
                    ROUTE_DISTANCE_HOLDING_DISTANCE_OR_TIME = None
                elif proc_se.group(27).rstrip(string.digits) == "T":
                    ROUTE_DISTANCE_HOLDING_DISTANCE_OR_TIME = 'T' + str(float(proc_se.group(27).lstrip('T'))/10)
                else:
                    ROUTE_DISTANCE_HOLDING_DISTANCE_OR_TIME = float(proc_se.group(27))/10
                RECOMMENDED_NAVAID_SECT_SUBS = proc_se.group(28) + proc_se.group(29).strip()
                if proc_se.group(32).strip() == '':
                    ALT_DESCR = None
                elif proc_se.group(30).strip() == '':
                    ALT_DESCR = '@'
                else:
                    ALT_DESCR = proc_se.group(30)
                if proc_se.group(32).strip() == '':
                    ALT_1 = None
                elif proc_se.group(32).rstrip(string.digits) == "FL":
                    ALT_1 = proc_se.group(32).lstrip(string.ascii_uppercase) + "00"
                else:
                    ALT_1 = proc_se.group(32).lstrip('0')
                if proc_se.group(33).strip() == '':
                    ALT_2 = None
                elif proc_se.group(33).rstrip(string.digits) == "FL":
                    ALT_2 = proc_se.group(33).lstrip(string.ascii_uppercase) + "00"
                else:
                    ALT_2 = proc_se.group(33).lstrip('0')
                if proc_se.group(34).strip() == '':
                    TRANSITION_ALT = None
                else:
                    TRANSITION_ALT = proc_se.group(34).lstrip('0')
                SPEED_LIMIT = proc_se.group(35)
                if proc_se.group(36).strip() == '':
                    VERTICAL_ANGLE = None
                else:
                    VERTICAL_ANGLE = round(float(proc_se.group(36))/100, 2)
                CENTER_FIX_OR_TAA_PROCEDURE_TURN_IND = proc_se.group(37)
                MULTIPLE_CODE_OR_TAA_SECTOR_IDENT = proc_se.group(38).strip()
                MULTIPLE_CODE_OR_TAA_SECTOR_SECT_SUBS = proc_se.group(40) + proc_se.group(41).strip()
                SPEED_LIMIT_DESCR = proc_se.group(43)
                APCH_ROUTE_QUALIFIER_1 = proc_se.group(44)
                APCH_ROUTE_QUALIFIER_2 = proc_se.group(45)
                proc_jh.append((SECT_SUBS,ARPT_IDENT,ARPT_ICAO_CODE,PROC_IDENT,ROUTE_TYPE,TRANSITION_IDENT,SEQ_NR,FIX_IDENT,FIX_ICAO_CODE,FIX_SECT_SUBS,WAYPOINT_DESCR_CODE,TURN_DIR,TURN_DIR_VALID,RNP,PATH_AND_TERMINATION,RECOMMENDED_NAVAID,RECOMMENDED_NAVAID_ICAO_CODE,RECOMMENDED_NAVAID_SECT_SUBS,ARC_RADIUS,THETA,RHO,MAG_COURSE,ROUTE_DISTANCE_HOLDING_DISTANCE_OR_TIME,ALT_DESCR,ALT_1,ALT_2,SPEED_LIMIT,SPEED_LIMIT_DESCR,VERTICAL_ANGLE,CENTER_FIX_OR_TAA_PROCEDURE_TURN_IND,TRANSITION_ALT,MULTIPLE_CODE_OR_TAA_SECTOR_IDENT,MULTIPLE_CODE_OR_TAA_SECTOR_SECT_SUBS,CONT_NR,APCH_ROUTE_QUALIFIER_1,APCH_ROUTE_QUALIFIER_2,''))
            if rwy_se:
                SECT_SUBS = rwy_se.group(3) + rwy_se.group(6)
                ARPT_IDENT = rwy_se.group(4)
                ARPT_ICAO_CODE = rwy_se.group(5)
                RUNWAY_IDENT = rwy_se.group(7).strip()
                RUNWAY_LENGTH = rwy_se.group(9)
                if rwy_se.group(10).lstrip(string.digits) == "T":
                    RUNWAY_MAG_BEARING = rwy_se.group(10).rstrip('T')
                else:
                    RUNWAY_MAG_BEARING = float(rwy_se.group(10))/10
                if rwy_se.group(11) == "N":
                    RUNWAY_LAT = str(Decimal(Decimal(rwy_se.group(12)) + Decimal(rwy_se.group(13))/60 + Decimal(rwy_se.group(14))/3600 + Decimal(rwy_se.group(15))/360000).quantize(Decimal('0.00000000'), rounding=ROUND_HALF_UP))
                else:
                    RUNWAY_LAT = str(Decimal(0 - Decimal(rwy_se.group(12)) - Decimal(rwy_se.group(13))/60 - Decimal(rwy_se.group(14))/3600 - Decimal(rwy_se.group(15))/360000).quantize(Decimal('0.00000000'), rounding=ROUND_HALF_UP))
                if rwy_se.group(16) == "E":
                    RUNWAY_LON = str(Decimal(Decimal(rwy_se.group(17)) + Decimal(rwy_se.group(18))/60 + Decimal(rwy_se.group(19))/3600 + Decimal(rwy_se.group(20))/360000).quantize(Decimal('0.00000000'), rounding=ROUND_HALF_UP))
                else:
                    RUNWAY_LON = str(Decimal(0 - Decimal(rwy_se.group(17)) - Decimal(rwy_se.group(18))/60 - Decimal(rwy_se.group(19))/3600 - Decimal(rwy_se.group(20))/360000).quantize(Decimal('0.00000000'), rounding=ROUND_HALF_UP))
                LANDING_THLD_ELEV = rwy_se.group(24).lstrip('0')
                if rwy_se.group(25).strip() == '':
                    DISPLACED_THLD_DISTANCE = "0"
                else:
                    DISPLACED_THLD_DISTANCE = rwy_se.group(25).lstrip('0')
                RUNWAY_WIDTH = rwy_se.group(27)
                LOCALIZER_MLS_GLS_REF_PATH_IDENT = rwy_se.group(29).strip()
                LOCALIZER_MLS_GLS_CATEGORY_CLASS = rwy_se.group(30)
                rwy_jh.append((SECT_SUBS,ARPT_IDENT,ARPT_ICAO_CODE,RUNWAY_IDENT,RUNWAY_LENGTH,DISPLACED_THLD_DISTANCE,RUNWAY_WIDTH,RUNWAY_LAT,RUNWAY_LON,LANDING_THLD_ELEV,RUNWAY_MAG_BEARING,LOCALIZER_MLS_GLS_REF_PATH_IDENT,LOCALIZER_MLS_GLS_CATEGORY_CLASS,''))
            if loc_se:
                SECT_SUBS = loc_se.group(3) + loc_se.group(6)
                ARPT_IDENT = loc_se.group(4)
                ARPT_ICAO_CODE = loc_se.group(5)
                LOCALIZER_IDENT = loc_se.group(7).strip()
                ILS_CATEGORY = loc_se.group(8)
                LOCALIZER_FREQ = float(loc_se.group(10))/100
                RUNWAY_IDENT = loc_se.group(11).strip()
                if loc_se.group(12).strip() == '':
                    LOCALIZER_LAT = None
                elif loc_se.group(12) == "N":
                    LOCALIZER_LAT = str(Decimal(Decimal(loc_se.group(13)) + Decimal(loc_se.group(14))/60 + Decimal(loc_se.group(15))/3600 + Decimal(loc_se.group(16))/360000).quantize(Decimal('0.00000000'), rounding=ROUND_HALF_UP))
                else:
                    LOCALIZER_LAT = str(Decimal(0 - Decimal(loc_se.group(13)) - Decimal(loc_se.group(14))/60 - Decimal(loc_se.group(15))/3600 - Decimal(loc_se.group(16))/360000).quantize(Decimal('0.00000000'), rounding=ROUND_HALF_UP))
                if loc_se.group(17).strip() == '':
                    LOCALIZER_LON = None
                elif loc_se.group(17) == "E":
                    LOCALIZER_LON = str(Decimal(Decimal(loc_se.group(18)) + Decimal(loc_se.group(19))/60 + Decimal(loc_se.group(20))/3600 + Decimal(loc_se.group(21))/360000).quantize(Decimal('0.00000000'), rounding=ROUND_HALF_UP))
                else:
                    LOCALIZER_LON = str(Decimal(0 - Decimal(loc_se.group(18)) - Decimal(loc_se.group(19))/60 - Decimal(loc_se.group(20))/3600 - Decimal(loc_se.group(21))/360000).quantize(Decimal('0.00000000'), rounding=ROUND_HALF_UP))
                if loc_se.group(22).lstrip(string.digits) == "T":
                    LOCALIZER_BEARING = str(Decimal(loc_se.group(22).rstrip('T')))
                else:
                    LOCALIZER_BEARING = str(Decimal(loc_se.group(22)[0:3] + '.' + loc_se.group(22)[3:]).quantize(Decimal('0'), rounding=ROUND_HALF_UP))
                if loc_se.group(23).strip() == '':
                    GLIDE_SLOPE_LAT = None
                elif loc_se.group(23) == "N":
                    GLIDE_SLOPE_LAT = str(Decimal(Decimal(loc_se.group(24)) + Decimal(loc_se.group(25))/60 + Decimal(loc_se.group(26))/3600 + Decimal(loc_se.group(27))/360000).quantize(Decimal('0.00000000'), rounding=ROUND_HALF_UP))
                else:
                    GLIDE_SLOPE_LAT = str(Decimal(0 - Decimal(loc_se.group(24)) - Decimal(loc_se.group(25))/60 - Decimal(loc_se.group(26))/3600 - Decimal(loc_se.group(27))/360000).quantize(Decimal('0.00000000'), rounding=ROUND_HALF_UP))
                if loc_se.group(28).strip() == '':
                    GLIDE_SLOPE_LON = None
                elif loc_se.group(28) == "E":
                    GLIDE_SLOPE_LON = str(Decimal(Decimal(loc_se.group(29)) + Decimal(loc_se.group(30))/60 + Decimal(loc_se.group(31))/3600 + Decimal(loc_se.group(32))/360000).quantize(Decimal('0.00000000'), rounding=ROUND_HALF_UP))
                else:
                    GLIDE_SLOPE_LON = str(Decimal(0 - Decimal(loc_se.group(29)) - Decimal(loc_se.group(30))/60 - Decimal(loc_se.group(31))/3600 - Decimal(loc_se.group(32))/360000).quantize(Decimal('0.00000000'), rounding=ROUND_HALF_UP))
                if loc_se.group(37).strip() == '':
                    GLIDE_SLOPE_ANGLE = None
                else:
                    GLIDE_SLOPE_ANGLE = str(Decimal(loc_se.group(37)[0:1] + '.' + loc_se.group(37)[1:]).quantize(Decimal('0.00'), rounding=ROUND_HALF_UP))
                STATION_DECLINATION = loc_se.group(38).strip(string.digits) + str(float(loc_se.group(38).strip('EW'))/10)
                if loc_se.group(40).strip() == '':
                    GLIDE_SLOPE_ELEV = None
                else:
                    GLIDE_SLOPE_ELEV = loc_se.group(40)
                ils_jh.append((SECT_SUBS,ARPT_IDENT,ARPT_ICAO_CODE,LOCALIZER_IDENT,ILS_CATEGORY,LOCALIZER_FREQ,RUNWAY_IDENT,LOCALIZER_LAT,LOCALIZER_LON,LOCALIZER_BEARING,GLIDE_SLOPE_LAT,GLIDE_SLOPE_LON,GLIDE_SLOPE_ANGLE,GLIDE_SLOPE_ELEV,STATION_DECLINATION,''))
            if mkr_se:
                SECT_SUBS = mkr_se.group(3) + mkr_se.group(6)
                ARPT_IDENT = mkr_se.group(4)
                ARPT_ICAO_CODE = mkr_se.group(5)
                MARKER_IDENT = mkr_se.group(7).strip()
                MARKER_TYPE = mkr_se.group(8)
                RUNWAY_IDENT = mkr_se.group(11).strip()
                if mkr_se.group(12).strip() == '':
                    MARKER_LAT = None
                elif mkr_se.group(12) == "N":
                    MARKER_LAT = str(Decimal(Decimal(mkr_se.group(13)) + Decimal(mkr_se.group(14))/60 + Decimal(mkr_se.group(15))/3600 + Decimal(mkr_se.group(16))/360000).quantize(Decimal('0.00000000'), rounding=ROUND_HALF_UP))
                else:
                    MARKER_LAT = str(Decimal(0 - Decimal(mkr_se.group(13)) - Decimal(mkr_se.group(14))/60 - Decimal(mkr_se.group(15))/3600 - Decimal(mkr_se.group(16))/360000).quantize(Decimal('0.00000000'), rounding=ROUND_HALF_UP))
                if mkr_se.group(17).strip() == '':
                    MARKER_LON = None
                elif mkr_se.group(17) == "E":
                    MARKER_LON = str(Decimal(Decimal(mkr_se.group(18)) + Decimal(mkr_se.group(19))/60 + Decimal(mkr_se.group(20))/3600 + Decimal(mkr_se.group(21))/360000).quantize(Decimal('0.00000000'), rounding=ROUND_HALF_UP))
                else:
                    MARKER_LON = str(Decimal(0 - Decimal(mkr_se.group(18)) - Decimal(mkr_se.group(19))/60 - Decimal(mkr_se.group(20))/3600 - Decimal(mkr_se.group(21))/360000).quantize(Decimal('0.00000000'), rounding=ROUND_HALF_UP))
                mkr_jh.append((SECT_SUBS,ARPT_IDENT,ARPT_ICAO_CODE,MARKER_IDENT,MARKER_TYPE,RUNWAY_IDENT,MARKER_LAT,MARKER_LON,''))
            if pn_se:
                SECT_SUBS = pn_se.group(3) + pn_se.group(4)
                NDB_IDENT = pn_se.group(7).strip()
                NDB_ICAO_CODE = pn_se.group(8)
                NDB_FREQ = float(pn_se.group(10))/10
                NDB_CLASS = pn_se.group(11)
                if pn_se.group(12) == "N":
                    NDB_LAT = str(Decimal(Decimal(pn_se.group(13)) + Decimal(pn_se.group(14))/60 + Decimal(pn_se.group(15))/3600 + Decimal(pn_se.group(16))/360000).quantize(Decimal('0.00000000'), rounding=ROUND_HALF_UP))
                else:
                    NDB_LAT = str(Decimal(0 - Decimal(pn_se.group(13)) - Decimal(pn_se.group(14))/60 - Decimal(pn_se.group(15))/3600 - Decimal(pn_se.group(16))/360000).quantize(Decimal('0.00000000'), rounding=ROUND_HALF_UP))
                if pn_se.group(17) == "E":
                    NDB_LON = str(Decimal(Decimal(pn_se.group(18)) + Decimal(pn_se.group(19))/60 + Decimal(pn_se.group(20))/3600 + Decimal(pn_se.group(21))/360000).quantize(Decimal('0.00000000'), rounding=ROUND_HALF_UP))
                else:
                    NDB_LON = str(Decimal(0 - Decimal(pn_se.group(18)) - Decimal(pn_se.group(19))/60 - Decimal(pn_se.group(20))/3600 - Decimal(pn_se.group(21))/360000).quantize(Decimal('0.00000000'), rounding=ROUND_HALF_UP))
                NDB_NAME = pn_se.group(25).replace('NDB','').rstrip().replace('(','').replace(')','')
                ndb_jh.append((SECT_SUBS,NDB_IDENT,NDB_ICAO_CODE,NDB_FREQ,NDB_CLASS,NDB_LAT,NDB_LON,NDB_NAME,''))
            if gls_se:
                SECT_SUBS = gls_se.group(3) + gls_se.group(6)
                ARPT_IDENT = gls_se.group(4)
                ARPT_ICAO_CODE = gls_se.group(5)
                GLS_PATH_IDENT = gls_se.group(7).strip()
                GLS_CATEGORY = gls_se.group(8)
                GLS_CHANNEL = float(gls_se.group(10))/100
                RUNWAY_IDENT = gls_se.group(11).strip()
                if gls_se.group(12).lstrip(string.digits) == "T":
                    GLS_APCH_BEARING = str(Decimal(gls_se.group(12).rstrip('T')))
                else:
                    GLS_APCH_BEARING = str(Decimal(gls_se.group(12)[0:3] + '.' + gls_se.group(12)[3:]).quantize(Decimal('0.0'), rounding=ROUND_HALF_UP))
                if gls_se.group(13).strip() == '':
                    GLS_LAT = None
                elif gls_se.group(13) == "N":
                    GLS_LAT = str(Decimal(Decimal(gls_se.group(14)) + Decimal(gls_se.group(15))/60 + Decimal(gls_se.group(16))/3600 + Decimal(gls_se.group(17))/360000).quantize(Decimal('0.00000000'), rounding=ROUND_HALF_UP))
                else:
                    GLS_LAT = str(Decimal(0 - Decimal(gls_se.group(14)) - Decimal(gls_se.group(15))/60 - Decimal(gls_se.group(16))/3600 - Decimal(gls_se.group(17))/360000).quantize(Decimal('0.00000000'), rounding=ROUND_HALF_UP))
                if gls_se.group(18).strip() == '':
                    GLS_LON = None
                elif gls_se.group(18) == "E":
                    GLS_LON = str(Decimal(Decimal(gls_se.group(19)) + Decimal(gls_se.group(20))/60 + Decimal(gls_se.group(21))/3600 + Decimal(gls_se.group(22))/360000).quantize(Decimal('0.00000000'), rounding=ROUND_HALF_UP))
                else:
                    GLS_LON = str(Decimal(0 - Decimal(gls_se.group(19)) - Decimal(gls_se.group(20))/60 - Decimal(gls_se.group(21))/3600 - Decimal(gls_se.group(22))/360000).quantize(Decimal('0.00000000'), rounding=ROUND_HALF_UP))
                GLS_IDENT = gls_se.group(23).strip()
                if gls_se.group(26).strip() == '':
                    GLS_SLOPE_ANGLE = None
                else:
                    GLS_SLOPE_ANGLE = str(Decimal(gls_se.group(26)[0:1] + '.' + gls_se.group(26)[1:]).quantize(Decimal('0.00'), rounding=ROUND_HALF_UP))
                MAG_VAR = gls_se.group(27).strip(string.digits) + str(float(gls_se.group(27).strip('EW'))/10)
                if gls_se.group(28).strip() == '':
                    GLS_ELEV = None
                else:
                    GLS_ELEV = int(gls_se.group(28))
                gls_jh.append((SECT_SUBS,ARPT_IDENT,ARPT_ICAO_CODE,GLS_PATH_IDENT,GLS_IDENT,GLS_CATEGORY,GLS_CHANNEL,RUNWAY_IDENT,GLS_LAT,GLS_LON,GLS_APCH_BEARING,GLS_SLOPE_ANGLE,GLS_ELEV,'',MAG_VAR,''))
        # Convert list to Pandas DataFrame
        mora_jh_tb = pandas.DataFrame(mora_jh, columns=['SECT_SUBS','MORA_LAT','MORA_LON','MORA_00','MORA_01','MORA_02','MORA_03','MORA_04','MORA_05','MORA_06','MORA_07','MORA_08','MORA_09','MORA_10','MORA_11','MORA_12','MORA_13','MORA_14','MORA_15','MORA_16','MORA_17','MORA_18','MORA_19','MORA_20','MORA_21','MORA_22','MORA_23','MORA_24','MORA_25','MORA_26','MORA_27','MORA_28','MORA_29','END'])
        mora_jh_tb = mora_jh_tb.sort_values(['MORA_LAT','MORA_LON'])
        vhf_jh_tb = pandas.DataFrame(vhf_jh, columns=['SECT_SUBS','VOR_IDENT','VHF_ICAO_CODE','VOR_FREQ','NAVAID_CLASS','VOR_LAT','VOR_LON','DME_LAT','DME_LON','STATION_DECLINATION','DME_ELEV','ILS_DME_BIAS','VOR_NAME','END'])
        vhf_jh_tb = vhf_jh_tb.sort_values(['VOR_IDENT','VHF_ICAO_CODE'])
        ndb_jh_tb = pandas.DataFrame(ndb_jh, columns=['SECT_SUBS','NDB_IDENT','NDB_ICAO_CODE','NDB_FREQ','NDB_CLASS','NDB_LAT','NDB_LON','NDB_NAME','END'])
        ndb_jh_tb = ndb_jh_tb.sort_values(['NDB_IDENT','NDB_ICAO_CODE'])
        wpt_jh_tb = pandas.DataFrame(wpt_jh, columns=['SECT_SUBS','WAYPOINT_IDENT','WAYPOINT_ICAO_CODE','WAYPOINT_TYPE','WAYPOINT_LAT','WAYPOINT_LON','REGION_CODE','WAYPOINT_NAME_DESCR','END'])
        wpt_jh_tb = wpt_jh_tb.sort_values(['SECT_SUBS','WAYPOINT_IDENT','WAYPOINT_ICAO_CODE'])
        hold_jh_tb = pandas.DataFrame(hold_jh, columns=['SECT_SUBS','FIX_IDENT','FIX_ICAO_CODE','FIX_SECT_SUBS','DUPLICATE_IDENT','INBOUND_HOLDING_COURSE','TURN_DIR','LEG_LENGTH','LEG_TIME','HOLDING_SPEED','END'])
        hold_jh_tb = hold_jh_tb.sort_values(['FIX_IDENT','FIX_ICAO_CODE','DUPLICATE_IDENT'])
        arpt_jh_tb = pandas.DataFrame(arpt_jh, columns=['SECT_SUBS','ARPT_IDENT','ARPT_ICAO_CODE','ARPT_LAT','ARPT_LON','ARPT_ELEV','SPEED_LIMIT','SPEED_LIMIT_ALT','TRANSITIONS_ALT','ARPT_NAME','END'])
        arpt_jh_tb = arpt_jh_tb.sort_values(['ARPT_IDENT','ARPT_ICAO_CODE'])
        proc_jh_tb = pandas.DataFrame(proc_jh, columns=['SECT_SUBS','ARPT_IDENT','ARPT_ICAO_CODE','PROC_IDENT','ROUTE_TYPE','TRANSITION_IDENT','SEQ_NR','FIX_IDENT','FIX_ICAO_CODE','FIX_SECT_SUBS','WAYPOINT_DESCR_CODE','TURN_DIR','TURN_DIR_VALID','RNP','PATH_AND_TERMINATION','RECOMMENDED_NAVAID','RECOMMENDED_NAVAID_ICAO_CODE','RECOMMENDED_NAVAID_SECT_SUBS','ARC_RADIUS','THETA','RHO','MAG_COURSE','ROUTE_DISTANCE_HOLDING_DISTANCE_OR_TIME','ALT_DESCR','ALT_1','ALT_2','SPEED_LIMIT','SPEED_LIMIT_DESCR','VERTICAL_ANGLE','CENTER_FIX_OR_TAA_PROCEDURE_TURN_IND','TRANSITION_ALT','MULTIPLE_CODE_OR_TAA_SECTOR_IDENT','MULTIPLE_CODE_OR_TAA_SECTOR_SECT_SUBS','CONT_NR','APCH_ROUTE_QUALIFIER_1','APCH_ROUTE_QUALIFIER_2','END'])
        proc_jh_tb = proc_jh_tb.sort_values(['SECT_SUBS','ARPT_IDENT','PROC_IDENT','ROUTE_TYPE','TRANSITION_IDENT','SEQ_NR'])
        rwy_jh_tb = pandas.DataFrame(rwy_jh, columns=['SECT_SUBS','ARPT_IDENT','ARPT_ICAO_CODE','RUNWAY_IDENT','RUNWAY_LENGTH','DISPLACED_THLD_DISTANCE','RUNWAY_WIDTH','RUNWAY_LAT','RUNWAY_LON','LANDING_THLD_ELEV','RUNWAY_MAG_BEARING','LOCALIZER_MLS_GLS_REF_PATH_IDENT','LOCALIZER_MLS_GLS_CATEGORY_CLASS','END'])
        rwy_jh_tb = rwy_jh_tb.sort_values(['ARPT_IDENT','RUNWAY_IDENT'])
        ils_jh_tb = pandas.DataFrame(ils_jh, columns=['SECT_SUBS','ARPT_IDENT','ARPT_ICAO_CODE','LOCALIZER_IDENT','ILS_CATEGORY','LOCALIZER_FREQ','RUNWAY_IDENT','LOCALIZER_LAT','LOCALIZER_LON','LOCALIZER_BEARING','GLIDE_SLOPE_LAT','GLIDE_SLOPE_LON','GLIDE_SLOPE_ANGLE','GLIDE_SLOPE_ELEV','STATION_DECLINATION','END'])
        ils_jh_tb = ils_jh_tb.sort_values(['ARPT_IDENT','LOCALIZER_IDENT'])
        mkr_jh_tb = pandas.DataFrame(mkr_jh, columns=['SECT_SUBS','ARPT_IDENT','ARPT_ICAO_CODE','MARKER_IDENT','MARKER_TYPE','RUNWAY_IDENT','MARKER_LAT','MARKER_LON','END'])
        mkr_jh_tb = mkr_jh_tb.sort_values(['ARPT_IDENT','RUNWAY_IDENT'])
        gls_jh_tb = pandas.DataFrame(gls_jh, columns=['SECT_SUBS','ARPT_IDENT','ARPT_ICAO_CODE','GLS_PATH_IDENT','GLS_IDENT','GLS_CATEGORY','GLS_CHANNEL','RUNWAY_IDENT','GLS_LAT','GLS_LON','GLS_APCH_BEARING','GLS_SLOPE_ANGLE','GLS_ELEV','UNK1','MAG_VAR','END'])
        gls_jh_tb = gls_jh_tb.sort_values(['ARPT_IDENT','RUNWAY_IDENT','GLS_PATH_IDENT'])
        print(time.strftime("%H:%M:%S", time.localtime()) , 'JeeHell: Data Process Part #1 Finished')
    # Process of JeeHell Part #2
    with open('Input/' + Datafile, 'r', encoding='ISO-8859-1') as ARINC424:
        for line in ARINC424:
            awy_se = re.search(er, line)
            if jh_output_navrecord:
                # AWY of JeeHell
                if awy_se:
                    SECT_SUBS = awy_se.group(3) + awy_se.group(4)
                    ROUTE_IDENT = awy_se.group(5).strip()
                    SEQ_NR = awy_se.group(6)
                    FIX_IDENT = awy_se.group(7).strip()
                    FIX_ICAO_CODE = awy_se.group(8)
                    FIX_SECT_SUBS = awy_se.group(9) + awy_se.group(10)
                    WAYPOINT_DESCR_CODE = awy_se.group(12)
                    if FIX_SECT_SUBS == "EA":
                        FIX_ID = wpt_jh_tb.loc[(wpt_jh_tb['SECT_SUBS'] == FIX_SECT_SUBS) & (wpt_jh_tb['WAYPOINT_IDENT'] == FIX_IDENT) & (wpt_jh_tb['WAYPOINT_ICAO_CODE'] == FIX_ICAO_CODE)].index.tolist()
                        if FIX_ID != []:
                            FIX_LAT = wpt_jh_tb.at[FIX_ID[0], 'WAYPOINT_LAT']
                            FIX_LON = wpt_jh_tb.at[FIX_ID[0], 'WAYPOINT_LON']
                        else:
                            if jh_create_blank != True:
                                FIX_ID = fixes_in_jh_nd_tb.loc[(fixes_in_jh_nd_tb['FIX_SECT_SUBS'] == FIX_SECT_SUBS) & (fixes_in_jh_nd_tb['FIX_IDENT'] == FIX_IDENT) & (fixes_in_jh_nd_tb['FIX_ICAO_CODE'] == FIX_ICAO_CODE)].index.tolist()
                                if FIX_ID != []:
                                    FIX_LAT = fixes_in_jh_nd_tb.at[FIX_ID[0], 'FIX_LAT']
                                    FIX_LON = fixes_in_jh_nd_tb.at[FIX_ID[0], 'FIX_LON']
                            else:
                                print('         JeeHell: Airway ' + awy_se.group(5) + ' Fix ' + awy_se.group(7) + ' Not Found')
                                FIX_LAT = '00.00000000'
                                FIX_LON = '000.00000000'
                    elif FIX_SECT_SUBS == "DB":
                        FIX_ID = ndb_jh_tb.loc[(ndb_jh_tb['SECT_SUBS'] == FIX_SECT_SUBS) & (ndb_jh_tb['NDB_IDENT'] == FIX_IDENT) & (ndb_jh_tb['NDB_ICAO_CODE'] == FIX_ICAO_CODE)].index.tolist()
                        if FIX_ID != []:
                            FIX_LAT = ndb_jh_tb.at[FIX_ID[0], 'NDB_LAT']
                            FIX_LON = ndb_jh_tb.at[FIX_ID[0], 'NDB_LON']
                        else:
                            if jh_create_blank != True:
                                FIX_ID = fixes_in_jh_nd_tb.loc[(fixes_in_jh_nd_tb['FIX_SECT_SUBS'] == FIX_SECT_SUBS) & (fixes_in_jh_nd_tb['FIX_IDENT'] == FIX_IDENT) & (fixes_in_jh_nd_tb['FIX_ICAO_CODE'] == FIX_ICAO_CODE)].index.tolist()
                                if FIX_ID != []:
                                    FIX_LAT = fixes_in_jh_nd_tb.at[FIX_ID[0], 'FIX_LAT']
                                    FIX_LON = fixes_in_jh_nd_tb.at[FIX_ID[0], 'FIX_LON']
                            else:
                                print('         JeeHell: Airway ' + awy_se.group(5) + ' Fix ' + awy_se.group(7) + ' Not Found')
                                FIX_LAT = '00.00000000'
                                FIX_LON = '000.00000000'
                    else:
                        FIX_ID = vhf_jh_tb.loc[(vhf_jh_tb['SECT_SUBS'] == FIX_SECT_SUBS) & (vhf_jh_tb['VOR_IDENT'] == FIX_IDENT) & (vhf_jh_tb['VHF_ICAO_CODE'] == FIX_ICAO_CODE)].index.tolist()
                        if FIX_ID != []:
                            if vhf_jh_tb.at[FIX_ID[0], 'VOR_LAT'] != '':
                                FIX_LAT = vhf_jh_tb.at[FIX_ID[0], 'VOR_LAT']
                                FIX_LON = vhf_jh_tb.at[FIX_ID[0], 'VOR_LON']
                            else:
                                FIX_LAT = vhf_jh_tb.at[FIX_ID[0], 'DME_LAT']
                                FIX_LON = vhf_jh_tb.at[FIX_ID[0], 'DME_LON']
                        else:
                            if jh_create_blank != True:
                                FIX_ID = fixes_in_jh_nd_tb.loc[(fixes_in_jh_nd_tb['FIX_SECT_SUBS'] == FIX_SECT_SUBS) & (fixes_in_jh_nd_tb['FIX_IDENT'] == FIX_IDENT) & (fixes_in_jh_nd_tb['FIX_ICAO_CODE'] == FIX_ICAO_CODE)].index.tolist()
                                if FIX_ID != []:
                                    FIX_LAT = fixes_in_jh_nd_tb.at[FIX_ID[0], 'FIX_LAT']
                                    FIX_LON = fixes_in_jh_nd_tb.at[FIX_ID[0], 'FIX_LON']
                            else:
                                print('         JeeHell: Airway ' + awy_se.group(5) + ' Fix ' + awy_se.group(7) + ' Not Found')
                                FIX_LAT = '00.00000000'
                                FIX_LON = '000.00000000'
                    LEVEL = awy_se.group(15).strip()
                    DIRECTION_RESTRICTION = awy_se.group(16).strip()
                    if awy_se.group(27).strip() == '':
                        MINIMUM_ALT = None
                    elif awy_se.group(27).rstrip(string.digits) == "FL":
                        MINIMUM_ALT = awy_se.group(27).lstrip(string.ascii_uppercase) + "00"
                    else:
                        MINIMUM_ALT = awy_se.group(27)
                    if awy_se.group(28).strip() == '':
                        MINIMUM_ALT_1 = None
                    elif awy_se.group(28).rstrip(string.digits) == "FL":
                        MINIMUM_ALT_1 = awy_se.group(28).lstrip(string.ascii_uppercase) + "00"
                    else:
                        MINIMUM_ALT_1 = awy_se.group(28)
                    if awy_se.group(29).strip() == '':
                        MAXIMUM_ALT = None
                    elif awy_se.group(29).rstrip(string.digits) == "FL":
                        MAXIMUM_ALT = awy_se.group(29).lstrip(string.ascii_uppercase) + "00"
                    else:
                        MAXIMUM_ALT = awy_se.group(29)
                    awy_jh.append((SECT_SUBS,ROUTE_IDENT,SEQ_NR,FIX_IDENT,FIX_ICAO_CODE,FIX_SECT_SUBS,WAYPOINT_DESCR_CODE,'','','',FIX_LAT,FIX_LON,LEVEL,DIRECTION_RESTRICTION,MINIMUM_ALT,MINIMUM_ALT_1,MAXIMUM_ALT,'',''))
        # Convert list to Pandas DataFrame
        awy_jh_tb = pandas.DataFrame(awy_jh, columns=['SECT_SUBS','ROUTE_IDENT','SEQ_NR','FIX_IDENT','FIX_ICAO_CODE','FIX_SECT_SUBS','WAYPOINT_DESCR_CODE','UNK1','UNK2','UNK3','FIX_LAT','FIX_LON','LEVEL','DIRECTION_RESTRICTION','MINIMUM_ALT','MINIMUM_ALT_1','MAXIMUM_ALT','UNK4','END'])
        awy_jh_tb = awy_jh_tb.sort_values(['ROUTE_IDENT','SEQ_NR'])
        print(time.strftime("%H:%M:%S", time.localtime()) , 'JeeHell: Data Process Part #2 Finished')
    # Write Data
    if jh_output_navrecord:
        vhf_jh_tb.to_csv(path_or_buf = Cycle_Ident + '/JeeHell/navrecord.dat', mode = 'a', sep = '|', header = False, index = False)
        ndb_jh_tb.to_csv(path_or_buf = Cycle_Ident + '/JeeHell/navrecord.dat', mode = 'a', sep = '|', header = False, index = False)
        wpt_jh_tb.to_csv(path_or_buf = Cycle_Ident + '/JeeHell/navrecord.dat', mode = 'a', sep = '|', header = False, index = False)
        hold_jh_tb.to_csv(path_or_buf = Cycle_Ident + '/JeeHell/navrecord.dat', mode = 'a', sep = '|', header = False, index = False)
        awy_jh_tb.to_csv(path_or_buf = Cycle_Ident + '/JeeHell/navrecord.dat', mode = 'a', sep = '|', header = False, index = False)
        arpt_jh_tb.to_csv(path_or_buf = Cycle_Ident + '/JeeHell/navrecord.dat', mode = 'a', sep = '|', header = False, index = False)
        rwy_jh_tb.to_csv(path_or_buf = Cycle_Ident + '/JeeHell/navrecord.dat', mode = 'a', sep = '|', header = False, index = False)
        ils_jh_tb.to_csv(path_or_buf = Cycle_Ident + '/JeeHell/navrecord.dat', mode = 'a', sep = '|', header = False, index = False)
        gls_jh_tb.to_csv(path_or_buf = Cycle_Ident + '/JeeHell/navrecord.dat', mode = 'a', sep = '|', header = False, index = False)
        mkr_jh_tb.to_csv(path_or_buf = Cycle_Ident + '/JeeHell/navrecord.dat', mode = 'a', sep = '|', header = False, index = False)
        proc_jh_tb.to_csv(path_or_buf = Cycle_Ident + '/JeeHell/navrecord.dat', mode = 'a', sep = '|', header = False, index = False)
        mora_jh_tb.to_csv(path_or_buf = Cycle_Ident + '/JeeHell/navrecord.dat', mode = 'a', sep = '|', header = False, index = False)
    if jh_output_additional:
        vhf_jh_tb.to_csv(path_or_buf = Cycle_Ident + '/JeeHell/additional.txt', mode = 'a', sep = '|', header = False, index = False)
        ndb_jh_tb.to_csv(path_or_buf = Cycle_Ident + '/JeeHell/additional.txt', mode = 'a', sep = '|', header = False, index = False)
        wpt_jh_tb.to_csv(path_or_buf = Cycle_Ident + '/JeeHell/additional.txt', mode = 'a', sep = '|', header = False, index = False)
        hold_jh_tb.to_csv(path_or_buf = Cycle_Ident + '/JeeHell/additional.txt', mode = 'a', sep = '|', header = False, index = False)
        arpt_jh_tb.to_csv(path_or_buf = Cycle_Ident + '/JeeHell/additional.txt', mode = 'a', sep = '|', header = False, index = False)
        rwy_jh_tb.to_csv(path_or_buf = Cycle_Ident + '/JeeHell/additional.txt', mode = 'a', sep = '|', header = False, index = False)
        ils_jh_tb.to_csv(path_or_buf = Cycle_Ident + '/JeeHell/additional.txt', mode = 'a', sep = '|', header = False, index = False)
        gls_jh_tb.to_csv(path_or_buf = Cycle_Ident + '/JeeHell/additional.txt', mode = 'a', sep = '|', header = False, index = False)
        mkr_jh_tb.to_csv(path_or_buf = Cycle_Ident + '/JeeHell/additional.txt', mode = 'a', sep = '|', header = False, index = False)
        proc_jh_tb.to_csv(path_or_buf = Cycle_Ident + '/JeeHell/additional.txt', mode = 'a', sep = '|', header = False, index = False)
    print(time.strftime("%H:%M:%S", time.localtime()) , 'JeeHell: Data Export Finished')

# MJC (Completed)
if MJC:
    print(time.strftime("%H:%M:%S", time.localtime()) , 'MJC: Start Processing DataFile')
    os.makedirs(Cycle_Ident + '/MJC')
    # Set Variable of MJC
    if 1:
        AirportLookup_mjc = []
        Airports_mjc = []
        AirwayLegs_mjc = []
        AirwayLegs_reversed_mjc = []
        AirwayLookup_mjc = []
        Airways_mjc = []
        ILSes_mjc = []
        NavaidLookup_mjc = []
        Navaids_mjc = []
        Runways_mjc = []
        TerminalLegs_mjc = []
        TerminalLegsEx_mjc = []
        Terminals_mjc = []
        WaypointLookup_mjc = []
        Waypoints_mjc = []
        AirwayIdent = ''
        TerminalName = ''
        if add_tailored:
            AirportID = 15000
            AirwayID = 10000
            AirwayLegsID = 200000
            ILSID = 5000
            NavaidID = 15000
            RunwayID = 40000
            TerminalID = 100000
            TerminalLegsID = 800000
            WaypointID = 300000
        else:
            AirportID = 0
            AirwayID = 0
            AirwayLegsID = 0
            ILSID = 0
            NavaidID = 0
            RunwayID = 0
            TerminalID = 0
            TerminalLegsID = 0
            WaypointID = 0
    # Load MJC's nd.db3
    if mjc_create_blank != True:
        shutil.copy2('Input/MJC/nd.db3', Cycle_Ident + '/MJC/nd.db3')
        rom = sqlite3.connect(Cycle_Ident + '/MJC/nd.db3')
        rom_cur = rom.cursor()
        if 1:
            Airports_in_mjc_nd_tb = pandas.read_sql_query('SELECT * FROM Airports', con = rom)
            Airways_in_mjc_nd_tb = pandas.read_sql_query('SELECT * FROM Airways', con = rom)
            NavaidLookup_in_mjc_nd_tb = pandas.read_sql_query('SELECT * FROM NavaidLookup', con = rom)
            Navaids_in_mjc_nd_tb = pandas.read_sql_query('SELECT * FROM Navaids', con = rom)
            Runways_in_mjc_nd_tb = pandas.read_sql_query('SELECT * FROM Runways', con = rom)
            WaypointLookup_in_mjc_nd_tb = pandas.read_sql_query('SELECT * FROM WaypointLookup', con = rom)
            Waypoints_in_mjc_nd_tb = pandas.read_sql_query('SELECT * FROM Waypoints', con = rom)
            TerminalWpts_in_mjc_nd_tb = pandas.read_sql_query('SELECT TerminalID, WptID, WptLat, WptLon FROM TerminalLegs', con = rom)
            Terminals_in_mjc_nd_tb = pandas.read_sql_query('SELECT * FROM Terminals', con = rom)
            ILSes_in_mjc_nd_tb = pandas.read_sql_query('SELECT * FROM ILSes', con = rom)
        rom_cur.close()
        rom.close()
        if 1:
            ILSes_in_mjc_nd_tb['AirportID'] = ILSes_in_mjc_nd_tb['RunwayID'].apply(lambda RunwayID : Runways_in_mjc_nd_tb.loc[Runways_in_mjc_nd_tb.loc[(Runways_in_mjc_nd_tb['ID'] == RunwayID)].index.tolist() , 'AirportID'].values[0])
            TerminalWpts_in_mjc_nd_tb.drop_duplicates(subset=['WptID'], keep='first', inplace=True)
            TerminalWpts_in_mjc_nd_tb = TerminalWpts_in_mjc_nd_tb.dropna()
            TerminalWpts_in_mjc_nd_tb['WptIdent'] = TerminalWpts_in_mjc_nd_tb['WptID'].apply(lambda WptID : WaypointLookup_in_mjc_nd_tb.loc[WaypointLookup_in_mjc_nd_tb.loc[(WaypointLookup_in_mjc_nd_tb['ID'] == WptID)].index.tolist() , 'Ident'].values[0])
            TerminalWpts_in_mjc_nd_tb['REGION_CODE'] = TerminalWpts_in_mjc_nd_tb['TerminalID'].apply(lambda TerminalID : Terminals_in_mjc_nd_tb.loc[Terminals_in_mjc_nd_tb.loc[(Terminals_in_mjc_nd_tb['ID'] == TerminalID)].index.tolist() , 'ICAO'].values[0])
        print(time.strftime("%H:%M:%S", time.localtime()) , 'MJC: nd.db3 Load Completed')
    # Process of MJC Part #1
    with open('Input/' + Datafile, 'r', encoding='ISO-8859-1') as ARINC424:
        for line in ARINC424: 
            vhf_se = re.search(vor, line)
            db_se = re.search(db, line)
            ea_se = re.search(ea, line)
            awy_se = re.search(er, line)
            arpt_se = re.search(pa, line)
            pc_se = re.search(pc, line)
            rwy_se = re.search(pg, line)
            pn_se = re.search(pn, line)
            if vhf_se:
                ARPT_IDENT = vhf_se.group(5).strip()
                NAV_IDENT = vhf_se.group(7).strip()
                WAYPOINT_IDENT = NAV_IDENT
                NAV_ICAO_CODE = vhf_se.group(8).strip()
                WAYPOINT_ICAO_CODE = NAV_ICAO_CODE
                NAV_FREQ = int('0x' + str(int(vhf_se.group(10))*100) , 16)
                if vhf_se.group(11)[0:1] == "V":
                    if vhf_se.group(11)[1:2] == " ":
                        NAVAID_TYPE = 1
                    elif vhf_se.group(11)[1:2] == "T":
                        NAVAID_TYPE = 2
                    elif vhf_se.group(11)[1:2] == "M":
                        NAVAID_TYPE = 2
                    elif vhf_se.group(11)[1:2] == "D":
                        NAVAID_TYPE = 4
                    else:
                        NAVAID_TYPE = 1
                else:
                    if vhf_se.group(11)[1:2] == "T":
                        NAVAID_TYPE = 3
                    elif vhf_se.group(11)[1:2] == "M":
                        NAVAID_TYPE = 3
                    elif vhf_se.group(11)[1:2] == "I":
                        NAVAID_TYPE = 8          
                    elif vhf_se.group(11)[1:2] == "D":
                        NAVAID_TYPE = 9
                    else:
                        NAVAID_TYPE = 9
                if vhf_se.group(11)[2:3] == "U":
                    NAVAID_CLASS = "B"
                else :
                    NAVAID_CLASS = vhf_se.group(11)[2:3]
                if vhf_se.group(12).strip() != '':
                    if vhf_se.group(12) == "N":
                        NAV_LAT = round(float(vhf_se.group(13)) + float(vhf_se.group(14))/60 + float(vhf_se.group(15))/3600 + float(vhf_se.group(16))/360000, 8)
                    else:
                        NAV_LAT = 0 - round(float(vhf_se.group(13)) - float(vhf_se.group(14))/60 - float(vhf_se.group(15))/3600 - float(vhf_se.group(16))/360000, 8)
                else:
                    if vhf_se.group(23) == "N":
                        NAV_LAT = round(float(vhf_se.group(24)) + float(vhf_se.group(25))/60 + float(vhf_se.group(26))/3600 + float(vhf_se.group(27))/360000, 8)
                    else:
                        NAV_LAT = 0 - round(float(vhf_se.group(24)) - float(vhf_se.group(25))/60 - float(vhf_se.group(26))/3600 - float(vhf_se.group(27))/360000, 8)
                WAYPOINT_LAT = NAV_LAT
                if vhf_se.group(17).strip() != '':
                    if vhf_se.group(17) == "E":
                        NAV_LON = round(float(vhf_se.group(18)) + float(vhf_se.group(19))/60 + float(vhf_se.group(20))/3600 + float(vhf_se.group(21))/360000, 8)
                    else:
                        NAV_LON = 0 - round(float(vhf_se.group(18)) - float(vhf_se.group(19))/60 - float(vhf_se.group(20))/3600 - float(vhf_se.group(21))/360000, 8)
                else:
                    if vhf_se.group(28) == "E":
                        NAV_LON = round(float(vhf_se.group(29)) + float(vhf_se.group(30))/60 + float(vhf_se.group(31))/3600 + float(vhf_se.group(32))/360000, 8)
                    else:
                        NAV_LON = 0 - round(float(vhf_se.group(29)) - float(vhf_se.group(30))/60 - float(vhf_se.group(31))/3600 - float(vhf_se.group(32))/360000, 8)
                WAYPOINT_LON = NAV_LON
                if vhf_se.group(33).strip() == '':
                    MAG_VAR = 0
                elif vhf_se.group(33).strip(string.digits) == "E":
                    MAG_VAR = float(vhf_se.group(33).strip('EW'))/10
                else:
                    MAG_VAR = 0 - float(vhf_se.group(33).strip('EW'))/10
                if vhf_se.group(34).strip() == '':
                    ELEV = 0
                else:
                    ELEV = vhf_se.group(34)
                NAV_NAME = vhf_se.group(39).replace('VOR/DME','').rstrip().replace('(','').replace(')','')
                WAYPOINT_NAME_DESCR = NAV_NAME
                NavaidID = NavaidID + 1
                WaypointID = WaypointID + 1
                NavaidLookup_mjc.append((NAV_IDENT,NAVAID_TYPE,NAV_ICAO_CODE,1,NavaidID))
                Navaids_mjc.append((NavaidID,NAV_IDENT,NAVAID_TYPE,NAV_NAME,NAV_FREQ,None,NAVAID_CLASS,NAV_LAT,NAV_LON,ELEV,MAG_VAR))
                WaypointLookup_mjc.append((WAYPOINT_IDENT,WAYPOINT_ICAO_CODE,WaypointID,ARPT_IDENT))
                Waypoints_mjc.append((WaypointID,WAYPOINT_IDENT,1,WAYPOINT_NAME_DESCR,WAYPOINT_LAT,WAYPOINT_LON,int(NavaidID)))
            if db_se:
                ARPT_IDENT = db_se.group(5).strip()
                NAV_IDENT = db_se.group(7).strip()
                WAYPOINT_IDENT = NAV_IDENT
                NAV_ICAO_CODE = db_se.group(8).strip()
                WAYPOINT_ICAO_CODE = NAV_ICAO_CODE
                NAV_FREQ = int('0x' + str(int(db_se.group(10))*1000) , 16)
                NAVAID_TYPE = 5
                if db_se.group(11)[2:3] == "L":
                    NAVAID_CLASS = "T"
                elif db_se.group(11)[2:3] == "M":
                    NAVAID_CLASS = "L"
                else :
                    NAVAID_CLASS = "H"
                if db_se.group(12) == "N":
                    NAV_LAT = round(float(db_se.group(13)) + float(db_se.group(14))/60 + float(db_se.group(15))/3600 + float(db_se.group(16))/360000, 8)
                else:
                    NAV_LAT = 0 - round(float(db_se.group(13)) - float(db_se.group(14))/60 - float(db_se.group(15))/3600 - float(db_se.group(16))/360000, 8)
                WAYPOINT_LAT = NAV_LAT
                if db_se.group(17) == "E":
                    NAV_LON = round(float(db_se.group(18)) + float(db_se.group(19))/60 + float(db_se.group(20))/3600 + float(db_se.group(21))/360000, 8)
                else:
                    NAV_LON = 0 - round(float(db_se.group(18)) - float(db_se.group(19))/60 - float(db_se.group(20))/3600 - float(db_se.group(21))/360000, 8)
                WAYPOINT_LON = NAV_LON
                if db_se.group(22) == "E":
                    MAG_VAR = float(db_se.group(23))/10
                else:
                    MAG_VAR = 0 - float(db_se.group(23))/10
                ELEV = 0
                NAV_NAME = db_se.group(25).replace('NDB','').rstrip().replace('(','').replace(')','')
                WAYPOINT_NAME_DESCR = NAV_NAME
                NavaidID = NavaidID + 1
                WaypointID = WaypointID + 1
                NavaidLookup_mjc.append((NAV_IDENT,NAVAID_TYPE,NAV_ICAO_CODE,1,NavaidID))
                Navaids_mjc.append((NavaidID,NAV_IDENT,NAVAID_TYPE,NAV_NAME,NAV_FREQ,None,NAVAID_CLASS,NAV_LAT,NAV_LON,ELEV,MAG_VAR))
                WaypointLookup_mjc.append((WAYPOINT_IDENT,WAYPOINT_ICAO_CODE,WaypointID,ARPT_IDENT))
                Waypoints_mjc.append((WaypointID,WAYPOINT_IDENT,1,WAYPOINT_NAME_DESCR,WAYPOINT_LAT,WAYPOINT_LON,int(NavaidID)))
            if ea_se:
                REGION_CODE = ea_se.group(5)
                WAYPOINT_IDENT = ea_se.group(8).strip()
                WAYPOINT_ICAO_CODE = ea_se.group(9).strip()
                if ea_se.group(13) == "N":
                    WAYPOINT_LAT = round(float(ea_se.group(14)) + float(ea_se.group(15))/60 + float(ea_se.group(16))/3600 + float(ea_se.group(17))/360000, 8)
                else:
                    WAYPOINT_LAT = round(0 - float(ea_se.group(14)) - float(ea_se.group(15))/60 - float(ea_se.group(16))/3600 - float(ea_se.group(17))/360000, 8)
                if ea_se.group(18) == "E":
                    WAYPOINT_LON = round(float(ea_se.group(19)) + float(ea_se.group(20))/60 + float(ea_se.group(21))/3600 + float(ea_se.group(22))/360000, 8)
                else:
                    WAYPOINT_LON = round(0 - float(ea_se.group(19)) - float(ea_se.group(20))/60 - float(ea_se.group(21))/3600 - float(ea_se.group(22))/360000, 8)
                WAYPOINT_NAME_DESCR = ea_se.group(26).rstrip().replace('(','').replace(')','')
                WaypointID = WaypointID + 1
                WaypointLookup_mjc.append((WAYPOINT_IDENT,WAYPOINT_ICAO_CODE,WaypointID,REGION_CODE))
                Waypoints_mjc.append((WaypointID,WAYPOINT_IDENT,0,WAYPOINT_NAME_DESCR,WAYPOINT_LAT,WAYPOINT_LON,None))
            if awy_se:
                if awy_se.group(5).strip() != AirwayIdent:
                    if mjc_create_blank != True:
                        if Airways_in_mjc_nd_tb.loc[(Airways_in_mjc_nd_tb['Ident'] == awy_se.group(5).strip())].index.tolist() == []:
                            AirwayID = AirwayID + 1
                            AirwayIdent = awy_se.group(5).strip()
                            Airways_mjc.append((AirwayID, AirwayIdent))
                    else:
                        AirwayID = AirwayID + 1
                        AirwayIdent = awy_se.group(5).strip()
                        Airways_mjc.append((AirwayID, AirwayIdent))
            if arpt_se:
                ARPT_IDENT = arpt_se.group(4)
                ARPT_ICAO_CODE = arpt_se.group(5)
                if arpt_se.group(13) == "N":
                    ARPT_LAT = round(float(arpt_se.group(14)) + float(arpt_se.group(15))/60 + float(arpt_se.group(16))/3600 + float(arpt_se.group(17))/360000, 8)
                else:
                    ARPT_LAT = round(0 - float(arpt_se.group(14)) - float(arpt_se.group(15))/60 - float(arpt_se.group(16))/3600 - float(arpt_se.group(17))/360000, 8)
                if arpt_se.group(18) == "E":
                    ARPT_LON = round(float(arpt_se.group(19)) + float(arpt_se.group(20))/60 + float(arpt_se.group(21))/3600 + float(arpt_se.group(22))/360000, 8)
                else:
                    ARPT_LON = round(0 - float(arpt_se.group(19)) - float(arpt_se.group(20))/60 - float(arpt_se.group(21))/3600 - float(arpt_se.group(22))/360000, 8)
                ARPT_ELEV = int(arpt_se.group(25))
                ARPT_NAME = arpt_se.group(36).rstrip().replace('(','').replace(')','')
                if arpt_se.group(23) == "E":
                    ARPT_MAG_VAR = float(arpt_se.group(24))/10
                else:
                    ARPT_MAG_VAR = 0 - float(arpt_se.group(24))/10
                AirportID = AirportID + 1
                Airports_mjc.append((AirportID,ARPT_NAME,ARPT_IDENT,None,ARPT_LAT,ARPT_LON,ARPT_ELEV))
                AirportLookup_mjc.append((ARPT_ICAO_CODE + ARPT_IDENT,AirportID))
            if pc_se:
                REGION_CODE = pc_se.group(5)
                WAYPOINT_IDENT = pc_se.group(8).strip()
                WAYPOINT_ICAO_CODE = pc_se.group(9).strip()
                if pc_se.group(13) == "N":
                    WAYPOINT_LAT = round(float(pc_se.group(14)) + float(pc_se.group(15))/60 + float(pc_se.group(16))/3600 + float(pc_se.group(17))/360000, 8)
                else:
                    WAYPOINT_LAT = round(0 - float(pc_se.group(14)) - float(pc_se.group(15))/60 - float(pc_se.group(16))/3600 - float(pc_se.group(17))/360000, 8)
                if pc_se.group(18) == "E":
                    WAYPOINT_LON = round(float(pc_se.group(19)) + float(pc_se.group(20))/60 + float(pc_se.group(21))/3600 + float(pc_se.group(22))/360000, 8)
                else:
                    WAYPOINT_LON = round(0 - float(pc_se.group(19)) - float(pc_se.group(20))/60 - float(pc_se.group(21))/3600 - float(pc_se.group(22))/360000, 8)
                WAYPOINT_NAME_DESCR = pc_se.group(26).rstrip().replace('(','').replace(')','')
                WaypointID = WaypointID + 1
                WaypointLookup_mjc.append((WAYPOINT_IDENT,WAYPOINT_ICAO_CODE,WaypointID,REGION_CODE))
                Waypoints_mjc.append((WaypointID,WAYPOINT_IDENT,0,WAYPOINT_NAME_DESCR,WAYPOINT_LAT,WAYPOINT_LON,None))
            if rwy_se:
                RunwayID = RunwayID + 1
                if rwy_se.group(4) != ARPT_IDENT:
                    print(rwy_se.group(4) + " Not Found")
                    AirportID = None
                Ident = rwy_se.group(7)[2:5].strip()
                if rwy_se.group(10).lstrip(string.digits) == "T":
                    TrueHeading = rwy_se.group(10).rstrip('T')
                else:
                    TrueHeading = float(rwy_se.group(10))/10 + ARPT_MAG_VAR
                Length = rwy_se.group(9)
                Width = rwy_se.group(27)
                if rwy_se.group(11) == "N":
                    Latitude = round(float(rwy_se.group(12)) + float(rwy_se.group(13))/60 + float(rwy_se.group(14))/3600 + float(rwy_se.group(15))/360000, 8)
                else:
                    Latitude = round(0 - float(rwy_se.group(12)) - float(rwy_se.group(13))/60 - float(rwy_se.group(14))/3600 - float(rwy_se.group(15))/360000, 8)
                if rwy_se.group(16) == "E":
                    Longtitude = round(float(rwy_se.group(17)) + float(rwy_se.group(18))/60 + float(rwy_se.group(19))/3600 + float(rwy_se.group(20))/360000, 8)
                else:
                    Longtitude = round(0 - float(rwy_se.group(17)) - float(rwy_se.group(18))/60 - float(rwy_se.group(19))/3600 - float(rwy_se.group(20))/360000, 8)
                Elevation = rwy_se.group(24).lstrip('0')
                Runways_mjc.append((RunwayID,AirportID,Ident,TrueHeading,Length,Width,'U  ',Latitude,Longtitude,Elevation))
            if pn_se:
                ARPT_IDENT = pn_se.group(5).strip()
                NAV_IDENT = pn_se.group(7).strip()
                WAYPOINT_IDENT = NAV_IDENT
                NAV_ICAO_CODE = pn_se.group(8).strip()
                WAYPOINT_ICAO_CODE = NAV_ICAO_CODE
                NAV_FREQ = int('0x' + str(int(pn_se.group(10))*1000) , 16)
                NAVAID_TYPE = 5
                if pn_se.group(11)[2:3] == "L":
                    NAVAID_CLASS = "T"
                elif pn_se.group(11)[2:3] == "M":
                    NAVAID_CLASS = "L"
                else :
                    NAVAID_CLASS = "H"
                if pn_se.group(12) == "N":
                    NAV_LAT = round(float(pn_se.group(13)) + float(pn_se.group(14))/60 + float(pn_se.group(15))/3600 + float(pn_se.group(16))/360000, 8)
                else:
                    NAV_LAT = 0 - round(float(pn_se.group(13)) - float(pn_se.group(14))/60 - float(pn_se.group(15))/3600 - float(pn_se.group(16))/360000, 8)
                WAYPOINT_LAT = NAV_LAT
                if pn_se.group(17) == "E":
                    NAV_LON = round(float(pn_se.group(18)) + float(pn_se.group(19))/60 + float(pn_se.group(20))/3600 + float(pn_se.group(21))/360000, 8)
                else:
                    NAV_LON = 0 - round(float(pn_se.group(18)) - float(pn_se.group(19))/60 - float(pn_se.group(20))/3600 - float(pn_se.group(21))/360000, 8)
                WAYPOINT_LON = NAV_LON
                if pn_se.group(22) == "E":
                    MAG_VAR = float(pn_se.group(23))/10
                else:
                    MAG_VAR = 0 - float(pn_se.group(23))/10
                NAV_NAME = pn_se.group(25).replace('NDB','').rstrip().replace('(','').replace(')','')
                WAYPOINT_NAME_DESCR = NAV_NAME
                NavaidID = NavaidID + 1
                WaypointID = WaypointID + 1
                NavaidLookup_mjc.append((NAV_IDENT,NAVAID_TYPE,NAV_ICAO_CODE,1,NavaidID))
                Navaids_mjc.append((NavaidID,NAV_IDENT,NAVAID_TYPE,NAV_NAME,NAV_FREQ,None,NAVAID_CLASS,NAV_LAT,NAV_LON,ELEV,MAG_VAR))
                WaypointLookup_mjc.append((WAYPOINT_IDENT,WAYPOINT_ICAO_CODE,WaypointID,ARPT_IDENT))
                Waypoints_mjc.append((WaypointID,WAYPOINT_IDENT,1,WAYPOINT_NAME_DESCR,WAYPOINT_LAT,WAYPOINT_LON,int(NavaidID)))
        # Convert list to Pandas DataFrame
        Airports_mjc_tb = pandas.DataFrame(Airports_mjc, columns=['ID','Name','ICAO','PrimaryID','Latitude','Longtitude','Elevation'])
        AirportLookup_mjc_tb = pandas.DataFrame(AirportLookup_mjc, columns=['extID','ID'])
        Navaids_mjc_tb = pandas.DataFrame(Navaids_mjc, columns=['ID','Ident','Type','Name','Freq','Channel','Usage','Latitude','Longtitude','Elevation','SlavedVar'])
        NavaidLookup_mjc_tb = pandas.DataFrame(NavaidLookup_mjc, columns=['Ident','Type','Country','NavKeyCode','ID'])
        Waypoints_mjc_tb = pandas.DataFrame(Waypoints_mjc, columns=['ID','Ident','Collocated','Name','Latitude','Longtitude','NavaidID'])
        WaypointLookup_mjc_tb = pandas.DataFrame(WaypointLookup_mjc, columns=['Ident','Country','ID','REGION_CODE'])
        Runways_mjc_tb = pandas.DataFrame(Runways_mjc, columns=['ID','AirportID','Ident','TrueHeading','Length','Width','Surface','Latitude','Longtitude','Elevation'])
        # Count Duplicate Navaids (NavKeyCode)
        if mjc_create_blank != True:
            NavaidLookup_mjc_tb = pandas.concat([NavaidLookup_in_mjc_nd_tb,NavaidLookup_mjc_tb])
            NavaidLookup_mjc_tb['NavKeyCode'] = NavaidLookup_mjc_tb.groupby(['Ident','Country'], as_index=False)[['Ident','Country']].cumcount() + 1
            NavaidLookup_mjc_tb = NavaidLookup_mjc_tb[NavaidLookup_mjc_tb['ID'] > 15000]
        else:  
            NavaidLookup_mjc_tb['NavKeyCode'] = NavaidLookup_mjc_tb.groupby(['Ident','Country'], as_index=False)[['Ident','Country']].cumcount() + 1
        # Make Airway List
        Airways_mjc_tb = pandas.DataFrame(Airways_mjc, columns=['ID','Ident'])
        Airways_mjc_tb = Airways_mjc_tb.sort_values('Ident')
        Airways_mjc_tb.drop_duplicates(subset=['Ident'], keep='first', inplace=True)
        Airways_mjc_tb = Airways_mjc_tb.reset_index(drop=True)
        if add_tailored:
            Airways_mjc_tb['ID'] = Airways_mjc_tb.index + 10001
        else:
            Airways_mjc_tb['ID'] = Airways_mjc_tb.index + 1
        AirwayIdent = ''
        print(time.strftime("%H:%M:%S", time.localtime()) , 'MJC: Data Process Part #1 Finished')
    # Process of MJC Part #2
    with open('Input/' + Datafile, 'r', encoding='ISO-8859-1') as ARINC424:
        for line in ARINC424:
            awy_se = re.search(er, line)
            loc_se = re.search(pi, line)
            # AWY of MJC (Part #1)
            if awy_se:
                CUST_AREA = awy_se.group(2)
                ROUTE_IDENT = awy_se.group(5).strip()
                AirwayID = Airways_mjc_tb.loc[(Airways_mjc_tb['Ident'] == ROUTE_IDENT)].index.tolist()
                if AirwayID != []:
                    AirwayID = Airways_mjc_tb.at[AirwayID[0], 'ID']
                else:
                    AirwayID = Airways_in_mjc_nd_tb.loc[(Airways_in_mjc_nd_tb['Ident'] == ROUTE_IDENT)].index.tolist()
                    if mjc_create_blank != True:
                        if AirwayID != []:
                            AirwayID = Airways_in_mjc_nd_tb.at[AirwayID[0], 'ID']
                        else:
                            print('         MJC: Airway '+ awy_se.group(5) + ' Not Found')
                            AirwayID = None
                    else:
                        print('         MJC: Airway '+ awy_se.group(5) + ' Not Found')
                        AirwayID = None
                SEQ_NR = awy_se.group(6)
                FIX_IDENT = awy_se.group(7).strip()
                FIX_ICAO_CODE = awy_se.group(8).strip()
                FIX_ID = WaypointLookup_mjc_tb.loc[(WaypointLookup_mjc_tb['Ident'] == FIX_IDENT) & (WaypointLookup_mjc_tb['Country'] == FIX_ICAO_CODE)].index.tolist()
                if FIX_ID != []:
                    FIX_ID = WaypointLookup_mjc_tb.at[FIX_ID[0], 'ID']
                else:
                    if mjc_create_blank != True:
                        FIX_ID = WaypointLookup_in_mjc_nd_tb.loc[(WaypointLookup_in_mjc_nd_tb['Ident'] == FIX_IDENT) & (WaypointLookup_in_mjc_nd_tb['Country'] == FIX_ICAO_CODE)].index.tolist()
                        if FIX_ID != []:
                            FIX_ID = WaypointLookup_in_mjc_nd_tb.at[FIX_ID[0], 'ID']
                        else:
                            print('         MJC: Airway ' + awy_se.group(5) + ' Fix ' + awy_se.group(7) + ' Not Found')
                            FIX_ID = '0'
                    else:
                        print('         MJC: Airway ' + awy_se.group(5) + ' Fix ' + awy_se.group(7) + ' Not Found')
                        FIX_ID = '0'
                EndOfRoute = awy_se.group(12)[1:2]
                LEVEL = awy_se.group(15)
                DIRECTION_RESTRICTION = awy_se.group(16)
                AirwayLookup_mjc.append((CUST_AREA,ROUTE_IDENT,AirwayID,SEQ_NR,FIX_IDENT,FIX_ICAO_CODE,FIX_ID,EndOfRoute,LEVEL,DIRECTION_RESTRICTION))
            # ILS of MJC
            if loc_se:
                ICAO = loc_se.group(4)
                if Airports_mjc_tb.loc[(Airports_mjc_tb['ICAO'] == ICAO)].index.tolist() != []:
                    AirportID = Airports_mjc_tb.loc[Airports_mjc_tb.loc[(Airports_mjc_tb['ICAO'] == ICAO)].index.tolist() , 'ID'].values[0]
                else:
                    if mjc_create_blank != True:
                        if Airports_in_mjc_nd_tb.loc[(Airports_in_mjc_nd_tb['ICAO'] == ICAO)].index.tolist() != []:
                            AirportID = Airports_in_mjc_nd_tb.loc[Airports_in_mjc_nd_tb.loc[(Airports_in_mjc_nd_tb['ICAO'] == ICAO)].index.tolist() , 'ID'].values[0]
                        else:
                            print('         MJC: Airport ' + ICAO + ' ID Not Found')
                            AirportID = None
                    else:
                        print('         MJC: Airport ' + ICAO + ' ID Not Found')
                        AirportID = None
                if loc_se.group(11)[0:2] != 'RW':
                    Rwy = None
                    RwyID = None
                    print('         MJC: Runway ' + ICAO + ' ' + loc_se.group(11) + ' Ident Not Correct')
                else:
                    Rwy = loc_se.group(11).strip().replace('RW','')
                    if Runways_mjc_tb.loc[(Runways_mjc_tb['AirportID'] == AirportID) & (Runways_mjc_tb['Ident'] == Rwy)].index.tolist() != []:
                        RwyID = Runways_mjc_tb.loc[Runways_mjc_tb.loc[(Runways_mjc_tb['AirportID'] == AirportID) & (Runways_mjc_tb['Ident'] == Rwy)].index.tolist() , 'ID'].values[0]
                    else:
                        if mjc_create_blank != True:
                            if Runways_in_mjc_nd_tb.loc[(Runways_in_mjc_nd_tb['AirportID'] == AirportID) & (Runways_in_mjc_nd_tb['Ident'] == Rwy)].index.tolist() != []:
                                RwyID = Runways_in_mjc_nd_tb.loc[Runways_in_mjc_nd_tb.loc[(Runways_in_mjc_nd_tb['AirportID'] == AirportID) & (Runways_in_mjc_nd_tb['Ident'] == Rwy)].index.tolist() , 'ID'].values[0]
                            else:
                                print('         MJC: Runway ' + ICAO + ' ' + Rwy + ' ID Not Found')
                                RwyID = None
                        else:
                            print('         MJC: Runway ' + ICAO + ' ' + Rwy + ' ID Not Found')
                            RwyID = None
                NAV_FREQ = int('0x' + str(int(loc_se.group(10))*100) , 16)
                if loc_se.group(37).strip() == '':
                    GLIDE_SLOPE_ANGLE = 0
                else:
                    GLIDE_SLOPE_ANGLE = float(loc_se.group(37))/100
                if loc_se.group(12).strip() == '':
                    LOCALIZER_LAT = None
                elif loc_se.group(12) == "N":
                    LOCALIZER_LAT = round(float(loc_se.group(13)) + float(loc_se.group(14))/60 + float(loc_se.group(15))/3600 + float(loc_se.group(16))/360000, 8)
                else:
                    LOCALIZER_LAT = 0 - round(float(loc_se.group(13)) - float(loc_se.group(14))/60 - float(loc_se.group(15))/3600 - float(loc_se.group(16))/360000, 8)
                if loc_se.group(17).strip() == '':
                    LOCALIZER_LON = None
                elif loc_se.group(17) == "E":
                    LOCALIZER_LON = round(float(loc_se.group(18)) + float(loc_se.group(19))/60 + float(loc_se.group(20))/3600 + float(loc_se.group(21))/360000, 8)
                else:
                    LOCALIZER_LON = 0 - round(float(loc_se.group(18)) - float(loc_se.group(19))/60 - float(loc_se.group(20))/3600 - float(loc_se.group(21))/360000, 8)
                if loc_se.group(8).strip(string.digits) == '':
                    ILS_CATEGORY = loc_se.group(8)
                else:
                    ILS_CATEGORY = 0
                LOCALIZER_IDENT = loc_se.group(7).strip()
                if loc_se.group(22).lstrip(string.digits) == "T":
                    LOCALIZER_BEARING = float(Decimal(loc_se.group(22).rstrip('T')))
                else:
                    LOCALIZER_BEARING = float(Decimal(loc_se.group(22)[0:3] + '.' + loc_se.group(22)[3:]).quantize(Decimal('0'), rounding=ROUND_HALF_UP))
                if loc_se.group(39).strip() == '':
                    GLIDE_SLOPE_HEIGHT_AT_LANDING_THLD = 50
                else:
                    GLIDE_SLOPE_HEIGHT_AT_LANDING_THLD = loc_se.group(39)
                if loc_se.group(40).strip() == '':
                    GLIDE_SLOPE_ELEV = 0
                else:
                    GLIDE_SLOPE_ELEV = loc_se.group(40)
                if loc_se.group(23).strip() == '':
                    GLIDE_SLOPE_LAT = None
                elif loc_se.group(23) == "N":
                    GLIDE_SLOPE_LAT = round(float(loc_se.group(24)) + float(loc_se.group(25))/60 + float(loc_se.group(26))/3600 + float(loc_se.group(27))/360000, 8)
                else:
                    GLIDE_SLOPE_LAT = 0 - round(float(loc_se.group(24)) - float(loc_se.group(25))/60 - float(loc_se.group(26))/3600 - float(loc_se.group(27))/360000, 8)
                if loc_se.group(28).strip() == '':
                    GLIDE_SLOPE_LON = None
                elif loc_se.group(28) == "E":
                    GLIDE_SLOPE_LON = round(float(loc_se.group(29)) + float(loc_se.group(30))/60 + float(loc_se.group(31))/3600 + float(loc_se.group(32))/360000, 8)
                else:
                    GLIDE_SLOPE_LON = 0 - round(float(loc_se.group(29)) - float(loc_se.group(30))/60 - float(loc_se.group(31))/3600 - float(loc_se.group(32))/360000, 8)
                DME_ID = WaypointLookup_mjc_tb.loc[(WaypointLookup_mjc_tb['Ident'] == loc_se.group(7).strip()) & (WaypointLookup_mjc_tb['REGION_CODE'] == ICAO)].index.tolist()
                if DME_ID != []:
                    HasDme = 1
                    DME_ID = WaypointLookup_mjc_tb.at[DME_ID[0], 'ID']
                    DME_LAT = Waypoints_mjc_tb.loc[Waypoints_mjc_tb.loc[(Waypoints_mjc_tb['ID'] == DME_ID)].index.tolist() , 'Latitude'].values[0]
                    DME_LON = Waypoints_mjc_tb.loc[Waypoints_mjc_tb.loc[(Waypoints_mjc_tb['ID'] == DME_ID)].index.tolist() , 'Longtitude'].values[0]
                else:
                    if mjc_create_blank != True:
                        DME_ID = WaypointLookup_in_mjc_nd_tb.loc[(WaypointLookup_in_mjc_nd_tb['Ident'] == loc_se.group(7).strip()) & (WaypointLookup_in_mjc_nd_tb['Country'] == loc_se.group(5))].index.tolist()
                        if DME_ID != []:
                            HasDme = 1
                            DME_ID = Waypoints_in_mjc_nd_tb.at[DME_ID[0], 'ID']
                            DME_LAT = Waypoints_in_mjc_nd_tb.loc[Waypoints_in_mjc_nd_tb.loc[(Waypoints_in_mjc_nd_tb['ID'] == DME_ID)].index.tolist() , 'Latitude'].values[0]
                            DME_LON = Waypoints_in_mjc_nd_tb.loc[Waypoints_in_mjc_nd_tb.loc[(Waypoints_in_mjc_nd_tb['ID'] == DME_ID)].index.tolist() , 'Longtitude'].values[0]
                        else:
                            HasDme = 0
                            DME_LAT = None
                            DME_LON = None
                    else:
                        HasDme = 0
                        DME_LAT = None
                        DME_LON = None
                ILSID = ILSID + 1
                ILSes_mjc.append((ILSID,AirportID,RwyID,NAV_FREQ,GLIDE_SLOPE_ANGLE,LOCALIZER_LAT,LOCALIZER_LON,ILS_CATEGORY,LOCALIZER_IDENT,LOCALIZER_BEARING,GLIDE_SLOPE_HEIGHT_AT_LANDING_THLD,HasDme,GLIDE_SLOPE_ELEV,GLIDE_SLOPE_LAT,GLIDE_SLOPE_LON,DME_LAT,DME_LON))
        # Convert list to Pandas DataFrame
        AirwayLookup_mjc_tb = pandas.DataFrame(AirwayLookup_mjc, columns=['CUST_AREA','ROUTE_IDENT','AirwayID','SEQ_NR','FIX_IDENT','FIX_ICAO_CODE','FIX_ID','EndOfRoute','LEVEL','DIRECTION_RESTRICTION'])
        AirwayLookup_mjc_tb = AirwayLookup_mjc_tb.sort_values(['ROUTE_IDENT','SEQ_NR','EndOfRoute'], ascending=[1,1,0])
        AirwayLookup_mjc_tb = AirwayLookup_mjc_tb.reset_index(drop=True)
        AirwayLookup_mjc = []
        ILSes_mjc_tb = pandas.DataFrame(ILSes_mjc, columns=['ID','AirportID','RunwayID','Freq','GsAngle','Latitude','Longtitude','Category','Ident','LocCourse','CrossingHeight','HasDme','Elevation','LatitudeGS','LongtitudeGS','LatitudeDME','LongtitudeDME'])
        print(time.strftime("%H:%M:%S", time.localtime()) , 'MJC: Data Process Part #2 Finished')
    # Process of MJC Part #3 (AWY Part #2)
    if len(AirwayLookup_mjc_tb.index) != 0:
        for index, row in AirwayLookup_mjc_tb.iterrows():
            if index + 1 != len(AirwayLookup_mjc_tb.index):
                if row['FIX_ID'] != AirwayLookup_mjc_tb.at[index + 1, 'FIX_ID']:
                    AirwayLookup_mjc.append((row['CUST_AREA'],row['ROUTE_IDENT'],row['AirwayID'],row['SEQ_NR'],row['FIX_IDENT'],row['FIX_ICAO_CODE'],row['FIX_ID'],row['EndOfRoute'],row['LEVEL'],row['DIRECTION_RESTRICTION']))
                else:
                    if row['AirwayID'] != AirwayLookup_mjc_tb.at[index + 1, 'AirwayID']:
                        AirwayLookup_mjc.append((row['CUST_AREA'],row['ROUTE_IDENT'],row['AirwayID'],row['SEQ_NR'],row['FIX_IDENT'],row['FIX_ICAO_CODE'],row['FIX_ID'],row['EndOfRoute'],row['LEVEL'],row['DIRECTION_RESTRICTION']))
            else:
                AirwayLookup_mjc.append((row['CUST_AREA'],row['ROUTE_IDENT'],row['AirwayID'],row['SEQ_NR'],row['FIX_IDENT'],row['FIX_ICAO_CODE'],row['FIX_ID'],row['EndOfRoute'],row['LEVEL'],row['DIRECTION_RESTRICTION']))
        AirwayLookup_mjc_tb = pandas.DataFrame(AirwayLookup_mjc, columns=['CUST_AREA','ROUTE_IDENT','AirwayID','SEQ_NR','FIX_IDENT','FIX_ICAO_CODE','FIX_ID','EndOfRoute','LEVEL','DIRECTION_RESTRICTION'])
        AirwayLookup_mjc_tb = AirwayLookup_mjc_tb.sort_values(['ROUTE_IDENT','SEQ_NR'])
        AirwayLookup_mjc_tb = AirwayLookup_mjc_tb.reset_index(drop=True)
        AirwayLookup_reversed_mjc_tb = AirwayLookup_mjc_tb.sort_index(ascending=False)
        AirwayLookup_reversed_mjc_tb = AirwayLookup_reversed_mjc_tb.reset_index(drop=True)
        for index, row in AirwayLookup_mjc_tb.iterrows():
            if index + 1 != len(AirwayLookup_mjc_tb.index):
                Waypoint1ID = row['FIX_ID']
                Waypoint2ID = AirwayLookup_mjc_tb.at[index + 1, 'FIX_ID']
                LEVEL = row['LEVEL']
                if row['ROUTE_IDENT'] == AirwayIdent:
                    if WasEnd == 0:
                        IsStart = 0
                    else:
                        IsStart = 1
                elif row['DIRECTION_RESTRICTION'] == 'B':
                    IsStart = 0
                else:
                    IsStart = 1
                    AirwayID = row['AirwayID']
                    AirwayIdent = row['ROUTE_IDENT']
                if index + 2  == len(AirwayLookup_mjc_tb.index):
                    IsEnd = 1
                elif AirwayLookup_mjc_tb.at[index + 1, 'DIRECTION_RESTRICTION'] == 'B':
                    IsEnd = 1
                elif AirwayLookup_mjc_tb.at[index + 2, 'ROUTE_IDENT'] != row['ROUTE_IDENT']:
                    IsEnd = 1
                elif AirwayLookup_mjc_tb.at[index + 1, 'EndOfRoute'] == 'E':
                    IsEnd = 1
                else:
                    IsEnd = 0
                if AirwayLookup_mjc_tb.at[index + 1, 'ROUTE_IDENT'] == row['ROUTE_IDENT']:
                    if row['DIRECTION_RESTRICTION'] != 'B':
                        if row['EndOfRoute'] != 'E':
                            AirwayLegsID = AirwayLegsID + 1
                            AirwayLegs_mjc.append((AirwayLegsID,AirwayID,LEVEL,Waypoint1ID,Waypoint2ID,IsStart,IsEnd))
                            WasEnd = IsEnd
        for index, row in AirwayLookup_reversed_mjc_tb.iterrows():
            if index + 1 != len(AirwayLookup_reversed_mjc_tb.index):
                Waypoint1ID = row['FIX_ID']
                Waypoint2ID = AirwayLookup_reversed_mjc_tb.at[index + 1, 'FIX_ID']
                LEVEL = row['LEVEL']
                if row['ROUTE_IDENT'] == AirwayIdent:
                    if row['EndOfRoute'] == 'E':
                        IsStart = 1
                    elif WasEnd == 1:
                        IsStart = 1
                    else:
                        IsStart = 0
                elif AirwayLookup_reversed_mjc_tb.at[index + 1, 'DIRECTION_RESTRICTION'] != 'F':
                    IsStart = 1
                    AirwayID = row['AirwayID']
                    AirwayIdent = row['ROUTE_IDENT']
                else:
                    IsStart = 0
                if index + 2  == len(AirwayLookup_reversed_mjc_tb.index):
                    IsEnd = 1
                elif AirwayLookup_reversed_mjc_tb.at[index + 2, 'DIRECTION_RESTRICTION'] == 'F':
                    IsEnd = 1
                elif AirwayLookup_reversed_mjc_tb.at[index + 2, 'ROUTE_IDENT'] != row['ROUTE_IDENT']:
                    IsEnd = 1
                elif AirwayLookup_reversed_mjc_tb.at[index + 2, 'EndOfRoute'] == 'E':
                    IsEnd = 1
                else:
                    IsEnd = 0
                if AirwayLookup_reversed_mjc_tb.at[index + 1, 'ROUTE_IDENT'] == row['ROUTE_IDENT']:
                    if AirwayLookup_reversed_mjc_tb.at[index + 1, 'DIRECTION_RESTRICTION'] != 'F':
                        if AirwayLookup_reversed_mjc_tb.at[index + 1, 'EndOfRoute'] != 'E':
                            AirwayLegsID = AirwayLegsID + 1
                            AirwayLegs_reversed_mjc.append((AirwayLegsID,AirwayID,LEVEL,Waypoint1ID,Waypoint2ID,IsStart,IsEnd))
                            WasEnd = IsEnd
        # Convert list to Pandas DataFrame
        AirwayLegs_mjc_tb = pandas.DataFrame(AirwayLegs_mjc, columns=['ID','AirwayID','Level','Waypoint1ID','Waypoint2ID','IsStart','IsEnd'])
        AirwayLegs_reversed_mjc_tb = pandas.DataFrame(AirwayLegs_reversed_mjc, columns=['ID','AirwayID','Level','Waypoint1ID','Waypoint2ID','IsStart','IsEnd'])
        AirwayLegs_mjc_tb = pandas.concat([AirwayLegs_mjc_tb, AirwayLegs_reversed_mjc_tb], axis=0, ignore_index=True)
        AirwayLegs_mjc_tb = AirwayLegs_mjc_tb.sort_values(['AirwayID','ID'])
        AirwayLegs_mjc_tb = AirwayLegs_mjc_tb.reset_index(drop=True)
        if add_tailored:
            AirwayLegs_mjc_tb['ID'] = AirwayLegs_mjc_tb.index + 200001
        else:
            AirwayLegs_mjc_tb['ID'] = AirwayLegs_mjc_tb.index + 1
        print(time.strftime("%H:%M:%S", time.localtime()) , 'MJC: Data Process Part #3 Finished')
    # Process of MJC Part #4
    with open('Input/' + Datafile, 'r', encoding='ISO-8859-1') as ARINC424:
        for line in ARINC424:
            proc_se = re.search(pdef, line)
            # PROC of MJC
            if proc_se:
                ICAO = proc_se.group(4)
                Type = proc_se.group(8)
                if proc_se.group(4) + proc_se.group(7).strip() != ICAO + TerminalName:
                    TerminalID = TerminalID + 1
                    AirportID = Airports_mjc_tb.loc[(Airports_mjc_tb['ICAO'] == ICAO)].index.tolist()
                    if AirportID != []:
                        AirportID = Airports_mjc_tb.at[AirportID[0], 'ID']
                    else:
                        if mjc_create_blank != True:
                            AirportID = Airports_in_mjc_nd_tb.loc[(Airports_in_mjc_nd_tb['ICAO'] == ICAO)].index.tolist()
                            if AirportID != []:
                                AirportID = Airports_in_mjc_nd_tb.at[AirportID[0], 'ID']
                            else:
                                print('         MJC: Airport ' + ICAO + ' ID Not Found')
                                AirportID = None
                        else:
                            print('         MJC: Airport ' + ICAO + ' ID Not Found')
                            AirportID = None
                    if proc_se.group(6) == 'D':
                        Proc = '2'
                    elif proc_se.group(6) == 'E':
                        Proc = '1'
                    else:
                        Proc = '3'
                    if Proc == '1' or Proc == '2':
                        if Type == '2' or Type == '5':
                            if proc_se.group(9)[0:2] + proc_se.group(9)[4:5].strip('LRCT ') == 'RW':
                                Rwy = proc_se.group(9).strip().replace('RW','')
                                RwyID = Runways_mjc_tb.loc[(Runways_mjc_tb['AirportID'] == AirportID) & (Runways_mjc_tb['Ident'] == Rwy)].index.tolist()
                                if RwyID != []:
                                    RwyID = Runways_mjc_tb.at[RwyID[0], 'ID']
                                else:
                                    if tfdi_create_blank != True:
                                        RwyID = Runways_in_mjc_nd_tb.loc[(Runways_in_mjc_nd_tb['AirportID'] == AirportID) & (Runways_in_mjc_nd_tb['Ident'] == Rwy)].index.tolist()
                                        if RwyID != []:
                                            RwyID = Runways_in_mjc_nd_tb.at[RwyID[0], 'ID']
                                        else:
                                            print('         MJC: Procedure ' + ICAO + ' '  + proc_se.group(7) + ' Runway ' + Rwy + ' Not Found')
                                            RwyID = None
                                    else:
                                        print('         MJC: Procedure ' + ICAO + ' '  + proc_se.group(7) + ' Runway ' + Rwy + ' Not Found')
                                        RwyID = None
                            else:
                                Rwy = None
                                RwyID = None
                        else:
                            Rwy = None
                            RwyID = None
                        Name = proc_se.group(7).strip()
                        FullName = Name
                        TerminalName = Name
                    else:
                        if proc_se.group(7)[1:3].strip('- ').strip(string.digits) == '':
                            Rwy = proc_se.group(7)[1:4].strip('- ')
                            RwyID = Runways_mjc_tb.loc[(Runways_mjc_tb['AirportID'] == AirportID) & (Runways_mjc_tb['Ident'] == Rwy)].index.tolist()
                            if RwyID != []:
                                RwyID = Runways_mjc_tb.at[RwyID[0], 'ID']
                            else:
                                if tfdi_create_blank != True:
                                    RwyID = Runways_in_mjc_nd_tb.loc[(Runways_in_mjc_nd_tb['AirportID'] == AirportID) & (Runways_in_mjc_nd_tb['Ident'] == Rwy)].index.tolist()
                                    if RwyID != []:
                                        RwyID = Runways_in_mjc_nd_tb.at[RwyID[0], 'ID']
                                    else:
                                        print('         MJC: Runway ' + ICAO + ' ' + Rwy + ' ID Not Found')
                                        RwyID = None
                                else:
                                    print('         MJC: Runway ' + ICAO + ' ' + Rwy + ' ID Not Found')
                                    RwyID = None
                        else:
                            Rwy = None
                            RwyID = None
                        Name = proc_se.group(7).strip()
                        FullName = proc_se.group(7) + ' APCH'
                        TerminalName = Name
                    # Duplicated Procedure Will Not Add into NDB
                    if mjc_create_blank != True:
                        if Terminals_in_mjc_nd_tb.loc[(Terminals_in_mjc_nd_tb['ICAO'] == ICAO) & (Terminals_in_mjc_nd_tb['Name'] == Name)].index.tolist() == []:
                            Terminals_mjc.append((TerminalID,AirportID,Proc,ICAO,FullName,Name,Rwy,RwyID))
                            Skip_Terminal = 0
                        else:
                            print('         MJC: Procedure ' + ICAO + ' ' + proc_se.group(7) + ' Skipped Due Duplicate')
                            Skip_Terminal = 1
                    else:
                        Terminals_mjc.append((TerminalID,AirportID,Proc,ICAO,FullName,Name,Rwy,RwyID))
                        Skip_Terminal = 0
                TerminalLegsID = TerminalLegsID + 1
                if proc_se.group(9).strip() == '':
                    Transition = None
                elif proc_se.group(9)[0:2] + proc_se.group(9)[4:5] == 'RWB':
                    Transition = 'ALL'
                else:
                    Transition = proc_se.group(9).strip()
                TrackCode = proc_se.group(19)
                if proc_se.group(13) + proc_se.group(14) == "  ":
                    WptLat = None
                    WptLon = None
                    WptID = None
                elif proc_se.group(13) + proc_se.group(14) == "PA":
                    WptID = Airports_mjc_tb.loc[(Airports_mjc_tb['ICAO'] == ICAO)].index.tolist()
                    if WptID != []:
                        WptLat = Airports_mjc_tb.at[WptID[0], 'Latitude']
                        WptLon = Airports_mjc_tb.at[WptID[0], 'Longtitude']
                        WptID = None
                    else:
                        if mjc_create_blank != True:
                            WptID = Airports_in_mjc_nd_tb.loc[(Airports_in_mjc_nd_tb['ICAO'] == ICAO)].index.tolist()
                            if WptID != []:
                                WptLat = Airports_in_mjc_nd_tb.at[WptID[0], 'Latitude']
                                WptLon = Airports_in_mjc_nd_tb.at[WptID[0], 'Longtitude']
                                WptID = None
                            else:
                                print('         MJC: Leg ' + ICAO + ' ' + proc_se.group(7) + ' Airport LAT/LON Not Found')
                                WptLat = None
                                WptLon = None
                                WptID = None
                        else:
                            print('         MJC: Leg ' + ICAO + ' ' + proc_se.group(7) + ' Airport LAT/LON Not Found')
                            WptLat = None
                            WptLon = None
                            WptID = None
                elif proc_se.group(13) + proc_se.group(14) == "PG":
                    WptID = Runways_mjc_tb.loc[(Runways_mjc_tb['AirportID'] == AirportID) & (Runways_mjc_tb['Ident'] == proc_se.group(11).strip().replace('RW',''))].index.tolist()
                    if WptID != []:
                        WptLat = Runways_mjc_tb.at[WptID[0], 'Latitude']
                        WptLon = Runways_mjc_tb.at[WptID[0], 'Longtitude']
                        WptID = None
                    else:
                        if mjc_create_blank != True:
                            WptID = Runways_in_mjc_nd_tb.loc[(Runways_in_mjc_nd_tb['AirportID'] == AirportID) & (Runways_in_mjc_nd_tb['Ident'] == proc_se.group(11).strip().replace('RW',''))].index.tolist()
                            if WptID != []:
                                WptLat = Runways_in_mjc_nd_tb.at[WptID[0], 'Latitude']
                                WptLon = Runways_in_mjc_nd_tb.at[WptID[0], 'Longtitude']
                                WptID = None
                            else:
                                print('         MJC: Leg ' + ICAO + ' ' + proc_se.group(7) + ' ' + Rwy + ' Rwy LAT/LON Not Found')
                                WptLat = None
                                WptLon = None
                                WptID = None
                        else:
                            print('         MJC: Leg ' + ICAO + ' ' + proc_se.group(7) + ' ' + Rwy + ' Rwy LAT/LON Not Found')
                            WptLat = None
                            WptLon = None
                            WptID = None
                elif proc_se.group(13) + proc_se.group(14) == "PC":
                    WptID = WaypointLookup_mjc_tb.loc[(WaypointLookup_mjc_tb['Ident'] == proc_se.group(11).strip()) & (WaypointLookup_mjc_tb['Country'] == proc_se.group(12)) & (WaypointLookup_mjc_tb['REGION_CODE'] == ICAO)].index.tolist()
                    if WptID != []:
                        WptID = WaypointLookup_mjc_tb.at[WptID[0], 'ID']
                        WptLat = Waypoints_mjc_tb.loc[Waypoints_mjc_tb.loc[(Waypoints_mjc_tb['ID'] == WptID)].index.tolist() , 'Latitude'].values[0]
                        WptLon = Waypoints_mjc_tb.loc[Waypoints_mjc_tb.loc[(Waypoints_mjc_tb['ID'] == WptID)].index.tolist() , 'Longtitude'].values[0]
                    else:
                        if mjc_create_blank != True:
                            WptID = TerminalWpts_in_mjc_nd_tb.loc[(TerminalWpts_in_mjc_nd_tb['WptIdent'] == proc_se.group(11).strip()) & (TerminalWpts_in_mjc_nd_tb['REGION_CODE'] == ICAO)].index.tolist()
                            if WptID != []:
                                WptID = TerminalWpts_in_mjc_nd_tb.at[WptID[0], 'WptID']
                                WptLat = Waypoints_in_mjc_nd_tb.loc[Waypoints_in_mjc_nd_tb.loc[(Waypoints_in_mjc_nd_tb['ID'] == WptID)].index.tolist() , 'Latitude'].values[0]
                                WptLon = Waypoints_in_mjc_nd_tb.loc[Waypoints_in_mjc_nd_tb.loc[(Waypoints_in_mjc_nd_tb['ID'] == WptID)].index.tolist() , 'Longtitude'].values[0]
                            else:
                                WptID = WaypointLookup_in_mjc_nd_tb.loc[(WaypointLookup_in_mjc_nd_tb['Ident'] == proc_se.group(11).strip()) & (WaypointLookup_in_mjc_nd_tb['Country'] == proc_se.group(12))].index.tolist()
                                if WptID != []:
                                    WptID = WaypointLookup_in_mjc_nd_tb.at[WptID[0], 'ID']
                                    WptLat = Waypoints_in_mjc_nd_tb.loc[Waypoints_in_mjc_nd_tb.loc[(Waypoints_in_mjc_nd_tb['ID'] == WptID)].index.tolist() , 'Latitude'].values[0]
                                    WptLon = Waypoints_in_mjc_nd_tb.loc[Waypoints_in_mjc_nd_tb.loc[(Waypoints_in_mjc_nd_tb['ID'] == WptID)].index.tolist() , 'Longtitude'].values[0]
                                else:
                                    print('         MJC: Leg ' + ICAO + ' ' + proc_se.group(7) + ' ' + proc_se.group(11) + ' WptID Not Found')
                                    WptID = None
                                    WptLat = None
                                    WptLon = None
                        else:
                            print('         MJC: Leg ' + ICAO + ' ' + proc_se.group(7) + ' ' + proc_se.group(11) + ' WptID Not Found')
                            WptID = None
                            WptLat = None
                            WptLon = None
                else:
                    WptID = WaypointLookup_mjc_tb.loc[(WaypointLookup_mjc_tb['Ident'] == proc_se.group(11).strip()) & (WaypointLookup_mjc_tb['Country'] == proc_se.group(12))].index.tolist()
                    if WptID != []:
                        WptID = WaypointLookup_mjc_tb.at[WptID[0], 'ID']
                        WptLat = Waypoints_mjc_tb.loc[Waypoints_mjc_tb.loc[(Waypoints_mjc_tb['ID'] == WptID)].index.tolist() , 'Latitude'].values[0]
                        WptLon = Waypoints_mjc_tb.loc[Waypoints_mjc_tb.loc[(Waypoints_mjc_tb['ID'] == WptID)].index.tolist() , 'Longtitude'].values[0]
                    else:
                        if mjc_create_blank != True:
                            WptID = WaypointLookup_in_mjc_nd_tb.loc[(WaypointLookup_in_mjc_nd_tb['Ident'] == proc_se.group(11).strip()) & (WaypointLookup_in_mjc_nd_tb['Country'] == proc_se.group(12))].index.tolist()
                            if WptID != []:
                                WptID = WaypointLookup_in_mjc_nd_tb.at[WptID[0], 'ID']
                                WptLat = Waypoints_in_mjc_nd_tb.loc[Waypoints_in_mjc_nd_tb.loc[(Waypoints_in_mjc_nd_tb['ID'] == WptID)].index.tolist() , 'Latitude'].values[0]
                                WptLon = Waypoints_in_mjc_nd_tb.loc[Waypoints_in_mjc_nd_tb.loc[(Waypoints_in_mjc_nd_tb['ID'] == WptID)].index.tolist() , 'Longtitude'].values[0]
                            else:
                                print('         MJC: Leg ' + ICAO + ' ' + proc_se.group(7) + ' ' + proc_se.group(11) + ' WptID Not Found')
                                WptID = None
                                WptLat = None
                                WptLon = None
                        else:
                            print('         MJC: Leg ' + ICAO + ' ' + proc_se.group(7) + ' ' + proc_se.group(11) + ' WptID Not Found')
                            WptID = None
                            WptLat = None
                            WptLon = None
                if proc_se.group(17).strip() == '':
                    TurnDir = None
                else:
                    TurnDir = proc_se.group(17)
                if proc_se.group(28) + proc_se.group(29) == '  ':
                    NavLat = None
                    NavLon = None
                    NavID = None
                elif proc_se.group(28) + proc_se.group(29) == 'PI':
                    ILSIdent = proc_se.group(21).strip()
                    NavID = ILSes_mjc_tb.loc[(ILSes_mjc_tb['AirportID'] == AirportID) & (ILSes_mjc_tb['Ident'] == ILSIdent)].index.tolist()
                    if NavID != []:
                        NavLat = ILSes_mjc_tb.at[NavID[0], 'Latitude']
                        NavLon = ILSes_mjc_tb.at[NavID[0], 'Longtitude']
                        NavID = None
                    else:
                        if mjc_create_blank != True:
                            NavID = ILSes_in_mjc_nd_tb.loc[(ILSes_in_mjc_nd_tb['AirportID'] == AirportID) & (ILSes_in_mjc_nd_tb['Ident'] == ILSIdent)].index.tolist()
                            if NavID != []:
                                NavLat = ILSes_in_mjc_nd_tb.at[NavID[0], 'Latitude']
                                NavLon = ILSes_in_mjc_nd_tb.at[NavID[0], 'Longtitude']
                                NavID = None
                            else:
                                print('         MJC: Leg ' + ICAO + ' ' + proc_se.group(7) + ' ' + proc_se.group(11) + ' Recommend ILS ' + proc_se.group(21) + ' Not Found')
                                NavLat = None
                                NavLon = None
                                NavID = None
                        else:
                            print('         MJC: Leg ' + ICAO + ' ' + proc_se.group(7) + ' ' + proc_se.group(11) + ' Recommend ILS ' + proc_se.group(21) + ' Not Found')
                            NavLat = None
                            NavLon = None
                            NavID = None
                elif proc_se.group(28) + proc_se.group(29) == 'PT':
                    NavLat = None
                    NavLon = None
                    NavID = None
                else:
                    NavID = NavaidLookup_mjc_tb.loc[(NavaidLookup_mjc_tb['Ident'] == proc_se.group(21).strip()) & (NavaidLookup_mjc_tb['Country'] == proc_se.group(22))].index.tolist()
                    if NavID != []:
                        NavID = NavaidLookup_mjc_tb.at[NavID[0], 'ID']
                        NavLat = Navaids_mjc_tb.loc[Navaids_mjc_tb.loc[(Navaids_mjc_tb['ID'] == NavID)].index.tolist() , 'Latitude'].values[0]
                        NavLon = Navaids_mjc_tb.loc[Navaids_mjc_tb.loc[(Navaids_mjc_tb['ID'] == NavID)].index.tolist() , 'Longtitude'].values[0]
                    else:
                        if mjc_create_blank != True:
                            NavID = NavaidLookup_in_mjc_nd_tb.loc[(NavaidLookup_in_mjc_nd_tb['Ident'] == proc_se.group(21).strip()) & (NavaidLookup_in_mjc_nd_tb['Country'] == proc_se.group(22))].index.tolist()
                            if NavID != []:
                                NavID = NavaidLookup_in_mjc_nd_tb.at[NavID[0], 'ID']
                                NavLat = Navaids_in_mjc_nd_tb.loc[Navaids_in_mjc_nd_tb.loc[(Navaids_in_mjc_nd_tb['ID'] == NavID)].index.tolist() , 'Latitude'].values[0]
                                NavLon = Navaids_in_mjc_nd_tb.loc[Navaids_in_mjc_nd_tb.loc[(Navaids_in_mjc_nd_tb['ID'] == NavID)].index.tolist() , 'Longtitude'].values[0]
                            else:
                                print('         MJC: Leg ' + ICAO + ' ' + proc_se.group(7) + ' ' + proc_se.group(11) + ' Recommend NavID ' + proc_se.group(21) + ' Not Found')
                                NavID = None
                                NavLat = None
                                NavLon = None
                        else:
                            print('         MJC: Leg ' + ICAO + ' ' + proc_se.group(7) + ' ' + proc_se.group(11) + ' Recommend NavID ' + proc_se.group(21) + ' Not Found')
                            NavID = None
                            NavLat = None
                            NavLon = None
                if proc_se.group(24).strip() == '':
                    NavBear = None
                else:
                    NavBear = float(proc_se.group(24))/10
                if proc_se.group(25).strip() == '':
                    NavDist = None
                else:
                    NavDist = float(proc_se.group(25))/10
                if proc_se.group(26).strip() == '':
                    Course = None
                elif proc_se.group(26).lstrip(string.digits) == "T":
                    Course = proc_se.group(26).rstrip('T')
                else:
                    Course = float(proc_se.group(26))/10
                if proc_se.group(27).strip() == '':
                    Distance = None
                elif proc_se.group(27).rstrip(string.digits) == "T":
                    Distance = float(proc_se.group(27).lstrip('T'))/2
                else:
                    Distance = float(proc_se.group(27))/10
                if proc_se.group(32).strip() == '':
                    ALT_1 = None
                elif proc_se.group(32).rstrip(string.digits) == "FL":
                    ALT_1 = proc_se.group(32).lstrip(string.ascii_uppercase) + "00"
                else:
                    ALT_1 = proc_se.group(32)
                if proc_se.group(33).strip() == '':
                    ALT_2 = None
                elif proc_se.group(33).rstrip(string.digits) == "FL":
                    ALT_2 = proc_se.group(33).lstrip(string.ascii_uppercase) + "00"
                else:
                    ALT_2 = proc_se.group(33)
                if proc_se.group(30).strip() == '+':
                    Alt = ALT_1 + 'A'
                elif proc_se.group(30).strip() == '-':
                    Alt = ALT_1 + 'B'
                elif proc_se.group(30).strip() == 'G':
                    Alt = ALT_2 + ''
                elif proc_se.group(30).strip() == 'H':
                    Alt = ALT_2 + 'A'
                elif proc_se.group(30).strip() == 'I':
                    Alt = ALT_1 + ''
                elif proc_se.group(30).strip() == 'J':
                    Alt = ALT_1 + 'A'
                else:
                    if proc_se.group(16)[3:4] == 'M':
                        Alt = 'MAP'
                    elif ALT_1 != None:
                        Alt = ALT_1 + ''
                    else:
                        Alt = None
                if proc_se.group(36).strip() == '':
                    Vnav = None
                else:
                    Vnav = round(float(proc_se.group(36)[1:4])/100, 2)
                if TrackCode == 'RF':
                    CenterID = WaypointLookup_mjc_tb.loc[(WaypointLookup_mjc_tb['Ident'] == proc_se.group(37).strip()) & (WaypointLookup_mjc_tb['Country'] == proc_se.group(39))].index.tolist()
                    if CenterID != []:
                        CenterID = WaypointLookup_mjc_tb.at[CenterID[0], 'ID']
                        CenterLat = Waypoints_mjc_tb.loc[Waypoints_mjc_tb.loc[(Waypoints_mjc_tb['ID'] == CenterID)].index.tolist() , 'Latitude'].values[0]
                        CenterLon = Waypoints_mjc_tb.loc[Waypoints_mjc_tb.loc[(Waypoints_mjc_tb['ID'] == CenterID)].index.tolist() , 'Longtitude'].values[0]
                    else:
                        if mjc_create_blank != True:
                            CenterID = WaypointLookup_in_mjc_nd_tb.loc[(WaypointLookup_in_mjc_nd_tb['Ident'] == proc_se.group(37).strip()) & (WaypointLookup_in_mjc_nd_tb['Country'] == proc_se.group(39))].index.tolist()
                            if CenterID != []:
                                CenterID = WaypointLookup_in_mjc_nd_tb.at[CenterID[0], 'ID']
                                CenterLat = Waypoints_in_mjc_nd_tb.loc[Waypoints_in_mjc_nd_tb.loc[(Waypoints_in_mjc_nd_tb['ID'] == CenterID)].index.tolist() , 'Latitude'].values[0]
                                CenterLon = Waypoints_in_mjc_nd_tb.loc[Waypoints_in_mjc_nd_tb.loc[(Waypoints_in_mjc_nd_tb['ID'] == CenterID)].index.tolist() , 'Longtitude'].values[0]
                            else:
                                print('         MJC: Leg ' + ICAO + ' ' + proc_se.group(7) + ' ' + proc_se.group(11) + ' CenterID ' + proc_se.group(37) + ' Not Found')
                                CenterID = None
                                CenterLat = None
                                CenterLon = None
                        else:
                            print('         MJC: Leg ' + ICAO + ' ' + proc_se.group(7) + ' ' + proc_se.group(11) + ' CenterID ' + proc_se.group(37) + ' Not Found')
                            CenterID = None
                            CenterLat = None
                            CenterLon = None
                else:
                    CenterID = None
                    CenterLat = None
                    CenterLon = None
                if proc_se.group(16)[1:2] == 'B':
                    IsFlyOver = 1
                elif proc_se.group(16)[1:2] == 'Y':
                    IsFlyOver = 1
                else:
                    IsFlyOver = 0
                if proc_se.group(35).strip() == '':
                    SpeedLimit = None
                else:
                    SpeedLimit = proc_se.group(35).strip()
                # Duplicated Procedure Will Not Add into NDB
                if Skip_Terminal == 0:
                    TerminalLegs_mjc.append((TerminalLegsID,TerminalID,Type,Transition,TrackCode,WptID,WptLat,WptLon,TurnDir,NavID,NavLat,NavLon,NavBear,NavDist,Course,Distance,Alt,Vnav,CenterID,CenterLat,CenterLon))
                    TerminalLegsEx_mjc.append((TerminalLegsID,IsFlyOver,SpeedLimit))
        # Convert list to Pandas DataFrame
        TerminalLegs_mjc_tb = pandas.DataFrame(TerminalLegs_mjc, columns=['ID','TerminalID','Type','Transition','TrackCode','WptID','WptLat','WptLon','TurnDir','NavID','NavLat','NavLon','NavBear','NavDist','Course','Distance','Alt','Vnav','CenterID','CenterLat','CenterLon'])
        TerminalLegsEx_mjc_tb = pandas.DataFrame(TerminalLegsEx_mjc, columns=['ID','IsFlyOver','SpeedLimit'])
        Terminals_mjc_tb = pandas.DataFrame(Terminals_mjc, columns=['ID','AirportID','Proc','ICAO','FullName','Name','Rwy','RwyID'])
        print(time.strftime("%H:%M:%S", time.localtime()) , 'MJC: Data Process Part #4 Finished')
    # Write Data
    if mjc_write:
        rom = sqlite3.connect(Cycle_Ident + '/MJC/nd.db3')
        rom_cur = rom.cursor()
        if mjc_create_blank:
            rom_cur.execute('DROP TABLE IF EXISTS AirportLookup;')
            rom_cur.execute('DROP TABLE IF EXISTS Airports;')
            rom_cur.execute('DROP TABLE IF EXISTS AirwayLegs;')
            rom_cur.execute('DROP TABLE IF EXISTS Airways;')
            rom_cur.execute('DROP TABLE IF EXISTS config;')
            rom_cur.execute('DROP TABLE IF EXISTS ILSes;')
            rom_cur.execute('DROP TABLE IF EXISTS NavaidLookup;')
            rom_cur.execute('DROP TABLE IF EXISTS Navaids;')
            rom_cur.execute('DROP TABLE IF EXISTS NavaidTypes;')
            rom_cur.execute('DROP TABLE IF EXISTS Runways;')
            rom_cur.execute('DROP TABLE IF EXISTS SurfaceTypes;')
            rom_cur.execute('DROP TABLE IF EXISTS TerminalLegs;')
            rom_cur.execute('DROP TABLE IF EXISTS TerminalLegsEx;')
            rom_cur.execute('DROP TABLE IF EXISTS Terminals;')
            rom_cur.execute('DROP TABLE IF EXISTS TrmLegTypes;')
            rom_cur.execute('DROP TABLE IF EXISTS WaypointLookup;')
            rom_cur.execute('DROP TABLE IF EXISTS Waypoints;')
            rom_cur.execute('CREATE TABLE "AirportLookup" ("extID" TEXT, "ID" INTEGER DEFAULT 0 REFERENCES "Airports" ("ID"), PRIMARY KEY("extID","ID"));')
            rom_cur.execute('CREATE TABLE "Airports" ("ID" INTEGER DEFAULT 0, "Name" TEXT, "ICAO" TEXT, "PrimaryID" INTEGER DEFAULT Null REFERENCES "Airports" ("ID"), "Latitude" DOUBLE DEFAULT 0, "Longtitude" DOUBLE DEFAULT 0, "Elevation" INTEGER DEFAULT 0, PRIMARY KEY("ID"));')
            rom_cur.execute('CREATE TABLE "AirwayLegs" ("ID" INTEGER DEFAULT 0, "AirwayID" INTEGER DEFAULT 0 REFERENCES "Airways" ("ID"), "Level" TEXT, "Waypoint1ID" INTEGER DEFAULT Null REFERENCES "Waypoints" ("ID"), "Waypoint2ID" INTEGER DEFAULT Null REFERENCES "Waypoints" ("ID"), "IsStart" BOOLEAN NOT NULL, "IsEnd" BOOLEAN NOT NULL, PRIMARY KEY("ID"));')
            rom_cur.execute('CREATE TABLE "Airways" ("ID" INTEGER DEFAULT 0, "Ident" TEXT NOT NULL, PRIMARY KEY("ID"));')
            rom_cur.execute('CREATE TABLE "config" ("key" TEXT, "val" TEXT, PRIMARY KEY("key"));')
            rom_cur.execute('CREATE TABLE "ILSes" ("ID" INTEGER NOT NULL DEFAULT 0, "RunwayID" INTEGER DEFAULT 0 REFERENCES "Runways" ("ID"), "Freq" INTEGER DEFAULT 0, "GsAngle" DOUBLE DEFAULT 0, "Latitude" DOUBLE DEFAULT 0, "Longtitude" DOUBLE DEFAULT 0, "Category" INTEGER DEFAULT 0, "Ident" TEXT, "LocCourse" DOUBLE DEFAULT 0, "CrossingHeight" INTEGER DEFAULT 0, "HasDme" BOOLEAN NOT NULL, "Elevation" INTEGER DEFAULT 0, "LatitudeGS" DOUBLE DEFAULT 0, "LongtitudeGS" DOUBLE DEFAULT 0, "LatitudeDME" DOUBLE DEFAULT 0, "LongtitudeDME" DOUBLE DEFAULT 0, PRIMARY KEY("ID"));')
            rom_cur.execute('CREATE TABLE "NavaidLookup" ("Ident" TEXT NOT NULL, "Type" INTEGER NOT NULL DEFAULT 0, "Country" TEXT NOT NULL, "NavKeyCode" INTEGER NOT NULL DEFAULT 0, "ID" INTEGER NOT NULL DEFAULT 0 REFERENCES "Navaids" ("ID"), PRIMARY KEY("Ident","Type","Country","NavKeyCode"));')
            rom_cur.execute('CREATE TABLE "Navaids" ("ID" INTEGER DEFAULT 0, "Ident" TEXT, "Type" INTEGER DEFAULT 0 REFERENCES "NavaidTypes" ("Type"), "Name" TEXT, "Freq" INTEGER, "Channel" TEXT, "Usage" TEXT, "Latitude" DOUBLE DEFAULT 0, "Longtitude" DOUBLE DEFAULT 0, "Elevation" INTEGER DEFAULT 0, "SlavedVar" DOUBLE DEFAULT 0, PRIMARY KEY("ID"));')
            rom_cur.execute('CREATE TABLE "NavaidTypes" ("Type" INTEGER DEFAULT 0, "Desc" TEXT, PRIMARY KEY("Type"));')
            rom_cur.execute('CREATE TABLE "Runways" ("ID" INTEGER DEFAULT 0, "AirportID" INTEGER DEFAULT 0 REFERENCES "Airports" ("ID"), "Ident" TEXT, "TrueHeading" DOUBLE DEFAULT 0, "Length" INTEGER DEFAULT 0, "Width" INTEGER DEFAULT 0, "Surface" TEXT REFERENCES "SurfaceTypes" ("SurfaceType"), "Latitude" DOUBLE DEFAULT 0, "Longtitude" DOUBLE DEFAULT 0, "Elevation" INTEGER DEFAULT 0, PRIMARY KEY("ID"));')
            rom_cur.execute('CREATE TABLE "SurfaceTypes" ("SurfaceType" TEXT, "Description" TEXT, PRIMARY KEY("SurfaceType"));')
            rom_cur.execute('CREATE TABLE "TerminalLegs" ("ID" INTEGER DEFAULT 0 REFERENCES "TerminalLegsEx" ("ID"), "TerminalID" INTEGER NOT NULL DEFAULT 0 REFERENCES "Terminals" ("ID"), "Type" TEXT, "Transition" TEXT, "TrackCode" TEXT REFERENCES "TrmLegTypes" ("Code"), "WptID" INTEGER DEFAULT 0 REFERENCES "Waypoints" ("ID"), "WptLat" DOUBLE DEFAULT 0, "WptLon" DOUBLE DEFAULT 0, "TurnDir" TEXT, "NavID" INTEGER DEFAULT 0 REFERENCES "Navaids" ("ID"), "NavLat" DOUBLE DEFAULT 0, "NavLon" DOUBLE DEFAULT 0, "NavBear" DOUBLE DEFAULT 0, "NavDist" DOUBLE DEFAULT 0, "Course" DOUBLE DEFAULT 0, "Distance" DOUBLE DEFAULT 0, "Alt" TEXT, "Vnav" DOUBLE DEFAULT 0, "CenterID" INTEGER DEFAULT 0, "CenterLat" DOUBLE DEFAULT 0, "CenterLon" DOUBLE DEFAULT 0, PRIMARY KEY("ID"));')
            rom_cur.execute('CREATE TABLE "TerminalLegsEx" ("ID" INTEGER DEFAULT 0, "IsFlyOver" BOOLEAN NOT NULL, "SpeedLimit" DOUBLE DEFAULT 0, PRIMARY KEY("ID"));')
            rom_cur.execute('CREATE TABLE "Terminals" ("ID" INTEGER DEFAULT 0, "AirportID" INTEGER DEFAULT 0 REFERENCES "Airports" ("ID"), "Proc" INTEGER DEFAULT 0, "ICAO" TEXT NOT NULL, "FullName" TEXT NOT NULL, "Name" TEXT NOT NULL, "Rwy" TEXT, "RwyID" INTEGER DEFAULT 0, PRIMARY KEY("ID"));')
            rom_cur.execute('CREATE TABLE "TrmLegTypes" ("Code" TEXT NOT NULL, "Description" TEXT, PRIMARY KEY("Code"));')
            rom_cur.execute('CREATE TABLE "WaypointLookup" ("Ident" TEXT NOT NULL, "Country" TEXT NOT NULL, "ID" INTEGER NOT NULL DEFAULT 0 REFERENCES "Waypoints" ("ID"), PRIMARY KEY("Ident","Country","ID"));')
            rom_cur.execute('CREATE TABLE "Waypoints" ("ID" INTEGER DEFAULT 0, "Ident" TEXT NOT NULL, "Collocated" BOOLEAN NOT NULL, "Name" TEXT, "Latitude" DOUBLE DEFAULT 0, "Longtitude" DOUBLE DEFAULT 0, "NavaidID" INTEGER DEFAULT Null REFERENCES "Navaids" ("ID"), PRIMARY KEY("ID"));')
            rom_cur.execute('INSERT INTO NavaidTypes VALUES(1, "VOR");')
            rom_cur.execute('INSERT INTO NavaidTypes VALUES(2, "VORTAC");')
            rom_cur.execute('INSERT INTO NavaidTypes VALUES(3, "TACAN");')
            rom_cur.execute('INSERT INTO NavaidTypes VALUES(4, "VOR-DME");')
            rom_cur.execute('INSERT INTO NavaidTypes VALUES(5, "NDB");')
            rom_cur.execute('INSERT INTO NavaidTypes VALUES(7, "NDB-DME");')
            rom_cur.execute('INSERT INTO NavaidTypes VALUES(8, "ILS-DME");')
            rom_cur.execute('INSERT INTO NavaidTypes VALUES(9, "DME (EXCLUDING ILS-DME)");')
            rom_cur.execute('INSERT INTO TrmLegTypes VALUES("AF", "Constant DME Arc to Fix");')
            rom_cur.execute('INSERT INTO TrmLegTypes VALUES("CA", "Course to an altitude (position unspecified)");')
            rom_cur.execute('INSERT INTO TrmLegTypes VALUES("CD", "Course to DME distance");')
            rom_cur.execute('INSERT INTO TrmLegTypes VALUES("CF", "Course to a fix");')
            rom_cur.execute('INSERT INTO TrmLegTypes VALUES("CI", "Course to next leg following by course oriented leg (interception point unspecified)");')
            rom_cur.execute('INSERT INTO TrmLegTypes VALUES("CR", "Course to a radial termination (intercept point unspecified)");')
            rom_cur.execute('INSERT INTO TrmLegTypes VALUES("DF", "Computed track direct to a fix");')
            rom_cur.execute('INSERT INTO TrmLegTypes VALUES("DS", "DISCONTINUITY");')
            rom_cur.execute('INSERT INTO TrmLegTypes VALUES("FA", "Course from a fix to an altitude");')
            rom_cur.execute('INSERT INTO TrmLegTypes VALUES("FC", "Course from a fix to a distance");')
            rom_cur.execute('INSERT INTO TrmLegTypes VALUES("FD", "Course from a fix to DME distance");')
            rom_cur.execute('INSERT INTO TrmLegTypes VALUES("FM", "Course from a fix to manual termination");')
            rom_cur.execute('INSERT INTO TrmLegTypes VALUES("HA", "Automatically at the fix after reaching an altitude");')
            rom_cur.execute('INSERT INTO TrmLegTypes VALUES("HF", "Automatically at the fix after one full circuit");')
            rom_cur.execute('INSERT INTO TrmLegTypes VALUES("HM", "Manually");')
            rom_cur.execute('INSERT INTO TrmLegTypes VALUES("IF", "Initial Fix");')
            rom_cur.execute('INSERT INTO TrmLegTypes VALUES("PI", "Procedure turn followed by a course to a fix (CF)");')
            rom_cur.execute('INSERT INTO TrmLegTypes VALUES("RF", "Constant radius to a fix");')
            rom_cur.execute('INSERT INTO TrmLegTypes VALUES("TF", "Track between two fixes (great circle)");')
            rom_cur.execute('INSERT INTO TrmLegTypes VALUES("VA", "heading to an altitude (position unspecified)");')
            rom_cur.execute('INSERT INTO TrmLegTypes VALUES("VD", "Heading to a DME Distance (position unspecified)");')
            rom_cur.execute('INSERT INTO TrmLegTypes VALUES("VI", "Heading to a next leg (position unspecified)");')
            rom_cur.execute('INSERT INTO TrmLegTypes VALUES("VM", "Heading to a manual termination");')
            rom_cur.execute('INSERT INTO TrmLegTypes VALUES("VR", "Heading to a radial termination");')
            rom_cur.execute('INSERT INTO SurfaceTypes VALUES("ASP", "ASPHALT, ASPHALTIC CONCRETE, TAR MACADAM, OR BITUMEN BOUND MACADAM (INCLUDING ANY OF THESE SURFACE TYPES WITH CONCRETE ENDS).");')
            rom_cur.execute('INSERT INTO SurfaceTypes VALUES("BIT", "BITUMINOUS, TAR OR ASPHALT MIXED IN PLACE, OILED.");')
            rom_cur.execute('INSERT INTO SurfaceTypes VALUES("BRI", "BRICK, LAID OR MORTARED.");')
            rom_cur.execute('INSERT INTO SurfaceTypes VALUES("CLA", "CLAY.");')
            rom_cur.execute('INSERT INTO SurfaceTypes VALUES("COM", "COMPOSITE, LESS THAN 50 PERCENT OF THE RUNWAY LENGTH IS PERMANENT.");')
            rom_cur.execute('INSERT INTO SurfaceTypes VALUES("CON", "CONCRETE.");')
            rom_cur.execute('INSERT INTO SurfaceTypes VALUES("COP", "COMPOSITE, 50 PERCENT OR MORE OF THE RUNWAY LENGTH IS PERMANENT.");')
            rom_cur.execute('INSERT INTO SurfaceTypes VALUES("COR", "CORAL.");')
            rom_cur.execute('INSERT INTO SurfaceTypes VALUES("GRE", "GRADED OR ROLLED EARTH, GRASS ON GRADED EARTH.");')
            rom_cur.execute('INSERT INTO SurfaceTypes VALUES("GRS", "GRASS OR EARTH NOT GRADED OR GRASS OR EARTH NOT GRADED OR ROLLED.");')
            rom_cur.execute('INSERT INTO SurfaceTypes VALUES("GVL", "GRAVEL.");')
            rom_cur.execute('INSERT INTO SurfaceTypes VALUES("ICE", "ICE.");')
            rom_cur.execute('INSERT INTO SurfaceTypes VALUES("LAT", "LATERITE.");')
            rom_cur.execute('INSERT INTO SurfaceTypes VALUES("MAC", "MACADAM - CRUSHED ROCK WATER BOUND.");')
            rom_cur.execute('INSERT INTO SurfaceTypes VALUES("MEM", "MEMBRANE - PLASTIC OR OTHER COATED FIBER MATERIAL.");')
            rom_cur.execute('INSERT INTO SurfaceTypes VALUES("MIX", "MIX IN PLACE USING NONBITUMIOUS BINDERS SUCH AS PORTLAND CEMENT.");')
            rom_cur.execute('INSERT INTO SurfaceTypes VALUES("PEM", "PART CONCRETE, PART ASPHALT, OR PART BITUMEN-BOUND MACADAM.");')
            rom_cur.execute('INSERT INTO SurfaceTypes VALUES("PER", "PERMANENT, SURFACE TYPE UNKNOWN.");')
            rom_cur.execute('INSERT INTO SurfaceTypes VALUES("PSP", "PIECED STEEL PLANKING.");')
            rom_cur.execute('INSERT INTO SurfaceTypes VALUES("SAN", "SAND, GRADED, ROLLED OR OILED.");')
            rom_cur.execute('INSERT INTO SurfaceTypes VALUES("SNO", "SNOW.");')
            rom_cur.execute('INSERT INTO SurfaceTypes VALUES("U", "SURFACE UNKNOWN.");')
            rom_cur.execute('INSERT INTO SurfaceTypes VALUES("WAT", "WATER");')
            config_mjc = [['CycleEndDate',CycleEndDate],['CycleName',CycleName],['CycleStartDate',CycleStartDate]]
            config_mjc_tb = pandas.DataFrame(config_mjc,columns=['key','val'])
            config_mjc_tb.to_sql('config', rom, if_exists='append', index=False)
        if mjc_write:
            Airports_mjc_tb.to_sql('Airports', rom, if_exists='append', index=False)
            AirportLookup_mjc_tb.to_sql('AirportLookup', rom, if_exists='append', index=False)
            Runways_mjc_tb.to_sql('Runways', rom, if_exists='append', index=False)
            Navaids_mjc_tb.to_sql('Navaids', rom, if_exists='append', index=False)
            NavaidLookup_mjc_tb.to_sql('NavaidLookup', rom, if_exists='append', index=False)
            Waypoints_mjc_tb.to_sql('Waypoints', rom, if_exists='append', index=False)
            WaypointLookup_mjc_tb = WaypointLookup_mjc_tb.drop(columns=['REGION_CODE'])
            WaypointLookup_mjc_tb.to_sql('WaypointLookup', rom, if_exists='append', index=False)
            Airways_mjc_tb.to_sql('Airways', rom, if_exists='append', index=False)
            AirwayLegs_mjc_tb.to_sql('AirwayLegs', rom, if_exists='append', index=False)
            ILSes_mjc_tb = ILSes_mjc_tb.drop(columns=['AirportID'])
            ILSes_mjc_tb.to_sql('ILSes', rom, if_exists='append', index=False)
            Terminals_mjc_tb.to_sql('Terminals', rom, if_exists='append', index=False)
            TerminalLegsEx_mjc_tb.to_sql('TerminalLegsEx', rom, if_exists='append', index=False)
            TerminalLegs_mjc_tb.to_sql('TerminalLegs', rom, if_exists='append', index=False)
        rom_cur.close()
        rom.commit()
        rom.close()
    print(time.strftime("%H:%M:%S", time.localtime()) , 'MJC: Data Export Finished')

# ProSim (Unfinish)
if ProSim:
    print(time.strftime("%H:%M:%S", time.localtime()) , 'ProSim: Start Processing DataFile')
    os.makedirs(Cycle_Ident + '/ProSim')
    # Set Variable of ProSim
    if 1:
        AirportLookup_prosim = []
        Airports_prosim = []
        AirwayLegs_prosim = []
        AirwayLegs_reversed_prosim = []
        AirwayLookup_prosim = []
        Airways_prosim = []
        ILSes_prosim = []
        NavaidLookup_prosim = []
        Navaids_prosim = []
        Runways_prosim = []
        TerminalLegs_prosim = []
        TerminalLegsEx_prosim = []
        Terminals_prosim = []
        WaypointLookup_prosim = []
        Waypoints_prosim = []
        AirwayIdent = ''
        TerminalName = ''
        if add_tailored:
            AirportID = 15000
            AirwayID = 10000
            AirwayLegsID = 200000
            ILSID = 5000
            NavaidID = 15000
            RunwayID = 40000
            TerminalID = 100000
            TerminalLegsID = 800000
            WaypointID = 300000
        else:
            AirportID = 0
            AirwayID = 0
            AirwayLegsID = 0
            ILSID = 0
            NavaidID = 0
            RunwayID = 0
            TerminalID = 0
            TerminalLegsID = 0
            WaypointID = 0
    # Load ProSim's nd.db3
    if prosim_create_blank != True:
        shutil.copy2('Input/ProSim/nd.db3', Cycle_Ident + '/ProSim/nd.db3')
        rom = sqlite3.connect(Cycle_Ident + '/ProSim/nd.db3')
        rom_cur = rom.cursor()
        if 1:
            Airports_in_prosim_nd_tb = pandas.read_sql_query('SELECT * FROM Airports', con = rom)
            Airways_in_prosim_nd_tb = pandas.read_sql_query('SELECT * FROM Airways', con = rom)
            NavaidLookup_in_prosim_nd_tb = pandas.read_sql_query('SELECT * FROM NavaidLookup', con = rom)
            Navaids_in_prosim_nd_tb = pandas.read_sql_query('SELECT * FROM Navaids', con = rom)
            Runways_in_prosim_nd_tb = pandas.read_sql_query('SELECT * FROM Runways', con = rom)
            WaypointLookup_in_prosim_nd_tb = pandas.read_sql_query('SELECT * FROM WaypointLookup', con = rom)
            Waypoints_in_prosim_nd_tb = pandas.read_sql_query('SELECT * FROM Waypoints', con = rom)
            TerminalWpts_in_prosim_nd_tb = pandas.read_sql_query('SELECT TerminalID, WptID, WptLat, WptLon FROM TerminalLegs', con = rom)
            Terminals_in_prosim_nd_tb = pandas.read_sql_query('SELECT * FROM Terminals', con = rom)
            ILSes_in_prosim_nd_tb = pandas.read_sql_query('SELECT * FROM ILSes', con = rom)
        rom_cur.close()
        rom.close()
        if 1:
            ILSes_in_prosim_nd_tb['AirportID'] = ILSes_in_prosim_nd_tb['RunwayID'].apply(lambda RunwayID : Runways_in_prosim_nd_tb.loc[Runways_in_prosim_nd_tb.loc[(Runways_in_prosim_nd_tb['ID'] == RunwayID)].index.tolist() , 'AirportID'].values[0])
            TerminalWpts_in_prosim_nd_tb.drop_duplicates(subset=['WptID'], keep='first', inplace=True)
            TerminalWpts_in_prosim_nd_tb = TerminalWpts_in_prosim_nd_tb.dropna()
            TerminalWpts_in_prosim_nd_tb['WptIdent'] = TerminalWpts_in_prosim_nd_tb['WptID'].apply(lambda WptID : WaypointLookup_in_prosim_nd_tb.loc[WaypointLookup_in_prosim_nd_tb.loc[(WaypointLookup_in_prosim_nd_tb['ID'] == WptID)].index.tolist() , 'Ident'].values[0])
            TerminalWpts_in_prosim_nd_tb['REGION_CODE'] = TerminalWpts_in_prosim_nd_tb['TerminalID'].apply(lambda TerminalID : Terminals_in_prosim_nd_tb.loc[Terminals_in_prosim_nd_tb.loc[(Terminals_in_prosim_nd_tb['ID'] == TerminalID)].index.tolist() , 'ICAO'].values[0])
        print(time.strftime("%H:%M:%S", time.localtime()) , 'ProSim: nd.db3 Load Completed')
    # Process of ProSim Part #1
    with open('Input/' + Datafile, 'r', encoding='ISO-8859-1') as ARINC424:
        for line in ARINC424: 
            vhf_se = re.search(vor, line)
            db_se = re.search(db, line)
            ea_se = re.search(ea, line)
            awy_se = re.search(er, line)
            arpt_se = re.search(pa, line)
            pc_se = re.search(pc, line)
            rwy_se = re.search(pg, line)
            pn_se = re.search(pn, line)
            if vhf_se:
                ARPT_IDENT = vhf_se.group(5).strip()
                NAV_IDENT = vhf_se.group(7).strip()
                WAYPOINT_IDENT = NAV_IDENT
                NAV_ICAO_CODE = vhf_se.group(8).strip()
                WAYPOINT_ICAO_CODE = NAV_ICAO_CODE
                NAV_FREQ = int('0x' + str(int(vhf_se.group(10))*100) , 16)
                if vhf_se.group(11)[0:1] == "V":
                    if vhf_se.group(11)[1:2] == " ":
                        NAVAID_TYPE = 1
                    elif vhf_se.group(11)[1:2] == "T":
                        NAVAID_TYPE = 2
                    elif vhf_se.group(11)[1:2] == "M":
                        NAVAID_TYPE = 2
                    elif vhf_se.group(11)[1:2] == "D":
                        NAVAID_TYPE = 4
                    else:
                        NAVAID_TYPE = 1
                else:
                    if vhf_se.group(11)[1:2] == "T":
                        NAVAID_TYPE = 3
                    elif vhf_se.group(11)[1:2] == "M":
                        NAVAID_TYPE = 3
                    elif vhf_se.group(11)[1:2] == "I":
                        NAVAID_TYPE = 8          
                    elif vhf_se.group(11)[1:2] == "D":
                        NAVAID_TYPE = 9
                    else:
                        NAVAID_TYPE = 9
                if vhf_se.group(11)[2:3] == "U":
                    NAVAID_CLASS = "B"
                    NAV_RANGE = 125
                else :
                    NAVAID_CLASS = vhf_se.group(11)[2:3]
                    if NAVAID_CLASS == 'H':
                        NAV_RANGE = 130
                    elif NAVAID_CLASS == 'L':
                        NAV_RANGE = 40
                    else:
                        NAV_RANGE = 25
                if vhf_se.group(12).strip() != '':
                    if vhf_se.group(12) == "N":
                        NAV_LAT = round(float(vhf_se.group(13)) + float(vhf_se.group(14))/60 + float(vhf_se.group(15))/3600 + float(vhf_se.group(16))/360000, 8)
                    else:
                        NAV_LAT = 0 - round(float(vhf_se.group(13)) - float(vhf_se.group(14))/60 - float(vhf_se.group(15))/3600 - float(vhf_se.group(16))/360000, 8)
                else:
                    if vhf_se.group(23) == "N":
                        NAV_LAT = round(float(vhf_se.group(24)) + float(vhf_se.group(25))/60 + float(vhf_se.group(26))/3600 + float(vhf_se.group(27))/360000, 8)
                    else:
                        NAV_LAT = 0 - round(float(vhf_se.group(24)) - float(vhf_se.group(25))/60 - float(vhf_se.group(26))/3600 - float(vhf_se.group(27))/360000, 8)
                WAYPOINT_LAT = NAV_LAT
                if vhf_se.group(17).strip() != '':
                    if vhf_se.group(17) == "E":
                        NAV_LON = round(float(vhf_se.group(18)) + float(vhf_se.group(19))/60 + float(vhf_se.group(20))/3600 + float(vhf_se.group(21))/360000, 8)
                    else:
                        NAV_LON = 0 - round(float(vhf_se.group(18)) - float(vhf_se.group(19))/60 - float(vhf_se.group(20))/3600 - float(vhf_se.group(21))/360000, 8)
                else:
                    if vhf_se.group(28) == "E":
                        NAV_LON = round(float(vhf_se.group(29)) + float(vhf_se.group(30))/60 + float(vhf_se.group(31))/3600 + float(vhf_se.group(32))/360000, 8)
                    else:
                        NAV_LON = 0 - round(float(vhf_se.group(29)) - float(vhf_se.group(30))/60 - float(vhf_se.group(31))/3600 - float(vhf_se.group(32))/360000, 8)
                WAYPOINT_LON = NAV_LON
                if vhf_se.group(33).strip() == '':
                    MAG_VAR = 0
                elif vhf_se.group(33).strip(string.digits) == "E":
                    MAG_VAR = float(vhf_se.group(33).strip('EW'))/10
                else:
                    MAG_VAR = 0 - float(vhf_se.group(33).strip('EW'))/10
                if vhf_se.group(34).strip() == '':
                    ELEV = 0
                else:
                    ELEV = vhf_se.group(34)
                NAV_NAME = vhf_se.group(39).replace('VOR/DME','').rstrip().replace('(','').replace(')','')
                WAYPOINT_NAME_DESCR = NAV_NAME
                NavaidID = NavaidID + 1
                WaypointID = WaypointID + 1
                NavaidLookup_prosim.append((NAV_IDENT,NAVAID_TYPE,NAV_ICAO_CODE,1,NavaidID))
                Navaids_prosim.append((NavaidID,NAV_IDENT,NAVAID_TYPE,NAV_NAME,NAV_FREQ,None,NAVAID_CLASS,NAV_LAT,NAV_LON,ELEV,MAG_VAR,MAG_VAR,NAV_RANGE))
                WaypointLookup_prosim.append((WAYPOINT_IDENT,WAYPOINT_ICAO_CODE,WaypointID,ARPT_IDENT))
                Waypoints_prosim.append((WaypointID,WAYPOINT_IDENT,1,WAYPOINT_NAME_DESCR,WAYPOINT_LAT,WAYPOINT_LON,int(NavaidID)))
            if db_se:
                ARPT_IDENT = db_se.group(5).strip()
                NAV_IDENT = db_se.group(7).strip()
                WAYPOINT_IDENT = NAV_IDENT
                NAV_ICAO_CODE = db_se.group(8).strip()
                WAYPOINT_ICAO_CODE = NAV_ICAO_CODE
                NAV_FREQ = int('0x' + str(int(db_se.group(10))*1000) , 16)
                NAVAID_TYPE = 5
                if db_se.group(11)[2:3] == "L":
                    NAVAID_CLASS = "T"
                    NAV_RANGE = 25
                elif db_se.group(11)[2:3] == "M":
                    NAVAID_CLASS = "L"
                    NAV_RANGE = 50
                else :
                    NAVAID_CLASS = "H"
                    NAV_RANGE = 75
                if db_se.group(12) == "N":
                    NAV_LAT = round(float(db_se.group(13)) + float(db_se.group(14))/60 + float(db_se.group(15))/3600 + float(db_se.group(16))/360000, 8)
                else:
                    NAV_LAT = 0 - round(float(db_se.group(13)) - float(db_se.group(14))/60 - float(db_se.group(15))/3600 - float(db_se.group(16))/360000, 8)
                WAYPOINT_LAT = NAV_LAT
                if db_se.group(17) == "E":
                    NAV_LON = round(float(db_se.group(18)) + float(db_se.group(19))/60 + float(db_se.group(20))/3600 + float(db_se.group(21))/360000, 8)
                else:
                    NAV_LON = 0 - round(float(db_se.group(18)) - float(db_se.group(19))/60 - float(db_se.group(20))/3600 - float(db_se.group(21))/360000, 8)
                WAYPOINT_LON = NAV_LON
                if db_se.group(22) == "E":
                    MAG_VAR = float(db_se.group(23))/10
                else:
                    MAG_VAR = 0 - float(db_se.group(23))/10
                ELEV = 0
                NAV_NAME = db_se.group(25).replace('NDB','').rstrip().replace('(','').replace(')','')
                WAYPOINT_NAME_DESCR = NAV_NAME
                NavaidID = NavaidID + 1
                WaypointID = WaypointID + 1
                NavaidLookup_prosim.append((NAV_IDENT,NAVAID_TYPE,NAV_ICAO_CODE,1,NavaidID))
                Navaids_prosim.append((NavaidID,NAV_IDENT,NAVAID_TYPE,NAV_NAME,NAV_FREQ,None,NAVAID_CLASS,NAV_LAT,NAV_LON,ELEV,MAG_VAR,MAG_VAR,NAV_RANGE))
                WaypointLookup_prosim.append((WAYPOINT_IDENT,WAYPOINT_ICAO_CODE,WaypointID,ARPT_IDENT))
                Waypoints_prosim.append((WaypointID,WAYPOINT_IDENT,1,WAYPOINT_NAME_DESCR,WAYPOINT_LAT,WAYPOINT_LON,int(NavaidID)))
            if ea_se:
                REGION_CODE = ea_se.group(5)
                WAYPOINT_IDENT = ea_se.group(8).strip()
                WAYPOINT_ICAO_CODE = ea_se.group(9).strip()
                if ea_se.group(13) == "N":
                    WAYPOINT_LAT = round(float(ea_se.group(14)) + float(ea_se.group(15))/60 + float(ea_se.group(16))/3600 + float(ea_se.group(17))/360000, 8)
                else:
                    WAYPOINT_LAT = round(0 - float(ea_se.group(14)) - float(ea_se.group(15))/60 - float(ea_se.group(16))/3600 - float(ea_se.group(17))/360000, 8)
                if ea_se.group(18) == "E":
                    WAYPOINT_LON = round(float(ea_se.group(19)) + float(ea_se.group(20))/60 + float(ea_se.group(21))/3600 + float(ea_se.group(22))/360000, 8)
                else:
                    WAYPOINT_LON = round(0 - float(ea_se.group(19)) - float(ea_se.group(20))/60 - float(ea_se.group(21))/3600 - float(ea_se.group(22))/360000, 8)
                WAYPOINT_NAME_DESCR = ea_se.group(26).rstrip().replace('(','').replace(')','')
                WaypointID = WaypointID + 1
                WaypointLookup_prosim.append((WAYPOINT_IDENT,WAYPOINT_ICAO_CODE,WaypointID,REGION_CODE))
                Waypoints_prosim.append((WaypointID,WAYPOINT_IDENT,0,WAYPOINT_NAME_DESCR,WAYPOINT_LAT,WAYPOINT_LON,None))
            if awy_se:
                if awy_se.group(5).strip() != AirwayIdent:
                    if prosim_create_blank != True:
                        if Airways_in_prosim_nd_tb.loc[(Airways_in_prosim_nd_tb['Ident'] == awy_se.group(5).strip())].index.tolist() == []:
                            AirwayID = AirwayID + 1
                            AirwayIdent = awy_se.group(5).strip()
                            Airways_prosim.append((AirwayID, AirwayIdent))
                    else:
                        AirwayID = AirwayID + 1
                        AirwayIdent = awy_se.group(5).strip()
                        Airways_prosim.append((AirwayID, AirwayIdent))
            if arpt_se:
                ARPT_IDENT = arpt_se.group(4)
                ARPT_ICAO_CODE = arpt_se.group(5)
                if arpt_se.group(13) == "N":
                    ARPT_LAT = round(float(arpt_se.group(14)) + float(arpt_se.group(15))/60 + float(arpt_se.group(16))/3600 + float(arpt_se.group(17))/360000, 8)
                else:
                    ARPT_LAT = round(0 - float(arpt_se.group(14)) - float(arpt_se.group(15))/60 - float(arpt_se.group(16))/3600 - float(arpt_se.group(17))/360000, 8)
                if arpt_se.group(18) == "E":
                    ARPT_LON = round(float(arpt_se.group(19)) + float(arpt_se.group(20))/60 + float(arpt_se.group(21))/3600 + float(arpt_se.group(22))/360000, 8)
                else:
                    ARPT_LON = round(0 - float(arpt_se.group(19)) - float(arpt_se.group(20))/60 - float(arpt_se.group(21))/3600 - float(arpt_se.group(22))/360000, 8)
                ARPT_ELEV = int(arpt_se.group(25))
                ARPT_NAME = arpt_se.group(36).rstrip().replace('(','').replace(')','')
                if arpt_se.group(23) == "E":
                    ARPT_MAG_VAR = float(arpt_se.group(24))/10
                else:
                    ARPT_MAG_VAR = 0 - float(arpt_se.group(24))/10
                if arpt_se.group(29).strip() == '':
                    TRANSITIONS_ALT = None
                else:
                    TRANSITIONS_ALT = arpt_se.group(29)
                if arpt_se.group(30).strip() == '':
                    TRANSITION_LEVEL = None
                else:
                    TRANSITION_LEVEL = arpt_se.group(30)
                AirportID = AirportID + 1
                Airports_prosim.append((AirportID,ARPT_NAME,ARPT_IDENT,None,ARPT_LAT,ARPT_LON,ARPT_ELEV,TRANSITIONS_ALT,TRANSITION_LEVEL))
                AirportLookup_prosim.append((ARPT_ICAO_CODE + ARPT_IDENT,AirportID))
            if pc_se:
                REGION_CODE = pc_se.group(5)
                WAYPOINT_IDENT = pc_se.group(8).strip()
                WAYPOINT_ICAO_CODE = pc_se.group(9).strip()
                if pc_se.group(13) == "N":
                    WAYPOINT_LAT = round(float(pc_se.group(14)) + float(pc_se.group(15))/60 + float(pc_se.group(16))/3600 + float(pc_se.group(17))/360000, 8)
                else:
                    WAYPOINT_LAT = round(0 - float(pc_se.group(14)) - float(pc_se.group(15))/60 - float(pc_se.group(16))/3600 - float(pc_se.group(17))/360000, 8)
                if pc_se.group(18) == "E":
                    WAYPOINT_LON = round(float(pc_se.group(19)) + float(pc_se.group(20))/60 + float(pc_se.group(21))/3600 + float(pc_se.group(22))/360000, 8)
                else:
                    WAYPOINT_LON = round(0 - float(pc_se.group(19)) - float(pc_se.group(20))/60 - float(pc_se.group(21))/3600 - float(pc_se.group(22))/360000, 8)
                WAYPOINT_NAME_DESCR = pc_se.group(26).rstrip().replace('(','').replace(')','')
                WaypointID = WaypointID + 1
                WaypointLookup_prosim.append((WAYPOINT_IDENT,WAYPOINT_ICAO_CODE,WaypointID,REGION_CODE))
                Waypoints_prosim.append((WaypointID,WAYPOINT_IDENT,0,WAYPOINT_NAME_DESCR,WAYPOINT_LAT,WAYPOINT_LON,None))
            if rwy_se:
                RunwayID = RunwayID + 1
                if rwy_se.group(4) != ARPT_IDENT:
                    print(rwy_se.group(4) + " Not Found")
                    AirportID = None
                Ident = rwy_se.group(7)[2:5].strip()
                if rwy_se.group(10).lstrip(string.digits) == "T":
                    TrueHeading = rwy_se.group(10).rstrip('T')
                else:
                    TrueHeading = float(rwy_se.group(10))/10 + ARPT_MAG_VAR
                Length = rwy_se.group(9)
                Width = rwy_se.group(27)
                if rwy_se.group(11) == "N":
                    Latitude = round(float(rwy_se.group(12)) + float(rwy_se.group(13))/60 + float(rwy_se.group(14))/3600 + float(rwy_se.group(15))/360000, 8)
                else:
                    Latitude = round(0 - float(rwy_se.group(12)) - float(rwy_se.group(13))/60 - float(rwy_se.group(14))/3600 - float(rwy_se.group(15))/360000, 8)
                if rwy_se.group(16) == "E":
                    Longtitude = round(float(rwy_se.group(17)) + float(rwy_se.group(18))/60 + float(rwy_se.group(19))/3600 + float(rwy_se.group(20))/360000, 8)
                else:
                    Longtitude = round(0 - float(rwy_se.group(17)) - float(rwy_se.group(18))/60 - float(rwy_se.group(19))/3600 - float(rwy_se.group(20))/360000, 8)
                Elevation = rwy_se.group(24).lstrip('0')
                Runways_prosim.append((RunwayID,AirportID,Ident,TrueHeading,Length,Width,'U  ',Latitude,Longtitude,Elevation))
            if pn_se:
                ARPT_IDENT = pn_se.group(5).strip()
                NAV_IDENT = pn_se.group(7).strip()
                WAYPOINT_IDENT = NAV_IDENT
                NAV_ICAO_CODE = pn_se.group(8).strip()
                WAYPOINT_ICAO_CODE = NAV_ICAO_CODE
                NAV_FREQ = int('0x' + str(int(pn_se.group(10))*1000) , 16)
                NAVAID_TYPE = 5
                if pn_se.group(11)[2:3] == "L":
                    NAVAID_CLASS = "T"
                    NAV_RANGE = 25
                elif pn_se.group(11)[2:3] == "M":
                    NAVAID_CLASS = "L"
                    NAV_RANGE = 50
                else :
                    NAVAID_CLASS = "H"
                    NAV_RANGE = 75
                if pn_se.group(12) == "N":
                    NAV_LAT = round(float(pn_se.group(13)) + float(pn_se.group(14))/60 + float(pn_se.group(15))/3600 + float(pn_se.group(16))/360000, 8)
                else:
                    NAV_LAT = 0 - round(float(pn_se.group(13)) - float(pn_se.group(14))/60 - float(pn_se.group(15))/3600 - float(pn_se.group(16))/360000, 8)
                WAYPOINT_LAT = NAV_LAT
                if pn_se.group(17) == "E":
                    NAV_LON = round(float(pn_se.group(18)) + float(pn_se.group(19))/60 + float(pn_se.group(20))/3600 + float(pn_se.group(21))/360000, 8)
                else:
                    NAV_LON = 0 - round(float(pn_se.group(18)) - float(pn_se.group(19))/60 - float(pn_se.group(20))/3600 - float(pn_se.group(21))/360000, 8)
                WAYPOINT_LON = NAV_LON
                if pn_se.group(22) == "E":
                    MAG_VAR = float(pn_se.group(23))/10
                else:
                    MAG_VAR = 0 - float(pn_se.group(23))/10
                NAV_NAME = pn_se.group(25).replace('NDB','').rstrip().replace('(','').replace(')','')
                WAYPOINT_NAME_DESCR = NAV_NAME
                NavaidID = NavaidID + 1
                WaypointID = WaypointID + 1
                NavaidLookup_prosim.append((NAV_IDENT,NAVAID_TYPE,NAV_ICAO_CODE,1,NavaidID))
                Navaids_prosim.append((NavaidID,NAV_IDENT,NAVAID_TYPE,NAV_NAME,NAV_FREQ,None,NAVAID_CLASS,NAV_LAT,NAV_LON,ELEV,MAG_VAR,MAG_VAR,NAV_RANGE))
                WaypointLookup_prosim.append((WAYPOINT_IDENT,WAYPOINT_ICAO_CODE,WaypointID,ARPT_IDENT))
                Waypoints_prosim.append((WaypointID,WAYPOINT_IDENT,1,WAYPOINT_NAME_DESCR,WAYPOINT_LAT,WAYPOINT_LON,int(NavaidID)))
        # Convert list to Pandas DataFrame
        Airports_prosim_tb = pandas.DataFrame(Airports_prosim, columns=['ID','Name','ICAO','PrimaryID','Latitude','Longtitude','Elevation','TransitionAltitude','TransitionLevel'])
        AirportLookup_prosim_tb = pandas.DataFrame(AirportLookup_prosim, columns=['extID','ID'])
        Navaids_prosim_tb = pandas.DataFrame(Navaids_prosim, columns=['ID','Ident','Type','Name','Freq','Channel','Usage','Latitude','Longtitude','Elevation','SlavedVar','MagneticVariation','Range'])
        NavaidLookup_prosim_tb = pandas.DataFrame(NavaidLookup_prosim, columns=['Ident','Type','Country','NavKeyCode','ID'])
        Waypoints_prosim_tb = pandas.DataFrame(Waypoints_prosim, columns=['ID','Ident','Collocated','Name','Latitude','Longtitude','NavaidID'])
        WaypointLookup_prosim_tb = pandas.DataFrame(WaypointLookup_prosim, columns=['Ident','Country','ID','REGION_CODE'])
        Runways_prosim_tb = pandas.DataFrame(Runways_prosim, columns=['ID','AirportID','Ident','TrueHeading','Length','Width','Surface','Latitude','Longtitude','Elevation'])
        # Count Duplicate Navaids (NavKeyCode)
        if prosim_create_blank != True:
            NavaidLookup_prosim_tb = pandas.concat([NavaidLookup_in_prosim_nd_tb,NavaidLookup_prosim_tb])
            NavaidLookup_prosim_tb['NavKeyCode'] = NavaidLookup_prosim_tb.groupby(['Ident','Country'], as_index=False)[['Ident','Country']].cumcount() + 1
            NavaidLookup_prosim_tb = NavaidLookup_prosim_tb[NavaidLookup_prosim_tb['ID'] > 15000]
        else:  
            NavaidLookup_prosim_tb['NavKeyCode'] = NavaidLookup_prosim_tb.groupby(['Ident','Country'], as_index=False)[['Ident','Country']].cumcount() + 1
        # Make Airway List
        Airways_prosim_tb = pandas.DataFrame(Airways_prosim, columns=['ID','Ident'])
        Airways_prosim_tb = Airways_prosim_tb.sort_values('Ident')
        Airways_prosim_tb.drop_duplicates(subset=['Ident'], keep='first', inplace=True)
        Airways_prosim_tb = Airways_prosim_tb.reset_index(drop=True)
        if add_tailored:
            Airways_prosim_tb['ID'] = Airways_prosim_tb.index + 10001
        else:
            Airways_prosim_tb['ID'] = Airways_prosim_tb.index + 1
        AirwayIdent = ''
        print(time.strftime("%H:%M:%S", time.localtime()) , 'ProSim: Data Process Part #1 Finished')
    # Process of ProSim Part #2
    with open('Input/' + Datafile, 'r', encoding='ISO-8859-1') as ARINC424:
        for line in ARINC424:
            awy_se = re.search(er, line)
            loc_se = re.search(pi, line)
            # AWY of ProSim (Part #1)
            if awy_se:
                CUST_AREA = awy_se.group(2)
                ROUTE_IDENT = awy_se.group(5).strip()
                AirwayID = Airways_prosim_tb.loc[(Airways_prosim_tb['Ident'] == ROUTE_IDENT)].index.tolist()
                if AirwayID != []:
                    AirwayID = Airways_prosim_tb.at[AirwayID[0], 'ID']
                else:
                    AirwayID = Airways_in_prosim_nd_tb.loc[(Airways_in_prosim_nd_tb['Ident'] == ROUTE_IDENT)].index.tolist()
                    if prosim_create_blank != True:
                        if AirwayID != []:
                            AirwayID = Airways_in_prosim_nd_tb.at[AirwayID[0], 'ID']
                        else:
                            print('         ProSim: Airway '+ awy_se.group(5) + ' Not Found')
                            AirwayID = None
                    else:
                        print('         ProSim: Airway '+ awy_se.group(5) + ' Not Found')
                        AirwayID = None
                SEQ_NR = awy_se.group(6)
                FIX_IDENT = awy_se.group(7).strip()
                FIX_ICAO_CODE = awy_se.group(8).strip()
                FIX_ID = WaypointLookup_prosim_tb.loc[(WaypointLookup_prosim_tb['Ident'] == FIX_IDENT) & (WaypointLookup_prosim_tb['Country'] == FIX_ICAO_CODE)].index.tolist()
                if FIX_ID != []:
                    FIX_ID = WaypointLookup_prosim_tb.at[FIX_ID[0], 'ID']
                else:
                    if prosim_create_blank != True:
                        FIX_ID = WaypointLookup_in_prosim_nd_tb.loc[(WaypointLookup_in_prosim_nd_tb['Ident'] == FIX_IDENT) & (WaypointLookup_in_prosim_nd_tb['Country'] == FIX_ICAO_CODE)].index.tolist()
                        if FIX_ID != []:
                            FIX_ID = WaypointLookup_in_prosim_nd_tb.at[FIX_ID[0], 'ID']
                        else:
                            print('         ProSim: Airway ' + awy_se.group(5) + ' Fix ' + awy_se.group(7) + ' Not Found')
                            FIX_ID = '0'
                    else:
                        print('         ProSim: Airway ' + awy_se.group(5) + ' Fix ' + awy_se.group(7) + ' Not Found')
                        FIX_ID = '0'
                EndOfRoute = awy_se.group(12)[1:2]
                LEVEL = awy_se.group(15)
                DIRECTION_RESTRICTION = awy_se.group(16)
                AirwayLookup_prosim.append((CUST_AREA,ROUTE_IDENT,AirwayID,SEQ_NR,FIX_IDENT,FIX_ICAO_CODE,FIX_ID,EndOfRoute,LEVEL,DIRECTION_RESTRICTION))
            # ILS of ProSim
            if loc_se:
                ICAO = loc_se.group(4)
                if Airports_prosim_tb.loc[(Airports_prosim_tb['ICAO'] == ICAO)].index.tolist() != []:
                    AirportID = Airports_prosim_tb.loc[Airports_prosim_tb.loc[(Airports_prosim_tb['ICAO'] == ICAO)].index.tolist() , 'ID'].values[0]
                else:
                    if prosim_create_blank != True:
                        if Airports_in_prosim_nd_tb.loc[(Airports_in_prosim_nd_tb['ICAO'] == ICAO)].index.tolist() != []:
                            AirportID = Airports_in_prosim_nd_tb.loc[Airports_in_prosim_nd_tb.loc[(Airports_in_prosim_nd_tb['ICAO'] == ICAO)].index.tolist() , 'ID'].values[0]
                        else:
                            print('         ProSim: Airport ' + ICAO + ' ID Not Found')
                            AirportID = None
                    else:
                        print('         ProSim: Airport ' + ICAO + ' ID Not Found')
                        AirportID = None
                if loc_se.group(11)[0:2] != 'RW':
                    Rwy = None
                    RwyID = None
                    print('         ProSim: Runway ' + ICAO + ' ' + loc_se.group(11) + ' Ident Not Correct')
                else:
                    Rwy = loc_se.group(11).strip().replace('RW','')
                    if Runways_prosim_tb.loc[(Runways_prosim_tb['AirportID'] == AirportID) & (Runways_prosim_tb['Ident'] == Rwy)].index.tolist() != []:
                        RwyID = Runways_prosim_tb.loc[Runways_prosim_tb.loc[(Runways_prosim_tb['AirportID'] == AirportID) & (Runways_prosim_tb['Ident'] == Rwy)].index.tolist() , 'ID'].values[0]
                    else:
                        if prosim_create_blank != True:
                            if Runways_in_prosim_nd_tb.loc[(Runways_in_prosim_nd_tb['AirportID'] == AirportID) & (Runways_in_prosim_nd_tb['Ident'] == Rwy)].index.tolist() != []:
                                RwyID = Runways_in_prosim_nd_tb.loc[Runways_in_prosim_nd_tb.loc[(Runways_in_prosim_nd_tb['AirportID'] == AirportID) & (Runways_in_prosim_nd_tb['Ident'] == Rwy)].index.tolist() , 'ID'].values[0]
                            else:
                                print('         ProSim: Runway ' + ICAO + ' ' + Rwy + ' ID Not Found')
                                RwyID = None
                        else:
                            print('         ProSim: Runway ' + ICAO + ' ' + Rwy + ' ID Not Found')
                            RwyID = None
                NAV_FREQ = int('0x' + str(int(loc_se.group(10))*100) , 16)
                if loc_se.group(37).strip() == '':
                    GLIDE_SLOPE_ANGLE = 3
                else:
                    GLIDE_SLOPE_ANGLE = float(loc_se.group(37))/100
                if loc_se.group(12).strip() == '':
                    LOCALIZER_LAT = None
                elif loc_se.group(12) == "N":
                    LOCALIZER_LAT = round(float(loc_se.group(13)) + float(loc_se.group(14))/60 + float(loc_se.group(15))/3600 + float(loc_se.group(16))/360000, 8)
                else:
                    LOCALIZER_LAT = 0 - round(float(loc_se.group(13)) - float(loc_se.group(14))/60 - float(loc_se.group(15))/3600 - float(loc_se.group(16))/360000, 8)
                if loc_se.group(17).strip() == '':
                    LOCALIZER_LON = None
                elif loc_se.group(17) == "E":
                    LOCALIZER_LON = round(float(loc_se.group(18)) + float(loc_se.group(19))/60 + float(loc_se.group(20))/3600 + float(loc_se.group(21))/360000, 8)
                else:
                    LOCALIZER_LON = 0 - round(float(loc_se.group(18)) - float(loc_se.group(19))/60 - float(loc_se.group(20))/3600 - float(loc_se.group(21))/360000, 8)
                if loc_se.group(8).strip(string.digits) == '':
                    ILS_CATEGORY = loc_se.group(8)
                else:
                    ILS_CATEGORY = 0
                LOCALIZER_IDENT = loc_se.group(7).strip()
                if loc_se.group(22).lstrip(string.digits) == "T":
                    LOCALIZER_BEARING = float(Decimal(loc_se.group(22).rstrip('T')))
                else:
                    LOCALIZER_BEARING = float(Decimal(loc_se.group(22)[0:3] + '.' + loc_se.group(22)[3:]).quantize(Decimal('0'), rounding=ROUND_HALF_UP))
                if loc_se.group(39).strip() == '':
                    GLIDE_SLOPE_HEIGHT_AT_LANDING_THLD = 50
                else:
                    GLIDE_SLOPE_HEIGHT_AT_LANDING_THLD = loc_se.group(39)
                if loc_se.group(40).strip() == '':
                    GLIDE_SLOPE_ELEV = 0
                else:
                    GLIDE_SLOPE_ELEV = loc_se.group(40)
                DME_ID = WaypointLookup_prosim_tb.loc[(WaypointLookup_prosim_tb['Ident'] == loc_se.group(7).strip()) & (WaypointLookup_prosim_tb['REGION_CODE'] == ICAO)].index.tolist()
                if DME_ID != []:
                    HasDme = 1
                    DME_ID = WaypointLookup_prosim_tb.at[DME_ID[0], 'ID']
                else:
                    if prosim_create_blank != True:
                        DME_ID = WaypointLookup_in_prosim_nd_tb.loc[(WaypointLookup_in_prosim_nd_tb['Ident'] == loc_se.group(7).strip()) & (WaypointLookup_in_prosim_nd_tb['Country'] == loc_se.group(5))].index.tolist()
                        if DME_ID != []:
                            HasDme = 1
                            DME_ID = Waypoints_in_prosim_nd_tb.at[DME_ID[0], 'ID']
                        else:
                            HasDme = 0
                    else:
                        HasDme = 0
                ILSID = ILSID + 1
                ILSes_prosim.append((ILSID,AirportID,RwyID,NAV_FREQ,GLIDE_SLOPE_ANGLE,LOCALIZER_LAT,LOCALIZER_LON,ILS_CATEGORY,LOCALIZER_IDENT,LOCALIZER_BEARING,GLIDE_SLOPE_HEIGHT_AT_LANDING_THLD,HasDme,GLIDE_SLOPE_ELEV))
        # Convert list to Pandas DataFrame
        AirwayLookup_prosim_tb = pandas.DataFrame(AirwayLookup_prosim, columns=['CUST_AREA','ROUTE_IDENT','AirwayID','SEQ_NR','FIX_IDENT','FIX_ICAO_CODE','FIX_ID','EndOfRoute','LEVEL','DIRECTION_RESTRICTION'])
        AirwayLookup_prosim_tb = AirwayLookup_prosim_tb.sort_values(['ROUTE_IDENT','SEQ_NR','EndOfRoute'], ascending=[1,1,0])
        AirwayLookup_prosim_tb = AirwayLookup_prosim_tb.reset_index(drop=True)
        AirwayLookup_prosim = []
        ILSes_prosim_tb = pandas.DataFrame(ILSes_prosim, columns=['ID','AirportID','RunwayID','Freq','GsAngle','Latitude','Longtitude','Category','Ident','LocCourse','CrossingHeight','HasDme','Elevation'])
        print(time.strftime("%H:%M:%S", time.localtime()) , 'ProSim: Data Process Part #2 Finished')
    # Process of ProSim Part #3 (AWY Part #2)
    if len(AirwayLookup_prosim_tb.index) != 0:
        for index, row in AirwayLookup_prosim_tb.iterrows():
            if index + 1 != len(AirwayLookup_prosim_tb.index):
                if row['FIX_ID'] != AirwayLookup_prosim_tb.at[index + 1, 'FIX_ID']:
                    AirwayLookup_prosim.append((row['CUST_AREA'],row['ROUTE_IDENT'],row['AirwayID'],row['SEQ_NR'],row['FIX_IDENT'],row['FIX_ICAO_CODE'],row['FIX_ID'],row['EndOfRoute'],row['LEVEL'],row['DIRECTION_RESTRICTION']))
                else:
                    if row['AirwayID'] != AirwayLookup_prosim_tb.at[index + 1, 'AirwayID']:
                        AirwayLookup_prosim.append((row['CUST_AREA'],row['ROUTE_IDENT'],row['AirwayID'],row['SEQ_NR'],row['FIX_IDENT'],row['FIX_ICAO_CODE'],row['FIX_ID'],row['EndOfRoute'],row['LEVEL'],row['DIRECTION_RESTRICTION']))
            else:
                AirwayLookup_prosim.append((row['CUST_AREA'],row['ROUTE_IDENT'],row['AirwayID'],row['SEQ_NR'],row['FIX_IDENT'],row['FIX_ICAO_CODE'],row['FIX_ID'],row['EndOfRoute'],row['LEVEL'],row['DIRECTION_RESTRICTION']))
        AirwayLookup_prosim_tb = pandas.DataFrame(AirwayLookup_prosim, columns=['CUST_AREA','ROUTE_IDENT','AirwayID','SEQ_NR','FIX_IDENT','FIX_ICAO_CODE','FIX_ID','EndOfRoute','LEVEL','DIRECTION_RESTRICTION'])
        AirwayLookup_prosim_tb = AirwayLookup_prosim_tb.sort_values(['ROUTE_IDENT','SEQ_NR'])
        AirwayLookup_prosim_tb = AirwayLookup_prosim_tb.reset_index(drop=True)
        AirwayLookup_reversed_prosim_tb = AirwayLookup_prosim_tb.sort_index(ascending=False)
        AirwayLookup_reversed_prosim_tb = AirwayLookup_reversed_prosim_tb.reset_index(drop=True)
        for index, row in AirwayLookup_prosim_tb.iterrows():
            if index + 1 != len(AirwayLookup_prosim_tb.index):
                Waypoint1ID = row['FIX_ID']
                Waypoint2ID = AirwayLookup_prosim_tb.at[index + 1, 'FIX_ID']
                LEVEL = row['LEVEL']
                if row['ROUTE_IDENT'] == AirwayIdent:
                    if WasEnd == 0:
                        IsStart = 0
                    else:
                        IsStart = 1
                elif row['DIRECTION_RESTRICTION'] == 'B':
                    IsStart = 0
                else:
                    IsStart = 1
                    AirwayID = row['AirwayID']
                    AirwayIdent = row['ROUTE_IDENT']
                if index + 2  == len(AirwayLookup_prosim_tb.index):
                    IsEnd = 1
                elif AirwayLookup_prosim_tb.at[index + 1, 'DIRECTION_RESTRICTION'] == 'B':
                    IsEnd = 1
                elif AirwayLookup_prosim_tb.at[index + 2, 'ROUTE_IDENT'] != row['ROUTE_IDENT']:
                    IsEnd = 1
                elif AirwayLookup_prosim_tb.at[index + 1, 'EndOfRoute'] == 'E':
                    IsEnd = 1
                else:
                    IsEnd = 0
                if AirwayLookup_prosim_tb.at[index + 1, 'ROUTE_IDENT'] == row['ROUTE_IDENT']:
                    if row['DIRECTION_RESTRICTION'] != 'B':
                        if row['EndOfRoute'] != 'E':
                            AirwayLegsID = AirwayLegsID + 1
                            AirwayLegs_prosim.append((AirwayLegsID,AirwayID,LEVEL,Waypoint1ID,Waypoint2ID,IsStart,IsEnd))
                            WasEnd = IsEnd
        for index, row in AirwayLookup_reversed_prosim_tb.iterrows():
            if index + 1 != len(AirwayLookup_reversed_prosim_tb.index):
                Waypoint1ID = row['FIX_ID']
                Waypoint2ID = AirwayLookup_reversed_prosim_tb.at[index + 1, 'FIX_ID']
                LEVEL = row['LEVEL']
                if row['ROUTE_IDENT'] == AirwayIdent:
                    if row['EndOfRoute'] == 'E':
                        IsStart = 1
                    elif WasEnd == 1:
                        IsStart = 1
                    else:
                        IsStart = 0
                elif AirwayLookup_reversed_prosim_tb.at[index + 1, 'DIRECTION_RESTRICTION'] != 'F':
                    IsStart = 1
                    AirwayID = row['AirwayID']
                    AirwayIdent = row['ROUTE_IDENT']
                else:
                    IsStart = 0
                if index + 2  == len(AirwayLookup_reversed_prosim_tb.index):
                    IsEnd = 1
                elif AirwayLookup_reversed_prosim_tb.at[index + 2, 'DIRECTION_RESTRICTION'] == 'F':
                    IsEnd = 1
                elif AirwayLookup_reversed_prosim_tb.at[index + 2, 'ROUTE_IDENT'] != row['ROUTE_IDENT']:
                    IsEnd = 1
                elif AirwayLookup_reversed_prosim_tb.at[index + 2, 'EndOfRoute'] == 'E':
                    IsEnd = 1
                else:
                    IsEnd = 0
                if AirwayLookup_reversed_prosim_tb.at[index + 1, 'ROUTE_IDENT'] == row['ROUTE_IDENT']:
                    if AirwayLookup_reversed_prosim_tb.at[index + 1, 'DIRECTION_RESTRICTION'] != 'F':
                        if AirwayLookup_reversed_prosim_tb.at[index + 1, 'EndOfRoute'] != 'E':
                            AirwayLegsID = AirwayLegsID + 1
                            AirwayLegs_reversed_prosim.append((AirwayLegsID,AirwayID,LEVEL,Waypoint1ID,Waypoint2ID,IsStart,IsEnd))
                            WasEnd = IsEnd
        # Convert list to Pandas DataFrame
        AirwayLegs_prosim_tb = pandas.DataFrame(AirwayLegs_prosim, columns=['ID','AirwayID','Level','Waypoint1ID','Waypoint2ID','IsStart','IsEnd'])
        AirwayLegs_reversed_prosim_tb = pandas.DataFrame(AirwayLegs_reversed_prosim, columns=['ID','AirwayID','Level','Waypoint1ID','Waypoint2ID','IsStart','IsEnd'])
        AirwayLegs_prosim_tb = pandas.concat([AirwayLegs_prosim_tb, AirwayLegs_reversed_prosim_tb], axis=0, ignore_index=True)
        AirwayLegs_prosim_tb = AirwayLegs_prosim_tb.sort_values(['AirwayID','ID'])
        AirwayLegs_prosim_tb = AirwayLegs_prosim_tb.reset_index(drop=True)
        if add_tailored:
            AirwayLegs_prosim_tb['ID'] = AirwayLegs_prosim_tb.index + 200001
        else:
            AirwayLegs_prosim_tb['ID'] = AirwayLegs_prosim_tb.index + 1
        print(time.strftime("%H:%M:%S", time.localtime()) , 'ProSim: Data Process Part #3 Finished')
    # Process of ProSim Part #4
    with open('Input/' + Datafile, 'r', encoding='ISO-8859-1') as ARINC424:
        for line in ARINC424:
            proc_se = re.search(pdef, line)
            # PROC of ProSim
            if proc_se:
                ICAO = proc_se.group(4)
                Type = proc_se.group(8)
                if proc_se.group(4) + proc_se.group(7).strip() != ICAO + TerminalName:
                    TerminalID = TerminalID + 1
                    AirportID = Airports_prosim_tb.loc[(Airports_prosim_tb['ICAO'] == ICAO)].index.tolist()
                    if AirportID != []:
                        AirportID = Airports_prosim_tb.at[AirportID[0], 'ID']
                    else:
                        if prosim_create_blank != True:
                            AirportID = Airports_in_prosim_nd_tb.loc[(Airports_in_prosim_nd_tb['ICAO'] == ICAO)].index.tolist()
                            if AirportID != []:
                                AirportID = Airports_in_prosim_nd_tb.at[AirportID[0], 'ID']
                            else:
                                print('         ProSim: Airport ' + ICAO + ' ID Not Found')
                                AirportID = None
                        else:
                            print('         ProSim: Airport ' + ICAO + ' ID Not Found')
                            AirportID = None
                    if proc_se.group(6) == 'D':
                        Proc = '2'
                    elif proc_se.group(6) == 'E':
                        Proc = '1'
                    else:
                        Proc = '3'
                    if Proc == '1' or Proc == '2':
                        if Type == '2' or Type == '5':
                            if proc_se.group(9)[0:2] + proc_se.group(9)[4:5].strip('LRCT ') == 'RW':
                                Rwy = proc_se.group(9).strip().replace('RW','')
                                RwyID = Runways_prosim_tb.loc[(Runways_prosim_tb['AirportID'] == AirportID) & (Runways_prosim_tb['Ident'] == Rwy)].index.tolist()
                                if RwyID != []:
                                    RwyID = Runways_prosim_tb.at[RwyID[0], 'ID']
                                else:
                                    if tfdi_create_blank != True:
                                        RwyID = Runways_in_prosim_nd_tb.loc[(Runways_in_prosim_nd_tb['AirportID'] == AirportID) & (Runways_in_prosim_nd_tb['Ident'] == Rwy)].index.tolist()
                                        if RwyID != []:
                                            RwyID = Runways_in_prosim_nd_tb.at[RwyID[0], 'ID']
                                        else:
                                            print('         ProSim: Procedure ' + ICAO + ' '  + proc_se.group(7) + ' Runway ' + Rwy + ' Not Found')
                                            RwyID = None
                                    else:
                                        print('         ProSim: Procedure ' + ICAO + ' '  + proc_se.group(7) + ' Runway ' + Rwy + ' Not Found')
                                        RwyID = None
                            else:
                                Rwy = None
                                RwyID = None
                        else:
                            Rwy = None
                            RwyID = None
                        Name = proc_se.group(7).strip()
                        FullName = Name
                        TerminalName = Name
                    else:
                        if proc_se.group(7)[1:3].strip('- ').strip(string.digits) == '':
                            Rwy = proc_se.group(7)[1:4].strip('- ')
                            RwyID = Runways_prosim_tb.loc[(Runways_prosim_tb['AirportID'] == AirportID) & (Runways_prosim_tb['Ident'] == Rwy)].index.tolist()
                            if RwyID != []:
                                RwyID = Runways_prosim_tb.at[RwyID[0], 'ID']
                            else:
                                if tfdi_create_blank != True:
                                    RwyID = Runways_in_prosim_nd_tb.loc[(Runways_in_prosim_nd_tb['AirportID'] == AirportID) & (Runways_in_prosim_nd_tb['Ident'] == Rwy)].index.tolist()
                                    if RwyID != []:
                                        RwyID = Runways_in_prosim_nd_tb.at[RwyID[0], 'ID']
                                    else:
                                        print('         ProSim: Runway ' + ICAO + ' ' + Rwy + ' ID Not Found')
                                        RwyID = None
                                else:
                                    print('         ProSim: Runway ' + ICAO + ' ' + Rwy + ' ID Not Found')
                                    RwyID = None
                        else:
                            Rwy = None
                            RwyID = None
                        Name = proc_se.group(7).strip()
                        FullName = proc_se.group(7) + ' APCH'
                        TerminalName = Name
                    # Duplicated Procedure Will Not Add into NDB
                    if prosim_create_blank != True:
                        if Terminals_in_prosim_nd_tb.loc[(Terminals_in_prosim_nd_tb['ICAO'] == ICAO) & (Terminals_in_prosim_nd_tb['Name'] == Name)].index.tolist() == []:
                            Terminals_prosim.append((TerminalID,AirportID,Proc,ICAO,FullName,Name,Rwy,RwyID,None))
                            Skip_Terminal = 0
                        else:
                            print('         ProSim: Procedure ' + ICAO + ' ' + proc_se.group(7) + ' Skipped Due Duplicate')
                            Skip_Terminal = 1
                    else:
                        Terminals_prosim.append((TerminalID,AirportID,Proc,ICAO,FullName,Name,Rwy,RwyID,None))
                        Skip_Terminal = 0
                TerminalLegsID = TerminalLegsID + 1
                if proc_se.group(9).strip() == '':
                    Transition = None
                elif proc_se.group(9)[0:2] + proc_se.group(9)[4:5] == 'RWB':
                    Transition = 'ALL'
                else:
                    Transition = proc_se.group(9).strip()
                TrackCode = proc_se.group(19)
                if proc_se.group(13) + proc_se.group(14) == "  ":
                    WptLat = None
                    WptLon = None
                    WptID = None
                elif proc_se.group(13) + proc_se.group(14) == "PA":
                    WptID = Airports_prosim_tb.loc[(Airports_prosim_tb['ICAO'] == ICAO)].index.tolist()
                    if WptID != []:
                        WptLat = Airports_prosim_tb.at[WptID[0], 'Latitude']
                        WptLon = Airports_prosim_tb.at[WptID[0], 'Longtitude']
                        WptID = None
                    else:
                        if prosim_create_blank != True:
                            WptID = Airports_in_prosim_nd_tb.loc[(Airports_in_prosim_nd_tb['ICAO'] == ICAO)].index.tolist()
                            if WptID != []:
                                WptLat = Airports_in_prosim_nd_tb.at[WptID[0], 'Latitude']
                                WptLon = Airports_in_prosim_nd_tb.at[WptID[0], 'Longtitude']
                                WptID = None
                            else:
                                print('         ProSim: Leg ' + ICAO + ' ' + proc_se.group(7) + ' Airport LAT/LON Not Found')
                                WptLat = None
                                WptLon = None
                                WptID = None
                        else:
                            print('         ProSim: Leg ' + ICAO + ' ' + proc_se.group(7) + ' Airport LAT/LON Not Found')
                            WptLat = None
                            WptLon = None
                            WptID = None
                elif proc_se.group(13) + proc_se.group(14) == "PG":
                    WptID = Runways_prosim_tb.loc[(Runways_prosim_tb['AirportID'] == AirportID) & (Runways_prosim_tb['Ident'] == proc_se.group(11).strip().replace('RW',''))].index.tolist()
                    if WptID != []:
                        WptLat = Runways_prosim_tb.at[WptID[0], 'Latitude']
                        WptLon = Runways_prosim_tb.at[WptID[0], 'Longtitude']
                        WptID = None
                    else:
                        if prosim_create_blank != True:
                            WptID = Runways_in_prosim_nd_tb.loc[(Runways_in_prosim_nd_tb['AirportID'] == AirportID) & (Runways_in_prosim_nd_tb['Ident'] == proc_se.group(11).strip().replace('RW',''))].index.tolist()
                            if WptID != []:
                                WptLat = Runways_in_prosim_nd_tb.at[WptID[0], 'Latitude']
                                WptLon = Runways_in_prosim_nd_tb.at[WptID[0], 'Longtitude']
                                WptID = None
                            else:
                                print('         ProSim: Leg ' + ICAO + ' ' + proc_se.group(7) + ' ' + Rwy + ' Rwy LAT/LON Not Found')
                                WptLat = None
                                WptLon = None
                                WptID = None
                        else:
                            print('         ProSim: Leg ' + ICAO + ' ' + proc_se.group(7) + ' ' + Rwy + ' Rwy LAT/LON Not Found')
                            WptLat = None
                            WptLon = None
                            WptID = None
                elif proc_se.group(13) + proc_se.group(14) == "PC":
                    WptID = WaypointLookup_prosim_tb.loc[(WaypointLookup_prosim_tb['Ident'] == proc_se.group(11).strip()) & (WaypointLookup_prosim_tb['Country'] == proc_se.group(12)) & (WaypointLookup_prosim_tb['REGION_CODE'] == ICAO)].index.tolist()
                    if WptID != []:
                        WptID = WaypointLookup_prosim_tb.at[WptID[0], 'ID']
                        WptLat = Waypoints_prosim_tb.loc[Waypoints_prosim_tb.loc[(Waypoints_prosim_tb['ID'] == WptID)].index.tolist() , 'Latitude'].values[0]
                        WptLon = Waypoints_prosim_tb.loc[Waypoints_prosim_tb.loc[(Waypoints_prosim_tb['ID'] == WptID)].index.tolist() , 'Longtitude'].values[0]
                    else:
                        if prosim_create_blank != True:
                            WptID = TerminalWpts_in_prosim_nd_tb.loc[(TerminalWpts_in_prosim_nd_tb['WptIdent'] == proc_se.group(11).strip()) & (TerminalWpts_in_prosim_nd_tb['REGION_CODE'] == ICAO)].index.tolist()
                            if WptID != []:
                                WptID = TerminalWpts_in_prosim_nd_tb.at[WptID[0], 'WptID']
                                WptLat = Waypoints_in_prosim_nd_tb.loc[Waypoints_in_prosim_nd_tb.loc[(Waypoints_in_prosim_nd_tb['ID'] == WptID)].index.tolist() , 'Latitude'].values[0]
                                WptLon = Waypoints_in_prosim_nd_tb.loc[Waypoints_in_prosim_nd_tb.loc[(Waypoints_in_prosim_nd_tb['ID'] == WptID)].index.tolist() , 'Longtitude'].values[0]
                            else:
                                WptID = WaypointLookup_in_prosim_nd_tb.loc[(WaypointLookup_in_prosim_nd_tb['Ident'] == proc_se.group(11).strip()) & (WaypointLookup_in_prosim_nd_tb['Country'] == proc_se.group(12))].index.tolist()
                                if WptID != []:
                                    WptID = WaypointLookup_in_prosim_nd_tb.at[WptID[0], 'ID']
                                    WptLat = Waypoints_in_prosim_nd_tb.loc[Waypoints_in_prosim_nd_tb.loc[(Waypoints_in_prosim_nd_tb['ID'] == WptID)].index.tolist() , 'Latitude'].values[0]
                                    WptLon = Waypoints_in_prosim_nd_tb.loc[Waypoints_in_prosim_nd_tb.loc[(Waypoints_in_prosim_nd_tb['ID'] == WptID)].index.tolist() , 'Longtitude'].values[0]
                                else:
                                    print('         ProSim: Leg ' + ICAO + ' ' + proc_se.group(7) + ' ' + proc_se.group(11) + ' WptID Not Found')
                                    WptID = None
                                    WptLat = None
                                    WptLon = None
                        else:
                            print('         ProSim: Leg ' + ICAO + ' ' + proc_se.group(7) + ' ' + proc_se.group(11) + ' WptID Not Found')
                            WptID = None
                            WptLat = None
                            WptLon = None
                else:
                    WptID = WaypointLookup_prosim_tb.loc[(WaypointLookup_prosim_tb['Ident'] == proc_se.group(11).strip()) & (WaypointLookup_prosim_tb['Country'] == proc_se.group(12))].index.tolist()
                    if WptID != []:
                        WptID = WaypointLookup_prosim_tb.at[WptID[0], 'ID']
                        WptLat = Waypoints_prosim_tb.loc[Waypoints_prosim_tb.loc[(Waypoints_prosim_tb['ID'] == WptID)].index.tolist() , 'Latitude'].values[0]
                        WptLon = Waypoints_prosim_tb.loc[Waypoints_prosim_tb.loc[(Waypoints_prosim_tb['ID'] == WptID)].index.tolist() , 'Longtitude'].values[0]
                    else:
                        if prosim_create_blank != True:
                            WptID = WaypointLookup_in_prosim_nd_tb.loc[(WaypointLookup_in_prosim_nd_tb['Ident'] == proc_se.group(11).strip()) & (WaypointLookup_in_prosim_nd_tb['Country'] == proc_se.group(12))].index.tolist()
                            if WptID != []:
                                WptID = WaypointLookup_in_prosim_nd_tb.at[WptID[0], 'ID']
                                WptLat = Waypoints_in_prosim_nd_tb.loc[Waypoints_in_prosim_nd_tb.loc[(Waypoints_in_prosim_nd_tb['ID'] == WptID)].index.tolist() , 'Latitude'].values[0]
                                WptLon = Waypoints_in_prosim_nd_tb.loc[Waypoints_in_prosim_nd_tb.loc[(Waypoints_in_prosim_nd_tb['ID'] == WptID)].index.tolist() , 'Longtitude'].values[0]
                            else:
                                print('         ProSim: Leg ' + ICAO + ' ' + proc_se.group(7) + ' ' + proc_se.group(11) + ' WptID Not Found')
                                WptID = None
                                WptLat = None
                                WptLon = None
                        else:
                            print('         ProSim: Leg ' + ICAO + ' ' + proc_se.group(7) + ' ' + proc_se.group(11) + ' WptID Not Found')
                            WptID = None
                            WptLat = None
                            WptLon = None
                if proc_se.group(17).strip() == '':
                    TurnDir = None
                else:
                    TurnDir = proc_se.group(17)
                if proc_se.group(28) + proc_se.group(29) == '  ':
                    NavLat = None
                    NavLon = None
                    NavID = None
                elif proc_se.group(28) + proc_se.group(29) == 'PI':
                    ILSIdent = proc_se.group(21).strip()
                    NavID = ILSes_prosim_tb.loc[(ILSes_prosim_tb['AirportID'] == AirportID) & (ILSes_prosim_tb['Ident'] == ILSIdent)].index.tolist()
                    if NavID != []:
                        NavLat = ILSes_prosim_tb.at[NavID[0], 'Latitude']
                        NavLon = ILSes_prosim_tb.at[NavID[0], 'Longtitude']
                        NavID = None
                    else:
                        if prosim_create_blank != True:
                            NavID = ILSes_in_prosim_nd_tb.loc[(ILSes_in_prosim_nd_tb['AirportID'] == AirportID) & (ILSes_in_prosim_nd_tb['Ident'] == ILSIdent)].index.tolist()
                            if NavID != []:
                                NavLat = ILSes_in_prosim_nd_tb.at[NavID[0], 'Latitude']
                                NavLon = ILSes_in_prosim_nd_tb.at[NavID[0], 'Longtitude']
                                NavID = None
                            else:
                                print('         ProSim: Leg ' + ICAO + ' ' + proc_se.group(7) + ' ' + proc_se.group(11) + ' Recommend ILS ' + proc_se.group(21) + ' Not Found')
                                NavLat = None
                                NavLon = None
                                NavID = None
                        else:
                            print('         ProSim: Leg ' + ICAO + ' ' + proc_se.group(7) + ' ' + proc_se.group(11) + ' Recommend ILS ' + proc_se.group(21) + ' Not Found')
                            NavLat = None
                            NavLon = None
                            NavID = None
                elif proc_se.group(28) + proc_se.group(29) == 'PT':
                    NavLat = None
                    NavLon = None
                    NavID = None
                else:
                    NavID = NavaidLookup_prosim_tb.loc[(NavaidLookup_prosim_tb['Ident'] == proc_se.group(21).strip()) & (NavaidLookup_prosim_tb['Country'] == proc_se.group(22))].index.tolist()
                    if NavID != []:
                        NavID = NavaidLookup_prosim_tb.at[NavID[0], 'ID']
                        NavLat = Navaids_prosim_tb.loc[Navaids_prosim_tb.loc[(Navaids_prosim_tb['ID'] == NavID)].index.tolist() , 'Latitude'].values[0]
                        NavLon = Navaids_prosim_tb.loc[Navaids_prosim_tb.loc[(Navaids_prosim_tb['ID'] == NavID)].index.tolist() , 'Longtitude'].values[0]
                    else:
                        if prosim_create_blank != True:
                            NavID = NavaidLookup_in_prosim_nd_tb.loc[(NavaidLookup_in_prosim_nd_tb['Ident'] == proc_se.group(21).strip()) & (NavaidLookup_in_prosim_nd_tb['Country'] == proc_se.group(22))].index.tolist()
                            if NavID != []:
                                NavID = NavaidLookup_in_prosim_nd_tb.at[NavID[0], 'ID']
                                NavLat = Navaids_in_prosim_nd_tb.loc[Navaids_in_prosim_nd_tb.loc[(Navaids_in_prosim_nd_tb['ID'] == NavID)].index.tolist() , 'Latitude'].values[0]
                                NavLon = Navaids_in_prosim_nd_tb.loc[Navaids_in_prosim_nd_tb.loc[(Navaids_in_prosim_nd_tb['ID'] == NavID)].index.tolist() , 'Longtitude'].values[0]
                            else:
                                print('         ProSim: Leg ' + ICAO + ' ' + proc_se.group(7) + ' ' + proc_se.group(11) + ' Recommend NavID ' + proc_se.group(21) + ' Not Found')
                                NavID = None
                                NavLat = None
                                NavLon = None
                        else:
                            print('         ProSim: Leg ' + ICAO + ' ' + proc_se.group(7) + ' ' + proc_se.group(11) + ' Recommend NavID ' + proc_se.group(21) + ' Not Found')
                            NavID = None
                            NavLat = None
                            NavLon = None
                if proc_se.group(24).strip() == '':
                    NavBear = None
                else:
                    NavBear = float(proc_se.group(24))/10
                if proc_se.group(25).strip() == '':
                    NavDist = None
                else:
                    NavDist = float(proc_se.group(25))/10
                if proc_se.group(26).strip() == '':
                    Course = None
                elif proc_se.group(26).lstrip(string.digits) == "T":
                    Course = proc_se.group(26).rstrip('T')
                else:
                    Course = float(proc_se.group(26))/10
                if proc_se.group(27).strip() == '':
                    Distance = None
                elif proc_se.group(27).rstrip(string.digits) == "T":
                    Distance = float(proc_se.group(27).lstrip('T'))/2
                else:
                    Distance = float(proc_se.group(27))/10
                if proc_se.group(32).strip() == '':
                    ALT_1 = None
                elif proc_se.group(32).rstrip(string.digits) == "FL":
                    ALT_1 = proc_se.group(32).lstrip(string.ascii_uppercase) + "00"
                else:
                    ALT_1 = proc_se.group(32)
                if proc_se.group(33).strip() == '':
                    ALT_2 = None
                elif proc_se.group(33).rstrip(string.digits) == "FL":
                    ALT_2 = proc_se.group(33).lstrip(string.ascii_uppercase) + "00"
                else:
                    ALT_2 = proc_se.group(33)
                if proc_se.group(30).strip() == '+':
                    Alt = ALT_1 + 'A'
                elif proc_se.group(30).strip() == '-':
                    Alt = ALT_1 + 'B'
                elif proc_se.group(30).strip() == 'G':
                    Alt = ALT_2 + ''
                elif proc_se.group(30).strip() == 'H':
                    Alt = ALT_2 + 'A'
                elif proc_se.group(30).strip() == 'I':
                    Alt = ALT_1 + ''
                elif proc_se.group(30).strip() == 'J':
                    Alt = ALT_1 + 'A'
                else:
                    if proc_se.group(16)[3:4] == 'M':
                        Alt = 'MAP'
                    elif ALT_1 != None:
                        Alt = ALT_1 + ''
                    else:
                        Alt = None
                if proc_se.group(36).strip() == '':
                    Vnav = None
                else:
                    Vnav = round(float(proc_se.group(36)[1:4])/100, 2)
                if TrackCode == 'RF':
                    CenterID = WaypointLookup_prosim_tb.loc[(WaypointLookup_prosim_tb['Ident'] == proc_se.group(37).strip()) & (WaypointLookup_prosim_tb['Country'] == proc_se.group(39))].index.tolist()
                    if CenterID != []:
                        CenterID = WaypointLookup_prosim_tb.at[CenterID[0], 'ID']
                        CenterLat = Waypoints_prosim_tb.loc[Waypoints_prosim_tb.loc[(Waypoints_prosim_tb['ID'] == CenterID)].index.tolist() , 'Latitude'].values[0]
                        CenterLon = Waypoints_prosim_tb.loc[Waypoints_prosim_tb.loc[(Waypoints_prosim_tb['ID'] == CenterID)].index.tolist() , 'Longtitude'].values[0]
                    else:
                        if prosim_create_blank != True:
                            CenterID = WaypointLookup_in_prosim_nd_tb.loc[(WaypointLookup_in_prosim_nd_tb['Ident'] == proc_se.group(37).strip()) & (WaypointLookup_in_prosim_nd_tb['Country'] == proc_se.group(39))].index.tolist()
                            if CenterID != []:
                                CenterID = WaypointLookup_in_prosim_nd_tb.at[CenterID[0], 'ID']
                                CenterLat = Waypoints_in_prosim_nd_tb.loc[Waypoints_in_prosim_nd_tb.loc[(Waypoints_in_prosim_nd_tb['ID'] == CenterID)].index.tolist() , 'Latitude'].values[0]
                                CenterLon = Waypoints_in_prosim_nd_tb.loc[Waypoints_in_prosim_nd_tb.loc[(Waypoints_in_prosim_nd_tb['ID'] == CenterID)].index.tolist() , 'Longtitude'].values[0]
                            else:
                                print('         ProSim: Leg ' + ICAO + ' ' + proc_se.group(7) + ' ' + proc_se.group(11) + ' CenterID ' + proc_se.group(37) + ' Not Found')
                                CenterID = None
                                CenterLat = None
                                CenterLon = None
                        else:
                            print('         ProSim: Leg ' + ICAO + ' ' + proc_se.group(7) + ' ' + proc_se.group(11) + ' CenterID ' + proc_se.group(37) + ' Not Found')
                            CenterID = None
                            CenterLat = None
                            CenterLon = None
                else:
                    CenterID = None
                    CenterLat = None
                    CenterLon = None
                if proc_se.group(16)[1:2] == 'B':
                    IsFlyOver = 1
                elif proc_se.group(16)[1:2] == 'Y':
                    IsFlyOver = 1
                else:
                    IsFlyOver = 0
                if proc_se.group(35).strip() == '':
                    SpeedLimit = None
                else:
                    SpeedLimit = proc_se.group(35).strip()
                # Duplicated Procedure Will Not Add into NDB
                if Skip_Terminal == 0:
                    TerminalLegs_prosim.append((TerminalLegsID,TerminalID,Type,Transition,TrackCode,WptID,WptLat,WptLon,TurnDir,NavID,NavLat,NavLon,NavBear,NavDist,Course,Distance,Alt,Vnav,CenterID,CenterLat,CenterLon))
                    TerminalLegsEx_prosim.append((TerminalLegsID,IsFlyOver,SpeedLimit))
        # Convert list to Pandas DataFrame
        TerminalLegs_prosim_tb = pandas.DataFrame(TerminalLegs_prosim, columns=['ID','TerminalID','Type','Transition','TrackCode','WptID','WptLat','WptLon','TurnDir','NavID','NavLat','NavLon','NavBear','NavDist','Course','Distance','Alt','Vnav','CenterID','CenterLat','CenterLon'])
        TerminalLegsEx_prosim_tb = pandas.DataFrame(TerminalLegsEx_prosim, columns=['ID','IsFlyOver','SpeedLimit'])
        Terminals_prosim_tb = pandas.DataFrame(Terminals_prosim, columns=['ID','AirportID','Proc','ICAO','FullName','Name','Rwy','RwyID','IlsID'])
        print(time.strftime("%H:%M:%S", time.localtime()) , 'ProSim: Data Process Part #4 Finished')
    # Write Data
    if prosim_write:
        rom = sqlite3.connect(Cycle_Ident + '/ProSim/nd.db3')
        rom_cur = rom.cursor()
        if prosim_create_blank:
            rom_cur.execute('DROP TABLE IF EXISTS AirportLookup;')
            rom_cur.execute('DROP TABLE IF EXISTS Airports;')
            rom_cur.execute('DROP TABLE IF EXISTS AirwayLegs;')
            rom_cur.execute('DROP TABLE IF EXISTS Airways;')
            rom_cur.execute('DROP TABLE IF EXISTS config;')
            rom_cur.execute('DROP TABLE IF EXISTS ILSes;')
            rom_cur.execute('DROP TABLE IF EXISTS Markers;')
            rom_cur.execute('DROP TABLE IF EXISTS MarkerTypes;')
            rom_cur.execute('DROP TABLE IF EXISTS NavaidLookup;')
            rom_cur.execute('DROP TABLE IF EXISTS Navaids;')
            rom_cur.execute('DROP TABLE IF EXISTS NavaidTypes;')
            rom_cur.execute('DROP TABLE IF EXISTS Runways;')
            rom_cur.execute('DROP TABLE IF EXISTS SurfaceTypes;')
            rom_cur.execute('DROP TABLE IF EXISTS TerminalLegs;')
            rom_cur.execute('DROP TABLE IF EXISTS TerminalLegsEx;')
            rom_cur.execute('DROP TABLE IF EXISTS Terminals;')
            rom_cur.execute('DROP TABLE IF EXISTS TrmLegTypes;')
            rom_cur.execute('DROP TABLE IF EXISTS WaypointLookup;')
            rom_cur.execute('DROP TABLE IF EXISTS Waypoints;')
            rom_cur.execute('CREATE TABLE "AirportLookup" ("extID" TEXT, "ID" INTEGER DEFAULT 0 REFERENCES "Airports" ("ID"), PRIMARY KEY("extID","ID"));')
            rom_cur.execute('CREATE TABLE "Airports" ("ID" INTEGER NOT NULL DEFAULT 0, "Name" TEXT, "ICAO" TEXT, "PrimaryID" INTEGER DEFAULT Null REFERENCES "Airports" ("ID"), "Latitude" DOUBLE DEFAULT 0, "Longtitude" DOUBLE DEFAULT 0, "Elevation" INTEGER DEFAULT 0, "TransitionAltitude" INTEGER DEFAULT 0, "TransitionLevel" INTEGER DEFAULT 0, PRIMARY KEY("ID"));')
            rom_cur.execute('CREATE TABLE "AirwayLegs" ("ID" INTEGER DEFAULT 0, "AirwayID" INTEGER DEFAULT 0 REFERENCES "Airways" ("ID"), "Level" TEXT, "Waypoint1ID" INTEGER DEFAULT Null REFERENCES "Waypoints" ("ID"), "Waypoint2ID" INTEGER DEFAULT Null REFERENCES "Waypoints" ("ID"), "IsStart" BOOLEAN NOT NULL, "IsEnd" BOOLEAN NOT NULL, PRIMARY KEY("ID"));')
            rom_cur.execute('CREATE TABLE "Airways" ("ID" INTEGER DEFAULT 0, "Ident" TEXT NOT NULL, PRIMARY KEY("ID"));')
            rom_cur.execute('CREATE TABLE "config" ("key" TEXT, "val" TEXT, PRIMARY KEY("key"));')
            rom_cur.execute('CREATE TABLE "ILSes" ("ID" INTEGER DEFAULT 0, "RunwayID" INTEGER DEFAULT 0 REFERENCES "Runways" ("ID"), "Freq" INTEGER DEFAULT 0, "GsAngle" DOUBLE DEFAULT 0, "Latitude" DOUBLE DEFAULT 0, "Longtitude" DOUBLE DEFAULT 0, "Category" INTEGER DEFAULT 0, "Ident" TEXT, "LocCourse" DOUBLE DEFAULT 0, "CrossingHeight" INTEGER DEFAULT 0, "HasDme" BOOLEAN NOT NULL, "Elevation" INTEGER DEFAULT 0, PRIMARY KEY("ID"));')
            rom_cur.execute('CREATE TABLE "Markers" ("ID" INTEGER NOT NULL DEFAULT 0, "AirportID" INTEGER NOT NULL DEFAULT 0 REFERENCES "Airports" ("ID"), "RunwayID" INTEGER DEFAULT 0 REFERENCES "Runways" ("ID"), "LLZIdent" TEXT, "MarkerIdent" TEXT, "Type" INTEGER DEFAULT Null REFERENCES "MarkerTypes" ("Type"), "Latitude" DOUBLE DEFAULT 0, "Longitude" DOUBLE DEFAULT 0, PRIMARY KEY("ID"));')
            rom_cur.execute('CREATE TABLE "MarkerTypes" ("Type" INTEGER NOT NULL DEFAULT 0, "Desc" TEXT, PRIMARY KEY("Type"));')
            rom_cur.execute('CREATE TABLE "NavaidLookup" ("Ident" TEXT NOT NULL, "Type" INTEGER NOT NULL DEFAULT 0, "Country" TEXT NOT NULL, "NavKeyCode" INTEGER NOT NULL DEFAULT 0, "ID" INTEGER NOT NULL DEFAULT 0 REFERENCES "Navaids" ("ID"), PRIMARY KEY("Ident","Type","Country","NavKeyCode"));')
            rom_cur.execute('CREATE TABLE "Navaids" ("ID" INTEGER NOT NULL DEFAULT 0, "Ident" TEXT, "Type" INTEGER DEFAULT 0 REFERENCES "NavaidTypes" ("Type"), "Name" TEXT, "Freq" INTEGER, "Channel" TEXT, "Usage" TEXT, "Latitude" DOUBLE DEFAULT 0, "Longtitude" DOUBLE DEFAULT 0, "Elevation" INTEGER DEFAULT 0, "SlavedVar" DOUBLE DEFAULT 0, "MagneticVariation" DOUBLE DEFAULT Null, "Range" INTEGER DEFAULT Null, PRIMARY KEY("ID"));')
            rom_cur.execute('CREATE TABLE "NavaidTypes" ("Type" INTEGER DEFAULT 0, "Desc" TEXT, PRIMARY KEY("Type"));')
            rom_cur.execute('CREATE TABLE "Runways" ("ID" INTEGER DEFAULT 0, "AirportID" INTEGER DEFAULT 0 REFERENCES "Airports" ("ID"), "Ident" TEXT, "TrueHeading" DOUBLE DEFAULT 0, "Length" INTEGER DEFAULT 0, "Width" INTEGER DEFAULT 0, "Surface" TEXT REFERENCES "SurfaceTypes" ("SurfaceType"), "Latitude" DOUBLE DEFAULT 0, "Longtitude" DOUBLE DEFAULT 0, "Elevation" INTEGER DEFAULT 0, PRIMARY KEY("ID"));')
            rom_cur.execute('CREATE TABLE "SurfaceTypes" ("SurfaceType" TEXT, "Description" TEXT, PRIMARY KEY("SurfaceType"));')
            rom_cur.execute('CREATE TABLE "TerminalLegs" ("ID" INTEGER DEFAULT 0 REFERENCES "TerminalLegsEx" ("ID"), "TerminalID" INTEGER NOT NULL DEFAULT 0 REFERENCES "Terminals" ("ID"), "Type" TEXT, "Transition" TEXT, "TrackCode" TEXT REFERENCES "TrmLegTypes" ("Code"), "WptID" INTEGER DEFAULT 0 REFERENCES "Waypoints" ("ID"), "WptLat" DOUBLE DEFAULT 0, "WptLon" DOUBLE DEFAULT 0, "TurnDir" TEXT, "NavID" INTEGER DEFAULT 0 REFERENCES "Navaids" ("ID"), "NavLat" DOUBLE DEFAULT 0, "NavLon" DOUBLE DEFAULT 0, "NavBear" DOUBLE DEFAULT 0, "NavDist" DOUBLE DEFAULT 0, "Course" DOUBLE DEFAULT 0, "Distance" DOUBLE DEFAULT 0, "Alt" TEXT, "Vnav" DOUBLE DEFAULT 0, "CenterID" INTEGER DEFAULT 0, "CenterLat" DOUBLE DEFAULT 0, "CenterLon" DOUBLE DEFAULT 0, PRIMARY KEY("ID"));')
            rom_cur.execute('CREATE TABLE "TerminalLegsEx" ("ID" INTEGER DEFAULT 0, "IsFlyOver" BOOLEAN NOT NULL, "SpeedLimit" DOUBLE DEFAULT 0, PRIMARY KEY("ID"));')
            rom_cur.execute('CREATE TABLE "Terminals" ("ID" INTEGER NOT NULL DEFAULT 0, "AirportID" INTEGER DEFAULT 0 REFERENCES "Airports" ("ID"), "Proc" INTEGER DEFAULT 0, "ICAO" TEXT NOT NULL, "FullName" TEXT NOT NULL, "Name" TEXT NOT NULL, "Rwy" TEXT, "RwyID" INTEGER DEFAULT 0, "IlsID" INTEGER DEFAULT 0, PRIMARY KEY("ID"));')
            rom_cur.execute('CREATE TABLE "TrmLegTypes" ("Code" TEXT NOT NULL, "Description" TEXT, PRIMARY KEY("Code"));')
            rom_cur.execute('CREATE TABLE "WaypointLookup" ("Ident" TEXT NOT NULL, "Country" TEXT NOT NULL, "ID" INTEGER NOT NULL DEFAULT 0 REFERENCES "Waypoints" ("ID"), PRIMARY KEY("Ident","Country","ID"));')
            rom_cur.execute('CREATE TABLE "Waypoints" ("ID" INTEGER DEFAULT 0, "Ident" TEXT NOT NULL, "Collocated" BOOLEAN NOT NULL, "Name" TEXT, "Latitude" DOUBLE DEFAULT 0, "Longtitude" DOUBLE DEFAULT 0, "NavaidID" INTEGER DEFAULT Null REFERENCES "Navaids" ("ID"), PRIMARY KEY("ID"));')
            rom_cur.execute('INSERT INTO MarkerTypes VALUES(1, "IM");')
            rom_cur.execute('INSERT INTO MarkerTypes VALUES(2, "MM");')
            rom_cur.execute('INSERT INTO MarkerTypes VALUES(3, "OM");')
            rom_cur.execute('INSERT INTO MarkerTypes VALUES(4, "BM");')
            rom_cur.execute('INSERT INTO MarkerTypes VALUES(5, "LIM");')
            rom_cur.execute('INSERT INTO MarkerTypes VALUES(6, "LMM");')
            rom_cur.execute('INSERT INTO MarkerTypes VALUES(7, "LOM");')
            rom_cur.execute('INSERT INTO MarkerTypes VALUES(8, "LBM");')
            rom_cur.execute('INSERT INTO NavaidTypes VALUES(1, "VOR");')
            rom_cur.execute('INSERT INTO NavaidTypes VALUES(2, "VORTAC");')
            rom_cur.execute('INSERT INTO NavaidTypes VALUES(3, "TACAN");')
            rom_cur.execute('INSERT INTO NavaidTypes VALUES(4, "VOR-DME");')
            rom_cur.execute('INSERT INTO NavaidTypes VALUES(5, "NDB");')
            rom_cur.execute('INSERT INTO NavaidTypes VALUES(7, "NDB-DME");')
            rom_cur.execute('INSERT INTO NavaidTypes VALUES(8, "ILS-DME");')
            rom_cur.execute('INSERT INTO NavaidTypes VALUES(9, "DME (EXCLUDING ILS-DME)");')
            rom_cur.execute('INSERT INTO TrmLegTypes VALUES("AF", "Constant DME Arc to Fix");')
            rom_cur.execute('INSERT INTO TrmLegTypes VALUES("CA", "Course to an altitude (position unspecified)");')
            rom_cur.execute('INSERT INTO TrmLegTypes VALUES("CD", "Course to DME distance");')
            rom_cur.execute('INSERT INTO TrmLegTypes VALUES("CF", "Course to a fix");')
            rom_cur.execute('INSERT INTO TrmLegTypes VALUES("CI", "Course to next leg following by course oriented leg (interception point unspecified)");')
            rom_cur.execute('INSERT INTO TrmLegTypes VALUES("CR", "Course to a radial termination (intercept point unspecified)");')
            rom_cur.execute('INSERT INTO TrmLegTypes VALUES("DF", "Computed track direct to a fix");')
            rom_cur.execute('INSERT INTO TrmLegTypes VALUES("DS", "DISCONTINUITY");')
            rom_cur.execute('INSERT INTO TrmLegTypes VALUES("FA", "Course from a fix to an altitude");')
            rom_cur.execute('INSERT INTO TrmLegTypes VALUES("FC", "Course from a fix to a distance");')
            rom_cur.execute('INSERT INTO TrmLegTypes VALUES("FD", "Course from a fix to DME distance");')
            rom_cur.execute('INSERT INTO TrmLegTypes VALUES("FM", "Course from a fix to manual termination");')
            rom_cur.execute('INSERT INTO TrmLegTypes VALUES("HA", "Automatically at the fix after reaching an altitude");')
            rom_cur.execute('INSERT INTO TrmLegTypes VALUES("HF", "Automatically at the fix after one full circuit");')
            rom_cur.execute('INSERT INTO TrmLegTypes VALUES("HM", "Manually");')
            rom_cur.execute('INSERT INTO TrmLegTypes VALUES("IF", "Initial Fix");')
            rom_cur.execute('INSERT INTO TrmLegTypes VALUES("PI", "Procedure turn followed by a course to a fix (CF)");')
            rom_cur.execute('INSERT INTO TrmLegTypes VALUES("RF", "Constant radius to a fix");')
            rom_cur.execute('INSERT INTO TrmLegTypes VALUES("TF", "Track between two fixes (great circle)");')
            rom_cur.execute('INSERT INTO TrmLegTypes VALUES("VA", "heading to an altitude (position unspecified)");')
            rom_cur.execute('INSERT INTO TrmLegTypes VALUES("VD", "Heading to a DME Distance (position unspecified)");')
            rom_cur.execute('INSERT INTO TrmLegTypes VALUES("VI", "Heading to a next leg (position unspecified)");')
            rom_cur.execute('INSERT INTO TrmLegTypes VALUES("VM", "Heading to a manual termination");')
            rom_cur.execute('INSERT INTO TrmLegTypes VALUES("VR", "Heading to a radial termination");')
            rom_cur.execute('INSERT INTO SurfaceTypes VALUES("ASP", "ASPHALT, ASPHALTIC CONCRETE, TAR MACADAM, OR BITUMEN BOUND MACADAM (INCLUDING ANY OF THESE SURFACE TYPES WITH CONCRETE ENDS).");')
            rom_cur.execute('INSERT INTO SurfaceTypes VALUES("BIT", "BITUMINOUS, TAR OR ASPHALT MIXED IN PLACE, OILED.");')
            rom_cur.execute('INSERT INTO SurfaceTypes VALUES("BRI", "BRICK, LAID OR MORTARED.");')
            rom_cur.execute('INSERT INTO SurfaceTypes VALUES("CLA", "CLAY.");')
            rom_cur.execute('INSERT INTO SurfaceTypes VALUES("COM", "COMPOSITE, LESS THAN 50 PERCENT OF THE RUNWAY LENGTH IS PERMANENT.");')
            rom_cur.execute('INSERT INTO SurfaceTypes VALUES("CON", "CONCRETE.");')
            rom_cur.execute('INSERT INTO SurfaceTypes VALUES("COP", "COMPOSITE, 50 PERCENT OR MORE OF THE RUNWAY LENGTH IS PERMANENT.");')
            rom_cur.execute('INSERT INTO SurfaceTypes VALUES("COR", "CORAL.");')
            rom_cur.execute('INSERT INTO SurfaceTypes VALUES("GRE", "GRADED OR ROLLED EARTH, GRASS ON GRADED EARTH.");')
            rom_cur.execute('INSERT INTO SurfaceTypes VALUES("GRS", "GRASS OR EARTH NOT GRADED OR GRASS OR EARTH NOT GRADED OR ROLLED.");')
            rom_cur.execute('INSERT INTO SurfaceTypes VALUES("GVL", "GRAVEL.");')
            rom_cur.execute('INSERT INTO SurfaceTypes VALUES("ICE", "ICE.");')
            rom_cur.execute('INSERT INTO SurfaceTypes VALUES("LAT", "LATERITE.");')
            rom_cur.execute('INSERT INTO SurfaceTypes VALUES("MAC", "MACADAM - CRUSHED ROCK WATER BOUND.");')
            rom_cur.execute('INSERT INTO SurfaceTypes VALUES("MEM", "MEMBRANE - PLASTIC OR OTHER COATED FIBER MATERIAL.");')
            rom_cur.execute('INSERT INTO SurfaceTypes VALUES("MIX", "MIX IN PLACE USING NONBITUMIOUS BINDERS SUCH AS PORTLAND CEMENT.");')
            rom_cur.execute('INSERT INTO SurfaceTypes VALUES("PEM", "PART CONCRETE, PART ASPHALT, OR PART BITUMEN-BOUND MACADAM.");')
            rom_cur.execute('INSERT INTO SurfaceTypes VALUES("PER", "PERMANENT, SURFACE TYPE UNKNOWN.");')
            rom_cur.execute('INSERT INTO SurfaceTypes VALUES("PSP", "PIECED STEEL PLANKING.");')
            rom_cur.execute('INSERT INTO SurfaceTypes VALUES("SAN", "SAND, GRADED, ROLLED OR OILED.");')
            rom_cur.execute('INSERT INTO SurfaceTypes VALUES("SNO", "SNOW.");')
            rom_cur.execute('INSERT INTO SurfaceTypes VALUES("U", "SURFACE UNKNOWN.");')
            rom_cur.execute('INSERT INTO SurfaceTypes VALUES("WAT", "WATER");')
            config_prosim = [['CycleEndDate',CycleEndDate],['CycleName',CycleName],['CycleStartDate',CycleStartDate]]
            config_prosim_tb = pandas.DataFrame(config_prosim,columns=['key','val'])
            config_prosim_tb.to_sql('config', rom, if_exists='append', index=False)
        if prosim_write:
            Airports_prosim_tb.to_sql('Airports', rom, if_exists='append', index=False)
            AirportLookup_prosim_tb.to_sql('AirportLookup', rom, if_exists='append', index=False)
            Runways_prosim_tb.to_sql('Runways', rom, if_exists='append', index=False)
            Navaids_prosim_tb.to_sql('Navaids', rom, if_exists='append', index=False)
            NavaidLookup_prosim_tb.to_sql('NavaidLookup', rom, if_exists='append', index=False)
            Waypoints_prosim_tb.to_sql('Waypoints', rom, if_exists='append', index=False)
            WaypointLookup_prosim_tb = WaypointLookup_prosim_tb.drop(columns=['REGION_CODE'])
            WaypointLookup_prosim_tb.to_sql('WaypointLookup', rom, if_exists='append', index=False)
            Airways_prosim_tb.to_sql('Airways', rom, if_exists='append', index=False)
            AirwayLegs_prosim_tb.to_sql('AirwayLegs', rom, if_exists='append', index=False)
            ILSes_prosim_tb = ILSes_prosim_tb.drop(columns=['AirportID'])
            ILSes_prosim_tb.to_sql('ILSes', rom, if_exists='append', index=False)
            Terminals_prosim_tb.to_sql('Terminals', rom, if_exists='append', index=False)
            TerminalLegsEx_prosim_tb.to_sql('TerminalLegsEx', rom, if_exists='append', index=False)
            TerminalLegs_prosim_tb.to_sql('TerminalLegs', rom, if_exists='append', index=False)
        rom_cur.close()
        rom.commit()
        rom.close()
    print(time.strftime("%H:%M:%S", time.localtime()) , 'ProSim: Data Export Finished')

# TFDi (Completed)
if TFDi:
    print(time.strftime("%H:%M:%S", time.localtime()) , 'TFDi: Start Processing DataFile')
    os.makedirs(Cycle_Ident + '/TFDi')
    # Set Variable of TFDi
    if 1:
        AirportLookup_tfdi = []
        Airports_tfdi = []
        AirwayLegs_tfdi = []
        AirwayLegs_reversed_tfdi = []
        AirwayLookup_tfdi = []
        Airways_tfdi = []
        ILSes_tfdi = []
        NavaidLookup_tfdi = []
        Navaids_tfdi = []
        Runways_tfdi = []
        TerminalLegs_tfdi = []
        TerminalLegsEx_tfdi = []
        Terminals_tfdi = []
        WaypointLookup_tfdi = []
        Waypoints_tfdi = []
        AirwayIdent = ''
        TerminalName = ''
        if add_tailored:
            AirportID = 15000
            AirwayID = 10000
            AirwayLegsID = 200000
            ILSID = 5000
            NavaidID = 15000
            RunwayID = 40000
            TerminalID = 100000
            TerminalLegsID = 800000
            WaypointID = 300000
        else:
            AirportID = 0
            AirwayID = 0
            AirwayLegsID = 0
            ILSID = 0
            NavaidID = 0
            RunwayID = 0
            TerminalID = 0
            TerminalLegsID = 0
            WaypointID = 0
    # Load TFDi's navigationdata.mdb
    if tfdi_create_blank != True:
        path = os.path.abspath(Cycle_Ident + '/TFDi/navigationdata.mdb')
        shutil.copy2('Input/TFDi/navigationdata.mdb', Cycle_Ident + '/TFDi/navigationdata.mdb')
        mdb_odbc = (r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)}; DBQ=' + path + '; ExtendedAnsiSQL=1;')
        mdb_eng = create_engine(f'access+pyodbc:///?odbc_connect={urllib.parse.quote_plus(mdb_odbc)}')
        mdb = mdb_eng.connect()
        Airports_in_tfdi_nd_tb = pandas.read_sql_query('SELECT * FROM Airports', con = mdb)
        Airways_in_tfdi_nd_tb = pandas.read_sql_query('SELECT * FROM Airways', con = mdb)
        NavaidLookup_in_tfdi_nd_tb = pandas.read_sql_query('SELECT * FROM NavaidLookup', con = mdb)
        Navaids_in_tfdi_nd_tb = pandas.read_sql_query('SELECT * FROM Navaids', con = mdb)
        Runways_in_tfdi_nd_tb = pandas.read_sql_query('SELECT * FROM Runways', con = mdb)
        WaypointLookup_in_tfdi_nd_tb = pandas.read_sql_query('SELECT * FROM WaypointLookup', con = mdb)
        Waypoints_in_tfdi_nd_tb = pandas.read_sql_query('SELECT * FROM Waypoints', con = mdb)
        TerminalWpts_in_tfdi_nd_tb = pandas.read_sql_query('SELECT TerminalID, WptID, WptLat, WptLon FROM TerminalLegs', con = mdb)
        Terminals_in_tfdi_nd_tb = pandas.read_sql_query('SELECT * FROM Terminals', con = mdb)
        ILSes_in_tfdi_nd_tb = pandas.read_sql_query('SELECT * FROM ILSes', con = mdb)
        ILSes_in_tfdi_nd_tb['AirportID'] = ILSes_in_tfdi_nd_tb['RunwayID'].apply(lambda RunwayID : Runways_in_tfdi_nd_tb.loc[Runways_in_tfdi_nd_tb.loc[(Runways_in_tfdi_nd_tb['ID'] == RunwayID)].index.tolist() , 'AirportID'].values[0])
        TerminalWpts_in_tfdi_nd_tb.drop_duplicates(subset=['WptID'], keep='first', inplace=True)
        TerminalWpts_in_tfdi_nd_tb = TerminalWpts_in_tfdi_nd_tb.dropna()
        TerminalWpts_in_tfdi_nd_tb['WptIdent'] = TerminalWpts_in_tfdi_nd_tb['WptID'].apply(lambda WptID : WaypointLookup_in_tfdi_nd_tb.loc[WaypointLookup_in_tfdi_nd_tb.loc[(WaypointLookup_in_tfdi_nd_tb['ID'] == WptID)].index.tolist() , 'Ident'].values[0])
        TerminalWpts_in_tfdi_nd_tb['REGION_CODE'] = TerminalWpts_in_tfdi_nd_tb['TerminalID'].apply(lambda TerminalID : Terminals_in_tfdi_nd_tb.loc[Terminals_in_tfdi_nd_tb.loc[(Terminals_in_tfdi_nd_tb['ID'] == TerminalID)].index.tolist() , 'ICAO'].values[0])
        print(time.strftime("%H:%M:%S", time.localtime()) , 'TFDi: navigationdata.mdb Load Completed')
    else:
        path = os.path.abspath(Cycle_Ident + '/TFDi/navigationdata.mdb')
        msaccessdb.create(Cycle_Ident + '/TFDi/navigationdata.mdb')
        # shutil.copy2('Input/tfdi_empty.mdb', path)
        mdb_odbc = (r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)}; DBQ=' + path + '; ExtendedAnsiSQL=1;')
        mdb_eng = create_engine(f'access+pyodbc:///?odbc_connect={urllib.parse.quote_plus(mdb_odbc)}')
        mdb = mdb_eng.connect()
    # Process of TFDi Part #1
    with open('Input/' + Datafile, 'r', encoding='ISO-8859-1') as ARINC424:
        for line in ARINC424: 
            vhf_se = re.search(vor, line)
            db_se = re.search(db, line)
            ea_se = re.search(ea, line)
            awy_se = re.search(er, line)
            arpt_se = re.search(pa, line)
            pc_se = re.search(pc, line)
            rwy_se = re.search(pg, line)
            pn_se = re.search(pn, line)
            if vhf_se:
                ARPT_IDENT = vhf_se.group(5).strip()
                NAV_IDENT = vhf_se.group(7).strip()
                WAYPOINT_IDENT = NAV_IDENT
                NAV_ICAO_CODE = vhf_se.group(8).strip()
                WAYPOINT_ICAO_CODE = NAV_ICAO_CODE
                NAV_FREQ = int('0x' + str(int(vhf_se.group(10))*100) , 16)
                if vhf_se.group(11)[0:1] == "V":
                    if vhf_se.group(11)[1:2] == " ":
                        NAVAID_TYPE = 1
                    elif vhf_se.group(11)[1:2] == "T":
                        NAVAID_TYPE = 2
                    elif vhf_se.group(11)[1:2] == "M":
                        NAVAID_TYPE = 2
                    elif vhf_se.group(11)[1:2] == "D":
                        NAVAID_TYPE = 4
                    else:
                        NAVAID_TYPE = 1
                else:
                    if vhf_se.group(11)[1:2] == "T":
                        NAVAID_TYPE = 3
                    elif vhf_se.group(11)[1:2] == "M":
                        NAVAID_TYPE = 3
                    elif vhf_se.group(11)[1:2] == "I":
                        NAVAID_TYPE = 8          
                    elif vhf_se.group(11)[1:2] == "D":
                        NAVAID_TYPE = 9
                    else:
                        NAVAID_TYPE = 9
                if vhf_se.group(11)[2:3] == "U":
                    NAVAID_CLASS = "B"
                else :
                    NAVAID_CLASS = vhf_se.group(11)[2:3]
                if vhf_se.group(12).strip() != '':
                    if vhf_se.group(12) == "N":
                        NAV_LAT = round(float(vhf_se.group(13)) + float(vhf_se.group(14))/60 + float(vhf_se.group(15))/3600 + float(vhf_se.group(16))/360000, 8)
                    else:
                        NAV_LAT = 0 - round(float(vhf_se.group(13)) - float(vhf_se.group(14))/60 - float(vhf_se.group(15))/3600 - float(vhf_se.group(16))/360000, 8)
                else:
                    if vhf_se.group(23) == "N":
                        NAV_LAT = round(float(vhf_se.group(24)) + float(vhf_se.group(25))/60 + float(vhf_se.group(26))/3600 + float(vhf_se.group(27))/360000, 8)
                    else:
                        NAV_LAT = 0 - round(float(vhf_se.group(24)) - float(vhf_se.group(25))/60 - float(vhf_se.group(26))/3600 - float(vhf_se.group(27))/360000, 8)
                WAYPOINT_LAT = NAV_LAT
                if vhf_se.group(17).strip() != '':
                    if vhf_se.group(17) == "E":
                        NAV_LON = round(float(vhf_se.group(18)) + float(vhf_se.group(19))/60 + float(vhf_se.group(20))/3600 + float(vhf_se.group(21))/360000, 8)
                    else:
                        NAV_LON = 0 - round(float(vhf_se.group(18)) - float(vhf_se.group(19))/60 - float(vhf_se.group(20))/3600 - float(vhf_se.group(21))/360000, 8)
                else:
                    if vhf_se.group(28) == "E":
                        NAV_LON = round(float(vhf_se.group(29)) + float(vhf_se.group(30))/60 + float(vhf_se.group(31))/3600 + float(vhf_se.group(32))/360000, 8)
                    else:
                        NAV_LON = 0 - round(float(vhf_se.group(29)) - float(vhf_se.group(30))/60 - float(vhf_se.group(31))/3600 - float(vhf_se.group(32))/360000, 8)
                WAYPOINT_LON = NAV_LON
                if vhf_se.group(33).strip() == '':
                    MAG_VAR = 0
                elif vhf_se.group(33).strip(string.digits) == "E":
                    MAG_VAR = float(vhf_se.group(33).strip('EW'))/10
                else:
                    MAG_VAR = 0 - float(vhf_se.group(33).strip('EW'))/10
                if vhf_se.group(34).strip() == '':
                    ELEV = 0
                else:
                    ELEV = vhf_se.group(34)
                NAV_NAME = vhf_se.group(39).replace('VOR/DME','').rstrip().replace('(','').replace(')','')
                WAYPOINT_NAME_DESCR = NAV_NAME[0:24]
                NavaidID = NavaidID + 1
                WaypointID = WaypointID + 1
                NavaidLookup_tfdi.append((NAV_IDENT,NAVAID_TYPE,NAV_ICAO_CODE,1,NavaidID))
                Navaids_tfdi.append((NavaidID,NAV_IDENT,NAVAID_TYPE,NAV_NAME,NAV_FREQ,None,NAVAID_CLASS,NAV_LAT,NAV_LON,ELEV,MAG_VAR))
                WaypointLookup_tfdi.append((WAYPOINT_IDENT,WAYPOINT_ICAO_CODE,WaypointID,ARPT_IDENT))
                Waypoints_tfdi.append((WaypointID,WAYPOINT_IDENT,1,WAYPOINT_NAME_DESCR,WAYPOINT_LAT,WAYPOINT_LON,int(NavaidID)))
            if db_se:
                ARPT_IDENT = db_se.group(5).strip()
                NAV_IDENT = db_se.group(7).strip()
                WAYPOINT_IDENT = NAV_IDENT
                NAV_ICAO_CODE = db_se.group(8).strip()
                WAYPOINT_ICAO_CODE = NAV_ICAO_CODE
                NAV_FREQ = int('0x' + str(int(db_se.group(10))*1000) , 16)
                NAVAID_TYPE = 5
                if db_se.group(11)[2:3] == "L":
                    NAVAID_CLASS = "T"
                elif db_se.group(11)[2:3] == "M":
                    NAVAID_CLASS = "L"
                else :
                    NAVAID_CLASS = "H"
                if db_se.group(12) == "N":
                    NAV_LAT = round(float(db_se.group(13)) + float(db_se.group(14))/60 + float(db_se.group(15))/3600 + float(db_se.group(16))/360000, 8)
                else:
                    NAV_LAT = 0 - round(float(db_se.group(13)) - float(db_se.group(14))/60 - float(db_se.group(15))/3600 - float(db_se.group(16))/360000, 8)
                WAYPOINT_LAT = NAV_LAT
                if db_se.group(17) == "E":
                    NAV_LON = round(float(db_se.group(18)) + float(db_se.group(19))/60 + float(db_se.group(20))/3600 + float(db_se.group(21))/360000, 8)
                else:
                    NAV_LON = 0 - round(float(db_se.group(18)) - float(db_se.group(19))/60 - float(db_se.group(20))/3600 - float(db_se.group(21))/360000, 8)
                WAYPOINT_LON = NAV_LON
                if db_se.group(22) == "E":
                    MAG_VAR = float(db_se.group(23))/10
                else:
                    MAG_VAR = 0 - float(db_se.group(23))/10
                ELEV = 0
                NAV_NAME = db_se.group(25).replace('NDB','').rstrip().replace('(','').replace(')','')
                WAYPOINT_NAME_DESCR = NAV_NAME[0:24]
                NavaidID = NavaidID + 1
                WaypointID = WaypointID + 1
                NavaidLookup_tfdi.append((NAV_IDENT,NAVAID_TYPE,NAV_ICAO_CODE,1,NavaidID))
                Navaids_tfdi.append((NavaidID,NAV_IDENT,NAVAID_TYPE,NAV_NAME,NAV_FREQ,None,NAVAID_CLASS,NAV_LAT,NAV_LON,ELEV,MAG_VAR))
                WaypointLookup_tfdi.append((WAYPOINT_IDENT,WAYPOINT_ICAO_CODE,WaypointID,ARPT_IDENT))
                Waypoints_tfdi.append((WaypointID,WAYPOINT_IDENT,1,WAYPOINT_NAME_DESCR,WAYPOINT_LAT,WAYPOINT_LON,int(NavaidID)))
            if ea_se:
                REGION_CODE = ea_se.group(5)
                WAYPOINT_IDENT = ea_se.group(8).strip()
                WAYPOINT_ICAO_CODE = ea_se.group(9).strip()
                if ea_se.group(13) == "N":
                    WAYPOINT_LAT = round(float(ea_se.group(14)) + float(ea_se.group(15))/60 + float(ea_se.group(16))/3600 + float(ea_se.group(17))/360000, 8)
                else:
                    WAYPOINT_LAT = round(0 - float(ea_se.group(14)) - float(ea_se.group(15))/60 - float(ea_se.group(16))/3600 - float(ea_se.group(17))/360000, 8)
                if ea_se.group(18) == "E":
                    WAYPOINT_LON = round(float(ea_se.group(19)) + float(ea_se.group(20))/60 + float(ea_se.group(21))/3600 + float(ea_se.group(22))/360000, 8)
                else:
                    WAYPOINT_LON = round(0 - float(ea_se.group(19)) - float(ea_se.group(20))/60 - float(ea_se.group(21))/3600 - float(ea_se.group(22))/360000, 8)
                WAYPOINT_NAME_DESCR = ea_se.group(26).rstrip().replace('(','').replace(')','')[0:24]
                WaypointID = WaypointID + 1
                WaypointLookup_tfdi.append((WAYPOINT_IDENT,WAYPOINT_ICAO_CODE,WaypointID,REGION_CODE))
                Waypoints_tfdi.append((WaypointID,WAYPOINT_IDENT,0,WAYPOINT_NAME_DESCR,WAYPOINT_LAT,WAYPOINT_LON,None))
            if awy_se:
                if awy_se.group(5).strip() != AirwayIdent:
                    if tfdi_create_blank != True:
                        if Airways_in_tfdi_nd_tb.loc[(Airways_in_tfdi_nd_tb['Ident'] == awy_se.group(5).strip())].index.tolist() == []:
                            AirwayID = AirwayID + 1
                            AirwayIdent = awy_se.group(5).strip()
                            Airways_tfdi.append((AirwayID, AirwayIdent))
                    else:
                        AirwayID = AirwayID + 1
                        AirwayIdent = awy_se.group(5).strip()
                        Airways_tfdi.append((AirwayID, AirwayIdent))
            if arpt_se:
                ARPT_IDENT = arpt_se.group(4)
                ARPT_ICAO_CODE = arpt_se.group(5)
                if arpt_se.group(13) == "N":
                    ARPT_LAT = round(float(arpt_se.group(14)) + float(arpt_se.group(15))/60 + float(arpt_se.group(16))/3600 + float(arpt_se.group(17))/360000, 8)
                else:
                    ARPT_LAT = round(0 - float(arpt_se.group(14)) - float(arpt_se.group(15))/60 - float(arpt_se.group(16))/3600 - float(arpt_se.group(17))/360000, 8)
                if arpt_se.group(18) == "E":
                    ARPT_LON = round(float(arpt_se.group(19)) + float(arpt_se.group(20))/60 + float(arpt_se.group(21))/3600 + float(arpt_se.group(22))/360000, 8)
                else:
                    ARPT_LON = round(0 - float(arpt_se.group(19)) - float(arpt_se.group(20))/60 - float(arpt_se.group(21))/3600 - float(arpt_se.group(22))/360000, 8)
                ARPT_ELEV = int(arpt_se.group(25))
                ARPT_NAME = arpt_se.group(36).rstrip().replace('(','').replace(')','')
                if arpt_se.group(23) == "E":
                    ARPT_MAG_VAR = float(arpt_se.group(24))/10
                else:
                    ARPT_MAG_VAR = 0 - float(arpt_se.group(24))/10
                if arpt_se.group(29).strip() == '':
                    TRANSITIONS_ALT = None
                else:
                    TRANSITIONS_ALT = arpt_se.group(29)
                AirportID = AirportID + 1
                Airports_tfdi.append((AirportID,ARPT_NAME,ARPT_IDENT,None,ARPT_LAT,ARPT_LON,ARPT_ELEV,TRANSITIONS_ALT))
                AirportLookup_tfdi.append((ARPT_ICAO_CODE + ARPT_IDENT,AirportID))
            if pc_se:
                REGION_CODE = pc_se.group(5)
                WAYPOINT_IDENT = pc_se.group(8).strip()
                WAYPOINT_ICAO_CODE = pc_se.group(9).strip()
                if pc_se.group(13) == "N":
                    WAYPOINT_LAT = round(float(pc_se.group(14)) + float(pc_se.group(15))/60 + float(pc_se.group(16))/3600 + float(pc_se.group(17))/360000, 8)
                else:
                    WAYPOINT_LAT = round(0 - float(pc_se.group(14)) - float(pc_se.group(15))/60 - float(pc_se.group(16))/3600 - float(pc_se.group(17))/360000, 8)
                if pc_se.group(18) == "E":
                    WAYPOINT_LON = round(float(pc_se.group(19)) + float(pc_se.group(20))/60 + float(pc_se.group(21))/3600 + float(pc_se.group(22))/360000, 8)
                else:
                    WAYPOINT_LON = round(0 - float(pc_se.group(19)) - float(pc_se.group(20))/60 - float(pc_se.group(21))/3600 - float(pc_se.group(22))/360000, 8)
                WAYPOINT_NAME_DESCR = pc_se.group(26).rstrip().replace('(','').replace(')','')[0:24]
                WaypointID = WaypointID + 1
                WaypointLookup_tfdi.append((WAYPOINT_IDENT,WAYPOINT_ICAO_CODE,WaypointID,REGION_CODE))
                Waypoints_tfdi.append((WaypointID,WAYPOINT_IDENT,0,WAYPOINT_NAME_DESCR,WAYPOINT_LAT,WAYPOINT_LON,None))
            if rwy_se:
                RunwayID = RunwayID + 1
                if rwy_se.group(4) != ARPT_IDENT:
                    print(rwy_se.group(4) + " Not Found")
                    AirportID = None
                Ident = rwy_se.group(7)[2:5].strip()
                if rwy_se.group(10).lstrip(string.digits) == "T":
                    TrueHeading = rwy_se.group(10).rstrip('T')
                else:
                    TrueHeading = float(rwy_se.group(10))/10 + ARPT_MAG_VAR
                Length = rwy_se.group(9)
                Width = rwy_se.group(27)
                if rwy_se.group(11) == "N":
                    Latitude = round(float(rwy_se.group(12)) + float(rwy_se.group(13))/60 + float(rwy_se.group(14))/3600 + float(rwy_se.group(15))/360000, 8)
                else:
                    Latitude = round(0 - float(rwy_se.group(12)) - float(rwy_se.group(13))/60 - float(rwy_se.group(14))/3600 - float(rwy_se.group(15))/360000, 8)
                if rwy_se.group(16) == "E":
                    Longitude = round(float(rwy_se.group(17)) + float(rwy_se.group(18))/60 + float(rwy_se.group(19))/3600 + float(rwy_se.group(20))/360000, 8)
                else:
                    Longitude = round(0 - float(rwy_se.group(17)) - float(rwy_se.group(18))/60 - float(rwy_se.group(19))/3600 - float(rwy_se.group(20))/360000, 8)
                Elevation = rwy_se.group(24).lstrip('0')
                Runways_tfdi.append((RunwayID,AirportID,Ident,TrueHeading,Length,Width,'U  ',Latitude,Longitude,Elevation))
            if pn_se:
                ARPT_IDENT = pn_se.group(5).strip()
                NAV_IDENT = pn_se.group(7).strip()
                WAYPOINT_IDENT = NAV_IDENT
                NAV_ICAO_CODE = pn_se.group(8).strip()
                WAYPOINT_ICAO_CODE = NAV_ICAO_CODE
                NAV_FREQ = int('0x' + str(int(pn_se.group(10))*1000) , 16)
                NAVAID_TYPE = 5
                if pn_se.group(11)[2:3] == "L":
                    NAVAID_CLASS = "T"
                elif pn_se.group(11)[2:3] == "M":
                    NAVAID_CLASS = "L"
                else :
                    NAVAID_CLASS = "H"
                if pn_se.group(12) == "N":
                    NAV_LAT = round(float(pn_se.group(13)) + float(pn_se.group(14))/60 + float(pn_se.group(15))/3600 + float(pn_se.group(16))/360000, 8)
                else:
                    NAV_LAT = 0 - round(float(pn_se.group(13)) - float(pn_se.group(14))/60 - float(pn_se.group(15))/3600 - float(pn_se.group(16))/360000, 8)
                WAYPOINT_LAT = NAV_LAT
                if pn_se.group(17) == "E":
                    NAV_LON = round(float(pn_se.group(18)) + float(pn_se.group(19))/60 + float(pn_se.group(20))/3600 + float(pn_se.group(21))/360000, 8)
                else:
                    NAV_LON = 0 - round(float(pn_se.group(18)) - float(pn_se.group(19))/60 - float(pn_se.group(20))/3600 - float(pn_se.group(21))/360000, 8)
                WAYPOINT_LON = NAV_LON
                if pn_se.group(22) == "E":
                    MAG_VAR = float(pn_se.group(23))/10
                else:
                    MAG_VAR = 0 - float(pn_se.group(23))/10
                NAV_NAME = pn_se.group(25).replace('NDB','').rstrip().replace('(','').replace(')','')
                WAYPOINT_NAME_DESCR = NAV_NAME[0:24]
                NavaidID = NavaidID + 1
                WaypointID = WaypointID + 1
                NavaidLookup_tfdi.append((NAV_IDENT,NAVAID_TYPE,NAV_ICAO_CODE,1,NavaidID))
                Navaids_tfdi.append((NavaidID,NAV_IDENT,NAVAID_TYPE,NAV_NAME,NAV_FREQ,None,NAVAID_CLASS,NAV_LAT,NAV_LON,ELEV,MAG_VAR))
                WaypointLookup_tfdi.append((WAYPOINT_IDENT,WAYPOINT_ICAO_CODE,WaypointID,ARPT_IDENT))
                Waypoints_tfdi.append((WaypointID,WAYPOINT_IDENT,1,WAYPOINT_NAME_DESCR,WAYPOINT_LAT,WAYPOINT_LON,int(NavaidID)))
        # Convert list to Pandas DataFrame
        Airports_tfdi_tb = pandas.DataFrame(Airports_tfdi, columns=['ID','Name','ICAO','PrimaryID','Latitude','Longitude','Elevation','TransAlt'])
        AirportLookup_tfdi_tb = pandas.DataFrame(AirportLookup_tfdi, columns=['extID','ID'])
        Navaids_tfdi_tb = pandas.DataFrame(Navaids_tfdi, columns=['ID','Ident','Type','Name','Freq','Channel','Usage','Latitude','Longitude','Elevation','SlavedVar'])
        NavaidLookup_tfdi_tb = pandas.DataFrame(NavaidLookup_tfdi, columns=['Ident','Type','Country','NavKeyCode','ID'])
        Waypoints_tfdi_tb = pandas.DataFrame(Waypoints_tfdi, columns=['ID','Ident','Collocated','Name','Latitude','Longitude','NavaidID'])
        WaypointLookup_tfdi_tb = pandas.DataFrame(WaypointLookup_tfdi, columns=['Ident','Country','ID','REGION_CODE'])
        Runways_tfdi_tb = pandas.DataFrame(Runways_tfdi, columns=['ID','AirportID','Ident','TrueHeading','Length','Width','Surface','Latitude','Longitude','Elevation'])
        # Count Duplicate Navaids (NavKeyCode)
        if tfdi_create_blank != True:
            NavaidLookup_tfdi_tb = pandas.concat([NavaidLookup_in_tfdi_nd_tb,NavaidLookup_tfdi_tb])
            NavaidLookup_tfdi_tb['NavKeyCode'] = NavaidLookup_tfdi_tb.groupby(['Ident','Country'], as_index=False)[['Ident','Country']].cumcount() + 1
            NavaidLookup_tfdi_tb = NavaidLookup_tfdi_tb[NavaidLookup_tfdi_tb['ID'] > 15000]
        else:  
            NavaidLookup_tfdi_tb['NavKeyCode'] = NavaidLookup_tfdi_tb.groupby(['Ident','Country'], as_index=False)[['Ident','Country']].cumcount() + 1
        # Make Airway List
        Airways_tfdi_tb = pandas.DataFrame(Airways_tfdi, columns=['ID','Ident'])
        Airways_tfdi_tb = Airways_tfdi_tb.sort_values('Ident')
        Airways_tfdi_tb.drop_duplicates(subset=['Ident'], keep='first', inplace=True)
        Airways_tfdi_tb = Airways_tfdi_tb.reset_index(drop=True)
        if add_tailored:
            Airways_tfdi_tb['ID'] = Airways_tfdi_tb.index + 10001
        else:
            Airways_tfdi_tb['ID'] = Airways_tfdi_tb.index + 1
        AirwayIdent = ''
        print(time.strftime("%H:%M:%S", time.localtime()) , 'TFDi: Data Process Part #1 Finished')
    # Process of TFDi Part #2
    with open('Input/' + Datafile, 'r', encoding='ISO-8859-1') as ARINC424:
        for line in ARINC424:
            awy_se = re.search(er, line)
            loc_se = re.search(pi, line)
            # AWY of TFDi (Part #1)
            if awy_se:
                CUST_AREA = awy_se.group(2)
                ROUTE_IDENT = awy_se.group(5).strip()
                AirwayID = Airways_tfdi_tb.loc[(Airways_tfdi_tb['Ident'] == ROUTE_IDENT)].index.tolist()
                if AirwayID != []:
                    AirwayID = Airways_tfdi_tb.at[AirwayID[0], 'ID']
                else:
                    AirwayID = Airways_in_tfdi_nd_tb.loc[(Airways_in_tfdi_nd_tb['Ident'] == ROUTE_IDENT)].index.tolist()
                    if tfdi_create_blank != True:
                        if AirwayID != []:
                            AirwayID = Airways_in_tfdi_nd_tb.at[AirwayID[0], 'ID']
                        else:
                            print('         TFDi: Airway '+ awy_se.group(5) + ' Not Found')
                            AirwayID = None
                    else:
                        print('         TFDi: Airway '+ awy_se.group(5) + ' Not Found')
                        AirwayID = None
                SEQ_NR = awy_se.group(6)
                FIX_IDENT = awy_se.group(7).strip()
                FIX_ICAO_CODE = awy_se.group(8).strip()
                FIX_ID = WaypointLookup_tfdi_tb.loc[(WaypointLookup_tfdi_tb['Ident'] == FIX_IDENT) & (WaypointLookup_tfdi_tb['Country'] == FIX_ICAO_CODE)].index.tolist()
                if FIX_ID != []:
                    FIX_ID = WaypointLookup_tfdi_tb.at[FIX_ID[0], 'ID']
                else:
                    if tfdi_create_blank != True:
                        FIX_ID = WaypointLookup_in_tfdi_nd_tb.loc[(WaypointLookup_in_tfdi_nd_tb['Ident'] == FIX_IDENT) & (WaypointLookup_in_tfdi_nd_tb['Country'] == FIX_ICAO_CODE)].index.tolist()
                        if FIX_ID != []:
                            FIX_ID = WaypointLookup_in_tfdi_nd_tb.at[FIX_ID[0], 'ID']
                        else:
                            print('         TFDi: Airway ' + awy_se.group(5) + ' Fix ' + awy_se.group(7) + ' Not Found')
                            FIX_ID = '0'
                    else:
                        print('         TFDi: Airway ' + awy_se.group(5) + ' Fix ' + awy_se.group(7) + ' Not Found')
                        FIX_ID = '0'
                EndOfRoute = awy_se.group(12)[1:2]
                LEVEL = awy_se.group(15)
                DIRECTION_RESTRICTION = awy_se.group(16)
                AirwayLookup_tfdi.append((CUST_AREA,ROUTE_IDENT,AirwayID,SEQ_NR,FIX_IDENT,FIX_ICAO_CODE,FIX_ID,EndOfRoute,LEVEL,DIRECTION_RESTRICTION))
            # ILS of TFDi
            if loc_se:
                ICAO = loc_se.group(4)
                if Airports_tfdi_tb.loc[(Airports_tfdi_tb['ICAO'] == ICAO)].index.tolist() != []:
                    AirportID = Airports_tfdi_tb.loc[Airports_tfdi_tb.loc[(Airports_tfdi_tb['ICAO'] == ICAO)].index.tolist() , 'ID'].values[0]
                else:
                    if tfdi_create_blank != True:
                        if Airports_in_tfdi_nd_tb.loc[(Airports_in_tfdi_nd_tb['ICAO'] == ICAO)].index.tolist() != []:
                            AirportID = Airports_in_tfdi_nd_tb.loc[Airports_in_tfdi_nd_tb.loc[(Airports_in_tfdi_nd_tb['ICAO'] == ICAO)].index.tolist() , 'ID'].values[0]
                        else:
                            print('         TFDi: Airport ' + ICAO + ' ID Not Found')
                            AirportID = None
                    else:
                        print('         TFDi: Airport ' + ICAO + ' ID Not Found')
                        AirportID = None
                if loc_se.group(11)[0:2] != 'RW':
                    Rwy = None
                    RwyID = None
                    print('         TFDi: Runway ' + ICAO + ' ' + loc_se.group(11) + ' Ident Not Correct')
                else:
                    Rwy = loc_se.group(11).strip().replace('RW','')
                    if Runways_tfdi_tb.loc[(Runways_tfdi_tb['AirportID'] == AirportID) & (Runways_tfdi_tb['Ident'] == Rwy)].index.tolist() != []:
                        RwyID = Runways_tfdi_tb.loc[Runways_tfdi_tb.loc[(Runways_tfdi_tb['AirportID'] == AirportID) & (Runways_tfdi_tb['Ident'] == Rwy)].index.tolist() , 'ID'].values[0]
                    else:
                        if tfdi_create_blank != True:
                            if Runways_in_tfdi_nd_tb.loc[(Runways_in_tfdi_nd_tb['AirportID'] == AirportID) & (Runways_in_tfdi_nd_tb['Ident'] == Rwy)].index.tolist() != []:
                                RwyID = Runways_in_tfdi_nd_tb.loc[Runways_in_tfdi_nd_tb.loc[(Runways_in_tfdi_nd_tb['AirportID'] == AirportID) & (Runways_in_tfdi_nd_tb['Ident'] == Rwy)].index.tolist() , 'ID'].values[0]
                            else:
                                print('         TFDi: Runway ' + ICAO + ' ' + Rwy + ' ID Not Found')
                                RwyID = None
                        else:
                            print('         TFDi: Runway ' + ICAO + ' ' + Rwy + ' ID Not Found')
                            RwyID = None
                NAV_FREQ = int('0x' + str(int(loc_se.group(10))*100) , 16)
                if loc_se.group(37).strip() == '':
                    GLIDE_SLOPE_ANGLE = 3
                else:
                    GLIDE_SLOPE_ANGLE = float(loc_se.group(37))/100
                if loc_se.group(12).strip() == '':
                    LOCALIZER_LAT = None
                elif loc_se.group(12) == "N":
                    LOCALIZER_LAT = round(float(loc_se.group(13)) + float(loc_se.group(14))/60 + float(loc_se.group(15))/3600 + float(loc_se.group(16))/360000, 8)
                else:
                    LOCALIZER_LAT = 0 - round(float(loc_se.group(13)) - float(loc_se.group(14))/60 - float(loc_se.group(15))/3600 - float(loc_se.group(16))/360000, 8)
                if loc_se.group(17).strip() == '':
                    LOCALIZER_LON = None
                elif loc_se.group(17) == "E":
                    LOCALIZER_LON = round(float(loc_se.group(18)) + float(loc_se.group(19))/60 + float(loc_se.group(20))/3600 + float(loc_se.group(21))/360000, 8)
                else:
                    LOCALIZER_LON = 0 - round(float(loc_se.group(18)) - float(loc_se.group(19))/60 - float(loc_se.group(20))/3600 - float(loc_se.group(21))/360000, 8)
                if loc_se.group(8).strip(string.digits) == '':
                    ILS_CATEGORY = loc_se.group(8)
                else:
                    ILS_CATEGORY = 0
                LOCALIZER_IDENT = loc_se.group(7).strip()
                if loc_se.group(22).lstrip(string.digits) == "T":
                    LOCALIZER_BEARING = float(Decimal(loc_se.group(22).rstrip('T')))
                else:
                    LOCALIZER_BEARING = float(Decimal(loc_se.group(22)[0:3] + '.' + loc_se.group(22)[3:]).quantize(Decimal('0'), rounding=ROUND_HALF_UP))
                if loc_se.group(39).strip() == '':
                    GLIDE_SLOPE_HEIGHT_AT_LANDING_THLD = 50
                else:
                    GLIDE_SLOPE_HEIGHT_AT_LANDING_THLD = loc_se.group(39)
                if loc_se.group(40).strip() == '':
                    GLIDE_SLOPE_ELEV = 0
                else:
                    GLIDE_SLOPE_ELEV = loc_se.group(40)
                DME_ID = WaypointLookup_tfdi_tb.loc[(WaypointLookup_tfdi_tb['Ident'] == loc_se.group(7).strip()) & (WaypointLookup_tfdi_tb['REGION_CODE'] == ICAO)].index.tolist()
                if DME_ID != []:
                    HasDme = 1
                    DME_ID = WaypointLookup_tfdi_tb.at[DME_ID[0], 'ID']
                else:
                    if tfdi_create_blank != True:
                        DME_ID = WaypointLookup_in_tfdi_nd_tb.loc[(WaypointLookup_in_tfdi_nd_tb['Ident'] == loc_se.group(7).strip()) & (WaypointLookup_in_tfdi_nd_tb['Country'] == loc_se.group(5))].index.tolist()
                        if DME_ID != []:
                            HasDme = 1
                            DME_ID = Waypoints_in_tfdi_nd_tb.at[DME_ID[0], 'ID']
                        else:
                            HasDme = 0
                    else:
                        HasDme = 0
                ILSID = ILSID + 1
                ILSes_tfdi.append((ILSID,AirportID,RwyID,NAV_FREQ,GLIDE_SLOPE_ANGLE,LOCALIZER_LAT,LOCALIZER_LON,ILS_CATEGORY,LOCALIZER_IDENT,LOCALIZER_BEARING,GLIDE_SLOPE_HEIGHT_AT_LANDING_THLD,GLIDE_SLOPE_ELEV,HasDme))
        # Convert list to Pandas DataFrame
        AirwayLookup_tfdi_tb = pandas.DataFrame(AirwayLookup_tfdi, columns=['CUST_AREA','ROUTE_IDENT','AirwayID','SEQ_NR','FIX_IDENT','FIX_ICAO_CODE','FIX_ID','EndOfRoute','LEVEL','DIRECTION_RESTRICTION'])
        AirwayLookup_tfdi_tb = AirwayLookup_tfdi_tb.sort_values(['ROUTE_IDENT','SEQ_NR','EndOfRoute'], ascending=[1,1,0])
        AirwayLookup_tfdi_tb = AirwayLookup_tfdi_tb.reset_index(drop=True)
        AirwayLookup_tfdi = []
        ILSes_tfdi_tb = pandas.DataFrame(ILSes_tfdi, columns=['ID','AirportID','RunwayID','Freq','GsAngle','Latitude','Longitude','Category','Ident','LocCourse','CrossingHeight','Elevation','HasDme'])
        print(time.strftime("%H:%M:%S", time.localtime()) , 'TFDi: Data Process Part #2 Finished')
    # Process of TFDi Part #3 (AWY Part #2)
    if len(AirwayLookup_tfdi_tb.index) != 0:
        for index, row in AirwayLookup_tfdi_tb.iterrows():
            if index + 1 != len(AirwayLookup_tfdi_tb.index):
                if row['FIX_ID'] != AirwayLookup_tfdi_tb.at[index + 1, 'FIX_ID']:
                    AirwayLookup_tfdi.append((row['CUST_AREA'],row['ROUTE_IDENT'],row['AirwayID'],row['SEQ_NR'],row['FIX_IDENT'],row['FIX_ICAO_CODE'],row['FIX_ID'],row['EndOfRoute'],row['LEVEL'],row['DIRECTION_RESTRICTION']))
                else:
                    if row['AirwayID'] != AirwayLookup_tfdi_tb.at[index + 1, 'AirwayID']:
                        AirwayLookup_tfdi.append((row['CUST_AREA'],row['ROUTE_IDENT'],row['AirwayID'],row['SEQ_NR'],row['FIX_IDENT'],row['FIX_ICAO_CODE'],row['FIX_ID'],row['EndOfRoute'],row['LEVEL'],row['DIRECTION_RESTRICTION']))
            else:
                AirwayLookup_tfdi.append((row['CUST_AREA'],row['ROUTE_IDENT'],row['AirwayID'],row['SEQ_NR'],row['FIX_IDENT'],row['FIX_ICAO_CODE'],row['FIX_ID'],row['EndOfRoute'],row['LEVEL'],row['DIRECTION_RESTRICTION']))
        AirwayLookup_tfdi_tb = pandas.DataFrame(AirwayLookup_tfdi, columns=['CUST_AREA','ROUTE_IDENT','AirwayID','SEQ_NR','FIX_IDENT','FIX_ICAO_CODE','FIX_ID','EndOfRoute','LEVEL','DIRECTION_RESTRICTION'])
        AirwayLookup_tfdi_tb = AirwayLookup_tfdi_tb.sort_values(['ROUTE_IDENT','SEQ_NR'])
        AirwayLookup_tfdi_tb = AirwayLookup_tfdi_tb.reset_index(drop=True)
        AirwayLookup_reversed_tfdi_tb = AirwayLookup_tfdi_tb.sort_index(ascending=False)
        AirwayLookup_reversed_tfdi_tb = AirwayLookup_reversed_tfdi_tb.reset_index(drop=True)
        for index, row in AirwayLookup_tfdi_tb.iterrows():
            if index + 1 != len(AirwayLookup_tfdi_tb.index):
                Waypoint1 = row['FIX_IDENT']
                Waypoint1ID = row['FIX_ID']
                Waypoint2 = AirwayLookup_tfdi_tb.at[index + 1, 'FIX_IDENT']
                Waypoint2ID = AirwayLookup_tfdi_tb.at[index + 1, 'FIX_ID']
                LEVEL = row['LEVEL']
                if row['ROUTE_IDENT'] == AirwayIdent:
                    if WasEnd == 0:
                        IsStart = 0
                    else:
                        IsStart = 1
                elif row['DIRECTION_RESTRICTION'] == 'B':
                    IsStart = 0
                else:
                    IsStart = 1
                    AirwayID = row['AirwayID']
                    AirwayIdent = row['ROUTE_IDENT']
                if index + 2  == len(AirwayLookup_tfdi_tb.index):
                    IsEnd = 1
                elif AirwayLookup_tfdi_tb.at[index + 1, 'DIRECTION_RESTRICTION'] == 'B':
                    IsEnd = 1
                elif AirwayLookup_tfdi_tb.at[index + 2, 'ROUTE_IDENT'] != row['ROUTE_IDENT']:
                    IsEnd = 1
                elif AirwayLookup_tfdi_tb.at[index + 1, 'EndOfRoute'] == 'E':
                    IsEnd = 1
                else:
                    IsEnd = 0
                if AirwayLookup_tfdi_tb.at[index + 1, 'ROUTE_IDENT'] == row['ROUTE_IDENT']:
                    if row['DIRECTION_RESTRICTION'] != 'B':
                        if row['EndOfRoute'] != 'E':
                            AirwayLegsID = AirwayLegsID + 1
                            AirwayLegs_tfdi.append((AirwayLegsID,AirwayID,LEVEL,Waypoint1ID,Waypoint2ID,IsStart,IsEnd,Waypoint1,Waypoint2))
                            WasEnd = IsEnd
        for index, row in AirwayLookup_reversed_tfdi_tb.iterrows():
            if index + 1 != len(AirwayLookup_reversed_tfdi_tb.index):
                Waypoint1 = row['FIX_IDENT']
                Waypoint1ID = row['FIX_ID']
                Waypoint2 = AirwayLookup_reversed_tfdi_tb.at[index + 1, 'FIX_IDENT']
                Waypoint2ID = AirwayLookup_reversed_tfdi_tb.at[index + 1, 'FIX_ID']
                LEVEL = row['LEVEL']
                if row['ROUTE_IDENT'] == AirwayIdent:
                    if row['EndOfRoute'] == 'E':
                        IsStart = 1
                    elif WasEnd == 1:
                        IsStart = 1
                    else:
                        IsStart = 0
                elif AirwayLookup_reversed_tfdi_tb.at[index + 1, 'DIRECTION_RESTRICTION'] != 'F':
                    IsStart = 1
                    AirwayID = row['AirwayID']
                    AirwayIdent = row['ROUTE_IDENT']
                else:
                    IsStart = 0
                if index + 2  == len(AirwayLookup_reversed_tfdi_tb.index):
                    IsEnd = 1
                elif AirwayLookup_reversed_tfdi_tb.at[index + 2, 'DIRECTION_RESTRICTION'] == 'F':
                    IsEnd = 1
                elif AirwayLookup_reversed_tfdi_tb.at[index + 2, 'ROUTE_IDENT'] != row['ROUTE_IDENT']:
                    IsEnd = 1
                elif AirwayLookup_reversed_tfdi_tb.at[index + 2, 'EndOfRoute'] == 'E':
                    IsEnd = 1
                else:
                    IsEnd = 0
                if AirwayLookup_reversed_tfdi_tb.at[index + 1, 'ROUTE_IDENT'] == row['ROUTE_IDENT']:
                    if AirwayLookup_reversed_tfdi_tb.at[index + 1, 'DIRECTION_RESTRICTION'] != 'F':
                        if AirwayLookup_reversed_tfdi_tb.at[index + 1, 'EndOfRoute'] != 'E':
                            AirwayLegsID = AirwayLegsID + 1
                            AirwayLegs_reversed_tfdi.append((AirwayLegsID,AirwayID,LEVEL,Waypoint1ID,Waypoint2ID,IsStart,IsEnd,Waypoint1,Waypoint2))
                            WasEnd = IsEnd
        # Convert list to Pandas DataFrame
        AirwayLegs_tfdi_tb = pandas.DataFrame(AirwayLegs_tfdi, columns=['ID','AirwayID','Level','Waypoint1ID','Waypoint2ID','IsStart','IsEnd','Waypoint1','Waypoint2'])
        AirwayLegs_reversed_tfdi_tb = pandas.DataFrame(AirwayLegs_reversed_tfdi, columns=['ID','AirwayID','Level','Waypoint1ID','Waypoint2ID','IsStart','IsEnd','Waypoint1','Waypoint2'])
        AirwayLegs_tfdi_tb = pandas.concat([AirwayLegs_tfdi_tb, AirwayLegs_reversed_tfdi_tb], axis=0, ignore_index=True)
        AirwayLegs_tfdi_tb = AirwayLegs_tfdi_tb.sort_values(['AirwayID','ID'])
        AirwayLegs_tfdi_tb = AirwayLegs_tfdi_tb.reset_index(drop=True)
        if add_tailored:
            AirwayLegs_tfdi_tb['ID'] = AirwayLegs_tfdi_tb.index + 200001
        else:
            AirwayLegs_tfdi_tb['ID'] = AirwayLegs_tfdi_tb.index + 1
        print(time.strftime("%H:%M:%S", time.localtime()) , 'TFDi: Data Process Part #3 Finished')
    # Process of TFDi Part #4
    with open('Input/' + Datafile, 'r', encoding='ISO-8859-1') as ARINC424:
        for line in ARINC424:
            proc_se = re.search(pdef, line)
            # PROC of TFDi
            if proc_se:
                ICAO = proc_se.group(4)
                Type = proc_se.group(8)
                if proc_se.group(4) + proc_se.group(7).strip() != ICAO + TerminalName:
                    TerminalID = TerminalID + 1
                    AirportID = Airports_tfdi_tb.loc[(Airports_tfdi_tb['ICAO'] == ICAO)].index.tolist()
                    if AirportID != []:
                        AirportID = Airports_tfdi_tb.at[AirportID[0], 'ID']
                    else:
                        if tfdi_create_blank != True:
                            AirportID = Airports_in_tfdi_nd_tb.loc[(Airports_in_tfdi_nd_tb['ICAO'] == ICAO)].index.tolist()
                            if AirportID != []:
                                AirportID = Airports_in_tfdi_nd_tb.at[AirportID[0], 'ID']
                            else:
                                print('         TFDi: Airport ' + ICAO + ' ID Not Found')
                                AirportID = None
                        else:
                            print('         TFDi: Airport ' + ICAO + ' ID Not Found')
                            AirportID = None
                    if proc_se.group(6) == 'D':
                        Proc = '2'
                    elif proc_se.group(6) == 'E':
                        Proc = '1'
                    else:
                        Proc = '3'
                    if Proc == '1' or Proc == '2':
                        if Type == '2' or Type == '5':
                            if proc_se.group(9)[0:2] + proc_se.group(9)[4:5].strip('LRCT ') == 'RW':
                                Rwy = proc_se.group(9).strip().replace('RW','')
                                RwyID = Runways_tfdi_tb.loc[(Runways_tfdi_tb['AirportID'] == AirportID) & (Runways_tfdi_tb['Ident'] == Rwy)].index.tolist()
                                if RwyID != []:
                                    RwyID = Runways_tfdi_tb.at[RwyID[0], 'ID']
                                else:
                                    if tfdi_create_blank != True:
                                        RwyID = Runways_in_tfdi_nd_tb.loc[(Runways_in_tfdi_nd_tb['AirportID'] == AirportID) & (Runways_in_tfdi_nd_tb['Ident'] == Rwy)].index.tolist()
                                        if RwyID != []:
                                            RwyID = Runways_in_tfdi_nd_tb.at[RwyID[0], 'ID']
                                        else:
                                            print('         TFDi: Procedure ' + ICAO + ' '  + proc_se.group(7) + ' Runway ' + Rwy + ' Not Found')
                                            RwyID = None
                                    else:
                                        print('         TFDi: Procedure ' + ICAO + ' '  + proc_se.group(7) + ' Runway ' + Rwy + ' Not Found')
                                        RwyID = None
                            else:
                                Rwy = None
                                RwyID = None
                        else:
                            Rwy = None
                            RwyID = None
                        Name = proc_se.group(7).strip()
                        FullName = Name
                        TerminalName = Name
                    else:
                        if proc_se.group(7)[1:3].strip('- ').strip(string.digits) == '':
                            Rwy = proc_se.group(7)[1:4].strip('- ')
                            RwyID = Runways_tfdi_tb.loc[(Runways_tfdi_tb['AirportID'] == AirportID) & (Runways_tfdi_tb['Ident'] == Rwy)].index.tolist()
                            if RwyID != []:
                                RwyID = Runways_tfdi_tb.at[RwyID[0], 'ID']
                            else:
                                if tfdi_create_blank != True:
                                    RwyID = Runways_in_tfdi_nd_tb.loc[(Runways_in_tfdi_nd_tb['AirportID'] == AirportID) & (Runways_in_tfdi_nd_tb['Ident'] == Rwy)].index.tolist()
                                    if RwyID != []:
                                        RwyID = Runways_in_tfdi_nd_tb.at[RwyID[0], 'ID']
                                    else:
                                        print('         TFDi: Runway ' + ICAO + ' ' + Rwy + ' ID Not Found')
                                        RwyID = None
                                else:
                                    print('         TFDi: Runway ' + ICAO + ' ' + Rwy + ' ID Not Found')
                                    RwyID = None
                        else:
                            Rwy = None
                            RwyID = None
                        Name = proc_se.group(7).strip()
                        FullName = proc_se.group(7) + ' APCH'
                        TerminalName = Name
                    # Duplicated Procedure Will Not Add into NDB
                    if tfdi_create_blank != True:
                        if Terminals_in_tfdi_nd_tb.loc[(Terminals_in_tfdi_nd_tb['ICAO'] == ICAO) & (Terminals_in_tfdi_nd_tb['Name'] == Name)].index.tolist() == []:
                            Terminals_tfdi.append((TerminalID,AirportID,Proc,ICAO,FullName,Name,Rwy,RwyID))
                            Skip_Terminal = 0
                        else:
                            print('         TFDi: Procedure ' + ICAO + ' ' + proc_se.group(7) + ' Skipped Due Duplicate')
                            Skip_Terminal = 1
                    else:
                        Terminals_tfdi.append((TerminalID,AirportID,Proc,ICAO,FullName,Name,Rwy,RwyID))
                        Skip_Terminal = 0
                TerminalLegsID = TerminalLegsID + 1
                if proc_se.group(9).strip() == '':
                    Transition = None
                elif proc_se.group(9)[0:2] + proc_se.group(9)[4:5] == 'RWB':
                    Transition = 'ALL'
                else:
                    Transition = proc_se.group(9).strip()
                TrackCode = proc_se.group(19)
                if proc_se.group(13) + proc_se.group(14) == "  ":
                    WptLat = None
                    WptLon = None
                    WptID = None
                elif proc_se.group(13) + proc_se.group(14) == "PA":
                    WptID = Airports_tfdi_tb.loc[(Airports_tfdi_tb['ICAO'] == ICAO)].index.tolist()
                    if WptID != []:
                        WptLat = Airports_tfdi_tb.at[WptID[0], 'Latitude']
                        WptLon = Airports_tfdi_tb.at[WptID[0], 'Longitude']
                        WptID = None
                    else:
                        if tfdi_create_blank != True:
                            WptID = Airports_in_tfdi_nd_tb.loc[(Airports_in_tfdi_nd_tb['ICAO'] == ICAO)].index.tolist()
                            if WptID != []:
                                WptLat = Airports_in_tfdi_nd_tb.at[WptID[0], 'Latitude']
                                WptLon = Airports_in_tfdi_nd_tb.at[WptID[0], 'Longitude']
                                WptID = None
                            else:
                                print('         TFDi: Leg ' + ICAO + ' ' + proc_se.group(7) + ' Airport LAT/LON Not Found')
                                WptLat = None
                                WptLon = None
                                WptID = None
                        else:
                            print('         TFDi: Leg ' + ICAO + ' ' + proc_se.group(7) + ' Airport LAT/LON Not Found')
                            WptLat = None
                            WptLon = None
                            WptID = None
                elif proc_se.group(13) + proc_se.group(14) == "PG":
                    WptID = Runways_tfdi_tb.loc[(Runways_tfdi_tb['AirportID'] == AirportID) & (Runways_tfdi_tb['Ident'] == proc_se.group(11).strip().replace('RW',''))].index.tolist()
                    if WptID != []:
                        WptLat = Runways_tfdi_tb.at[WptID[0], 'Latitude']
                        WptLon = Runways_tfdi_tb.at[WptID[0], 'Longitude']
                        WptID = None
                    else:
                        if tfdi_create_blank != True:
                            WptID = Runways_in_tfdi_nd_tb.loc[(Runways_in_tfdi_nd_tb['AirportID'] == AirportID) & (Runways_in_tfdi_nd_tb['Ident'] == proc_se.group(11).strip().replace('RW',''))].index.tolist()
                            if WptID != []:
                                WptLat = Runways_in_tfdi_nd_tb.at[WptID[0], 'Latitude']
                                WptLon = Runways_in_tfdi_nd_tb.at[WptID[0], 'Longitude']
                                WptID = None
                            else:
                                print('         TFDi: Leg ' + ICAO + ' ' + proc_se.group(7) + ' ' + Rwy + ' Rwy LAT/LON Not Found')
                                WptLat = None
                                WptLon = None
                                WptID = None
                        else:
                            print('         TFDi: Leg ' + ICAO + ' ' + proc_se.group(7) + ' ' + Rwy + ' Rwy LAT/LON Not Found')
                            WptLat = None
                            WptLon = None
                            WptID = None
                elif proc_se.group(13) + proc_se.group(14) == "PC":
                    WptID = WaypointLookup_tfdi_tb.loc[(WaypointLookup_tfdi_tb['Ident'] == proc_se.group(11).strip()) & (WaypointLookup_tfdi_tb['Country'] == proc_se.group(12)) & (WaypointLookup_tfdi_tb['REGION_CODE'] == ICAO)].index.tolist()
                    if WptID != []:
                        WptID = WaypointLookup_tfdi_tb.at[WptID[0], 'ID']
                        WptLat = Waypoints_tfdi_tb.loc[Waypoints_tfdi_tb.loc[(Waypoints_tfdi_tb['ID'] == WptID)].index.tolist() , 'Latitude'].values[0]
                        WptLon = Waypoints_tfdi_tb.loc[Waypoints_tfdi_tb.loc[(Waypoints_tfdi_tb['ID'] == WptID)].index.tolist() , 'Longitude'].values[0]
                    else:
                        if tfdi_create_blank != True:
                            WptID = TerminalWpts_in_tfdi_nd_tb.loc[(TerminalWpts_in_tfdi_nd_tb['WptIdent'] == proc_se.group(11).strip()) & (TerminalWpts_in_tfdi_nd_tb['REGION_CODE'] == ICAO)].index.tolist()
                            if WptID != []:
                                WptID = TerminalWpts_in_tfdi_nd_tb.at[WptID[0], 'WptID']
                                WptLat = Waypoints_in_tfdi_nd_tb.loc[Waypoints_in_tfdi_nd_tb.loc[(Waypoints_in_tfdi_nd_tb['ID'] == WptID)].index.tolist() , 'Latitude'].values[0]
                                WptLon = Waypoints_in_tfdi_nd_tb.loc[Waypoints_in_tfdi_nd_tb.loc[(Waypoints_in_tfdi_nd_tb['ID'] == WptID)].index.tolist() , 'Longitude'].values[0]
                            else:
                                WptID = WaypointLookup_in_tfdi_nd_tb.loc[(WaypointLookup_in_tfdi_nd_tb['Ident'] == proc_se.group(11).strip()) & (WaypointLookup_in_tfdi_nd_tb['Country'] == proc_se.group(12))].index.tolist()
                                if WptID != []:
                                    WptID = WaypointLookup_in_tfdi_nd_tb.at[WptID[0], 'ID']
                                    WptLat = Waypoints_in_tfdi_nd_tb.loc[Waypoints_in_tfdi_nd_tb.loc[(Waypoints_in_tfdi_nd_tb['ID'] == WptID)].index.tolist() , 'Latitude'].values[0]
                                    WptLon = Waypoints_in_tfdi_nd_tb.loc[Waypoints_in_tfdi_nd_tb.loc[(Waypoints_in_tfdi_nd_tb['ID'] == WptID)].index.tolist() , 'Longitude'].values[0]
                                else:
                                    print('         TFDi: Leg ' + ICAO + ' ' + proc_se.group(7) + ' ' + proc_se.group(11) + ' WptID Not Found')
                                    WptID = None
                                    WptLat = None
                                    WptLon = None
                        else:
                            print('         TFDi: Leg ' + ICAO + ' ' + proc_se.group(7) + ' ' + proc_se.group(11) + ' WptID Not Found')
                            WptID = None
                            WptLat = None
                            WptLon = None
                else:
                    WptID = WaypointLookup_tfdi_tb.loc[(WaypointLookup_tfdi_tb['Ident'] == proc_se.group(11).strip()) & (WaypointLookup_tfdi_tb['Country'] == proc_se.group(12))].index.tolist()
                    if WptID != []:
                        WptID = WaypointLookup_tfdi_tb.at[WptID[0], 'ID']
                        WptLat = Waypoints_tfdi_tb.loc[Waypoints_tfdi_tb.loc[(Waypoints_tfdi_tb['ID'] == WptID)].index.tolist() , 'Latitude'].values[0]
                        WptLon = Waypoints_tfdi_tb.loc[Waypoints_tfdi_tb.loc[(Waypoints_tfdi_tb['ID'] == WptID)].index.tolist() , 'Longitude'].values[0]
                    else:
                        if tfdi_create_blank != True:
                            WptID = WaypointLookup_in_tfdi_nd_tb.loc[(WaypointLookup_in_tfdi_nd_tb['Ident'] == proc_se.group(11).strip()) & (WaypointLookup_in_tfdi_nd_tb['Country'] == proc_se.group(12))].index.tolist()
                            if WptID != []:
                                WptID = WaypointLookup_in_tfdi_nd_tb.at[WptID[0], 'ID']
                                WptLat = Waypoints_in_tfdi_nd_tb.loc[Waypoints_in_tfdi_nd_tb.loc[(Waypoints_in_tfdi_nd_tb['ID'] == WptID)].index.tolist() , 'Latitude'].values[0]
                                WptLon = Waypoints_in_tfdi_nd_tb.loc[Waypoints_in_tfdi_nd_tb.loc[(Waypoints_in_tfdi_nd_tb['ID'] == WptID)].index.tolist() , 'Longitude'].values[0]
                            else:
                                print('         TFDi: Leg ' + ICAO + ' ' + proc_se.group(7) + ' ' + proc_se.group(11) + ' WptID Not Found')
                                WptID = None
                                WptLat = None
                                WptLon = None
                        else:
                            print('         TFDi: Leg ' + ICAO + ' ' + proc_se.group(7) + ' ' + proc_se.group(11) + ' WptID Not Found')
                            WptID = None
                            WptLat = None
                            WptLon = None
                if proc_se.group(17).strip() == '':
                    TurnDir = None
                else:
                    TurnDir = proc_se.group(17)
                if proc_se.group(28) + proc_se.group(29) == '  ':
                    NavLat = None
                    NavLon = None
                    NavID = None
                elif proc_se.group(28) + proc_se.group(29) == 'PI':
                    ILSIdent = proc_se.group(21).strip()
                    NavID = ILSes_tfdi_tb.loc[(ILSes_tfdi_tb['AirportID'] == AirportID) & (ILSes_tfdi_tb['Ident'] == ILSIdent)].index.tolist()
                    if NavID != []:
                        NavLat = ILSes_tfdi_tb.at[NavID[0], 'Latitude']
                        NavLon = ILSes_tfdi_tb.at[NavID[0], 'Longitude']
                        NavID = None
                    else:
                        if tfdi_create_blank != True:
                            NavID = ILSes_in_tfdi_nd_tb.loc[(ILSes_in_tfdi_nd_tb['AirportID'] == AirportID) & (ILSes_in_tfdi_nd_tb['Ident'] == ILSIdent)].index.tolist()
                            if NavID != []:
                                NavLat = ILSes_in_tfdi_nd_tb.at[NavID[0], 'Latitude']
                                NavLon = ILSes_in_tfdi_nd_tb.at[NavID[0], 'Longitude']
                                NavID = None
                            else:
                                print('         TFDi: Leg ' + ICAO + ' ' + proc_se.group(7) + ' ' + proc_se.group(11) + ' Recommend ILS ' + proc_se.group(21) + ' Not Found')
                                NavLat = None
                                NavLon = None
                                NavID = None
                        else:
                            print('         TFDi: Leg ' + ICAO + ' ' + proc_se.group(7) + ' ' + proc_se.group(11) + ' Recommend ILS ' + proc_se.group(21) + ' Not Found')
                            NavLat = None
                            NavLon = None
                            NavID = None
                elif proc_se.group(28) + proc_se.group(29) == 'PT':
                    NavLat = None
                    NavLon = None
                    NavID = None
                else:
                    NavID = NavaidLookup_tfdi_tb.loc[(NavaidLookup_tfdi_tb['Ident'] == proc_se.group(21).strip()) & (NavaidLookup_tfdi_tb['Country'] == proc_se.group(22))].index.tolist()
                    if NavID != []:
                        NavID = NavaidLookup_tfdi_tb.at[NavID[0], 'ID']
                        NavLat = Navaids_tfdi_tb.loc[Navaids_tfdi_tb.loc[(Navaids_tfdi_tb['ID'] == NavID)].index.tolist() , 'Latitude'].values[0]
                        NavLon = Navaids_tfdi_tb.loc[Navaids_tfdi_tb.loc[(Navaids_tfdi_tb['ID'] == NavID)].index.tolist() , 'Longitude'].values[0]
                    else:
                        if tfdi_create_blank != True:
                            NavID = NavaidLookup_in_tfdi_nd_tb.loc[(NavaidLookup_in_tfdi_nd_tb['Ident'] == proc_se.group(21).strip()) & (NavaidLookup_in_tfdi_nd_tb['Country'] == proc_se.group(22))].index.tolist()
                            if NavID != []:
                                NavID = NavaidLookup_in_tfdi_nd_tb.at[NavID[0], 'ID']
                                NavLat = Navaids_in_tfdi_nd_tb.loc[Navaids_in_tfdi_nd_tb.loc[(Navaids_in_tfdi_nd_tb['ID'] == NavID)].index.tolist() , 'Latitude'].values[0]
                                NavLon = Navaids_in_tfdi_nd_tb.loc[Navaids_in_tfdi_nd_tb.loc[(Navaids_in_tfdi_nd_tb['ID'] == NavID)].index.tolist() , 'Longitude'].values[0]
                            else:
                                print('         TFDi: Leg ' + ICAO + ' ' + proc_se.group(7) + ' ' + proc_se.group(11) + ' Recommend NavID ' + proc_se.group(21) + ' Not Found')
                                NavID = None
                                NavLat = None
                                NavLon = None
                        else:
                            print('         TFDi: Leg ' + ICAO + ' ' + proc_se.group(7) + ' ' + proc_se.group(11) + ' Recommend NavID ' + proc_se.group(21) + ' Not Found')
                            NavID = None
                            NavLat = None
                            NavLon = None
                if proc_se.group(24).strip() == '':
                    NavBear = None
                else:
                    NavBear = float(proc_se.group(24))/10
                if proc_se.group(25).strip() == '':
                    NavDist = None
                else:
                    NavDist = float(proc_se.group(25))/10
                if proc_se.group(26).strip() == '':
                    Course = None
                elif proc_se.group(26).lstrip(string.digits) == "T":
                    Course = proc_se.group(26).rstrip('T')
                else:
                    Course = float(proc_se.group(26))/10
                if proc_se.group(27).strip() == '':
                    Distance = None
                elif proc_se.group(27).rstrip(string.digits) == "T":
                    Distance = float(proc_se.group(27).lstrip('T'))/2
                else:
                    Distance = float(proc_se.group(27))/10
                if proc_se.group(32).strip() == '':
                    ALT_1 = None
                elif proc_se.group(32).rstrip(string.digits) == "FL":
                    ALT_1 = proc_se.group(32).lstrip(string.ascii_uppercase) + "00"
                else:
                    ALT_1 = proc_se.group(32)
                if proc_se.group(33).strip() == '':
                    ALT_2 = None
                elif proc_se.group(33).rstrip(string.digits) == "FL":
                    ALT_2 = proc_se.group(33).lstrip(string.ascii_uppercase) + "00"
                else:
                    ALT_2 = proc_se.group(33)
                if proc_se.group(30).strip() == '+':
                    Alt = ALT_1 + 'A'
                elif proc_se.group(30).strip() == '-':
                    Alt = ALT_1 + 'B'
                elif proc_se.group(30).strip() == 'G':
                    Alt = ALT_2 + ''
                elif proc_se.group(30).strip() == 'H':
                    Alt = ALT_2 + 'A'
                elif proc_se.group(30).strip() == 'I':
                    Alt = ALT_1 + ''
                elif proc_se.group(30).strip() == 'J':
                    Alt = ALT_1 + 'A'
                else:
                    if proc_se.group(16)[3:4] == 'M':
                        Alt = 'MAP'
                    elif ALT_1 != None:
                        Alt = ALT_1 + ''
                    else:
                        Alt = None
                if proc_se.group(16)[3:4] != 'F':
                    IsFAF = '0'
                else:
                    IsFAF = '-1'
                if proc_se.group(16)[3:4] != 'M':
                    IsMAP = '0'
                else:
                    IsMAP = '-1'
                if proc_se.group(36).strip() == '':
                    Vnav = None
                else:
                    Vnav = round(float(proc_se.group(36)[1:4])/100, 2)
                if TrackCode == 'RF':
                    CenterID = WaypointLookup_tfdi_tb.loc[(WaypointLookup_tfdi_tb['Ident'] == proc_se.group(37).strip()) & (WaypointLookup_tfdi_tb['Country'] == proc_se.group(39))].index.tolist()
                    if CenterID != []:
                        CenterID = WaypointLookup_tfdi_tb.at[CenterID[0], 'ID']
                        CenterLat = Waypoints_tfdi_tb.loc[Waypoints_tfdi_tb.loc[(Waypoints_tfdi_tb['ID'] == CenterID)].index.tolist() , 'Latitude'].values[0]
                        CenterLon = Waypoints_tfdi_tb.loc[Waypoints_tfdi_tb.loc[(Waypoints_tfdi_tb['ID'] == CenterID)].index.tolist() , 'Longitude'].values[0]
                    else:
                        if tfdi_create_blank != True:
                            CenterID = WaypointLookup_in_tfdi_nd_tb.loc[(WaypointLookup_in_tfdi_nd_tb['Ident'] == proc_se.group(37).strip()) & (WaypointLookup_in_tfdi_nd_tb['Country'] == proc_se.group(39))].index.tolist()
                            if CenterID != []:
                                CenterID = WaypointLookup_in_tfdi_nd_tb.at[CenterID[0], 'ID']
                                CenterLat = Waypoints_in_tfdi_nd_tb.loc[Waypoints_in_tfdi_nd_tb.loc[(Waypoints_in_tfdi_nd_tb['ID'] == CenterID)].index.tolist() , 'Latitude'].values[0]
                                CenterLon = Waypoints_in_tfdi_nd_tb.loc[Waypoints_in_tfdi_nd_tb.loc[(Waypoints_in_tfdi_nd_tb['ID'] == CenterID)].index.tolist() , 'Longitude'].values[0]
                            else:
                                print('         TFDi: Leg ' + ICAO + ' ' + proc_se.group(7) + ' ' + proc_se.group(11) + ' CenterID ' + proc_se.group(37) + ' Not Found')
                                CenterID = None
                                CenterLat = None
                                CenterLon = None
                        else:
                            print('         TFDi: Leg ' + ICAO + ' ' + proc_se.group(7) + ' ' + proc_se.group(11) + ' CenterID ' + proc_se.group(37) + ' Not Found')
                            CenterID = None
                            CenterLat = None
                            CenterLon = None
                else:
                    CenterID = None
                    CenterLat = None
                    CenterLon = None
                if proc_se.group(16)[1:2] == 'B':
                    IsFlyOver = '-1'
                elif proc_se.group(16)[1:2] == 'Y':
                    IsFlyOver = '-1'
                else:
                    IsFlyOver = '0'
                if proc_se.group(35).strip() == '':
                    SpeedLimit = None
                else:
                    SpeedLimit = proc_se.group(35).strip()
                # Duplicated Procedure Will Not Add into NDB
                if Skip_Terminal == 0:
                    TerminalLegs_tfdi.append((TerminalLegsID,TerminalID,Type,Transition,TrackCode,WptID,WptLat,WptLon,TurnDir,NavID,NavLat,NavLon,NavBear,NavDist,Course,Distance,Alt,Vnav,CenterID,CenterLat,CenterLon))
                    TerminalLegsEx_tfdi.append((TerminalLegsID,IsFlyOver,SpeedLimit,IsFAF,IsMAP))
        # Convert list to Pandas DataFrame
        TerminalLegs_tfdi_tb = pandas.DataFrame(TerminalLegs_tfdi, columns=['ID','TerminalID','Type','Transition','TrackCode','WptID','WptLat','WptLon','TurnDir','NavID','NavLat','NavLon','NavBear','NavDist','Course','Distance','Alt','Vnav','CenterID','CenterLat','CenterLon'])
        TerminalLegsEx_tfdi_tb = pandas.DataFrame(TerminalLegsEx_tfdi, columns=['ID','IsFlyOver','SpeedLimit','IsFAF','IsMAP'])
        Terminals_tfdi_tb = pandas.DataFrame(Terminals_tfdi, columns=['ID','AirportID','Proc','ICAO','FullName','Name','Rwy','RwyID'])
        print(time.strftime("%H:%M:%S", time.localtime()) , 'TFDi: Data Process Part #4 Finished')
    # Write Data
    if tfdi_write:
        if tfdi_create_blank:
            mdb.execute('CREATE TABLE "config" ("key" TEXT(50) WITH COMPRESSION, "val" TEXT(50) WITH COMPRESSION, CONSTRAINT "PrimaryKey" PRIMARY KEY ("key"));')
            mdb.execute('CREATE TABLE "NavaidTypes" ("Type" TINYINT DEFAULT 0, "Desc" TEXT(24) WITH COMPRESSION, CONSTRAINT "PrimaryKey" PRIMARY KEY ("Type"));')
            mdb.execute('CREATE TABLE "SurfaceTypes" ("SurfaceType" TEXT(3) WITH COMPRESSION, "Description" TEXT(255) WITH COMPRESSION, CONSTRAINT "PrimaryKey" PRIMARY KEY ("SurfaceType"));')
            mdb.execute('CREATE TABLE "TrmLegTypes" ("Code" TEXT(2) WITH COMPRESSION NOT NULL, "Description" TEXT(100) WITH COMPRESSION, CONSTRAINT "PrimaryKey" PRIMARY KEY ("Code"));')
            mdb.execute('CREATE TABLE "AirportLookup" ("extID" TEXT(7) WITH COMPRESSION, "ID" LONG DEFAULT 0, CONSTRAINT "PrimaryKey" PRIMARY KEY ("extID", "ID"));')
            mdb.execute('CREATE TABLE "Airports" ("ID" LONG NOT NULL DEFAULT 0, "Name" TEXT(38) WITH COMPRESSION, "ICAO" TEXT(4) WITH COMPRESSION, "PrimaryID" LONG, "Latitude" DOUBLE DEFAULT 0, "Longitude" DOUBLE, "Elevation" SMALLINT DEFAULT 0, "TransAlt" SMALLINT DEFAULT 0, CONSTRAINT "PrimaryKey" PRIMARY KEY ("ID"));')
            mdb.execute('CREATE TABLE "Runways" ("ID" LONG NOT NULL DEFAULT 0, "AirportID" LONG DEFAULT 0, "Ident" TEXT(3) WITH COMPRESSION, "TrueHeading" SINGLE DEFAULT 0, "Length" SMALLINT DEFAULT 0, "Width" SMALLINT DEFAULT 0, "Surface" TEXT(3) WITH COMPRESSION, "Latitude" DOUBLE DEFAULT 0, "Longitude" DOUBLE, "Elevation" SMALLINT DEFAULT 0, CONSTRAINT "PrimaryKey" PRIMARY KEY ("ID"));')
            mdb.execute('CREATE TABLE "Navaids" ("ID" LONG NOT NULL DEFAULT 0, "Ident" TEXT(4) WITH COMPRESSION, "Type" TINYINT DEFAULT 0, "Name" TEXT(38) WITH COMPRESSION, "Freq" LONG, "Channel" TEXT(4) WITH COMPRESSION, "Usage" TEXT(1) WITH COMPRESSION, "Latitude" DOUBLE DEFAULT 0, "Longitude" DOUBLE, "Elevation" SMALLINT DEFAULT 0, "SlavedVar" SINGLE DEFAULT 0, CONSTRAINT "PrimaryKey" PRIMARY KEY ("ID"));')
            mdb.execute('CREATE TABLE "NavaidLookup" ("Ident" TEXT(4) WITH COMPRESSION NOT NULL, "Type" TINYINT NOT NULL DEFAULT 0, "Country" TEXT(2) WITH COMPRESSION NOT NULL, "NavKeyCode" TINYINT NOT NULL DEFAULT 0, "ID" LONG NOT NULL DEFAULT 0, CONSTRAINT "NavaidsLookupExt" PRIMARY KEY ("Ident", "Type", "Country", "NavKeyCode"));')
            mdb.execute('CREATE TABLE "Waypoints" ("ID" LONG NOT NULL DEFAULT 0, "Ident" TEXT(5) WITH COMPRESSION NOT NULL, "Name" TEXT(25) WITH COMPRESSION, "Latitude" DOUBLE DEFAULT 0, "NavaidID" LONG, "Longitude" DOUBLE, "Collocated" LONG DEFAULT 0, CONSTRAINT "PrimaryKey" PRIMARY KEY ("ID"));')
            mdb.execute('CREATE TABLE "WaypointLookup" ("Ident" TEXT(5) WITH COMPRESSION NOT NULL, "Country" TEXT(2) WITH COMPRESSION NOT NULL, "ID" LONG NOT NULL DEFAULT 0, CONSTRAINT "PrimaryKey" PRIMARY KEY ("ID", "Ident", "Country"));')
            mdb.execute('CREATE TABLE "Airways" ("ID" LONG NOT NULL DEFAULT 0, "Ident" TEXT(6) WITH COMPRESSION NOT NULL, CONSTRAINT "PrimaryKey" PRIMARY KEY ("ID"));')
            mdb.execute('CREATE TABLE "AirwayLegs" ("ID" LONG NOT NULL DEFAULT 0, "AirwayID" LONG DEFAULT 0, "Level" TEXT(1) WITH COMPRESSION, "Waypoint1ID" LONG, "Waypoint2ID" LONG, "IsStart" LONG DEFAULT 0, "IsEnd" LONG DEFAULT 0, "Waypoint1" TEXT(255) WITH COMPRESSION, "Waypoint2" TEXT(255) WITH COMPRESSION, CONSTRAINT "PrimaryKey" PRIMARY KEY ("ID"));')
            mdb.execute('CREATE TABLE "ILSes" ("ID" LONG NOT NULL DEFAULT 0, "RunwayID" LONG DEFAULT 0, "Freq" LONG DEFAULT 0, "GsAngle" SINGLE DEFAULT 0, "Latitude" DOUBLE DEFAULT 0, "Longitude" DOUBLE, "Category" TINYINT DEFAULT 0, "Ident" TEXT(5) WITH COMPRESSION, "LocCourse" SINGLE DEFAULT 0, "CrossingHeight" TINYINT DEFAULT 0, "Elevation" SMALLINT DEFAULT 0, "HasDme" LONG DEFAULT 0, CONSTRAINT "PrimaryKey" PRIMARY KEY ("ID"));')
            mdb.execute('CREATE TABLE "Terminals" ("ID" LONG NOT NULL DEFAULT 0, "AirportID" LONG DEFAULT 0, "Proc" TINYINT DEFAULT 0, "ICAO" TEXT(4) WITH COMPRESSION NOT NULL, "FullName" TEXT(28) WITH COMPRESSION NOT NULL, "Name" TEXT(8) WITH COMPRESSION NOT NULL, "Rwy" TEXT(3) WITH COMPRESSION, "RwyID" LONG DEFAULT 0, CONSTRAINT "PrimaryKey" PRIMARY KEY ("ID"), CONSTRAINT "TrmId" UNIQUE ("AirportID", "Proc", "FullName"));')
            mdb.execute('CREATE TABLE "TerminalLegsEx" ("ID" LONG NOT NULL DEFAULT 0, "IsFlyOver" LONG DEFAULT 0, "SpeedLimit" SINGLE DEFAULT 0, "IsFAF" LONG DEFAULT 0, "IsMAP" LONG DEFAULT 0, CONSTRAINT "PrimaryKey" PRIMARY KEY ("ID"));')
            mdb.execute('CREATE TABLE "TerminalLegs" ("ID" LONG DEFAULT 0, "TerminalID" LONG NOT NULL DEFAULT 0, "Type" TEXT(1) WITH COMPRESSION, "Transition" TEXT(5) WITH COMPRESSION, "TrackCode" TEXT(2) WITH COMPRESSION, "WptID" LONG DEFAULT 0, "WptLat" DOUBLE DEFAULT 0, "WptLon" DOUBLE DEFAULT 0, "TurnDir" TEXT(1) WITH COMPRESSION, "NavID" LONG DEFAULT 0, "NavLat" DOUBLE DEFAULT 0, "NavLon" DOUBLE DEFAULT 0, "NavBear" SINGLE DEFAULT 0, "NavDist" SINGLE DEFAULT 0, "Course" SINGLE DEFAULT 0, "Distance" SINGLE DEFAULT 0, "Alt" TEXT(12) WITH COMPRESSION, "Vnav" SINGLE DEFAULT 0, "CenterID" LONG DEFAULT 0, "CenterLat" DOUBLE DEFAULT 0, "CenterLon" DOUBLE DEFAULT 0, CONSTRAINT "PrimaryKey" PRIMARY KEY ("ID"));')
            mdb.execute('CREATE INDEX "TerminalID" ON "TerminalLegs" ("TerminalID");')
            mdb.execute('CREATE INDEX "Code" ON "TrmLegTypes" ("Code");')
            mdb.execute('ALTER TABLE "AirportLookup" ADD CONSTRAINT "AirportAirportLookup" FOREIGN KEY ("ID") REFERENCES  "Airports" ("ID") ON UPDATE CASCADE ON DELETE CASCADE;')
            mdb.execute('ALTER TABLE "Airports" ADD CONSTRAINT "AirportsAirports" FOREIGN KEY ("PrimaryID") REFERENCES  "Airports" ("ID") ON UPDATE CASCADE ON DELETE CASCADE;')
            mdb.execute('ALTER TABLE "AirwayLegs" ADD CONSTRAINT "AirwaysAirwayLegs" FOREIGN KEY ("AirwayID") REFERENCES  "Airways" ("ID") ON UPDATE CASCADE ON DELETE CASCADE;')
            mdb.execute('ALTER TABLE "AirwayLegs" ADD CONSTRAINT "WaypointsAirwayLegs" FOREIGN KEY ("Waypoint1ID") REFERENCES  "Waypoints" ("ID") ON UPDATE CASCADE ON DELETE CASCADE;')
            mdb.execute('ALTER TABLE "AirwayLegs" ADD CONSTRAINT "WaypointsAirwayLegs1" FOREIGN KEY ("Waypoint2ID") REFERENCES  "Waypoints" ("ID") ON UPDATE CASCADE ON DELETE CASCADE;')
            mdb.execute('ALTER TABLE "ILSes" ADD CONSTRAINT "RunwaysILSes" FOREIGN KEY ("RunwayID") REFERENCES  "Runways" ("ID") ON UPDATE CASCADE ON DELETE CASCADE;')
            mdb.execute('ALTER TABLE "NavaidLookup" ADD CONSTRAINT "NavaidsNavaidsLookup" FOREIGN KEY ("ID") REFERENCES  "Navaids" ("ID") ON UPDATE CASCADE ON DELETE CASCADE;')
            mdb.execute('ALTER TABLE "Navaids" ADD CONSTRAINT "NavaidTypesNavaids" FOREIGN KEY ("Type") REFERENCES  "NavaidTypes" ("Type") ON UPDATE CASCADE ON DELETE CASCADE;')
            mdb.execute('ALTER TABLE "Runways" ADD CONSTRAINT "AirportsRunways" FOREIGN KEY ("AirportID") REFERENCES  "Airports" ("ID") ON UPDATE CASCADE ON DELETE CASCADE;')
            mdb.execute('ALTER TABLE "Runways" ADD CONSTRAINT "SurfaceTypesRunways" FOREIGN KEY ("Surface") REFERENCES  "SurfaceTypes" ("SurfaceType") ON UPDATE CASCADE ON DELETE CASCADE;')
            mdb.execute('ALTER TABLE "TerminalLegs" ADD CONSTRAINT "NavaidsTerminalLegs" FOREIGN KEY ("NavID") REFERENCES  "Navaids" ("ID") ON UPDATE CASCADE ON DELETE CASCADE;')
            mdb.execute('ALTER TABLE "TerminalLegs" ADD CONSTRAINT "TerminalLegsExTerminalLegs" FOREIGN KEY NO INDEX ("ID") REFERENCES  "TerminalLegsEx" ("ID");')
            mdb.execute('ALTER TABLE "TerminalLegs" ADD CONSTRAINT "TerminalsTerminalLegs" FOREIGN KEY ("TerminalID") REFERENCES  "Terminals" ("ID") ON UPDATE CASCADE ON DELETE CASCADE;')
            mdb.execute('ALTER TABLE "TerminalLegs" ADD CONSTRAINT "TrmLegTypesTerminalLegs" FOREIGN KEY ("TrackCode") REFERENCES  "TrmLegTypes" ("Code") ON UPDATE CASCADE ON DELETE CASCADE;')
            mdb.execute('ALTER TABLE "TerminalLegs" ADD CONSTRAINT "WaypointsTerminalLegs" FOREIGN KEY ("WptID") REFERENCES  "Waypoints" ("ID") ON UPDATE CASCADE ON DELETE CASCADE;')
            mdb.execute('ALTER TABLE "Terminals" ADD CONSTRAINT "AirportsTerminals" FOREIGN KEY ("AirportID") REFERENCES  "Airports" ("ID") ON UPDATE CASCADE ON DELETE CASCADE;')
            mdb.execute('ALTER TABLE "WaypointLookup" ADD CONSTRAINT "WaypointsWaypointsLookup" FOREIGN KEY ("ID") REFERENCES  "Waypoints" ("ID") ON UPDATE CASCADE ON DELETE CASCADE;')
            mdb.execute('ALTER TABLE "Waypoints" ADD CONSTRAINT "NavaidsWaypoints" FOREIGN KEY ("NavaidID") REFERENCES  "Navaids" ("ID") ON UPDATE CASCADE ON DELETE CASCADE;')
            NavaidTypes_tfdi = [[1, 'VOR']]
            NavaidTypes_tfdi.append([2, 'VORTAC'])
            NavaidTypes_tfdi.append([3, 'TACAN'])
            NavaidTypes_tfdi.append([4, 'VOR-DME'])
            NavaidTypes_tfdi.append([5, 'NDB'])
            NavaidTypes_tfdi.append([7, 'NDB-DME'])
            NavaidTypes_tfdi.append([8, 'ILS-DME'])
            NavaidTypes_tfdi.append([9, 'DME (EXCLUDING ILS-DME)'])
            NavaidTypes_tfdi_tb = pandas.DataFrame(NavaidTypes_tfdi,columns=['Type','Desc'])
            NavaidTypes_tfdi_tb.to_sql('NavaidTypes', mdb, if_exists='append', index=False)
            SurfaceTypes_tfdi = [['ASP', 'ASPHALT, ASPHALTIC CONCRETE, TAR MACADAM, OR BITUMEN BOUND MACADAM (INCLUDING ANY OF THESE SURFACE TYPES WITH CONCRETE ENDS).']]
            SurfaceTypes_tfdi.append(['BIT', 'BITUMINOUS, TAR OR ASPHALT MIXED IN PLACE, OILED.'])
            SurfaceTypes_tfdi.append(['BRI', 'BRICK, LAID OR MORTARED.'])
            SurfaceTypes_tfdi.append(['CLA', 'CLAY.'])
            SurfaceTypes_tfdi.append(['COM', 'COMPOSITE, LESS THAN 50 PERCENT OF THE RUNWAY LENGTH IS PERMANENT.'])
            SurfaceTypes_tfdi.append(['CON', 'CONCRETE.'])
            SurfaceTypes_tfdi.append(['COP', 'COMPOSITE, 50 PERCENT OR MORE OF THE RUNWAY LENGTH IS PERMANENT.'])
            SurfaceTypes_tfdi.append(['COR', 'CORAL.'])
            SurfaceTypes_tfdi.append(['GRE', 'GRADED OR ROLLED EARTH, GRASS ON GRADED EARTH.'])
            SurfaceTypes_tfdi.append(['GRS', 'GRASS OR EARTH NOT GRADED OR GRASS OR EARTH NOT GRADED OR ROLLED.'])
            SurfaceTypes_tfdi.append(['GVL', 'GRAVEL.'])
            SurfaceTypes_tfdi.append(['ICE', 'ICE.'])
            SurfaceTypes_tfdi.append(['LAT', 'LATERITE.'])
            SurfaceTypes_tfdi.append(['MAC', 'MACADAM - CRUSHED ROCK WATER BOUND.'])
            SurfaceTypes_tfdi.append(['MEM', 'MEMBRANE - PLASTIC OR OTHER COATED FIBER MATERIAL.'])
            SurfaceTypes_tfdi.append(['MIX', 'MIX IN PLACE USING NONBITUMIOUS BINDERS SUCH AS PORTLAND CEMENT.'])
            SurfaceTypes_tfdi.append(['PEM', 'PART CONCRETE, PART ASPHALT, OR PART BITUMEN-BOUND MACADAM.'])
            SurfaceTypes_tfdi.append(['PER', 'PERMANENT, SURFACE TYPE UNKNOWN.'])
            SurfaceTypes_tfdi.append(['PSP', 'PIECED STEEL PLANKING.'])
            SurfaceTypes_tfdi.append(['SAN', 'SAND, GRADED, ROLLED OR OILED.'])
            SurfaceTypes_tfdi.append(['SNO', 'SNOW.'])
            SurfaceTypes_tfdi.append(['U', 'SURFACE UNKNOWN.'])
            SurfaceTypes_tfdi.append(['WAT', 'WATER'])
            SurfaceTypes_tfdi_tb = pandas.DataFrame(SurfaceTypes_tfdi,columns=['SurfaceType','Description'])
            SurfaceTypes_tfdi_tb.to_sql('SurfaceTypes', mdb, if_exists='append', index=False)
            TrmLegTypes_tfdi = [['AF', 'Constant DME Arc to Fix']]
            TrmLegTypes_tfdi.append(['CA', 'Course to an altitude (position unspecified)'])
            TrmLegTypes_tfdi.append(['CD', 'Course to DME distance'])
            TrmLegTypes_tfdi.append(['CF', 'Course to a fix'])
            TrmLegTypes_tfdi.append(['CI', 'Course to next leg following by course oriented leg (interception point unspecified)'])
            TrmLegTypes_tfdi.append(['CR', 'Course to a radial termination (intercept point unspecified)'])
            TrmLegTypes_tfdi.append(['DF', 'Computed track direct to a fix'])
            TrmLegTypes_tfdi.append(['DS', 'DISCONTINUITY'])
            TrmLegTypes_tfdi.append(['FA', 'Course from a fix to an altitude'])
            TrmLegTypes_tfdi.append(['FC', 'Course from a fix to a distance'])
            TrmLegTypes_tfdi.append(['FD', 'Course from a fix to DME distance'])
            TrmLegTypes_tfdi.append(['FM', 'Course from a fix to manual termination'])
            TrmLegTypes_tfdi.append(['HA', 'Automatically at the fix after reaching an altitude'])
            TrmLegTypes_tfdi.append(['HF', 'Automatically at the fix after one full circuit'])
            TrmLegTypes_tfdi.append(['HM', 'Manually'])
            TrmLegTypes_tfdi.append(['IF', 'Initial Fix'])
            TrmLegTypes_tfdi.append(['PI', 'Procedure turn followed by a course to a fix (CF)'])
            TrmLegTypes_tfdi.append(['RF', 'Constant radius to a fix'])
            TrmLegTypes_tfdi.append(['TF', 'Track between two fixes (great circle)'])
            TrmLegTypes_tfdi.append(['VA', 'heading to an altitude (position unspecified)'])
            TrmLegTypes_tfdi.append(['VD', 'Heading to a DME Distance (position unspecified)'])
            TrmLegTypes_tfdi.append(['VI', 'Heading to a next leg (position unspecified)'])
            TrmLegTypes_tfdi.append(['VM', 'Heading to a manual termination'])
            TrmLegTypes_tfdi.append(['VR', 'Heading to a radial termination'])
            TrmLegTypes_tfdi_tb = pandas.DataFrame(TrmLegTypes_tfdi,columns=['Code','Description'])
            TrmLegTypes_tfdi_tb.to_sql('TrmLegTypes', mdb, if_exists='append', index=False)
            config_tfdi = [['CycleEndDate',CycleEndDate],['CycleName',CycleName],['CycleStartDate',CycleStartDate]]
            config_tfdi_tb = pandas.DataFrame(config_tfdi,columns=['key','val'])
            config_tfdi_tb.to_sql('config', mdb, if_exists='append', index=False)
        if tfdi_write:
            Airports_tfdi_tb.to_sql('Airports', mdb, if_exists='append', index=False)
            AirportLookup_tfdi_tb.to_sql('AirportLookup', mdb, if_exists='append', index=False)
            Runways_tfdi_tb.to_sql('Runways', mdb, if_exists='append', index=False)
            Navaids_tfdi_tb.to_sql('Navaids', mdb, if_exists='append', index=False)
            NavaidLookup_tfdi_tb.to_sql('NavaidLookup', mdb, if_exists='append', index=False)
            Waypoints_tfdi_tb.to_sql('Waypoints', mdb, if_exists='append', index=False)
            WaypointLookup_tfdi_tb = WaypointLookup_tfdi_tb.drop(columns=['REGION_CODE'])
            WaypointLookup_tfdi_tb.to_sql('WaypointLookup', mdb, if_exists='append', index=False)
            Airways_tfdi_tb.to_sql('Airways', mdb, if_exists='append', index=False)
            AirwayLegs_tfdi_tb.to_sql('AirwayLegs', mdb, if_exists='append', index=False)
            ILSes_tfdi_tb = ILSes_tfdi_tb.drop(columns=['AirportID'])
            ILSes_tfdi_tb.to_sql('ILSes', mdb, if_exists='append', index=False)
            Terminals_tfdi_tb.to_sql('Terminals', mdb, if_exists='append', index=False)
            TerminalLegsEx_tfdi_tb.to_sql('TerminalLegsEx', mdb, if_exists='append', index=False)
            TerminalLegs_tfdi_tb.to_sql('TerminalLegs', mdb, if_exists='append', index=False)
    print(time.strftime("%H:%M:%S", time.localtime()) , 'TFDi: Data Export Finished')

# Aerosoft (Acceptable)
if Aerosoft:
    print(time.strftime("%H:%M:%S", time.localtime()) , 'Aerosoft: Start Processing DataFile')
    os.makedirs(Cycle_Ident + '/Aerosoft')
    os.makedirs(Cycle_Ident + '/Aerosoft/PROC')
    # Set Variable of Aerosoft
    if 1:
        Airports_as = []
        Airports_in_as_nd = []
        Navaids_as = []
        Waypoints_as = []
        Runways_as = []
        Runways_in_as_nd = []
        Localizer_as = []
        ATS_as = []
        Airway_as = []
        AirwayLegs_as = []
        AirwayLegs_reversed_as = []
        ApproachesLookup_as = []
        AirwayIdent = ''
        AirwayLegsID = 0
        AirwaySegmentID = 0
        TerminalName = ''
        TerminalLegs_as = []
        MISSED_APCH = 0
        MISSED_APCH_COUNT = 0
    # Load Aerosoft's NAVData
    if as_create_blank != True:
        if os.path.isfile('Input/Aerosoft/Navaids.txt'):
            shutil.copy2('Input/Aerosoft/Navaids.txt', Cycle_Ident + '/Aerosoft/Navaids.txt')
            Navaids_in_as_nd_tb = pandas.read_csv(Cycle_Ident + '/Aerosoft/Navaids.txt', header=None, names=['NAV_IDENT','NAV_NAME','NAV_FREQ','have_vor','have_dme','195','NAV_LAT','NAV_LON','DME_ELEV','NAV_ICAO_CODE','UNK','is_ndb','STATION_DECLINATION','ILS_DME_BIAS'], dtype='str')
        if os.path.isfile('Input/Aerosoft/Waypoints.txt'):
            shutil.copy2('Input/Aerosoft/Waypoints.txt', Cycle_Ident + '/Aerosoft/Waypoints.txt')
            Waypoints_in_as_nd_tb = pandas.read_csv(Cycle_Ident + '/Aerosoft/Waypoints.txt', header=None, names=['WAYPOINT_IDENT','WAYPOINT_LAT','WAYPOINT_LON','WAYPOINT_ICAO_CODE'], dtype='str')
        if os.path.isfile('Input/Aerosoft/Airports.txt'):
            shutil.copy2('Input/Aerosoft/Airports.txt', Cycle_Ident + '/Aerosoft/Airports.txt')
            with open(Cycle_Ident + '/Aerosoft/Airports.txt', 'r', encoding='ISO-8859-1') as AS_nd:
                Airports_in_as_nd_tb = []
                for line in AS_nd:
                    pa_as_se = re.search(pa_as_nd, line)
                    pg_as_se = re.search(pg_as_nd, line)
                    if pa_as_se:
                        ARPT_IDENT = pa_as_se.group(1)
                        ARPT_NAME = pa_as_se.group(2)
                        ARPT_LAT = pa_as_se.group(3)
                        ARPT_LON = pa_as_se.group(4)
                        ARPT_ELEV = pa_as_se.group(5)
                        TRANSITIONS_ALT = pa_as_se.group(6)
                        TRANSITION_LEVEL = pa_as_se.group(7)
                        LONGEST_RUNWAY = pa_as_se.group(8)
                        MAG_TRUE = pa_as_se.group(9)
                        Airports_in_as_nd.append((ARPT_IDENT,ARPT_NAME,ARPT_LAT,ARPT_LON,ARPT_ELEV,TRANSITIONS_ALT,TRANSITION_LEVEL,LONGEST_RUNWAY,MAG_TRUE))
                    if pg_as_se:
                        RUNWAY_IDENT = pg_as_se.group(1)
                        RUNWAY_MAG_BEARING = pg_as_se.group(2)
                        RUNWAY_LENGTH = pg_as_se.group(3)
                        RUNWAY_WIDTH = pg_as_se.group(4)
                        have_ils = pg_as_se.group(5)
                        LOCALIZER_FREQ = pg_as_se.group(6)
                        LOCALIZER_BEARING = pg_as_se.group(7)
                        RUNWAY_LAT = pg_as_se.group(8)
                        RUNWAY_LON = pg_as_se.group(9)
                        LANDING_THLD_ELEV = pg_as_se.group(10)
                        GLIDE_SLOPE_ANGLE = pg_as_se.group(11)
                        THLD_CROSSING_HEIGHT = pg_as_se.group(12)
                        Runways_in_as_nd.append((ARPT_IDENT,RUNWAY_IDENT,RUNWAY_MAG_BEARING,RUNWAY_LENGTH,RUNWAY_WIDTH,have_ils,LOCALIZER_FREQ,LOCALIZER_BEARING,RUNWAY_LAT,RUNWAY_LON,LANDING_THLD_ELEV,GLIDE_SLOPE_ANGLE,THLD_CROSSING_HEIGHT))
                Airports_in_as_nd_tb = pandas.DataFrame(Airports_in_as_nd, columns=['ARPT_IDENT','ARPT_NAME','ARPT_LAT','ARPT_LON','ARPT_ELEV','TRANSITIONS_ALT','TRANSITION_LEVEL','LONGEST_RUNWAY','MAG_TRUE'])
                Runways_in_as_nd_tb = pandas.DataFrame(Runways_in_as_nd, columns=['ARPT_IDENT','RUNWAY_IDENT','RUNWAY_MAG_BEARING','RUNWAY_LENGTH','RUNWAY_WIDTH','have_ils','LOCALIZER_FREQ','LOCALIZER_BEARING','RUNWAY_LAT','RUNWAY_LON','LANDING_THLD_ELEV','GLIDE_SLOPE_ANGLE','THLD_CROSSING_HEIGHT'])
                print('', file=AS_nd)
        if os.path.isfile('Input/Aerosoft/ATS.txt'):
            shutil.copy2('Input/Aerosoft/ATS.txt', Cycle_Ident + '/Aerosoft/ATS.txt')
        print(time.strftime("%H:%M:%S", time.localtime()) , 'Aerosoft: NAVData Load Completed')
    # Process of Aerosoft Part #1
    with open('Input/' + Datafile, 'r', encoding='ISO-8859-1') as ARINC424:
        for line in ARINC424: 
            vhf_se = re.search(vor, line)
            db_se = re.search(db, line)
            ea_se = re.search(ea, line)
            pc_se = re.search(pc, line)
            proc_se = re.search(pdef, line)
            loc_se = re.search(pi, line)
            pn_se = re.search(pn, line)
            if vhf_se:
                ARPT_IDENT = vhf_se.group(5).strip()
                VHF_IDENT = vhf_se.group(7).strip()
                VHF_NAME = vhf_se.group(39).replace('VOR/DME','').rstrip().replace('(','').replace(')','')
                VHF_FREQ = str(Decimal(vhf_se.group(10)[0:3] + '.' + vhf_se.group(10)[3:]).quantize(Decimal('0.000'), rounding=ROUND_HALF_UP))
                if vhf_se.group(12).strip() == '':
                    is_vor = '0'
                else:
                    is_vor = '1'
                if vhf_se.group(23).strip() == '':
                    is_dme = '0'
                else:
                    is_dme = '1'
                if is_vor == '1':
                    if vhf_se.group(12) == "N":
                        VHF_LAT = str(Decimal(Decimal(vhf_se.group(13)) + Decimal(vhf_se.group(14))/60 + Decimal(vhf_se.group(15))/3600 + Decimal(vhf_se.group(16))/360000).quantize(Decimal('0.000000'), rounding=ROUND_HALF_UP))
                    else:
                        VHF_LAT = str(Decimal(0 - Decimal(vhf_se.group(13)) - Decimal(vhf_se.group(14))/60 - Decimal(vhf_se.group(15))/3600 - Decimal(vhf_se.group(16))/360000).quantize(Decimal('0.000000'), rounding=ROUND_HALF_UP))
                    if vhf_se.group(17) == "E":
                        VHF_LON = str(Decimal(Decimal(vhf_se.group(18)) + Decimal(vhf_se.group(19))/60 + Decimal(vhf_se.group(20))/3600 + Decimal(vhf_se.group(21))/360000).quantize(Decimal('0.000000'), rounding=ROUND_HALF_UP))
                    else:
                        VHF_LON = str(Decimal(0 - Decimal(vhf_se.group(18)) - Decimal(vhf_se.group(19))/60 - Decimal(vhf_se.group(20))/3600 - Decimal(vhf_se.group(21))/360000).quantize(Decimal('0.000000'), rounding=ROUND_HALF_UP))
                else:
                    if vhf_se.group(23) == "N":
                        VHF_LAT = str(Decimal(Decimal(vhf_se.group(24)) + Decimal(vhf_se.group(25))/60 + Decimal(vhf_se.group(26))/3600 + Decimal(vhf_se.group(27))/360000).quantize(Decimal('0.000000'), rounding=ROUND_HALF_UP))
                    else:
                        VHF_LAT = str(Decimal(0 - Decimal(vhf_se.group(24)) - Decimal(vhf_se.group(25))/60 - Decimal(vhf_se.group(26))/3600 - Decimal(vhf_se.group(27))/360000).quantize(Decimal('0.000000'), rounding=ROUND_HALF_UP))
                    if vhf_se.group(28) == "E":
                        VHF_LON = str(Decimal(Decimal(vhf_se.group(29)) + Decimal(vhf_se.group(30))/60 + Decimal(vhf_se.group(31))/3600 + Decimal(vhf_se.group(32))/360000).quantize(Decimal('0.000000'), rounding=ROUND_HALF_UP))
                    else:
                        VHF_LON = str(Decimal(0 - Decimal(vhf_se.group(29)) - Decimal(vhf_se.group(30))/60 - Decimal(vhf_se.group(31))/3600 - Decimal(vhf_se.group(32))/360000).quantize(Decimal('0.000000'), rounding=ROUND_HALF_UP))
                if vhf_se.group(34).strip() == '':
                    ELEV = 0
                else:
                    ELEV = int(vhf_se.group(34))
                VHF_ICAO_CODE = vhf_se.group(8)
                is_ndb = '0'
                if vhf_se.group(33).strip() == '':
                    MAG_VAR = '0.0'
                elif vhf_se.group(33).strip(string.digits) == "E":
                    MAG_VAR = float(vhf_se.group(33).strip('EW'))/10
                else:
                    MAG_VAR = 0 - float(vhf_se.group(33).strip('EW'))/10
                ILS_DME_BIAS = 0/10
                Navaids_as.append((ARPT_IDENT,VHF_IDENT,VHF_NAME,VHF_FREQ,is_vor,is_dme,'195',VHF_LAT,VHF_LON,ELEV,VHF_ICAO_CODE,'0',is_ndb,MAG_VAR,ILS_DME_BIAS))
            if db_se:
                ARPT_IDENT = db_se.group(5).strip()
                NDB_IDENT = db_se.group(7).strip()
                NDB_ICAO_CODE = db_se.group(8)
                NDB_NAME = db_se.group(25).replace('NDB','').rstrip().replace('(','').replace(')','')
                NDB_FREQ = str(Decimal(db_se.group(10)[0:4] + '.' + db_se.group(10)[4:]).quantize(Decimal('0.000'), rounding=ROUND_HALF_UP))
                is_vor = '0'
                is_dme = '0'
                if db_se.group(12) == "N":
                    NDB_LAT = str(Decimal(Decimal(db_se.group(13)) + Decimal(db_se.group(14))/60 + Decimal(db_se.group(15))/3600 + Decimal(db_se.group(16))/360000).quantize(Decimal('0.000000'), rounding=ROUND_HALF_UP))
                else:
                    NDB_LAT = str(Decimal(0 - Decimal(db_se.group(13)) - Decimal(db_se.group(14))/60 - Decimal(db_se.group(15))/3600 - Decimal(db_se.group(16))/360000).quantize(Decimal('0.000000'), rounding=ROUND_HALF_UP))
                if db_se.group(17) == "E":
                    NDB_LON = str(Decimal(Decimal(db_se.group(18)) + Decimal(db_se.group(19))/60 + Decimal(db_se.group(20))/3600 + Decimal(db_se.group(21))/360000).quantize(Decimal('0.000000'), rounding=ROUND_HALF_UP))
                else:
                    NDB_LON = str(Decimal(0 - Decimal(db_se.group(18)) - Decimal(db_se.group(19))/60 - Decimal(db_se.group(20))/3600 - Decimal(db_se.group(21))/360000).quantize(Decimal('0.000000'), rounding=ROUND_HALF_UP))
                ELEV = '0'
                is_ndb = '1'
                MAG_VAR = '0.0'
                ILS_DME_BIAS = '0.0'
                Navaids_as.append((ARPT_IDENT,NDB_IDENT,NDB_NAME,NDB_FREQ,is_vor,is_dme,'195',NDB_LAT,NDB_LON,ELEV,NDB_ICAO_CODE,'0',is_ndb,MAG_VAR,ILS_DME_BIAS))
            if ea_se:
                REGION_CODE = ea_se.group(5)
                WAYPOINT_IDENT = ea_se.group(8).strip()
                if ea_se.group(9) != 'P ':
                    WAYPOINT_ICAO_CODE = ea_se.group(9)
                else:
                    WAYPOINT_ICAO_CODE = ' P'
                if ea_se.group(13) == "N":
                    WAYPOINT_LAT = str(Decimal(Decimal(ea_se.group(14)) + Decimal(ea_se.group(15))/60 + Decimal(ea_se.group(16))/3600 + Decimal(ea_se.group(17))/360000).quantize(Decimal('0.000000'), rounding=ROUND_HALF_UP))
                else:
                    WAYPOINT_LAT = str(Decimal(0 - Decimal(ea_se.group(14)) - Decimal(ea_se.group(15))/60 - Decimal(ea_se.group(16))/3600 - Decimal(ea_se.group(17))/360000).quantize(Decimal('0.000000'), rounding=ROUND_HALF_UP))
                if ea_se.group(18) == "E":
                    WAYPOINT_LON = str(Decimal(Decimal(ea_se.group(19)) + Decimal(ea_se.group(20))/60 + Decimal(ea_se.group(21))/3600 + Decimal(ea_se.group(22))/360000).quantize(Decimal('0.000000'), rounding=ROUND_HALF_UP))
                else:
                    WAYPOINT_LON = str(Decimal(0 - Decimal(ea_se.group(19)) - Decimal(ea_se.group(20))/60 - Decimal(ea_se.group(21))/3600 - Decimal(ea_se.group(22))/360000).quantize(Decimal('0.000000'), rounding=ROUND_HALF_UP))
                Waypoints_as.append((REGION_CODE,WAYPOINT_IDENT,WAYPOINT_LAT,WAYPOINT_LON,WAYPOINT_ICAO_CODE))
            if pc_se:
                REGION_CODE = pc_se.group(5)
                WAYPOINT_IDENT = pc_se.group(8).strip()
                WAYPOINT_ICAO_CODE = pc_se.group(9)
                if pc_se.group(13) == "N":
                    WAYPOINT_LAT = str(Decimal(Decimal(pc_se.group(14)) + Decimal(pc_se.group(15))/60 + Decimal(pc_se.group(16))/3600 + Decimal(pc_se.group(17))/360000).quantize(Decimal('0.000000'), rounding=ROUND_HALF_UP))
                else:
                    WAYPOINT_LAT = str(Decimal(0 - Decimal(pc_se.group(14)) - Decimal(pc_se.group(15))/60 - Decimal(pc_se.group(16))/3600 - Decimal(pc_se.group(17))/360000).quantize(Decimal('0.000000'), rounding=ROUND_HALF_UP))
                if pc_se.group(18) == "E":
                    WAYPOINT_LON = str(Decimal(Decimal(pc_se.group(19)) + Decimal(pc_se.group(20))/60 + Decimal(pc_se.group(21))/3600 + Decimal(pc_se.group(22))/360000).quantize(Decimal('0.000000'), rounding=ROUND_HALF_UP))
                else:
                    WAYPOINT_LON = str(Decimal(0 - Decimal(pc_se.group(19)) - Decimal(pc_se.group(20))/60 - Decimal(pc_se.group(21))/3600 - Decimal(pc_se.group(22))/360000).quantize(Decimal('0.000000'), rounding=ROUND_HALF_UP))
                Waypoints_as.append((REGION_CODE,WAYPOINT_IDENT,WAYPOINT_LAT,WAYPOINT_LON,WAYPOINT_ICAO_CODE))
            if proc_se:
                ARPT_IDENT = proc_se.group(4)
                SUBS_CODE = proc_se.group(6)
                PROC_IDENT = proc_se.group(7).strip()
                ROUTE_TYPE = proc_se.group(8)
                if SUBS_CODE == 'F':
                    if ROUTE_TYPE != 'A':
                        if PROC_IDENT[0:1] != ROUTE_TYPE:
                            ROUTE_TYPE = 'R'
                        ApproachesLookup_as.append((ARPT_IDENT,PROC_IDENT,ROUTE_TYPE))
            if loc_se:
                ARPT_IDENT = loc_se.group(4)
                ARPT_ICAO_CODE = loc_se.group(5)
                LOCALIZER_IDENT = loc_se.group(7).strip()
                if loc_se.group(8).strip(string.digits) == '':
                    ILS_CATEGORY = loc_se.group(8)
                else:
                    ILS_CATEGORY = 0
                LOCALIZER_FREQ = str(Decimal(loc_se.group(10)[0:3] + '.' + loc_se.group(10)[3:]).quantize(Decimal('0.000'), rounding=ROUND_HALF_UP))
                RUNWAY_IDENT = loc_se.group(11).strip()
                if loc_se.group(12).strip() == '':
                    LOCALIZER_LAT = None
                elif loc_se.group(12) == "N":
                    LOCALIZER_LAT = str(Decimal(Decimal(loc_se.group(13)) + Decimal(loc_se.group(14))/60 + Decimal(loc_se.group(15))/3600 + Decimal(loc_se.group(16))/360000).quantize(Decimal('0.000000'), rounding=ROUND_HALF_UP))
                else:
                    LOCALIZER_LAT = str(Decimal(0 - Decimal(loc_se.group(13)) - Decimal(loc_se.group(14))/60 - Decimal(loc_se.group(15))/3600 - Decimal(loc_se.group(16))/360000).quantize(Decimal('0.000000'), rounding=ROUND_HALF_UP))
                if loc_se.group(17).strip() == '':
                    LOCALIZER_LON = None
                elif loc_se.group(17) == "E":
                    LOCALIZER_LON = str(Decimal(Decimal(loc_se.group(18)) + Decimal(loc_se.group(19))/60 + Decimal(loc_se.group(20))/3600 + Decimal(loc_se.group(21))/360000).quantize(Decimal('0.000000'), rounding=ROUND_HALF_UP))
                else:
                    LOCALIZER_LON = str(Decimal(0 - Decimal(loc_se.group(18)) - Decimal(loc_se.group(19))/60 - Decimal(loc_se.group(20))/3600 - Decimal(loc_se.group(21))/360000).quantize(Decimal('0.000000'), rounding=ROUND_HALF_UP))
                if loc_se.group(22).lstrip(string.digits) == "T":
                    LOCALIZER_BEARING = str(Decimal(loc_se.group(22).rstrip('T')))
                else:
                    LOCALIZER_BEARING = str(Decimal(loc_se.group(22)[0:3] + '.' + loc_se.group(22)[3:]).quantize(Decimal('0'), rounding=ROUND_HALF_UP))
                if loc_se.group(23).strip() == '':
                    GLIDE_SLOPE_LAT = None
                elif loc_se.group(23) == "N":
                    GLIDE_SLOPE_LAT = str(Decimal(Decimal(loc_se.group(24)) + Decimal(loc_se.group(25))/60 + Decimal(loc_se.group(26))/3600 + Decimal(loc_se.group(27))/360000).quantize(Decimal('0.000000'), rounding=ROUND_HALF_UP))
                else:
                    GLIDE_SLOPE_LAT = str(Decimal(0 - Decimal(loc_se.group(24)) - Decimal(loc_se.group(25))/60 - Decimal(loc_se.group(26))/3600 - Decimal(loc_se.group(27))/360000).quantize(Decimal('0.000000'), rounding=ROUND_HALF_UP))
                if loc_se.group(28).strip() == '':
                    GLIDE_SLOPE_LON = None
                elif loc_se.group(28) == "E":
                    GLIDE_SLOPE_LON = str(Decimal(Decimal(loc_se.group(29)) + Decimal(loc_se.group(30))/60 + Decimal(loc_se.group(31))/3600 + Decimal(loc_se.group(32))/360000).quantize(Decimal('0.000000'), rounding=ROUND_HALF_UP))
                else:
                    GLIDE_SLOPE_LON = str(Decimal(0 - Decimal(loc_se.group(29)) - Decimal(loc_se.group(30))/60 - Decimal(loc_se.group(31))/3600 - Decimal(loc_se.group(32))/360000).quantize(Decimal('0.000000'), rounding=ROUND_HALF_UP))
                LOCALIZER_POS = loc_se.group(33)
                LOCALIZER_POS_REFERENCE = loc_se.group(34)
                GLIDE_SLOPE_POS = loc_se.group(35)
                LOCALIZER_WIDTH = int(loc_se.group(36))
                if loc_se.group(37).strip() == '':
                    GLIDE_SLOPE_ANGLE = '0.00'
                    have_gs = 0
                else:
                    GLIDE_SLOPE_ANGLE = str(Decimal(loc_se.group(37)[0:1] + '.' + loc_se.group(37)[1:]).quantize(Decimal('0.00'), rounding=ROUND_HALF_UP))
                    have_gs = 1
                STATION_DECLINATION = loc_se.group(38)
                if loc_se.group(39).strip() == '':
                    GLIDE_SLOPE_HEIGHT_AT_LANDING_THLD = None
                else:
                    GLIDE_SLOPE_HEIGHT_AT_LANDING_THLD = loc_se.group(39)
                if loc_se.group(40).strip() == '':
                    GLIDE_SLOPE_ELEV = '0'
                else:
                    GLIDE_SLOPE_ELEV = loc_se.group(40)
                SUPPORTING_FACILITY_IDENT = loc_se.group(41)
                SUPPORTING_FACILITY_ICAO_CODE = loc_se.group(42)
                SUPPORTING_FACILITY_SECT_CODE = loc_se.group(43)
                SUPPORTING_FACILITY_SUBS_CODE = loc_se.group(44)
                Localizer_as.append((ARPT_IDENT,RUNWAY_IDENT,LOCALIZER_IDENT,ILS_CATEGORY,LOCALIZER_LAT,LOCALIZER_LON,LOCALIZER_BEARING,LOCALIZER_FREQ,have_gs,GLIDE_SLOPE_ANGLE))
            if pn_se:
                ARPT_IDENT = pn_se.group(5).strip()
                NDB_IDENT = pn_se.group(7).strip()
                NDB_ICAO_CODE = pn_se.group(8)
                NDB_NAME = pn_se.group(25).replace('NDB','').rstrip().replace('(','').replace(')','')
                NDB_FREQ = str(Decimal(pn_se.group(10)[0:4] + '.' + pn_se.group(10)[4:]).quantize(Decimal('0.000'), rounding=ROUND_HALF_UP))
                is_vor = '0'
                is_dme = '0'
                if pn_se.group(12) == "N":
                    NDB_LAT = str(Decimal(Decimal(pn_se.group(13)) + Decimal(pn_se.group(14))/60 + Decimal(pn_se.group(15))/3600 + Decimal(pn_se.group(16))/360000).quantize(Decimal('0.000000'), rounding=ROUND_HALF_UP))
                else:
                    NDB_LAT = str(Decimal(0 - Decimal(pn_se.group(13)) - Decimal(pn_se.group(14))/60 - Decimal(pn_se.group(15))/3600 - Decimal(pn_se.group(16))/360000).quantize(Decimal('0.000000'), rounding=ROUND_HALF_UP))
                if pn_se.group(17) == "E":
                    NDB_LON = str(Decimal(Decimal(pn_se.group(18)) + Decimal(pn_se.group(19))/60 + Decimal(pn_se.group(20))/3600 + Decimal(pn_se.group(21))/360000).quantize(Decimal('0.000000'), rounding=ROUND_HALF_UP))
                else:
                    NDB_LON = str(Decimal(0 - Decimal(pn_se.group(18)) - Decimal(pn_se.group(19))/60 - Decimal(pn_se.group(20))/3600 - Decimal(pn_se.group(21))/360000).quantize(Decimal('0.000000'), rounding=ROUND_HALF_UP))
                ELEV = '0'
                is_ndb = '1'
                MAG_VAR = '0.0'
                ILS_DME_BIAS = '0.0'
                Navaids_as.append((ARPT_IDENT,NDB_IDENT,NDB_NAME,NDB_FREQ,is_vor,is_dme,'195',NDB_LAT,NDB_LON,ELEV,NDB_ICAO_CODE,'0',is_ndb,MAG_VAR,ILS_DME_BIAS))
        # Convert list to Pandas DataFrame
        Navaids_as_tb = pandas.DataFrame(Navaids_as, columns=['ARPT_IDENT','NAV_IDENT','NAV_NAME','NAV_FREQ','is_vor','is_dme','RANGE','NAV_LAT','NAV_LON','ELEV','NAV_ICAO_CODE','UNK','is_ndb','MAG_VAR','ILS_DME_BIAS'])
        Waypoints_as_tb = pandas.DataFrame(Waypoints_as, columns=['REGION_CODE','WAYPOINT_IDENT','WAYPOINT_LAT','WAYPOINT_LON','WAYPOINT_ICAO_CODE'])
        Localizer_as_tb = pandas.DataFrame(Localizer_as, columns=['ARPT_IDENT','RUNWAY_IDENT','LOCALIZER_IDENT','ILS_CATEGORY','LOCALIZER_LAT','LOCALIZER_LON','LOCALIZER_BEARING','LOCALIZER_FREQ','have_gs','GLIDE_SLOPE_ANGLE'])
        ApproachesLookup_as_tb = pandas.DataFrame(ApproachesLookup_as, columns=['ARPT_IDENT','PROC_IDENT','ROUTE_TYPE'])
        print(time.strftime("%H:%M:%S", time.localtime()) , 'Aerosoft: Data Process Part #1 Finished')
    # Process of Aerosoft Part #2
    with open('Input/' + Datafile, 'r', encoding='ISO-8859-1') as ARINC424:
        for line in ARINC424: 
            awy_se = re.search(er, line)
            arpt_se = re.search(pa, line)
            rwy_se = re.search(pg, line)
            # AWY of Aerosoft (Part #1)
            if awy_se:
                CUST_AREA = awy_se.group(2)
                ROUTE_IDENT = awy_se.group(5).strip()
                SEQ_NR = awy_se.group(6)
                FIX_IDENT = awy_se.group(7).strip()
                if awy_se.group(8) != 'P ':
                    FIX_ICAO_CODE = awy_se.group(8)
                else:
                    FIX_ICAO_CODE = ' P'
                FIX_SECT_SUBS = awy_se.group(9) + awy_se.group(10)
                if FIX_SECT_SUBS == "EA":
                    FIX_ID = Waypoints_as_tb.loc[(Waypoints_as_tb['WAYPOINT_IDENT'] == FIX_IDENT) & (Waypoints_as_tb['WAYPOINT_ICAO_CODE'] == FIX_ICAO_CODE)].index.tolist()
                    if FIX_ID != []:
                        FIX_LAT = Waypoints_as_tb.at[FIX_ID[0], 'WAYPOINT_LAT']
                        FIX_LON = Waypoints_as_tb.at[FIX_ID[0], 'WAYPOINT_LON']
                    else:
                        if as_create_blank != True:
                            FIX_ID = Waypoints_in_as_nd_tb.loc[(Waypoints_in_as_nd_tb['WAYPOINT_IDENT'] == FIX_IDENT) & (Waypoints_in_as_nd_tb['WAYPOINT_ICAO_CODE'] == FIX_ICAO_CODE)].index.tolist()
                            if FIX_ID != []:
                                FIX_LAT = Waypoints_in_as_nd_tb.at[FIX_ID[0], 'WAYPOINT_LAT']
                                FIX_LON = Waypoints_in_as_nd_tb.at[FIX_ID[0], 'WAYPOINT_LON']
                            else:
                                print('         Aerosoft: Airway ' + awy_se.group(5) + ' Fix ' + awy_se.group(7) + ' Not Found')
                                FIX_LAT = '00.000000'
                                FIX_LON = '000.000000'
                        else:
                            print('         Aerosoft: Airway ' + awy_se.group(5) + ' Fix ' + awy_se.group(7) + ' Not Found')
                            FIX_LAT = '00.000000'
                            FIX_LON = '000.000000'
                elif FIX_SECT_SUBS == "DB":
                    FIX_ID = Navaids_as_tb.loc[(Navaids_as_tb['NAV_IDENT'] == FIX_IDENT) & (Navaids_as_tb['NAV_ICAO_CODE'] == FIX_ICAO_CODE) & (Navaids_as_tb['is_ndb'] == '1')].index.tolist()
                    if FIX_ID != []:
                        FIX_LAT = Navaids_as_tb.at[FIX_ID[0], 'NAV_LAT']
                        FIX_LON = Navaids_as_tb.at[FIX_ID[0], 'NAV_LON']
                    else:
                        if as_create_blank != True:
                            FIX_ID = Navaids_in_as_nd_tb.loc[(Navaids_in_as_nd_tb['NAV_IDENT'] == FIX_IDENT) & (Navaids_in_as_nd_tb['NAV_ICAO_CODE'] == FIX_ICAO_CODE) & (Navaids_in_as_nd_tb['is_ndb'] == '1')].index.tolist()
                            if FIX_ID != []:
                                FIX_LAT = Navaids_in_as_nd_tb.at[FIX_ID[0], 'NAV_LAT']
                                FIX_LON = Navaids_in_as_nd_tb.at[FIX_ID[0], 'NAV_LON']
                            else:
                                print('         Aerosoft: Airway ' + awy_se.group(5) + ' Fix ' + awy_se.group(7) + ' Not Found')
                                FIX_LAT = '00.000000'
                                FIX_LON = '000.000000'
                        else:
                            print('         Aerosoft: Airway ' + awy_se.group(5) + ' Fix ' + awy_se.group(7) + ' Not Found')
                            FIX_LAT = '00.000000'
                            FIX_LON = '000.000000'
                else:
                    FIX_ID = Navaids_as_tb.loc[(Navaids_as_tb['NAV_IDENT'] == FIX_IDENT) & (Navaids_as_tb['NAV_ICAO_CODE'] == FIX_ICAO_CODE) & (Navaids_as_tb['is_ndb'] == '0')].index.tolist()
                    if FIX_ID != []:
                        FIX_LAT = Navaids_as_tb.at[FIX_ID[0], 'NAV_LAT']
                        FIX_LON = Navaids_as_tb.at[FIX_ID[0], 'NAV_LON']
                    else:
                        if as_create_blank != True:
                            FIX_ID = Navaids_in_as_nd_tb.loc[(Navaids_in_as_nd_tb['NAV_IDENT'] == FIX_IDENT) & (Navaids_in_as_nd_tb['NAV_ICAO_CODE'] == FIX_ICAO_CODE) & (Navaids_in_as_nd_tb['is_ndb'] == '0')].index.tolist()
                            if FIX_ID != []:
                                FIX_LAT = Navaids_in_as_nd_tb.at[FIX_ID[0], 'NAV_LAT']
                                FIX_LON = Navaids_in_as_nd_tb.at[FIX_ID[0], 'NAV_LON']
                            else:
                                print('         Aerosoft: Airway ' + awy_se.group(5) + ' Fix ' + awy_se.group(7) + ' Not Found')
                                FIX_LAT = '00.000000'
                                FIX_LON = '000.000000'
                        else:
                            print('         Aerosoft: Airway ' + awy_se.group(5) + ' Fix ' + awy_se.group(7) + ' Not Found')
                            FIX_LAT = '00.000000'
                            FIX_LON = '000.000000'
                EndOfRoute = awy_se.group(12)[1:2]
                DIRECTION_RESTRICTION = awy_se.group(16)
                if awy_se.group(24).strip() == '':
                    OUTBOUND_MAG_COURSE = None
                elif awy_se.group(24).lstrip(string.digits) == "T":
                    OUTBOUND_MAG_COURSE = Decimal(awy_se.group(24).rstrip('T'))
                else:
                    OUTBOUND_MAG_COURSE = Decimal(awy_se.group(24)[0:3] + '.' + awy_se.group(24)[3:]).quantize(Decimal('0'), rounding=ROUND_HALF_UP)
                if awy_se.group(25).strip() == '':
                    ROUTE_DISTANCE_FROM = None
                else:
                    ROUTE_DISTANCE_FROM = Decimal(awy_se.group(25)[0:3] + '.' + awy_se.group(25)[3:]).quantize(Decimal('0.00'), rounding=ROUND_HALF_UP)
                if awy_se.group(26).strip() == '':
                    INBOUND_MAG_COURSE = None
                elif awy_se.group(26).lstrip(string.digits) == "T":
                    INBOUND_MAG_COURSE = Decimal(awy_se.group(26).rstrip('T'))
                else:
                    INBOUND_MAG_COURSE = Decimal(awy_se.group(26)[0:3] + '.' + awy_se.group(26)[3:]).quantize(Decimal('0'), rounding=ROUND_HALF_UP)
                Airway_as.append((CUST_AREA,ROUTE_IDENT,SEQ_NR,FIX_IDENT,FIX_ICAO_CODE,FIX_LAT,FIX_LON,EndOfRoute,DIRECTION_RESTRICTION,INBOUND_MAG_COURSE,OUTBOUND_MAG_COURSE,ROUTE_DISTANCE_FROM))
            if arpt_se:
                ARPT_IDENT = arpt_se.group(4)
                ARPT_NAME = arpt_se.group(36).rstrip().replace('(','').replace(')','')
                if arpt_se.group(13) == "N":
                    ARPT_LAT = str(Decimal(Decimal(arpt_se.group(14)) + Decimal(arpt_se.group(15))/60 + Decimal(arpt_se.group(16))/3600 + Decimal(arpt_se.group(17))/360000).quantize(Decimal('0.000000'), rounding=ROUND_HALF_UP))
                else:
                    ARPT_LAT = str(Decimal(0 - Decimal(arpt_se.group(14)) - Decimal(arpt_se.group(15))/60 - Decimal(arpt_se.group(16))/3600 - Decimal(arpt_se.group(17))/360000).quantize(Decimal('0.000000'), rounding=ROUND_HALF_UP))
                if arpt_se.group(18) == "E":
                    ARPT_LON = str(Decimal(Decimal(arpt_se.group(19)) + Decimal(arpt_se.group(20))/60 + Decimal(arpt_se.group(21))/3600 + Decimal(arpt_se.group(22))/360000).quantize(Decimal('0.000000'), rounding=ROUND_HALF_UP))
                else:
                    ARPT_LON = str(Decimal(0 - Decimal(arpt_se.group(19)) - Decimal(arpt_se.group(20))/60 - Decimal(arpt_se.group(21))/3600 - Decimal(arpt_se.group(22))/360000).quantize(Decimal('0.000000'), rounding=ROUND_HALF_UP))
                ARPT_ELEV = arpt_se.group(25).lstrip('0')
                if arpt_se.group(29).strip() == '':
                    TRANSITIONS_ALT = '0'
                else:
                    TRANSITIONS_ALT = int(arpt_se.group(29))
                if arpt_se.group(30).strip() == '':
                    TRANSITION_LEVEL = '0'
                else:
                    TRANSITION_LEVEL = int(arpt_se.group(30))
                LONGEST_RUNWAY = int(arpt_se.group(10))*100
                if arpt_se.group(34) == 'T':
                    MAG_TRUE = '1'
                else:
                    MAG_TRUE = '0'
                Airports_as.append((ARPT_IDENT,ARPT_NAME,ARPT_LAT,ARPT_LON,ARPT_ELEV,TRANSITIONS_ALT,TRANSITION_LEVEL,LONGEST_RUNWAY,MAG_TRUE))
                with open(Cycle_Ident + '/Aerosoft/Airports.txt', 'a') as Airports:
                    print('', file=Airports)
                    print(str('A') + ',' + str(ARPT_IDENT) + ',' + str(ARPT_NAME) + ',' + str(ARPT_LAT) + ',' + str(ARPT_LON) + ',' + str(ARPT_ELEV) + ',' + str(TRANSITIONS_ALT) + ',' + str(TRANSITION_LEVEL) + ',' + str(LONGEST_RUNWAY) + ',' + str(MAG_TRUE), file=Airports)
                with open(Cycle_Ident + '/Aerosoft/PROC/' + ARPT_IDENT + '.txt', 'a') as PROC:
                    print(str('A') + ',' + str(ARPT_IDENT) + ',' + str(ARPT_NAME) + ',' + str(ARPT_LAT) + ',' + str(ARPT_LON) + ',' + str(ARPT_ELEV) + ',' + str(TRANSITIONS_ALT) + ',' + str(TRANSITION_LEVEL) + ',' + str(LONGEST_RUNWAY) + ',' + str(MAG_TRUE), file=PROC)
            if rwy_se:
                ARPT_IDENT = rwy_se.group(4)
                RUNWAY_IDENT = rwy_se.group(7)[2:5].strip()
                if rwy_se.group(10).lstrip(string.digits) == "T":
                    RUNWAY_MAG_BEARING = int(Decimal(rwy_se.group(10).rstrip('T')))
                else:
                    RUNWAY_MAG_BEARING = int(Decimal(rwy_se.group(10)[0:3] + '.' + rwy_se.group(10)[3:]).quantize(Decimal('0'), rounding=ROUND_HALF_UP))
                RUNWAY_LENGTH = int(rwy_se.group(9))
                RUNWAY_WIDTH = int(rwy_se.group(27))
                if rwy_se.group(29).strip() != '':
                    have_ils = 1
                else:
                    have_ils = 0
                if Localizer_as_tb.loc[(Localizer_as_tb['ARPT_IDENT'] == ARPT_IDENT) & (Localizer_as_tb['RUNWAY_IDENT'] == rwy_se.group(7).strip())].index.tolist() == []: 
                    # print('         Aerosoft: Runway ' + ARPT_IDENT + ' ' + rwy_se.group(7) + ' Have No ILS')
                    LOCALIZER_FREQ = '0.000'
                    LOCALIZER_BEARING = '0'
                    GLIDE_SLOPE_ANGLE = '3.00'
                else:
                    LOCALIZER_FREQ = Localizer_as_tb.loc[Localizer_as_tb.loc[(Localizer_as_tb['ARPT_IDENT'] == ARPT_IDENT) & (Localizer_as_tb['RUNWAY_IDENT'] == rwy_se.group(7).strip())].index.tolist() , 'LOCALIZER_FREQ'].values[0]
                    LOCALIZER_BEARING = Localizer_as_tb.loc[Localizer_as_tb.loc[(Localizer_as_tb['ARPT_IDENT'] == ARPT_IDENT) & (Localizer_as_tb['RUNWAY_IDENT'] == rwy_se.group(7).strip())].index.tolist() , 'LOCALIZER_BEARING'].values[0]
                    GLIDE_SLOPE_ANGLE = Localizer_as_tb.loc[Localizer_as_tb.loc[(Localizer_as_tb['ARPT_IDENT'] == ARPT_IDENT) & (Localizer_as_tb['RUNWAY_IDENT'] == rwy_se.group(7).strip())].index.tolist() , 'GLIDE_SLOPE_ANGLE'].values[0]
                    if GLIDE_SLOPE_ANGLE == None:
                        GLIDE_SLOPE_ANGLE = '3.00'
                if rwy_se.group(11) == "N":
                    RUNWAY_LAT = str(Decimal(Decimal(rwy_se.group(12)) + Decimal(rwy_se.group(13))/60 + Decimal(rwy_se.group(14))/3600 + Decimal(rwy_se.group(15))/360000).quantize(Decimal('0.000000'), rounding=ROUND_HALF_UP))
                else:
                    RUNWAY_LAT = str(Decimal(0 - Decimal(rwy_se.group(12)) - Decimal(rwy_se.group(13))/60 - Decimal(rwy_se.group(14))/3600 - Decimal(rwy_se.group(15))/360000).quantize(Decimal('0.000000'), rounding=ROUND_HALF_UP))
                if rwy_se.group(16) == "E":
                    RUNWAY_LON = str(Decimal(Decimal(rwy_se.group(17)) + Decimal(rwy_se.group(18))/60 + Decimal(rwy_se.group(19))/3600 + Decimal(rwy_se.group(20))/360000).quantize(Decimal('0.000000'), rounding=ROUND_HALF_UP))
                else:
                    RUNWAY_LON = str(Decimal(0 - Decimal(rwy_se.group(17)) - Decimal(rwy_se.group(18))/60 - Decimal(rwy_se.group(19))/3600 - Decimal(rwy_se.group(20))/360000).quantize(Decimal('0.000000'), rounding=ROUND_HALF_UP))
                LANDING_THLD_ELEV = int(rwy_se.group(24))
                if rwy_se.group(26).strip() == '':
                    THLD_CROSSING_HEIGHT = "50"
                else:
                    THLD_CROSSING_HEIGHT = rwy_se.group(26)
                Runways_as.append((ARPT_IDENT,RUNWAY_IDENT,RUNWAY_MAG_BEARING,RUNWAY_LENGTH,RUNWAY_WIDTH,have_ils,LOCALIZER_FREQ,LOCALIZER_BEARING,RUNWAY_LAT,RUNWAY_LON,LANDING_THLD_ELEV,GLIDE_SLOPE_ANGLE,THLD_CROSSING_HEIGHT))
                with open(Cycle_Ident + '/Aerosoft/Airports.txt', 'a') as Airports:
                    print(str('R') + ',' + str(RUNWAY_IDENT) + ',' + str(RUNWAY_MAG_BEARING) + ',' + str(RUNWAY_LENGTH) + ',' + str(RUNWAY_WIDTH) + ',' + str(have_ils) + ',' + str(LOCALIZER_FREQ) + ',' + str(LOCALIZER_BEARING) + ',' + str(RUNWAY_LAT) + ',' + str(RUNWAY_LON) + ',' + str(LANDING_THLD_ELEV) + ',' + str(GLIDE_SLOPE_ANGLE) + ',' + str(THLD_CROSSING_HEIGHT) + ',' + str('1') + ',' + str('0'), file=Airports)
                with open(Cycle_Ident + '/Aerosoft/PROC/' + ARPT_IDENT + '.txt', 'a') as PROC:
                    print(str('R') + ',' + str(RUNWAY_IDENT) + ',' + str(RUNWAY_MAG_BEARING) + ',' + str(RUNWAY_LENGTH) + ',' + str(RUNWAY_WIDTH) + ',' + str(have_ils) + ',' + str(LOCALIZER_FREQ) + ',' + str(LOCALIZER_BEARING) + ',' + str(RUNWAY_LAT) + ',' + str(RUNWAY_LON) + ',' + str(LANDING_THLD_ELEV) + ',' + str(GLIDE_SLOPE_ANGLE) + ',' + str(THLD_CROSSING_HEIGHT) + ',' + str('1') + ',' + str('0'), file=PROC)
        # Convert list to Pandas DataFrame
        Airports_as_tb = pandas.DataFrame(Airports_as, columns=['ARPT_IDENT','ARPT_NAME','ARPT_LAT','ARPT_LON','ARPT_ELEV','TRANSITIONS_ALT','TRANSITION_LEVEL','LONGEST_RUNWAY','MAG_TRUE'])
        Runways_as_tb = pandas.DataFrame(Runways_as, columns=['ARPT_IDENT','RUNWAY_IDENT','RUNWAY_MAG_BEARING','RUNWAY_LENGTH','RUNWAY_WIDTH','have_ils','LOCALIZER_FREQ','LOCALIZER_BEARING','RUNWAY_LAT','RUNWAY_LON','LANDING_THLD_ELEV','GLIDE_SLOPE_ANGLE','THLD_CROSSING_HEIGHT'])
        Airway_as_tb = pandas.DataFrame(Airway_as, columns=['CUST_AREA','ROUTE_IDENT','SEQ_NR','FIX_IDENT','FIX_ICAO_CODE','FIX_LAT','FIX_LON','EndOfRoute','DIRECTION_RESTRICTION','INBOUND_MAG_COURSE','OUTBOUND_MAG_COURSE','ROUTE_DISTANCE_FROM'])
        Airway_as_tb = Airway_as_tb.sort_values(['ROUTE_IDENT','SEQ_NR','EndOfRoute'], ascending=[1,1,0])
        Airway_as_tb = Airway_as_tb.reset_index(drop=True)
        Airway_as = []
        print(time.strftime("%H:%M:%S", time.localtime()) , 'Aerosoft: Data Process Part #2 Finished')
    # Process of Aerosoft Part #3 (AWY Part #2)
    if len(Airway_as_tb.index) != 0:
        for index, row in Airway_as_tb.iterrows():
            if index + 1 != len(Airway_as_tb.index):
                if row['FIX_IDENT'] != Airway_as_tb.at[index + 1, 'FIX_IDENT']:
                    Airway_as.append((row['CUST_AREA'],row['ROUTE_IDENT'],row['SEQ_NR'],row['FIX_IDENT'],row['FIX_ICAO_CODE'],row['FIX_LAT'],row['FIX_LON'],row['EndOfRoute'],row['DIRECTION_RESTRICTION'],row['INBOUND_MAG_COURSE'],row['OUTBOUND_MAG_COURSE'],row['ROUTE_DISTANCE_FROM']))
                else:
                    if row['ROUTE_IDENT'] != Airway_as_tb.at[index + 1, 'ROUTE_IDENT']:
                        Airway_as.append((row['CUST_AREA'],row['ROUTE_IDENT'],row['SEQ_NR'],row['FIX_IDENT'],row['FIX_ICAO_CODE'],row['FIX_LAT'],row['FIX_LON'],row['EndOfRoute'],row['DIRECTION_RESTRICTION'],row['INBOUND_MAG_COURSE'],row['OUTBOUND_MAG_COURSE'],row['ROUTE_DISTANCE_FROM']))
            else:
                Airway_as.append((row['CUST_AREA'],row['ROUTE_IDENT'],row['SEQ_NR'],row['FIX_IDENT'],row['FIX_ICAO_CODE'],row['FIX_LAT'],row['FIX_LON'],row['EndOfRoute'],row['DIRECTION_RESTRICTION'],row['INBOUND_MAG_COURSE'],row['OUTBOUND_MAG_COURSE'],row['ROUTE_DISTANCE_FROM']))
        if 1:
            Airway_as_tb = pandas.DataFrame(Airway_as, columns=['CUST_AREA','ROUTE_IDENT','SEQ_NR','FIX_IDENT','FIX_ICAO_CODE','FIX_LAT','FIX_LON','EndOfRoute','DIRECTION_RESTRICTION','INBOUND_MAG_COURSE','OUTBOUND_MAG_COURSE','ROUTE_DISTANCE_FROM'])
            Airway_as_tb = Airway_as_tb.sort_values(['ROUTE_IDENT','SEQ_NR'])
            Airway_as_tb = Airway_as_tb.reset_index(drop=True)
            Airway_reversed_as_tb = Airway_as_tb.sort_index(ascending=False)
            Airway_reversed_as_tb = Airway_reversed_as_tb.reset_index(drop=True)
        for index, row in Airway_as_tb.iterrows():
            if index + 1  != len(Airway_as_tb.index):
                Waypoint1_IDENT = row['FIX_IDENT']
                Waypoint1_LAT = row['FIX_LAT']
                Waypoint1_LON = row['FIX_LON']
                Waypoint2_IDENT = Airway_as_tb.at[index + 1, 'FIX_IDENT']
                Waypoint2_LAT = Airway_as_tb.at[index + 1, 'FIX_LAT']
                Waypoint2_LON = Airway_as_tb.at[index + 1, 'FIX_LON']
                if row['ROUTE_IDENT'] == AirwayIdent:
                    if WasEnd == 0:
                        IsStart = 0
                    else:
                        IsStart = 1
                        AirwaySegmentID = AirwaySegmentID + 1
                elif row['DIRECTION_RESTRICTION'] == 'B':
                    IsStart = 0
                else:
                    IsStart = 1
                    AirwayIdent = row['ROUTE_IDENT']
                    AirwaySegmentID = AirwaySegmentID + 1
                if index + 2  == len(Airway_as_tb.index):
                    IsEnd = 1
                elif Airway_as_tb.at[index + 1, 'DIRECTION_RESTRICTION'] == 'B':
                    IsEnd = 1
                elif Airway_as_tb.at[index + 2, 'ROUTE_IDENT'] != row['ROUTE_IDENT']:
                    IsEnd = 1
                elif Airway_as_tb.at[index + 1, 'EndOfRoute'] == 'E':
                    IsEnd = 1
                else:
                    IsEnd = 0
                # For GeoBox Baseline
                if IsStart == 1:
                    INBOUND_MAG_COURSE = '0'
                    OUTBOUND_MAG_COURSE = row['OUTBOUND_MAG_COURSE']
                    ROUTE_DISTANCE_FROM = row['ROUTE_DISTANCE_FROM']
                else:
                    if IsEnd == 0:
                        INBOUND_MAG_COURSE = row['INBOUND_MAG_COURSE']
                        OUTBOUND_MAG_COURSE = row['OUTBOUND_MAG_COURSE']
                        ROUTE_DISTANCE_FROM = row['ROUTE_DISTANCE_FROM']
                    else:
                        INBOUND_MAG_COURSE = row['INBOUND_MAG_COURSE']
                        OUTBOUND_MAG_COURSE = '0'
                        ROUTE_DISTANCE_FROM = row['ROUTE_DISTANCE_FROM']
                if Airway_as_tb.at[index + 1, 'ROUTE_IDENT'] == row['ROUTE_IDENT']:
                    if row['DIRECTION_RESTRICTION'] != 'B':
                        if row['EndOfRoute'] != 'E':
                            AirwayLegsID = AirwayLegsID + 1
                            AirwayLegs_as.append(('S',AirwayIdent,AirwaySegmentID,AirwayLegsID,Waypoint1_IDENT,Waypoint1_LAT,Waypoint1_LON,Waypoint2_IDENT,Waypoint2_LAT,Waypoint2_LON,INBOUND_MAG_COURSE,OUTBOUND_MAG_COURSE,ROUTE_DISTANCE_FROM,IsStart,IsEnd))
                            WasEnd = IsEnd
                if WasEnd == 1:
                    AirwayLegsID = 0
        for index, row in Airway_reversed_as_tb.iterrows():
            if index + 1  != len(Airway_reversed_as_tb.index):
                Waypoint1_IDENT = row['FIX_IDENT']
                Waypoint1_LAT = row['FIX_LAT']
                Waypoint1_LON = row['FIX_LON']
                Waypoint2_IDENT = Airway_reversed_as_tb.at[index + 1, 'FIX_IDENT']
                Waypoint2_LAT = Airway_reversed_as_tb.at[index + 1, 'FIX_LAT']
                Waypoint2_LON = Airway_reversed_as_tb.at[index + 1, 'FIX_LON']
                if row['ROUTE_IDENT'] == AirwayIdent:
                    if row['EndOfRoute'] == 'E':
                        IsStart = 1
                        AirwaySegmentID = AirwaySegmentID + 1
                    elif WasEnd == 1:
                        IsStart = 1
                        AirwaySegmentID = AirwaySegmentID + 1
                    else:
                        IsStart = 0
                elif Airway_reversed_as_tb.at[index + 1, 'DIRECTION_RESTRICTION'] != 'F':
                    IsStart = 1
                    AirwayIdent = row['ROUTE_IDENT']
                    AirwaySegmentID = AirwaySegmentID + 1
                else:
                    IsStart = 0
                if index + 2  == len(Airway_reversed_as_tb.index):
                    IsEnd = 1
                elif Airway_reversed_as_tb.at[index + 2, 'DIRECTION_RESTRICTION'] == 'F':
                    IsEnd = 1
                elif Airway_reversed_as_tb.at[index + 2, 'ROUTE_IDENT'] != row['ROUTE_IDENT']:
                    IsEnd = 1
                elif Airway_reversed_as_tb.at[index + 2, 'EndOfRoute'] == 'E':
                    IsEnd = 1
                else:
                    IsEnd = 0
                # For GeoBox Baseline
                if IsStart == 1:
                    INBOUND_MAG_COURSE = '0'
                    OUTBOUND_MAG_COURSE = row['INBOUND_MAG_COURSE']
                    ROUTE_DISTANCE_FROM = Airway_reversed_as_tb.at[index + 1, 'ROUTE_DISTANCE_FROM']
                else:
                    if IsEnd == 0:
                        INBOUND_MAG_COURSE = row['OUTBOUND_MAG_COURSE']
                        OUTBOUND_MAG_COURSE = row['INBOUND_MAG_COURSE']
                        ROUTE_DISTANCE_FROM = Airway_reversed_as_tb.at[index + 1, 'ROUTE_DISTANCE_FROM']
                    else:
                        INBOUND_MAG_COURSE = row['INBOUND_MAG_COURSE']
                        OUTBOUND_MAG_COURSE = '0'
                        ROUTE_DISTANCE_FROM = Airway_reversed_as_tb.at[index + 1, 'ROUTE_DISTANCE_FROM']
                if Airway_reversed_as_tb.at[index + 1, 'ROUTE_IDENT'] == row['ROUTE_IDENT']:
                    if Airway_reversed_as_tb.at[index + 1, 'DIRECTION_RESTRICTION'] != 'F':
                        if Airway_reversed_as_tb.at[index + 1, 'EndOfRoute'] != 'E':
                            AirwayLegsID = AirwayLegsID + 1
                            AirwayLegs_reversed_as.append(('S',AirwayIdent,AirwaySegmentID,AirwayLegsID,Waypoint1_IDENT,Waypoint1_LAT,Waypoint1_LON,Waypoint2_IDENT,Waypoint2_LAT,Waypoint2_LON,INBOUND_MAG_COURSE,OUTBOUND_MAG_COURSE,ROUTE_DISTANCE_FROM,IsStart,IsEnd))
                            WasEnd = IsEnd
                if WasEnd == 1:
                    AirwayLegsID = 0
        AirwayLegs_as_tb = pandas.DataFrame(AirwayLegs_as, columns=['HDR','AirwayIdent','AirwaySegmentID','AirwayLegsID','Waypoint1_IDENT','Waypoint1_LAT','Waypoint1_LON','Waypoint2_IDENT','Waypoint2_LAT','Waypoint2_LON','INBOUND_MAG_COURSE','OUTBOUND_MAG_COURSE','ROUTE_DISTANCE_FROM','IsStart','IsEnd'])
        AirwayLegs_reversed_as_tb = pandas.DataFrame(AirwayLegs_reversed_as, columns=['HDR','AirwayIdent','AirwaySegmentID','AirwayLegsID','Waypoint1_IDENT','Waypoint1_LAT','Waypoint1_LON','Waypoint2_IDENT','Waypoint2_LAT','Waypoint2_LON','INBOUND_MAG_COURSE','OUTBOUND_MAG_COURSE','ROUTE_DISTANCE_FROM','IsStart','IsEnd'])
        AirwayLegs_as_tb = pandas.concat([AirwayLegs_as_tb, AirwayLegs_reversed_as_tb], axis=0, ignore_index=True)
        AirwayLegs_as_tb = AirwayLegs_as_tb.sort_values(['AirwayIdent','AirwaySegmentID','AirwayLegsID'])
        AirwayLegs_as_tb = AirwayLegs_as_tb.reset_index(drop=True)
        if os.path.isfile('Input/Aerosoft/ATS.txt'):
            with open(Cycle_Ident + '/Aerosoft/ATS.txt', 'a+') as ATS:
                for index, row in AirwayLegs_as_tb.iterrows():
                    if index + 1  != len(AirwayLegs_as_tb.index):
                        ATS_as.append((row['HDR'],row['Waypoint1_IDENT'],row['Waypoint1_LAT'],row['Waypoint1_LON'],row['Waypoint2_IDENT'],row['Waypoint2_LAT'],row['Waypoint2_LON'],row['INBOUND_MAG_COURSE'],row['OUTBOUND_MAG_COURSE'],row['ROUTE_DISTANCE_FROM']))
                        if AirwayLegs_as_tb.at[index + 1, 'AirwaySegmentID'] !=  row['AirwaySegmentID']:
                            print('A,' + row['AirwayIdent'] + ',' + str(row['AirwayLegsID']), file=ATS)
                            ATS_as_tb = pandas.DataFrame(ATS_as, columns=['HDR','Waypoint1_IDENT','Waypoint1_LAT','Waypoint1_LON','Waypoint2_IDENT','Waypoint2_LAT','Waypoint2_LON','INBOUND_MAG_COURSE','OUTBOUND_MAG_COURSE','ROUTE_DISTANCE_FROM'])
                            ATS_as_tb.to_csv(path_or_buf = ATS, mode = 'a+', index = False, header = False, line_terminator='\n')
                            ATS_as = []
                            print('', file=ATS)
                    else:
                        ATS_as.append((row['HDR'],row['Waypoint1_IDENT'],row['Waypoint1_LAT'],row['Waypoint1_LON'],row['Waypoint2_IDENT'],row['Waypoint2_LAT'],row['Waypoint2_LON'],row['INBOUND_MAG_COURSE'],row['OUTBOUND_MAG_COURSE'],row['ROUTE_DISTANCE_FROM']))
                        print('A,' + row['AirwayIdent'] + ',' + str(row['AirwayLegsID']), file=ATS)
                        ATS_as_tb = pandas.DataFrame(ATS_as, columns=['HDR','Waypoint1_IDENT','Waypoint1_LAT','Waypoint1_LON','Waypoint2_IDENT','Waypoint2_LAT','Waypoint2_LON','INBOUND_MAG_COURSE','OUTBOUND_MAG_COURSE','ROUTE_DISTANCE_FROM'])
                        ATS_as_tb.to_csv(path_or_buf = ATS, mode = 'a+', index = False, header = False, line_terminator='\n')
                        ATS_as = []
            if as_create_blank != True:
                for line in open(Cycle_Ident + '/Aerosoft/ATS.txt'):
                    if line[0:1] == 'A':
                        AirwayIdent = line.split(',', 2)[1]
                    ATS_as.append((AirwayIdent,line.strip('\n')))
                ATS_as_tb = pandas.DataFrame(ATS_as,columns=['AirwayIdent','Rows'])
                ATS_as_tb['index'] = ATS_as_tb.index + 1
                ATS_as_tb = ATS_as_tb.sort_values(['AirwayIdent','index'])
                ATS_as_tb = ATS_as_tb.reset_index(drop=True)
                ATS_as_tb = ATS_as_tb.drop(columns=['index','AirwayIdent'])
                with open(Cycle_Ident + '/Aerosoft/ATS_out.txt', 'w') as ATS:
                    for index, row in ATS_as_tb.iterrows():
                        print(row['Rows'].strip('"'), file = ATS)
                os.remove(Cycle_Ident + '/Aerosoft/ATS.txt')
                os.rename(Cycle_Ident + '/Aerosoft/ATS_out.txt', Cycle_Ident + '/Aerosoft/ATS.txt')
        print(time.strftime("%H:%M:%S", time.localtime()) , 'Aerosoft: Data Process Part #3 Finished')
    # Process of Aerosoft Part #4
    with open('Input/' + Datafile, 'r', encoding='ISO-8859-1') as ARINC424:
        ARPT_IDENT = ''
        for line in ARINC424: 
            proc_se = re.search(pdef, line)
            # PROC of AS
            if proc_se:
                if proc_se.group(4) != ARPT_IDENT:
                    ARPT_IDENT = proc_se.group(4)
                    if os.path.isfile(Cycle_Ident + '/Aerosoft/PROC/' + ARPT_IDENT + '.txt') == True:
                        with open(Cycle_Ident + '/Aerosoft/PROC/' + ARPT_IDENT + '.txt', 'a+') as PROC:
                            print('', file=PROC)
                    else:
                        if as_create_blank != True:
                            if os.path.isdir('Input/Aerosoft/PROC/'):
                                if os.path.isfile('Input/Aerosoft/PROC/' + ARPT_IDENT + '.txt'):
                                    shutil.copy2('Input/Aerosoft/PROC/' + ARPT_IDENT + '.txt', Cycle_Ident + '/Aerosoft/PROC/' + ARPT_IDENT + '.txt')
                                    with open(Cycle_Ident + '/Aerosoft/PROC/' + ARPT_IDENT + '.txt', 'a+') as PROC:
                                        print('', file=PROC)
                SUBS_CODE = proc_se.group(6)
                PROC_IDENT = proc_se.group(7).strip()
                ROUTE_TYPE = proc_se.group(8)
                if proc_se.group(9).strip() == '':
                    TRANSITION_IDENT = 'ALL'
                elif proc_se.group(9)[0:4].strip(string.digits) == 'RW':
                    TRANSITION_IDENT = proc_se.group(9)[2:5].strip()
                else:
                    TRANSITION_IDENT = proc_se.group(9).strip()
                if proc_se.group(11).strip() != '':
                    FIX_IDENT = proc_se.group(11).strip()
                else:
                    FIX_IDENT = ' '
                FIX_ICAO_CODE = proc_se.group(12)
                FIX_SECT_SUBS = proc_se.group(13) + proc_se.group(14)
                if FIX_SECT_SUBS == "  ":
                    FIX_LAT = '00.000000'
                    FIX_LON = '000.000000'
                elif FIX_SECT_SUBS == "PA":
                    ARPT_ID = Airports_as_tb.loc[(Airports_as_tb['ARPT_IDENT'] == ARPT_IDENT)].index.tolist()
                    if ARPT_ID != []:
                        FIX_LAT = Airports_as_tb.at[ARPT_ID[0], 'ARPT_LAT']
                        FIX_LON = Airports_as_tb.at[ARPT_ID[0], 'ARPT_LON']
                    else:
                        if as_create_blank != True:
                            ARPT_ID = Airports_in_as_nd_tb.loc[(Airports_in_as_nd_tb['ARPT_IDENT'] == ARPT_IDENT)].index.tolist()
                            if ARPT_ID != []:
                                FIX_LAT = Airports_in_as_nd_tb.at[ARPT_ID[0], 'ARPT_LAT']
                                FIX_LON = Airports_in_as_nd_tb.at[ARPT_ID[0], 'ARPT_LON']
                            else:
                                print('         Aerosoft: Leg ' + ARPT_IDENT + ' ' + proc_se.group(7) + ' ' + proc_se.group(11) + ' Airport LAT/LON Not Found')
                                FIX_LAT = '00.000000'
                                FIX_LON = '000.000000'
                        else:
                            print('         Aerosoft: Leg ' + ARPT_IDENT + ' ' + proc_se.group(7) + ' ' + proc_se.group(11) + ' Airport LAT/LON Not Found')
                            FIX_LAT = '00.000000'
                            FIX_LON = '000.000000'
                elif FIX_SECT_SUBS == "PC":
                    FIX_ID = Waypoints_as_tb.loc[(Waypoints_as_tb['WAYPOINT_IDENT'] == FIX_IDENT) & (Waypoints_as_tb['WAYPOINT_ICAO_CODE'] == FIX_ICAO_CODE)& (Waypoints_as_tb['REGION_CODE'] == ARPT_IDENT)].index.tolist()
                    if FIX_ID != []:
                        FIX_LAT = Waypoints_as_tb.at[FIX_ID[0], 'WAYPOINT_LAT']
                        FIX_LON = Waypoints_as_tb.at[FIX_ID[0], 'WAYPOINT_LON']
                    else:
                        if as_create_blank != True:
                            FIX_ID = Waypoints_in_as_nd_tb.loc[(Waypoints_in_as_nd_tb['WAYPOINT_IDENT'] == FIX_IDENT) & (Waypoints_in_as_nd_tb['WAYPOINT_ICAO_CODE'] == FIX_ICAO_CODE)].index.tolist()
                            if FIX_ID != []:
                                # Screen for duplicated Waypoint
                                if len(FIX_ID) == 1:
                                    FIX_LAT = Waypoints_in_as_nd_tb.at[FIX_ID[0], 'WAYPOINT_LAT']
                                    FIX_LON = Waypoints_in_as_nd_tb.at[FIX_ID[0], 'WAYPOINT_LON']
                                else:
                                    ARPT_ID = Airports_as_tb.loc[(Airports_as_tb['ARPT_IDENT'] == ARPT_IDENT)].index.tolist()
                                    if ARPT_ID != []:
                                        ARPT_LAT = Airports_as_tb.at[ARPT_ID[0], 'ARPT_LAT']
                                        ARPT_LON = Airports_as_tb.at[ARPT_ID[0], 'ARPT_LON']
                                    else:
                                        if as_create_blank != True:
                                            ARPT_ID = Airports_in_as_nd_tb.loc[(Airports_in_as_nd_tb['ARPT_IDENT'] == ARPT_IDENT)].index.tolist()
                                            if ARPT_ID != []:
                                                ARPT_LAT = Airports_in_as_nd_tb.at[ARPT_ID[0], 'ARPT_LAT']
                                                ARPT_LON = Airports_in_as_nd_tb.at[ARPT_ID[0], 'ARPT_LON']
                                            else:
                                                ARPT_LAT = '00.000000'
                                                ARPT_LON = '000.000000'
                                        else:
                                            ARPT_LAT = '00.000000'
                                            ARPT_LON = '000.000000'
                                    dist = 100
                                    for a in FIX_ID:
                                        b = ((float(Waypoints_in_as_nd_tb.at[a, 'WAYPOINT_LAT']) - float(ARPT_LAT)) ** 2 + (float(Waypoints_in_as_nd_tb.at[a, 'WAYPOINT_LON']) - float(ARPT_LON)) ** 2) ** 0.5
                                        if b < dist:
                                            dist = b
                                            FIX_LAT = Waypoints_in_as_nd_tb.at[a, 'WAYPOINT_LAT']
                                            FIX_LON = Waypoints_in_as_nd_tb.at[a, 'WAYPOINT_LON']
                            else:
                                print('         Aerosoft: Leg ' + ARPT_IDENT + ' ' + proc_se.group(7) + ' ' + proc_se.group(11) + ' Waypoint LAT/LON Not Found')
                                FIX_LAT = '00.000000'
                                FIX_LON = '000.000000'
                        else:
                            print('         Aerosoft: Leg ' + ARPT_IDENT + ' ' + proc_se.group(7) + ' ' + proc_se.group(11) + ' Waypoint LAT/LON Not Found')
                            FIX_LAT = '00.000000'
                            FIX_LON = '000.000000'
                elif FIX_SECT_SUBS == "PG":
                    RWY_ID = Runways_as_tb.loc[(Runways_as_tb['ARPT_IDENT'] == ARPT_IDENT) & (Runways_as_tb['RUNWAY_IDENT'] == proc_se.group(11).strip().replace('RW',''))].index.tolist()
                    if RWY_ID != []:
                        FIX_LAT = Runways_as_tb.at[RWY_ID[0], 'RUNWAY_LAT']
                        FIX_LON = Runways_as_tb.at[RWY_ID[0], 'RUNWAY_LON']
                    else:
                        if as_create_blank != True:
                            RWY_ID = Runways_in_as_nd_tb.loc[(Runways_in_as_nd_tb['ARPT_IDENT'] == ARPT_IDENT) & (Runways_in_as_nd_tb['RUNWAY_IDENT'] == proc_se.group(11).strip().replace('RW',''))].index.tolist()
                            if RWY_ID != []:
                                FIX_LAT = Runways_in_as_nd_tb.at[RWY_ID[0], 'RUNWAY_LAT']
                                FIX_LON = Runways_in_as_nd_tb.at[RWY_ID[0], 'RUNWAY_LON']
                            else:
                                print('         Aerosoft: Leg ' + ARPT_IDENT + ' ' + proc_se.group(7) + ' ' + proc_se.group(11) + ' Runway LAT/LON Not Found')
                                FIX_LAT = '00.000000'
                                FIX_LON = '000.000000'
                        else:
                            print('         Aerosoft: Leg ' + ARPT_IDENT + ' ' + proc_se.group(7) + ' ' + proc_se.group(11) + ' Runway LAT/LON Not Found')
                            FIX_LAT = '00.000000'
                            FIX_LON = '000.000000'
                elif FIX_SECT_SUBS == "EA":
                    FIX_ID = Waypoints_as_tb.loc[(Waypoints_as_tb['WAYPOINT_IDENT'] == FIX_IDENT) & (Waypoints_as_tb['WAYPOINT_ICAO_CODE'] == FIX_ICAO_CODE)].index.tolist()
                    if FIX_ID != []:
                        FIX_LAT = Waypoints_as_tb.at[FIX_ID[0], 'WAYPOINT_LAT']
                        FIX_LON = Waypoints_as_tb.at[FIX_ID[0], 'WAYPOINT_LON']
                    else:
                        if as_create_blank != True:
                            FIX_ID = Waypoints_in_as_nd_tb.loc[(Waypoints_in_as_nd_tb['WAYPOINT_IDENT'] == FIX_IDENT) & (Waypoints_in_as_nd_tb['WAYPOINT_ICAO_CODE'] == FIX_ICAO_CODE)].index.tolist()
                            if FIX_ID != []:
                                FIX_LAT = Waypoints_in_as_nd_tb.at[FIX_ID[0], 'WAYPOINT_LAT']
                                FIX_LON = Waypoints_in_as_nd_tb.at[FIX_ID[0], 'WAYPOINT_LON']
                            else:
                                print('         Aerosoft: Leg ' + ARPT_IDENT + ' ' + proc_se.group(7) + ' ' + proc_se.group(11) + ' Waypoint LAT/LON Not Found')
                                FIX_LAT = '00.000000'
                                FIX_LON = '000.000000'
                        else:
                            print('         Aerosoft: Leg ' + ARPT_IDENT + ' ' + proc_se.group(7) + ' ' + proc_se.group(11) + ' Waypoint LAT/LON Not Found')
                            FIX_LAT = '00.000000'
                            FIX_LON = '000.000000'
                elif FIX_SECT_SUBS == "D ":
                    FIX_ID = Navaids_as_tb.loc[(Navaids_as_tb['NAV_IDENT'] == FIX_IDENT) & (Navaids_as_tb['NAV_ICAO_CODE'] == FIX_ICAO_CODE) & (Navaids_as_tb['is_ndb'] == '0')].index.tolist()
                    if FIX_ID != []:
                        FIX_LAT = Navaids_as_tb.at[FIX_ID[0], 'NAV_LAT']
                        FIX_LON = Navaids_as_tb.at[FIX_ID[0], 'NAV_LON']
                    else:
                        if as_create_blank != True:
                            FIX_ID = Navaids_in_as_nd_tb.loc[(Navaids_in_as_nd_tb['NAV_IDENT'] == FIX_IDENT) & (Navaids_in_as_nd_tb['NAV_ICAO_CODE'] == FIX_ICAO_CODE) & (Navaids_in_as_nd_tb['is_ndb'] == '0')].index.tolist()
                            if FIX_ID != []:
                                FIX_LAT = Navaids_in_as_nd_tb.at[FIX_ID[0], 'NAV_LAT']
                                FIX_LON = Navaids_in_as_nd_tb.at[FIX_ID[0], 'NAV_LON']
                            else:
                                print('         Aerosoft: Leg ' + ARPT_IDENT + ' ' + proc_se.group(7) + ' ' + proc_se.group(11) + ' Navaid LAT/LON Not Found')
                                FIX_LAT = '00.000000'
                                FIX_LON = '000.000000'
                        else:
                            print('         Aerosoft: Leg ' + ARPT_IDENT + ' ' + proc_se.group(7) + ' ' + proc_se.group(11) + ' Navaid LAT/LON Not Found')
                            FIX_LAT = '00.000000'
                            FIX_LON = '000.000000'
                else:
                    FIX_ID = Navaids_as_tb.loc[(Navaids_as_tb['NAV_IDENT'] == FIX_IDENT) & (Navaids_as_tb['NAV_ICAO_CODE'] == FIX_ICAO_CODE) & (Navaids_as_tb['is_ndb'] == '1')].index.tolist()
                    if FIX_ID != []:
                        FIX_LAT = Navaids_as_tb.at[FIX_ID[0], 'NAV_LAT']
                        FIX_LON = Navaids_as_tb.at[FIX_ID[0], 'NAV_LON']
                    else:
                        if as_create_blank != True:
                            FIX_ID = Navaids_in_as_nd_tb.loc[(Navaids_in_as_nd_tb['NAV_IDENT'] == FIX_IDENT) & (Navaids_in_as_nd_tb['NAV_ICAO_CODE'] == FIX_ICAO_CODE) & (Navaids_in_as_nd_tb['is_ndb'] == '1')].index.tolist()
                            if FIX_ID != []:
                                FIX_LAT = Navaids_in_as_nd_tb.at[FIX_ID[0], 'NAV_LAT']
                                FIX_LON = Navaids_in_as_nd_tb.at[FIX_ID[0], 'NAV_LON']
                            else:
                                print('         Aerosoft: Leg ' + ARPT_IDENT + ' ' + proc_se.group(7) + ' ' + proc_se.group(11) + ' Navaid LAT/LON Not Found')
                                FIX_LAT = '00.000000'
                                FIX_LON = '000.000000'
                        else:
                            print('         Aerosoft: Leg ' + ARPT_IDENT + ' ' + proc_se.group(7) + ' ' + proc_se.group(11) + ' Navaid LAT/LON Not Found')
                            FIX_LAT = '00.000000'
                            FIX_LON = '000.000000'
                if proc_se.group(16)[3:4] == 'A':
                    fix_type = '1'
                elif proc_se.group(16)[3:4] == 'B':
                    fix_type = '1'
                elif proc_se.group(16)[3:4] == 'F':
                    fix_type = '2'
                elif proc_se.group(16)[3:4] == 'M':
                    fix_type = '3'
                else:
                    fix_type = '0'
                if proc_se.group(16)[1:2] == 'Y':
                    ovfly = '1'
                elif proc_se.group(16)[1:2] == 'B':
                    ovfly = '1'
                else:
                    ovfly = '0'
                if proc_se.group(17).strip() == 'L':
                    TURN_DIR = '1'
                elif proc_se.group(17) == 'R':
                    TURN_DIR = '2'
                else:
                    TURN_DIR = '0'
                PATH_AND_TERMINATION = proc_se.group(19)
                if PATH_AND_TERMINATION == 'RF':
                    RECOMMENDED_NAVAID = proc_se.group(37).strip()
                elif proc_se.group(21).strip() == '':
                    RECOMMENDED_NAVAID = ' '
                else:
                    RECOMMENDED_NAVAID = proc_se.group(21).strip()
                if proc_se.group(24).strip() == '':
                    THETA = 0.0
                else:
                    THETA = float(proc_se.group(24))/10
                if PATH_AND_TERMINATION == 'RF':
                    if proc_se.group(23).strip() == '':
                        print('         Aerosoft: Leg ' + ARPT_IDENT + ' ' + proc_se.group(7) + ' ' + proc_se.group(11) + ' RF Radius Not Exist')
                        RHO = '0.0'
                    else:
                        RHO = round(float(proc_se.group(23))/1000, 4)
                elif proc_se.group(25).strip() == '':
                    RHO = 0.0
                else:
                    RHO = float(proc_se.group(25))/10
                if proc_se.group(26).strip() == '':
                    MAG_COURSE = 0.0
                elif proc_se.group(26).lstrip(string.digits) == "T":
                    MAG_COURSE = float(Decimal(proc_se.group(26).rstrip('T')))
                else:
                    MAG_COURSE = float(Decimal(proc_se.group(26)[0:3] + '.' + proc_se.group(26)[3:]).quantize(Decimal('0'), rounding=ROUND_HALF_UP))
                if proc_se.group(27).strip() == '':
                    Distance = 0.0
                elif proc_se.group(27).rstrip(string.digits) == "T":
                    Distance = float(proc_se.group(27).lstrip('T'))/2
                else:
                    Distance = float(proc_se.group(27))/10
                if proc_se.group(32).strip() == '':
                    ALT_1 = 0
                elif proc_se.group(32).rstrip(string.digits) == "FL":
                    ALT_1 = int(proc_se.group(32).lstrip(string.ascii_uppercase) + "00")
                else:
                    ALT_1 = int(proc_se.group(32))
                if proc_se.group(33).strip() == '':
                    ALT_2 = 0
                elif proc_se.group(33).rstrip(string.digits) == "FL":
                    ALT_2 = int(proc_se.group(33).lstrip(string.ascii_uppercase) + "00")
                else:
                    ALT_2 = int(proc_se.group(33))
                if proc_se.group(30).strip() == '+':
                    ALT_DESCR = '2'
                elif proc_se.group(30).strip() == '-':
                    ALT_DESCR = '3'
                elif proc_se.group(30).strip() == 'B':
                    ALT_DESCR = '4'
                elif proc_se.group(30).strip() == 'G':
                    ALT_DESCR = '1'
                    ALT_1 = ALT_2
                    ALT_2 = 0
                elif proc_se.group(30).strip() == 'H':
                    ALT_DESCR = '1'
                    ALT_1 = ALT_2
                    ALT_2 = 0
                elif proc_se.group(30).strip() == 'I':
                    ALT_DESCR = '1'
                    ALT_2 = 0
                elif proc_se.group(30).strip() == 'J':
                    ALT_DESCR = '2'
                    ALT_2 = 0
                elif ALT_1 != 0:
                    ALT_DESCR = '1'
                else:
                    ALT_DESCR = '0'
                if proc_se.group(35).strip() == '':
                    SPEED_LIMIT = 0
                    SPEED_LIMIT_DESCR = '0'
                else:
                    SPEED_LIMIT = proc_se.group(35)
                    SPEED_LIMIT_DESCR = '1'
                if SUBS_CODE == 'F':
                    if ROUTE_TYPE != 'A':
                        if MISSED_APCH == 0:
                            MISSED_APCH_COUNT = MISSED_APCH_COUNT + 1
                            if proc_se.group(16)[3:4] == 'M':
                                MISSED_APCH = 1
                                if proc_se.group(36).strip() == '':
                                    VERTICAL_ANGLE = 0
                                    have_glide_path = 0
                                else:
                                    VERTICAL_ANGLE = 0 - float(proc_se.group(36))/100
                                    have_glide_path = 1
                if proc_se.group(18).strip() == '':
                    RNP = ' '
                else:
                    RNP = float(proc_se.group(18)[0:2])/(10 ** float(proc_se.group(18)[2:]))
                if proc_se.group(16)[1:2] == 'E':
                    IsEnd = 1
                elif proc_se.group(16)[1:2] == 'B':
                    IsEnd = 1
                else:
                    IsEnd = 0
                # Different Process of All 23 Path Terminator
                if PATH_AND_TERMINATION == 'AF':
                    TerminalLegs_as.append((str(PATH_AND_TERMINATION) + ',' + str(FIX_IDENT) + ',' + str(FIX_LAT) + ',' + str(FIX_LON) + ',' + str(TURN_DIR) + ',' + str(RECOMMENDED_NAVAID) + ',' + str(THETA) + ',' + str(RHO) + ',' + str(MAG_COURSE) + ',' + str(ALT_DESCR) + ',' + str(ALT_1) + ',' + str(ALT_2) + ',' + str(SPEED_LIMIT_DESCR) + ',' + str(SPEED_LIMIT) + ',' + '0' + ',' + str(fix_type) + ',' + str(ovfly) + ',' + str(RNP)))
                elif PATH_AND_TERMINATION == 'CA':
                    TerminalLegs_as.append((str(PATH_AND_TERMINATION) + ',' + str(TURN_DIR) + ',' + str(MAG_COURSE) + ',' + str(ALT_DESCR) + ',' + str(ALT_1) + ',' + str(ALT_2) + ',' + str(SPEED_LIMIT_DESCR) + ',' + str(SPEED_LIMIT) + ',' + '0' + ',' + str(fix_type) + ',' + str(ovfly) + ',' + str(RNP)))
                elif PATH_AND_TERMINATION == 'CD':
                    TerminalLegs_as.append((str(PATH_AND_TERMINATION) + ',' + str(FIX_IDENT) + ',' + '0.000000' + ',' + '0.000000' + ',' + str(TURN_DIR) + ',' + str(RECOMMENDED_NAVAID) + ',' + '0' + ',' + '0.0' + ',' + str(MAG_COURSE) + ',' + str(Distance) + ',' + str(ALT_DESCR) + ',' + str(ALT_1) + ',' + str(ALT_2) + ',' + str(SPEED_LIMIT_DESCR) + ',' + str(SPEED_LIMIT) + ',' + '0' + ',' + str(fix_type) + ',' + str(ovfly) + ',' + str(RNP)))
                elif PATH_AND_TERMINATION == 'CF':
                    TerminalLegs_as.append((str(PATH_AND_TERMINATION) + ',' + str(FIX_IDENT) + ',' + str(FIX_LAT) + ',' + str(FIX_LON) + ',' + str(TURN_DIR) + ',' + str(RECOMMENDED_NAVAID) + ',' + str(THETA) + ',' + str(RHO) + ',' + str(MAG_COURSE) + ',' + str(Distance) + ',' + str(ALT_DESCR) + ',' + str(ALT_1) + ',' + str(ALT_2) + ',' + str(SPEED_LIMIT_DESCR) + ',' + str(SPEED_LIMIT) + ',' + '0' + ',' + str(fix_type) + ',' + str(ovfly) + ',' + str(RNP)))
                elif PATH_AND_TERMINATION == 'CI':
                    TerminalLegs_as.append((str(PATH_AND_TERMINATION) + ',' + str(TURN_DIR) + ',' + str(RECOMMENDED_NAVAID) + ',' + str(RHO) + ',' + str(MAG_COURSE) + ',' + str(ALT_DESCR) + ',' + str(ALT_1) + ',' + str(ALT_2) + ',' + str(SPEED_LIMIT_DESCR) + ',' + str(SPEED_LIMIT) + ',' + '0' + ',' + str(fix_type) + ',' + str(ovfly) + ',' + str(RNP)))
                elif PATH_AND_TERMINATION == 'CR':
                    TerminalLegs_as.append((str(PATH_AND_TERMINATION) + ',' + str(TURN_DIR) + ',' + str(RECOMMENDED_NAVAID) + ',' + str(THETA) + ',' + str(MAG_COURSE) + ',' + str(ALT_DESCR) + ',' + str(ALT_1) + ',' + str(ALT_2) + ',' + str(SPEED_LIMIT_DESCR) + ',' + str(SPEED_LIMIT) + ',' + '0' + ',' + str(fix_type) + ',' + str(ovfly) + ',' + str(RNP)))
                elif PATH_AND_TERMINATION == 'DF':
                    TerminalLegs_as.append((str(PATH_AND_TERMINATION) + ',' + str(FIX_IDENT) + ',' + str(FIX_LAT) + ',' + str(FIX_LON) + ',' + str(TURN_DIR) + ',' + str(RECOMMENDED_NAVAID) + ',' + str(MAG_COURSE) + ',' + str(Distance) + ',' + str(ALT_DESCR) + ',' + str(ALT_1) + ',' + str(ALT_2) + ',' + str(SPEED_LIMIT_DESCR) + ',' + str(SPEED_LIMIT) + ',' + '0' + ',' + str(fix_type) + ',' + str(ovfly) + ',' + str(RNP)))
                elif PATH_AND_TERMINATION == 'FA':
                    TerminalLegs_as.append((str(PATH_AND_TERMINATION) + ',' + str(FIX_IDENT) + ',' + str(FIX_LAT) + ',' + str(FIX_LON) + ',' + str(TURN_DIR) + ',' + str(RECOMMENDED_NAVAID) + ',' + '0.0' + ',' + '0.0' + ',' + str(MAG_COURSE) + ',' + str(ALT_DESCR) + ',' + str(ALT_1) + ',' + str(ALT_2) + ',' + str(SPEED_LIMIT_DESCR) + ',' + str(SPEED_LIMIT) + ',' + '0' + ',' + str(fix_type) + ',' + str(ovfly) + ',' + str(RNP)))
                elif PATH_AND_TERMINATION == 'FC':
                    TerminalLegs_as.append((str(PATH_AND_TERMINATION) + ',' + str(FIX_IDENT) + ',' + str(FIX_LAT) + ',' + str(FIX_LON) + ',' + str(TURN_DIR) + ',' + str(RECOMMENDED_NAVAID) + ',' + '0.0' + ',' + '0.0' + ',' + str(MAG_COURSE) + ',' + str(Distance) + ',' + str(ALT_DESCR) + ',' + str(ALT_1) + ',' + str(ALT_2) + ',' + str(SPEED_LIMIT_DESCR) + ',' + str(SPEED_LIMIT) + ',' + '0' + ',' + str(fix_type) + ',' + str(ovfly) + ',' + str(RNP)))
                elif PATH_AND_TERMINATION == 'FD':
                    TerminalLegs_as.append((str(PATH_AND_TERMINATION) + ',' + str(FIX_IDENT) + ',' + str(FIX_LAT) + ',' + str(FIX_LON) + ',' + str(TURN_DIR) + ',' + str(RECOMMENDED_NAVAID) + ',' + '0.0' + ',' + str(Distance) + ',' + str(MAG_COURSE) + ',' + str(Distance) + ',' + str(ALT_DESCR) + ',' + str(ALT_1) + ',' + str(ALT_2) + ',' + str(SPEED_LIMIT_DESCR) + ',' + str(SPEED_LIMIT) + ',' + '0' + ',' + str(fix_type) + ',' + str(ovfly) + ',' + str(RNP)))
                elif PATH_AND_TERMINATION == 'FM':
                    TerminalLegs_as.append((str(PATH_AND_TERMINATION) + ',' + str(FIX_IDENT) + ',' + str(FIX_LAT) + ',' + str(FIX_LON) + ',' + str(TURN_DIR) + ',' + str(RECOMMENDED_NAVAID) + ',' + '0.0' + ',' + '0.0' + ',' + str(MAG_COURSE) + ',' + str(ALT_DESCR) + ',' + str(ALT_1) + ',' + str(ALT_2) + ',' + str(SPEED_LIMIT_DESCR) + ',' + str(SPEED_LIMIT) + ',' + '0' + ',' + str(fix_type) + ',' + str(ovfly) + ',' + str(RNP)))
                elif PATH_AND_TERMINATION == 'HA':
                    TerminalLegs_as.append((str(PATH_AND_TERMINATION) + ',' + str(FIX_IDENT) + ',' + str(FIX_LAT) + ',' + str(FIX_LON) + ',' + str(TURN_DIR) + ',' + str(RECOMMENDED_NAVAID) + ',' + str(THETA) + ',' + str(RHO) + ',' + str(MAG_COURSE) + ',' + str(Distance) + ',' + str(ALT_DESCR) + ',' + str(ALT_1) + ',' + str(ALT_2) + ',' + str(SPEED_LIMIT_DESCR) + ',' + str(SPEED_LIMIT) + ',' + '0' + ',' + str(fix_type) + ',' + str(ovfly) + ',' + ' ' + ',' + str(RNP)))
                elif PATH_AND_TERMINATION == 'HF':
                    TerminalLegs_as.append((str(PATH_AND_TERMINATION) + ',' + str(FIX_IDENT) + ',' + str(FIX_LAT) + ',' + str(FIX_LON) + ',' + str(TURN_DIR) + ',' + str(RECOMMENDED_NAVAID) + ',' + str(THETA) + ',' + str(RHO) + ',' + str(MAG_COURSE) + ',' + str(Distance) + ',' + str(ALT_DESCR) + ',' + str(ALT_1) + ',' + str(ALT_2) + ',' + str(SPEED_LIMIT_DESCR) + ',' + str(SPEED_LIMIT) + ',' + '0' + ',' + str(fix_type) + ',' + str(ovfly) + ',' + ' ' + ',' + str(RNP)))
                elif PATH_AND_TERMINATION == 'HM':
                    TerminalLegs_as.append((str(PATH_AND_TERMINATION) + ',' + str(FIX_IDENT) + ',' + str(FIX_LAT) + ',' + str(FIX_LON) + ',' + str(TURN_DIR) + ',' + str(RECOMMENDED_NAVAID) + ',' + str(THETA) + ',' + str(RHO) + ',' + str(MAG_COURSE) + ',' + str(Distance) + ',' + str(ALT_DESCR) + ',' + str(ALT_1) + ',' + str(ALT_2) + ',' + str(SPEED_LIMIT_DESCR) + ',' + str(SPEED_LIMIT) + ',' + '0' + ',' + str(fix_type) + ',' + str(ovfly) + ',' + ' ' + ',' + str(RNP)))
                elif PATH_AND_TERMINATION == 'IF':
                    TerminalLegs_as.append((str(PATH_AND_TERMINATION) + ',' + str(FIX_IDENT) + ',' + str(FIX_LAT) + ',' + str(FIX_LON) + ',' + str(RECOMMENDED_NAVAID) + ',' + str(THETA) + ',' + str(RHO) + ',' + str(ALT_DESCR) + ',' + str(ALT_1) + ',' + str(ALT_2) + ',' + str(SPEED_LIMIT_DESCR) + ',' + str(SPEED_LIMIT) + ',' + '0' + ',' + str(fix_type) + ',' + str(ovfly) + ',' + str(RNP)))
                elif PATH_AND_TERMINATION == 'PI':
                    THETA = MAG_COURSE
                    RHO = Distance
                    if TURN_DIR == 1:
                        MAG_COURSE = MAG_COURSE - 45
                        if MAG_COURSE < 0:
                            MAG_COURSE = MAG_COURSE + 360
                    else:
                        MAG_COURSE = MAG_COURSE + 45
                        if MAG_COURSE > 360:
                            MAG_COURSE = MAG_COURSE - 360
                    Distance = 5
                    TerminalLegs_as.append((str(PATH_AND_TERMINATION) + ',' + str(FIX_IDENT) + ',' + str(FIX_LAT) + ',' + str(FIX_LON) + ',' + str(TURN_DIR) + ',' + str(RECOMMENDED_NAVAID) + ',' + str(THETA) + ',' + str(RHO) + ',' + str(MAG_COURSE) + ',' + str(Distance) + ',' + str(ALT_DESCR) + ',' + str(ALT_1) + ',' + str(ALT_2) + ',' + str(SPEED_LIMIT_DESCR) + ',' + str(SPEED_LIMIT) + ',' + '0' + ',' + str(fix_type) + ',' + str(ovfly) + ',' + str(RNP)))
                elif PATH_AND_TERMINATION == 'RF':
                    TerminalLegs_as.append((str(PATH_AND_TERMINATION) + ',' + str(FIX_IDENT) + ',' + str(FIX_LAT) + ',' + str(FIX_LON) + ',' + str(TURN_DIR) + ',' + str(RECOMMENDED_NAVAID) + ',' + str(THETA) + ',' + str(RHO) + ',' + str(ALT_DESCR) + ',' + str(ALT_1) + ',' + str(ALT_2) + ',' + str(SPEED_LIMIT_DESCR) + ',' + str(SPEED_LIMIT) + ',' + '0' + ',' + str(fix_type) + ',' + str(ovfly) + ',' + str(RNP)))
                elif PATH_AND_TERMINATION == 'TF':
                    TerminalLegs_as.append((str(PATH_AND_TERMINATION) + ',' + str(FIX_IDENT) + ',' + str(FIX_LAT) + ',' + str(FIX_LON) + ',' + str(TURN_DIR) + ',' + str(RECOMMENDED_NAVAID) + ',' + str(THETA) + ',' + str(RHO) + ',' + str(MAG_COURSE) + ',' + str(Distance) + ',' + str(ALT_DESCR) + ',' + str(ALT_1) + ',' + str(ALT_2) + ',' + str(SPEED_LIMIT_DESCR) + ',' + str(SPEED_LIMIT) + ',' + '0' + ',' + str(fix_type) + ',' + str(ovfly) + ',' + str(RNP)))
                elif PATH_AND_TERMINATION == 'VA':
                    TerminalLegs_as.append((str(PATH_AND_TERMINATION) + ',' + str(TURN_DIR) + ',' + str(MAG_COURSE) + ',' + str(ALT_DESCR) + ',' + str(ALT_1) + ',' + str(ALT_2) + ',' + str(SPEED_LIMIT_DESCR) + ',' + str(SPEED_LIMIT) + ',' + '0' + ',' + str(fix_type) + ',' + str(ovfly) + ',' + str(RNP)))
                elif PATH_AND_TERMINATION == 'VD':
                    TerminalLegs_as.append((str(PATH_AND_TERMINATION) + ',' + str(FIX_IDENT) + ',' + str(FIX_LAT) + ',' + str(FIX_LON) + ',' + str(TURN_DIR) + ',' + str(RECOMMENDED_NAVAID) + ',' + str(THETA) + ',' + str(RHO) + ',' + str(MAG_COURSE) + ',' + str(Distance) + ',' + str(ALT_DESCR) + ',' + str(ALT_1) + ',' + str(ALT_2) + ',' + str(SPEED_LIMIT_DESCR) + ',' + str(SPEED_LIMIT) + ',' + '0' + ',' + str(fix_type) + ',' + str(ovfly) + ',' + str(RNP)))
                elif PATH_AND_TERMINATION == 'VI':
                    TerminalLegs_as.append((str(PATH_AND_TERMINATION) + ',' + str(TURN_DIR) + ',' + str(RECOMMENDED_NAVAID) + ',' + str(RHO) + ',' + str(MAG_COURSE) + ',' + str(ALT_DESCR) + ',' + str(ALT_1) + ',' + str(ALT_2) + ',' + str(SPEED_LIMIT_DESCR) + ',' + str(SPEED_LIMIT) + ',' + '0' + ',' + str(fix_type) + ',' + str(ovfly) + ',' + str(RNP)))
                elif PATH_AND_TERMINATION == 'VM':
                    TerminalLegs_as.append((str(PATH_AND_TERMINATION) + ',' + str(TURN_DIR) + ',' + str(THETA) + ',' + str(RHO) + ',' + str(MAG_COURSE) + ',' + str(ALT_DESCR) + ',' + str(ALT_1) + ',' + str(ALT_2) + ',' + str(SPEED_LIMIT_DESCR) + ',' + str(SPEED_LIMIT) + ',' + '0' + ',' + str(fix_type) + ',' + str(ovfly) + ',' + str(RNP)))
                elif PATH_AND_TERMINATION == 'VR':
                    TerminalLegs_as.append((str(PATH_AND_TERMINATION) + ',' + str(TURN_DIR) + ',' + str(RECOMMENDED_NAVAID) + ',' + str(THETA) + ',' + str(MAG_COURSE) + ',' + str(ALT_DESCR) + ',' + str(ALT_1) + ',' + str(ALT_2) + ',' + str(SPEED_LIMIT_DESCR) + ',' + str(SPEED_LIMIT) + ',' + '0' + ',' + str(fix_type) + ',' + str(ovfly) + ',' + str(RNP)))
                else:
                    print('         Aerosoft: Leg ' + ARPT_IDENT + ' ' + proc_se.group(7) + ' ' + proc_se.group(11) + ' Path Code Invalid')
                if IsEnd == 1:
                    TerminalLegs_as_tb = pandas.DataFrame(TerminalLegs_as, columns=['DATA'])
                    TerminalLegs_as = []
                    with open(Cycle_Ident + '/Aerosoft/PROC/' + ARPT_IDENT + '.txt', 'a+') as PROC:
                        if SUBS_CODE == 'D':
                            # Solution for 'RWxxB'
                            if proc_se.group(9)[0:2] + proc_se.group(9)[2:5].strip(string.digits) != 'RWB':
                                print('SID' + ',' + PROC_IDENT + ',' + TRANSITION_IDENT + ',' + ROUTE_TYPE, file=PROC)
                                TerminalLegs_as_tb.to_csv(path_or_buf = PROC, mode = 'a+', index = False, header = False, sep='|', line_terminator='\n')
                                print('', file=PROC)
                            else:
                                RWY_ID = Runways_as_tb.loc[(Runways_as_tb['ARPT_IDENT'] == ARPT_IDENT)].index.tolist()
                                if RWY_ID != []:
                                    for RWY in RWY_ID:
                                        if str(Runways_as_tb.at[RWY, 'RUNWAY_IDENT'])[0:2] == proc_se.group(9)[2:4].strip():
                                            print('SID' + ',' + PROC_IDENT + ',' + str(Runways_as_tb.at[RWY, 'RUNWAY_IDENT']) + ',' + ROUTE_TYPE, file=PROC)
                                            TerminalLegs_as_tb.to_csv(path_or_buf = PROC, mode = 'a+', index = False, header = False, sep='|', line_terminator='\n')
                                            print('', file=PROC)
                                else:
                                    if as_create_blank != True:
                                        RWY_ID = Runways_in_as_nd_tb.loc[(Runways_in_as_nd_tb['ARPT_IDENT'] == ARPT_IDENT)].index.tolist()
                                        if RWY_ID != []:
                                            for RWY in RWY_ID:
                                                if str(Runways_in_as_nd_tb.at[RWY, 'RUNWAY_IDENT'])[0:2] == proc_se.group(9)[2:4].strip():
                                                    print('SID' + ',' + PROC_IDENT + ',' + str(Runways_in_as_nd_tb.at[RWY, 'RUNWAY_IDENT']) + ',' + ROUTE_TYPE, file=PROC)
                                                    TerminalLegs_as_tb.to_csv(path_or_buf = PROC, mode = 'a+', index = False, header = False, sep='|', line_terminator='\n')
                                                    print('', file=PROC)
                                        else:
                                            print('         Aerosoft: Procedure ' + ARPT_IDENT + ' ' + proc_se.group(7) + ' Airport Not Found')
                                            print('SID' + ',' + PROC_IDENT + ',' + TRANSITION_IDENT + ',' + ROUTE_TYPE, file=PROC)
                                            TerminalLegs_as_tb.to_csv(path_or_buf = PROC, mode = 'a+', index = False, header = False, sep='|', line_terminator='\n')
                                            print('', file=PROC)
                                    else:
                                        print('         Aerosoft: Procedure ' + ARPT_IDENT + ' ' + proc_se.group(7) + ' Airport Not Found')
                                        print('SID' + ',' + PROC_IDENT + ',' + TRANSITION_IDENT + ',' + ROUTE_TYPE, file=PROC)
                                        TerminalLegs_as_tb.to_csv(path_or_buf = PROC, mode = 'a+', index = False, header = False, sep='|', line_terminator='\n')
                                        print('', file=PROC)
                        elif SUBS_CODE == 'E':
                            # Solution for 'RWxxB'
                            if proc_se.group(9)[0:2] + proc_se.group(9)[2:5].strip(string.digits) != 'RWB':
                                print('STAR' + ',' + PROC_IDENT + ',' + TRANSITION_IDENT + ',' + ROUTE_TYPE, file=PROC)
                                TerminalLegs_as_tb.to_csv(path_or_buf = PROC, mode = 'a+', index = False, header = False, sep='|', line_terminator='\n')
                                print('', file=PROC)
                            else:
                                RWY_ID = Runways_as_tb.loc[(Runways_as_tb['ARPT_IDENT'] == ARPT_IDENT)].index.tolist()
                                if RWY_ID != []:
                                    for RWY in RWY_ID:
                                        if str(Runways_as_tb.at[RWY, 'RUNWAY_IDENT'])[0:2] == proc_se.group(9)[2:4].strip():
                                            print('STAR' + ',' + PROC_IDENT + ',' + str(Runways_as_tb.at[RWY, 'RUNWAY_IDENT']) + ',' + ROUTE_TYPE, file=PROC)
                                            TerminalLegs_as_tb.to_csv(path_or_buf = PROC, mode = 'a+', index = False, header = False, sep='|', line_terminator='\n')
                                            print('', file=PROC)
                                else:
                                    if as_create_blank != True:
                                        RWY_ID = Runways_in_as_nd_tb.loc[(Runways_in_as_nd_tb['ARPT_IDENT'] == ARPT_IDENT)].index.tolist()
                                        if RWY_ID != []:
                                            for RWY in RWY_ID:
                                                if str(Runways_in_as_nd_tb.at[RWY, 'RUNWAY_IDENT'])[0:2] == proc_se.group(9)[2:4].strip():
                                                    print('STAR' + ',' + PROC_IDENT + ',' + str(Runways_in_as_nd_tb.at[RWY, 'RUNWAY_IDENT']) + ',' + ROUTE_TYPE, file=PROC)
                                                    TerminalLegs_as_tb.to_csv(path_or_buf = PROC, mode = 'a+', index = False, header = False, sep='|', line_terminator='\n')
                                                    print('', file=PROC)
                                        else:
                                            print('         Aerosoft: Procedure ' + ARPT_IDENT + ' ' + proc_se.group(7) + ' Airport Not Found')
                                            print('STAR' + ',' + PROC_IDENT + ',' + TRANSITION_IDENT + ',' + ROUTE_TYPE, file=PROC)
                                            TerminalLegs_as_tb.to_csv(path_or_buf = PROC, mode = 'a+', index = False, header = False, sep='|', line_terminator='\n')
                                            print('', file=PROC)
                                    else:
                                        print('         Aerosoft: Procedure ' + ARPT_IDENT + ' ' + proc_se.group(7) + ' Airport Not Found')
                                        print('STAR' + ',' + PROC_IDENT + ',' + TRANSITION_IDENT + ',' + ROUTE_TYPE, file=PROC)
                                        TerminalLegs_as_tb.to_csv(path_or_buf = PROC, mode = 'a+', index = False, header = False, sep='|', line_terminator='\n')
                                        print('', file=PROC)
                        elif ROUTE_TYPE == 'A':
                            # Solution for APCHs with no Runway Number
                            if proc_se.group(7)[1:3].strip('- ').strip(string.digits) == '':
                                RUNWAY_IDENT = proc_se.group(7)[1:4].strip('- ')
                                print('APPTR' + ',' + PROC_IDENT + ',' + RUNWAY_IDENT + ',' + TRANSITION_IDENT, file=PROC)
                                TerminalLegs_as_tb.to_csv(path_or_buf = PROC, mode = 'a+', index = False, header = False, sep='|', line_terminator='\n')
                                print('', file=PROC)
                            else:
                                RWY_ID = Runways_as_tb.loc[(Runways_as_tb['ARPT_IDENT'] == ARPT_IDENT)].index.tolist()
                                ROUTE_TYPE = ApproachesLookup_as_tb.loc[ApproachesLookup_as_tb.loc[(ApproachesLookup_as_tb['ARPT_IDENT'] == ARPT_IDENT) & (ApproachesLookup_as_tb['PROC_IDENT'] == PROC_IDENT)].index.tolist() , 'ROUTE_TYPE'].values[0]
                                if RWY_ID != []:
                                    for RWY in RWY_ID:
                                        if len(str(Runways_as_tb.at[RWY, 'RUNWAY_IDENT'])) == 2:
                                            PROC_IDENT = ROUTE_TYPE + str(Runways_as_tb.at[RWY, 'RUNWAY_IDENT']) + '-' + PROC_IDENT[-1]
                                        else:
                                            PROC_IDENT = ROUTE_TYPE + str(Runways_as_tb.at[RWY, 'RUNWAY_IDENT']) + '' + PROC_IDENT[-1]
                                        print('APPTR' + ',' + PROC_IDENT + ',' + str(Runways_as_tb.at[RWY, 'RUNWAY_IDENT']) + ',' + TRANSITION_IDENT, file=PROC)
                                        TerminalLegs_as_tb.to_csv(path_or_buf = PROC, mode = 'a+', index = False, header = False, sep='|', line_terminator='\n')
                                        print('', file=PROC)
                                else:
                                    if as_create_blank != True:
                                        RWY_ID = Runways_in_as_nd_tb.loc[(Runways_in_as_nd_tb['ARPT_IDENT'] == ARPT_IDENT)].index.tolist()
                                        if RWY_ID != []:
                                            for RWY in RWY_ID:
                                                if len(str(Runways_in_as_nd_tb.at[RWY, 'RUNWAY_IDENT'])) == 2:
                                                    PROC_IDENT = ROUTE_TYPE + str(Runways_in_as_nd_tb.at[RWY, 'RUNWAY_IDENT']) + '-' + PROC_IDENT[-1]
                                                else:
                                                    PROC_IDENT = ROUTE_TYPE + str(Runways_in_as_nd_tb.at[RWY, 'RUNWAY_IDENT']) + '' + PROC_IDENT[-1]
                                                print('APPTR' + ',' + PROC_IDENT + ',' + str(Runways_in_as_nd_tb.at[RWY, 'RUNWAY_IDENT']) + ',' + TRANSITION_IDENT, file=PROC)
                                                TerminalLegs_as_tb.to_csv(path_or_buf = PROC, mode = 'a+', index = False, header = False, sep='|', line_terminator='\n')
                                                print('', file=PROC)
                                        else:
                                            print('         Aerosoft: Procedure ' + ARPT_IDENT + ' ' + proc_se.group(7) + ' No Available Runway')
                                    else:
                                        print('         Aerosoft: Procedure ' + ARPT_IDENT + ' ' + proc_se.group(7) + ' No Available Runway')
                        else:
                            # Solution for APCHs with no Runway Number
                            if proc_se.group(7)[1:3].strip('- ').strip(string.digits) == '':
                                RUNWAY_IDENT = proc_se.group(7)[1:4].strip('- ')
                                print('FINAL' + ',' + PROC_IDENT + ',' + RUNWAY_IDENT + ',' + PROC_IDENT[0:1] + ',' + str(MISSED_APCH_COUNT) + ',' + '0' + ',' + ' ' + ',' + '0' + ',' + '0' + ',' + str(have_glide_path) + ',' + str(VERTICAL_ANGLE), file=PROC)
                                TerminalLegs_as_tb.to_csv(path_or_buf = PROC, mode = 'a+', index = False, header = False, sep='|', line_terminator='\n')
                                print('', file=PROC)
                            else:
                                RWY_ID = Runways_as_tb.loc[(Runways_as_tb['ARPT_IDENT'] == ARPT_IDENT)].index.tolist()
                                ROUTE_TYPE = ApproachesLookup_as_tb.loc[ApproachesLookup_as_tb.loc[(ApproachesLookup_as_tb['ARPT_IDENT'] == ARPT_IDENT) & (ApproachesLookup_as_tb['PROC_IDENT'] == PROC_IDENT)].index.tolist() , 'ROUTE_TYPE'].values[0]
                                if RWY_ID != []:
                                    for RWY in RWY_ID:
                                        if len(str(Runways_as_tb.at[RWY, 'RUNWAY_IDENT'])) == 2:
                                            PROC_IDENT = ROUTE_TYPE + str(Runways_as_tb.at[RWY, 'RUNWAY_IDENT']) + '-' + PROC_IDENT[-1]
                                        else:
                                            PROC_IDENT = ROUTE_TYPE + str(Runways_as_tb.at[RWY, 'RUNWAY_IDENT']) + '' + PROC_IDENT[-1]
                                        print('FINAL' + ',' + PROC_IDENT + ',' + str(Runways_as_tb.at[RWY, 'RUNWAY_IDENT']) + ',' + PROC_IDENT[0:1] + ',' + str(MISSED_APCH_COUNT) + ',' + '0' + ',' + ' ' + ',' + '0' + ',' + '0' + ',' + str(have_glide_path) + ',' + str(VERTICAL_ANGLE), file=PROC)
                                        TerminalLegs_as_tb.to_csv(path_or_buf = PROC, mode = 'a+', index = False, header = False, sep='|', line_terminator='\n')
                                        print('', file=PROC)
                                else:
                                    if as_create_blank != True:
                                        RWY_ID = Runways_in_as_nd_tb.loc[(Runways_in_as_nd_tb['ARPT_IDENT'] == ARPT_IDENT)].index.tolist()
                                        if RWY_ID != []:
                                            for RWY in RWY_ID:
                                                if len(str(Runways_in_as_nd_tb.at[RWY, 'RUNWAY_IDENT'])) == 2:
                                                    PROC_IDENT = ROUTE_TYPE + str(Runways_in_as_nd_tb.at[RWY, 'RUNWAY_IDENT']) + '-' + PROC_IDENT[-1]
                                                else:
                                                    PROC_IDENT = ROUTE_TYPE + str(Runways_in_as_nd_tb.at[RWY, 'RUNWAY_IDENT']) + '' + PROC_IDENT[-1]
                                                print('FINAL' + ',' + PROC_IDENT + ',' + str(Runways_in_as_nd_tb.at[RWY, 'RUNWAY_IDENT']) + ',' + PROC_IDENT[0:1] + ',' + str(MISSED_APCH_COUNT) + ',' + '0' + ',' + ' ' + ',' + '0' + ',' + '0' + ',' + str(have_glide_path) + ',' + str(VERTICAL_ANGLE), file=PROC)
                                                TerminalLegs_as_tb.to_csv(path_or_buf = PROC, mode = 'a+', index = False, header = False, sep='|', line_terminator='\n')
                                                print('', file=PROC)
                                        else:
                                            print('         Aerosoft: Procedure ' + ARPT_IDENT + ' ' + proc_se.group(7) + ' No Available Runway')
                                    else:
                                        print('         Aerosoft: Procedure ' + ARPT_IDENT + ' ' + proc_se.group(7) + ' No Available Runway')
                            MISSED_APCH = 0
                            MISSED_APCH_COUNT = 0
        print(time.strftime("%H:%M:%S", time.localtime()) , 'Aerosoft: Data Process Part #4 Finished')
    # Write Data
    if as_write:
        Navaids_as_tb = Navaids_as_tb.drop(columns=['ARPT_IDENT'])
        Navaids_as_tb = Navaids_as_tb.sort_values(['NAV_IDENT','NAV_LAT','NAV_LON'])
        Waypoints_as_tb = Waypoints_as_tb.drop(columns=['REGION_CODE'])
        Waypoints_as_tb = Waypoints_as_tb.sort_values(['WAYPOINT_IDENT','WAYPOINT_LAT','WAYPOINT_LON'])
        Navaids_as_tb.to_csv(path_or_buf = Cycle_Ident + '/Aerosoft/Navaids.txt', mode = 'a', header = False, index = False)
        Waypoints_as_tb.to_csv(path_or_buf = Cycle_Ident + '/Aerosoft/Waypoints.txt', mode = 'a', header = False, index = False)
        # Sort Waypoints and Navaids
        if as_create_blank != True:
            Waypoints_as_tb = pandas.read_csv(Cycle_Ident + '/Aerosoft/Waypoints.txt',header=None,index_col=False,names=['WAYPOINT_IDENT','WAYPOINT_LAT','WAYPOINT_LON','WAYPOINT_ICAO_CODE'],dtype='string',keep_default_na=False)
            Waypoints_as_tb = Waypoints_as_tb.sort_values(['WAYPOINT_IDENT','WAYPOINT_LAT','WAYPOINT_LON'])
            Waypoints_as_tb.to_csv(path_or_buf = Cycle_Ident + '/Aerosoft/Waypoints.txt', mode = 'w', index = False, header = False)
            Navaids_as_tb = pandas.read_csv(Cycle_Ident + '/Aerosoft/Navaids.txt',header=None,index_col=False,names=['NAV_IDENT','NAV_NAME','NAV_FREQ','is_vor','is_dme','RANGE','NAV_LAT','NAV_LON','ELEV','NAV_ICAO_CODE','UNK','is_ndb','MAG_VAR','ILS_DME_BIAS'],dtype='string',keep_default_na=False)
            Navaids_as_tb = Navaids_as_tb.sort_values(['NAV_IDENT','NAV_LAT','NAV_LON'])
            Navaids_as_tb.to_csv(path_or_buf = Cycle_Ident + '/Aerosoft/Navaids.txt', mode = 'w', index = False, header = False)
    print(time.strftime("%H:%M:%S", time.localtime()) , 'Aerosoft: Data Export Finished')

print(time.strftime("%H:%M:%S", time.localtime()) , 'System: DataFile Convert Finished')
